In [3]:
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import TimeoutException
import selenium.common.exceptions as Exception
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver
import socket

from xml.etree import ElementTree
from bs4 import BeautifulSoup
import multiprocessing as mp
import pandas as pd
import numpy as np
import pyautogui
import re as re

import geopy.distance
import tldextract
import requests
import time
import os

import csv
from datetime import datetime


In [5]:
chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--window-size=1920x1080")

driver = webdriver.Chrome(options=chrome_options)

### Webpage Wait Functions

In [4]:
def wait_then_do(by, locator, ec, func = lambda x: x):
    try:
        element = WebDriverWait(driver, 10).until(ec((by, locator)))
        return func(element)
    except:
        raise Exception(f'timed out waiting for {locator}')
        
def wait_all_then_get(by, locator):
    return wait_then_do(by, locator, EC.presence_of_all_elements_located)

def wait_present_then_do(by, locator, func):
    return wait_then_do(by, locator, EC.presence_of_element_located, func)
    
def wait_clickable_then_do(by, locator, func):
    return wait_then_do(by, locator, EC.element_to_be_clickable, func)
    
def wait_then_click_elem(by, locator):
    return wait_clickable_then_do(by, locator, lambda elem: elem.click())
    
def wait_then_send_keys(by, locator, keys):
    return wait_clickable_then_do(by, locator, lambda elem: elem.send_keys(keys))

### Functions to get Url List

In [10]:
def get_local_shop_urls(locality):
    url = "https://dutchie.com/dispensaries"
    # navigate to url
    driver.get(url)
    
    # wait for age verification, and click 'Yes'
    #wait_then_click_elem(By.XPATH, "/html/body/div[3]/div[3]/div[1]/div[1]/button")
    # wait for search area input to show, then enter locality into input (ex. Fairfield, CT)
    #wait_then_send_keys(By.ID, "enter-your-address-input", locality)
    # wait for results, then click top option to load local dispensaries
    #wait_then_click_elem(By.XPATH, "//li[@data-testid='addressAutocompleteOption']")
    # wait for age verification, and click 'Yes'
    wait_then_click_elem(By.CLASS_NAME, "age-confirmation-modal__StyledButton-di8wrk-0")
    # wait for search area input to show, then enter locality into input (ex. Fairfield, CT)
    wait_then_send_keys(By.ID, "enter-your-address-input", locality)
    # wait for results, then click top option to load local dispensaries
    wait_then_click_elem(By.CLASS_NAME, "option__StyledComboboxOption-sc-1e884xj-1")
    
    # wait for dispensary list, then retrieve and return
    dispensaries = wait_all_then_get(By.CLASS_NAME, "dispensary-card__Container-sc-1wd9p5b-0")
    # extract urls from list
    return [d.get_attribute("href") for d in dispensaries]

### Function to get Product Info (Flower)

In [7]:
# Retrieves product info (flower only) from given dispensary website
# 
# example usage:
#     info = get_products('https://dutchie.com/dispensary/verb-is-herb')
#     info = get_products('https://dutchie.com/dispensary/theory-wellness-chicopee-med')
# ------------------------------------------------
def get_products(shop_url):
    info = []
    try:
        # navigate to product page
        driver.get(f'{shop_url}/products/flower')
        time.sleep(2)
        # wait for product list to show
        wait_then_do(By.CLASS_NAME, "jTTcVS", EC.presence_of_all_elements_located)
        #wait_then_do(By.CLASS_NAME, "jTzrhu", EC.presence_of_all_elements_located)
        # scroll to lazy load list items
        html = driver.find_element(By.TAG_NAME, 'html')
        for i in range(25):
            html.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.02)

        # get name of dispenary
        dispo_name = wait_present_then_do(By.CLASS_NAME, "incqNo", lambda x: x.text)
        
        # get product list from webpage, then extract product data from each list item
        
        products = wait_all_then_get(By.CLASS_NAME, "jTzrhU")
        for p in products:
            product_url = p.find_element(By.CLASS_NAME, "jTTcVS").get_attribute("href")
            name = p.find_element(By.CLASS_NAME, "kjymBK").text
            #print("name", name)
            sih = p.find_elements(By.CLASS_NAME, "qkQlV")
            #print("sih", sih)
            thc = p.find_element(By.CLASS_NAME, "hdncuE").text
            #print(f"thc: {thc}")
            weights = p.find_elements(By.CLASS_NAME, "dLcrTv")
            weight_list = []
            for w in weights:
                if w != " - ":
                    w = w.text
                    weight_list.append(w)
            #print(f"weights:{weight_list}")
            prices = p.find_elements(By.CLASS_NAME, "jMsbLi")
            price_list = []
            for p in prices:
                if p != "$":
                    p = p.text
                    price_list.append(p)
            #print(f"prices:{price_list}")
            

            for i in range(len(weights)):
                info.append([name, sih[0].text if len(sih) > 0 else '', thc, weights[i].text.strip('-').strip(), prices[i].text, product_url, dispo_name])
                #print("Unsorted:", name, sih[0].text if len(sih) > 0 else '', thc, weights[i].text.strip('-').strip(), prices[i].text, product_url, dispo_name)
        
    except TimeoutException:
        info.append(["Error: Timeout Exception", "NA", "NA", "NA", "NA", "NA", "NA"])
        pass
    except TypeError:
        info.append(["Error: Type Error", "NA", "NA", "NA", "NA", "NA", "NA"])
        pass

    # product data extracted into 2d array
    return info

In [8]:
def ppo_(type, x, price):
    if type == "Flower":
        if x == "1 oz": a = 1
        elif x == "4 oz": a = 1/4
        elif x == "1/2 oz": a = 2
        elif x == "1/4 oz": a = 4
        elif x == "1/8 oz": a = 8
        elif x == "28g": a = 1
        elif x == "20g": a = 28.0/20.0
        elif x == "14.17g": a = 2
        elif x == "14g": a = 2
        elif x == "8.6g": a = 28.0/8.6
        elif x == "7g": a = 4
        elif x == "5g": a = 28/5
        elif x == "4g": a = 7
        elif x == "3.5g": a = 8
        elif x == "2.5g": a= 4
        elif x == "2g": a = 14
        elif x == "2.0g": a = 14
        elif x == "1g": a = 28
        elif x == "1.0g": a = 28
        else: a=0 
        return a*price

In [9]:
# Converts product info from 2d array to formatted pandas dataframe
# 
# example usage:
#     df = arr_to_df(info)
# ------------------------------------------------
def arr_to_df(arr):
    products = pd.DataFrame(columns=['Strain', 'Type', 'TAC', 'THC', 'Amount', 'Price', 'URL', 'Dispensary', 'PPO', 'mgp'])
    
    for row in arr:
        name = row[0]      #[:row[0].find('(')-1]
        
        t = row[1]
        if not t:
            t = row[0][len(name)+2: row[0].find(')')]
            t = t.replace('I', 'Indica').replace('S', 'Sativa').replace('H', 'Hybrid')
        
        potency = row[2].split('|')
        
        tac, thc = '', ''
        for v in potency:
            s = v.strip()
            if s[:3] == 'THC':
                thc = s[4:]
            elif s[:3] == 'TAC':
                tac = s[4:]
        tac = tac[tac.find(':')+1:].strip()
        thc = thc[thc.find(':')+1:].strip()
        
        o_i = row[3].find(' oz')
        weight_f = f'{row[3][:-1]}/28' if o_i < 0 else row[3][:o_i]
        s_i = weight_f.find('/')
        weight = float(weight_f) if s_i < 0 else float(weight_f[:s_i]) / float(weight_f[s_i+1:])
    
        price = float(row[4][1:])

        #---Price per OZ-------------------------------------------------------------------------------------------------------------------
        x = row[3].strip()
        ppo = ppo_("Flower", x, price)

        #---mg THC per Dollar---------------------------------------------------------------------------------------------------------------
        p = 0
        if thc:
            if thc[-1] == '%':
                p = float(thc[thc.rfind(' ')+1:-1]) *28
            elif thc[-1] == 'g':
                p = float(thc.split()[0])
            else:
                p = float(thc[:thc.find(' ')]) * 28
        elif p >= 1 and thc:
            if thc[-1] == '%':
                p = float(thc[thc.rfind(' ')+1:-1]) * 28
            else:
                p = float(thc[:thc.find(' ')]) * 28

        if ppo > 0:          
            mgp = p/ppo
        else: mgp=0
        '''p = 0
        if tac:
            if tac[-1] == '%':
                p = float(tac[tac.rfind(' ')+1:-1]) / 100
            else:
                p = float(tac[:tac.find(' ')]) / 100
        elif p >= 1 and thc:
            if thc[-1] == '%':
                p = float(thc[thc.rfind(' ')+1:-1]) / 100
            else:
                p = float(thc[:thc.find(' ')]) / 100

        if p >= 1:
            p = 0
                
        value = weight * p * 28000 / price'''
      
        
        products.loc[len(products)] = [name, t, tac, thc, row[3], price, row[5], row[6], ppo, mgp]
        #print("Sorted:", name, t, tac, thc, row[3], price, row[5], row[6], value)
        
    return products

In [11]:
urls = get_local_shop_urls('Chicopee, MA')
print(urls)

['https://dutchie.com/dispensary/311-page-springfield', 'https://dutchie.com/dispensary/dazed-cannabis', 'https://dutchie.com/dispensary/insa-inc-springfield-adult-use', 'https://dutchie.com/dispensary/holyoke', 'https://dutchie.com/dispensary/canna-provisions-holyoke', 'https://dutchie.com/dispensary/leaf-lux-retail-rec', 'https://dutchie.com/dispensary/enlite-indian-orchard', 'https://dutchie.com/dispensary/insa-easthampton-rec', 'https://dutchie.com/dispensary/insa-easthampton', 'https://dutchie.com/dispensary/dazed-retail-monson', 'https://dutchie.com/dispensary/cannabis-connection-west-springfield', 'https://dutchie.com/dispensary/theory-wellness-chicopee', 'https://dutchie.com/dispensary/theory-wellness-chicopee-med', 'https://dutchie.com/dispensary/6-brickss-springfield', 'https://dutchie.com/dispensary/east-coast-pharms', 'https://dutchie.com/dispensary/mass-alternative-care-med', 'https://dutchie.com/dispensary/mass-alternative-care', 'https://dutchie.com/dispensary/jimbuddys'

In [12]:
urls = ['https://dutchie.com/dispensary/cannabis-connection-west-springfield', 
        'https://dutchie.com/dispensary/311-page-springfield', 
        'https://dutchie.com/dispensary/theory-wellness-chicopee', 
        'https://dutchie.com/dispensary/theory-wellness-chicopee-med', 
        'https://dutchie.com/dispensary/6-brickss-springfield', 
        'https://dutchie.com/dispensary/east-coast-pharms', 
        'https://dutchie.com/dispensary/dazed-cannabis', 
        'https://dutchie.com/dispensary/mass-alternative-care-med', 
        'https://dutchie.com/dispensary/mass-alternative-care', 
        'https://dutchie.com/dispensary/jimbuddys', 
        'https://dutchie.com/dispensary/boston-bud-factory', 
        'https://dutchie.com/dispensary/insa-inc-springfield-adult-use', 
        'https://dutchie.com/dispensary/insa-springfield', 
        'https://dutchie.com/dispensary/holyoke', 
        'https://dutchie.com/dispensary/canna-provisions-holyoke', 
        'https://dutchie.com/dispensary/leaf-lux-retail-rec', 
        'https://dutchie.com/dispensary/liberty-springfield', 
        'https://dutchie.com/dispensary/heka', 
        'https://dutchie.com/dispensary/enlite-indian-orchard', 
        'https://dutchie.com/dispensary/cannabis-connection', 
        'https://dutchie.com/dispensary/dreamer', 
        'https://dutchie.com/dispensary/verb-is-herb', 
        #'https://dutchie.com/dispensary/insa-easthampton-rec', 
        'https://dutchie.com/dispensary/insa-easthampton', 
        'https://dutchie.com/dispensary/fyre-ants', 
        'https://dutchie.com/dispensary/neta-northampton', 
        'https://dutchie.com/dispensary/neta-northampton-med', 
        'https://dutchie.com/dispensary/resinate-northampton', 
        'https://dutchie.com/dispensary/honey-northhampton', 
        'https://dutchie.com/dispensary/cannabis-culture1', 
        'https://dutchie.com/dispensary/dazed-retail-monson', 
        'https://dutchie.com/dispensary/embr-northhampton', 
        'https://dutchie.com/dispensary/turning-leaf', 
        'https://dutchie.com/dispensary/enlite', 
        #'https://dutchie.com/dispensary/toroverde-northhampton', 
        'https://dutchie.com/dispensary/clear-sky-belchertown', 
        'https://dutchie.com/dispensary/silver-therapeutics-palmer-retail', 
        'https://dutchie.com/dispensary/liberty-easthampton', 
        'https://dutchie.com/dispensary/liberty-easthampton-ma-med']



In [75]:
# test_urls = ['https://dutchie.com/dispensary/verb-is-herb', 'https://dutchie.com/dispensary/theory-wellness-chicopee', 'https://dutchie.com/dispensary/east-coast-pharms', 'https://dutchie.com/dispensary/cannabis-connection']
#test_urls = urls
#urls = ['https://dutchie.com/dispensary/verb-is-herb']
frames = []
i=1
for u in urls:
    p = get_products(u)
    #print('p', p)
    q = arr_to_df(p)
    #print('q', q)
    frames.append(q)
    print(i)
    i+=1

combined = pd.concat(frames)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37


In [76]:
combined = pd.concat(frames)

In [88]:
#combined.sort_values('Value (mgTAC/$)', ascending=False, inplace=True)
# combined.sort_values(['TAC', 'THC', 'Price'], ascending=[False, False, True], inplace=True)
print(len(combined))
combined.head(30)

2754


,Strain,Type,TAC,THC,Amount,Price,URL,Dispensary,PPO,mgp
0,Valley Vixen,Hybrid,28.1%,26.9%,1/8 oz,35.0,https://dutchie.com/dispensary/cannabis-connec...,Cannabis Connection West Springfield,280.0,2.690000
1,Maya,Hybrid,29.4%,29%,1/8 oz,40.0,https://dutchie.com/dispensary/cannabis-connec...,Cannabis Connection West Springfield,320.0,2.537500
2,Larry Lovestein #1,Hybrid,31.4%,30.6%,1/8 oz,45.0,https://dutchie.com/dispensary/cannabis-connec...,Cannabis Connection West Springfield,360.0,2.380000
3,Poddy Mouth,Hybrid,29.4%,28.3%,1/8 oz,30.0,https://dutchie.com/dispensary/cannabis-connec...,Cannabis Connection West Springfield,240.0,3.301667
4,Blueberry Gas,Hybrid,25%,24%,1/8 oz,40.0,https://dutchie.com/dispensary/cannabis-connec...,Cannabis Connection West Springfield,320.0,2.100000
5,Glue Sniffer,Hybrid,26.2%,22.8%,1/8 oz,45.0,https://dutchie.com/dispensary/cannabis-connec...,Cannabis Connection West Springfield,360.0,1.773333
6,So F'n Gassy - Pre-Ground,Hybrid,37.2%,35.4%,1/4 oz,70.0,https://dutchie.com/dispensary/cannabis-connec...,Cannabis Connection West Springfield,280.0,3.540000
7,Super Silver Pupil W/Papers,Hybrid,25.9%,24.4%,1/2 oz,80.0,https://dutchie.com/dispensary/cannabis-connec...,Cannabis Connection West Springfield,160.0,4.270000
8,Super Yamba - Pre-Ground,Hybrid,22%,21.4%,1/2 oz,65.0,https://dutchie.com/dispensary/cannabis-connec...,Cannabis Connection West Springfield,130.0,4.609231
9,Apple Sherbert - Small Buds,Hybrid,27.6%,26.7%,1/4 oz,60.0,https://dutchie.com/dispensary/cannabis-connec...,Cannabis Connection West Springfield,240.0,3.115000


In [78]:
# Save results to csv file
combined.to_csv('scrape6-3.csv', index=False)

## Sorting

In [13]:
df = pd.read_csv('scrape6-3.csv')
df.head(5)

,Strain,Form,Type,TAC,THC,Amount,Price,URL,Dispensary,PPO,mgp
0,Valley Vixen,NaN,Hybrid,28.10%,26.90%,1/8 oz,35.0,https://dutchie.com/dispensary/cannabis-connec...,Cannabis Connection West Springfield,280.0,2.690000
1,Maya,NaN,Hybrid,29.40%,29%,1/8 oz,40.0,https://dutchie.com/dispensary/cannabis-connec...,Cannabis Connection West Springfield,320.0,2.537500
2,Larry Lovestein #1,NaN,Hybrid,31.40%,30.60%,1/8 oz,45.0,https://dutchie.com/dispensary/cannabis-connec...,Cannabis Connection West Springfield,360.0,2.380000
3,Poddy Mouth,NaN,Hybrid,29.40%,28.30%,1/8 oz,30.0,https://dutchie.com/dispensary/cannabis-connec...,Cannabis Connection West Springfield,240.0,3.301667
4,Blueberry Gas,NaN,Hybrid,25%,24%,1/8 oz,40.0,https://dutchie.com/dispensary/cannabis-connec...,Cannabis Connection West Springfield,320.0,2.100000


In [14]:
def remove_substring(row):
    if pd.notna(row['Type']):
        return row['Strain'].replace(row['Type'], '').strip(' - ').strip(' | ')
    if pd.notna(row['Form']):
        return row['Strain'].replace(row['Form'], '').strip(' - ').strip(' | ')
    if pd.notna(row['TAC']):
        return row['Strain'].replace(row['TAC'], '').strip(' - ').strip(' | ')
    if pd.notna(row['THC']):
        return row['Strain'].replace(row['THC'], '').strip(' - ').strip(' | ')
    if pd.notna(row['Amount']):
        return row['Strain'].replace(row['Amount'], '').strip(' - ').strip(' | ')
    if pd.notna(row['Price']):
        return row['Strain'].replace(row['Price'], '').strip(' - ').strip(' | ')
    else:
        return row['Strain']

# Apply the function to the dataframe
df['Strain'] = df.apply(remove_substring, axis=1)

In [15]:
df.head(20)

,Strain,Form,Type,TAC,THC,Amount,Price,URL,Dispensary,PPO,mgp
0,Valley Vixen,NaN,Hybrid,28.10%,26.90%,1/8 oz,35.0,https://dutchie.com/dispensary/cannabis-connec...,Cannabis Connection West Springfield,280.0,2.690000
1,Maya,NaN,Hybrid,29.40%,29%,1/8 oz,40.0,https://dutchie.com/dispensary/cannabis-connec...,Cannabis Connection West Springfield,320.0,2.537500
2,Larry Lovestein #1,NaN,Hybrid,31.40%,30.60%,1/8 oz,45.0,https://dutchie.com/dispensary/cannabis-connec...,Cannabis Connection West Springfield,360.0,2.380000
3,Poddy Mouth,NaN,Hybrid,29.40%,28.30%,1/8 oz,30.0,https://dutchie.com/dispensary/cannabis-connec...,Cannabis Connection West Springfield,240.0,3.301667
4,Blueberry Gas,NaN,Hybrid,25%,24%,1/8 oz,40.0,https://dutchie.com/dispensary/cannabis-connec...,Cannabis Connection West Springfield,320.0,2.100000
5,Glue Sniffer,NaN,Hybrid,26.20%,22.80%,1/8 oz,45.0,https://dutchie.com/dispensary/cannabis-connec...,Cannabis Connection West Springfield,360.0,1.773333
6,So F'n Gassy - Pre-Ground,Pre-Ground,Hybrid,37.20%,35.40%,1/4 oz,70.0,https://dutchie.com/dispensary/cannabis-connec...,Cannabis Connection West Springfield,280.0,3.540000
7,Super Silver Pupil W/Papers,NaN,Hybrid,25.90%,24.40%,1/2 oz,80.0,https://dutchie.com/dispensary/cannabis-connec...,Cannabis Connection West Springfield,160.0,4.270000
8,Super Yamba - Pre-Ground,Pre-Ground,Hybrid,22%,21.40%,1/2 oz,65.0,https://dutchie.com/dispensary/cannabis-connec...,Cannabis Connection West Springfield,130.0,4.609231
9,Apple Sherbert - Small Buds,Small Buds,Hybrid,27.60%,26.70%,1/4 oz,60.0,https://dutchie.com/dispensary/cannabis-connec...,Cannabis Connection West Springfield,240.0,3.115000


In [120]:
df.to_csv("scrapetest1.csv")

In [ ]:
df.head(20)

In [ ]:
for i in range(1,len(df)):
    

## Canna Scrape

In [79]:
def get_all_dispensary_urls():
    sitemap = 'https://dutchie.com/sitemap.xml'
    print('requesting')
    response = requests.get(sitemap)
    root = ElementTree.fromstring(response.content)
    
    dispensaries = pd.DataFrame(columns=['Name', 'Location', 'URL'])
        
    print('processing events')
    for child in root:
        if len(child) != 3:
            continue
        name = child[2][2].text if len(child[2]) >= 3 else ''
        geoloc = child[2][1].text if len(child[2]) >= 2 else ''
        url = child[0].text
        dispensaries.loc[len(dispensaries)] = [name, geoloc, url]
        
    return dispensaries

dispensaries = get_all_dispensary_urls()

requesting
processing events


In [80]:
print(f'Total # of dispensaries: {len(dispensaries)}')
dispensaries.head(10)

Total # of dispensaries: 4608


,Name,Location,URL
0,Mr. Nice Guy (Waldport),"Waldport, OR",https://dutchie.com/dispensary/mr-nice-guy-wal...
1,Oregon's Green Rush,"Eugene, OR",https://dutchie.com/dispensary/oregons-green-r...
2,Attis Trading Co. (Cully),"Portland, OR",https://dutchie.com/dispensary/attis-trading-c...
3,Cannabis Nation (Sunriver),"Bend, OR",https://dutchie.com/dispensary/cannabis-nation...
4,Tangent Farmacy,"Tangent, OR",https://dutchie.com/dispensary/tangent-farmacy
5,Substance (Empire Ave.),"Bend, OR",https://dutchie.com/dispensary/substance-empire
6,Mr. Nice Guy (Portland),"Portland, OR",https://dutchie.com/dispensary/mr-nice-guy-por...
7,Utopia Gardens,"Detroit, MI",https://dutchie.com/dispensary/utopia-gardens
8,Four Seasons,"Albany, OR",https://dutchie.com/dispensary/four-seasons
9,"Pakalolo (Holgate Blvd, PDX)","Portland, OR",https://dutchie.com/dispensary/pakalolo


In [81]:
# Save urls to csv file
dispensaries.to_csv('all-dispensaries.csv', index=False)

In [82]:
weedmaps_urls = [
    "<loc>https://weedmaps.com/dispensaries</loc>",
    "<loc>https://weedmaps.com/dispensaries/dc-collective</loc>",
    "<loc>https://weedmaps.com/dispensaries/santa-barbara-care-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/barccollective</loc>",
    "<loc>https://weedmaps.com/dispensaries/citycompassionatecaregivers</loc>",
    "<loc>https://weedmaps.com/dispensaries/downtown-patients-group-dtpg</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenlight-discount-pharmacy</loc>",
    "<loc>https://weedmaps.com/dispensaries/northhollywood</loc>",
    "<loc>https://weedmaps.com/dispensaries/compassionate-patient-resources</loc>",
    "<loc>https://weedmaps.com/dispensaries/mnrcoop</loc>",
    "<loc>https://weedmaps.com/dispensaries/california-caregivers-alliance</loc>",
    "<loc>https://weedmaps.com/dispensaries/hugs-alternative-care</loc>",
    "<loc>https://weedmaps.com/dispensaries/west-valley-caregivers</loc>",
    "<loc>https://weedmaps.com/dispensaries/redmoon-holistic-wellness-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/valley-health-options</loc>",
    "<loc>https://weedmaps.com/dispensaries/woodland-hills-treatment-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-reserve-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/divinewc</loc>",
    "<loc>https://weedmaps.com/dispensaries/numberoneclinic</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-country-healing</loc>",
    "<loc>https://weedmaps.com/dispensaries/abraxas-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/baba-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/bluebird-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/bulldog-no-90-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/dampkring-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/dolphins-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/420-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/grey-area-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenhouse-centrum-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-healing-canna</loc>",
    "<loc>https://weedmaps.com/dispensaries/kadinsky-2-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/kadinsky-3-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/prix-d-ami-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/get-down-to-it-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/starbuds-altermeds-louisville</loc>",
    "<loc>https://weedmaps.com/dispensaries/ajoya-louisville</loc>",
    "<loc>https://weedmaps.com/dispensaries/alternative-medicine-on-capitol-hill</loc>",
    "<loc>https://weedmaps.com/dispensaries/denver-kush-club-more-than-a-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-farm</loc>",
    "<loc>https://weedmaps.com/dispensaries/ganja-gourmet</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbal-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/better-health-group-collective</loc>",
    "<loc>https://weedmaps.com/dispensaries/emerald-fields-highlands</loc>",
    "<loc>https://weedmaps.com/dispensaries/mc-caregivers</loc>",
    "<loc>https://weedmaps.com/dispensaries/boulder-wellness-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/theclinik</loc>",
    "<loc>https://weedmaps.com/dispensaries/callies-cannabis-shoppe-rino</loc>",
    "<loc>https://weedmaps.com/dispensaries/natural-herbal-pain-relief</loc>",
    "<loc>https://weedmaps.com/dispensaries/a-cut-above-colorado-springs</loc>",
    "<loc>https://weedmaps.com/dispensaries/livwell-berthoud</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-dragon-edgewater</loc>",
    "<loc>https://weedmaps.com/dispensaries/rocky-mountain-high-denver</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-tree-medicinals-formerly-nature-s-medicine-of-longmont</loc>",
    "<loc>https://weedmaps.com/dispensaries/a-cut-above-denver</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannamart</loc>",
    "<loc>https://weedmaps.com/dispensaries/capitola-healing</loc>",
    "<loc>https://weedmaps.com/dispensaries/pure-medical-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/from-the-earth-port-hueneme</loc>",
    "<loc>https://weedmaps.com/dispensaries/organic-alternatives</loc>",
    "<loc>https://weedmaps.com/dispensaries/pure-medical-downtown</loc>",
    "<loc>https://weedmaps.com/dispensaries/todays-health-care</loc>",
    "<loc>https://weedmaps.com/dispensaries/briargate-wellness-center-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-station</loc>",
    "<loc>https://weedmaps.com/dispensaries/best-budz</loc>",
    "<loc>https://weedmaps.com/dispensaries/kind-pain-management</loc>",
    "<loc>https://weedmaps.com/dispensaries/back-to-the-garden-broadway</loc>",
    "<loc>https://weedmaps.com/dispensaries/starbuds-du</loc>",
    "<loc>https://weedmaps.com/dispensaries/growlife</loc>",
    "<loc>https://weedmaps.com/dispensaries/los-angeles-valley-caregivers</loc>",
    "<loc>https://weedmaps.com/dispensaries/desert-organic-solutions</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-healing-canna-west</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabicare</loc>",
    "<loc>https://weedmaps.com/dispensaries/breckenridge-organic-therapy</loc>",
    "<loc>https://weedmaps.com/dispensaries/long-beach-green-room-lab-tested-by-sc-labs</loc>",
    "<loc>https://weedmaps.com/dispensaries/hhc-heavenly-herbal-collective</loc>",
    "<loc>https://weedmaps.com/dispensaries/purple-lotus</loc>",
    "<loc>https://weedmaps.com/dispensaries/mercy-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/karing-kind</loc>",
    "<loc>https://weedmaps.com/dispensaries/super-clinik-yale</loc>",
    "<loc>https://weedmaps.com/dispensaries/euphoric-40-cap</loc>",
    "<loc>https://weedmaps.com/dispensaries/doctors-orders</loc>",
    "<loc>https://weedmaps.com/dispensaries/chronic-pain-releaf-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/unity-rd-boulder</loc>",
    "<loc>https://weedmaps.com/dispensaries/redwood-herbal-alliance</loc>",
    "<loc>https://weedmaps.com/dispensaries/pure-medical-dispensary-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/rcp</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-tops-colorado-springs</loc>",
    "<loc>https://weedmaps.com/dispensaries/nrc</loc>",
    "<loc>https://weedmaps.com/dispensaries/t-keteltje-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/bronx-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/black-star-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/flower-power-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/sanementereng-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/crush-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-noon-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-rookies-bar-coffeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/tops-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/easy-times-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/barney-s-lounge-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-bushdocter-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/het-ballonnetje-coffeeshop-amsterdam</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenhouse-namaste-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/reefer-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/betty-boop-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-other-side-coffeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/best-friends-ii</loc>",
    "<loc>https://weedmaps.com/dispensaries/la-tertulia-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/free-i-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/smokey-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/johnny-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/balou-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/andalucia-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/bushdocter-amstel-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-saint-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/kadinsky-i-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/tweede-kamer-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/rusland-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/basjoe-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/solo-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/softland-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/barney-s-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/barney-s-uptown-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/barney-s-farm-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenhouse-lounge-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/dampkring-2-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/popeye-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/amsterdamned</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-store-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/siberie-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/rockland-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/420-cafe-de-kuil-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-old-church-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/rick-s-cafe-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/hash-marihuana-hemp-museum-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/herban-medicinals-underground</loc>",
    "<loc>https://weedmaps.com/dispensaries/standing-akimbo-denver</loc>",
    "<loc>https://weedmaps.com/dispensaries/pure-medical-rockrimmon</loc>",
    "<loc>https://weedmaps.com/dispensaries/animas-herbal-wellness-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/ajoya-lakewood</loc>",
    "<loc>https://weedmaps.com/dispensaries/naturaleaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/golden-alternative-medicine</loc>",
    "<loc>https://weedmaps.com/dispensaries/cloud-nine-caregivers</loc>",
    "<loc>https://weedmaps.com/dispensaries/maggie-s-farm</loc>",
    "<loc>https://weedmaps.com/dispensaries/sherman-oaks-health-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/tacoma-greenthumb-port-orchard</loc>",
    "<loc>https://weedmaps.com/dispensaries/love-in-it-cooperative</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-dragon-caregivers-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/sensible-alternatives</loc>",
    "<loc>https://weedmaps.com/dispensaries/greengoddess</loc>",
    "<loc>https://weedmaps.com/dispensaries/ascend-cannabis-co-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/wcc-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-earth-collective</loc>",
    "<loc>https://weedmaps.com/dispensaries/a-wellness-centers</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-barn</loc>",
    "<loc>https://weedmaps.com/dispensaries/emerald-city-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-healing-center-2-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/soho-sherman-oaks-holistic-oasis</loc>",
    "<loc>https://weedmaps.com/dispensaries/santacruzmountainnaturals-santa-cruz</loc>",
    "<loc>https://weedmaps.com/dispensaries/depot-town-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/calm-and-collective</loc>",
    "<loc>https://weedmaps.com/dispensaries/river-city-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/tge</loc>",
    "<loc>https://weedmaps.com/dispensaries/svc-home-of-the-louie-xiii</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-apothecarium</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbal-relief-caregivers-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/pure-life</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbal-solutions-ypsi</loc>",
    "<loc>https://weedmaps.com/dispensaries/denver-clone-store-north</loc>",
    "<loc>https://weedmaps.com/dispensaries/cbcb</loc>",
    "<loc>https://weedmaps.com/dispensaries/pueblo-west-steel-city-meds</loc>",
    "<loc>https://weedmaps.com/dispensaries/nohocc-van-nuys-oasis</loc>",
    "<loc>https://weedmaps.com/dispensaries/third-day-apothecary</loc>",
    "<loc>https://weedmaps.com/dispensaries/cc101</loc>",
    "<loc>https://weedmaps.com/dispensaries/minerva-canna-group-albuquerque</loc>",
    "<loc>https://weedmaps.com/dispensaries/a-cut-off-the-top-2059-w-9th-ave-80204</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-sanctuary-sac</loc>",
    "<loc>https://weedmaps.com/dispensaries/todays-health-care-ii</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-remedy</loc>",
    "<loc>https://weedmaps.com/dispensaries/dank</loc>",
    "<loc>https://weedmaps.com/dispensaries/ballpark-holistic-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/creekside</loc>",
    "<loc>https://weedmaps.com/dispensaries/healthy-education-society</loc>",
    "<loc>https://weedmaps.com/dispensaries/californiahealinggardens</loc>",
    "<loc>https://weedmaps.com/dispensaries/highstreetgrowers</loc>",
    "<loc>https://weedmaps.com/dispensaries/olympus-incendias</loc>",
    "<loc>https://weedmaps.com/dispensaries/a-greener-today-seattle</loc>",
    "<loc>https://weedmaps.com/dispensaries/shakeandbake</loc>",
    "<loc>https://weedmaps.com/dispensaries/san-fernando-valley-patients-group</loc>",
    "<loc>https://weedmaps.com/dispensaries/818-brands-palm-springs</loc>",
    "<loc>https://weedmaps.com/dispensaries/compassionate-heart</loc>",
    "<loc>https://weedmaps.com/dispensaries/northwest-collective</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-cookie-co-831</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-loft-co-op</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-weed-project-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/hollywood-high-grade-hollywood-los-angeles</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-heart</loc>",
    "<loc>https://weedmaps.com/dispensaries/la-brea-collective-2-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/cloud-9-sacramento</loc>",
    "<loc>https://weedmaps.com/dispensaries/bmc-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/haze-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/tworiverssac</loc>",
    "<loc>https://weedmaps.com/dispensaries/highway-29-health-care</loc>",
    "<loc>https://weedmaps.com/dispensaries/7-stars-hhc</loc>",
    "<loc>https://weedmaps.com/dispensaries/fox-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/wellness-connection-main-thomaston</loc>",
    "<loc>https://weedmaps.com/dispensaries/wellness-connection-maine-portland</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbal-healing-safe-access</loc>",
    "<loc>https://weedmaps.com/dispensaries/healthy-life-pharmacy-hlp</loc>",
    "<loc>https://weedmaps.com/dispensaries/mecca</loc>",
    "<loc>https://weedmaps.com/dispensaries/plan-b-wellness-center-detroit</loc>",
    "<loc>https://weedmaps.com/dispensaries/beach-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/missionhillsorganic</loc>",
    "<loc>https://weedmaps.com/dispensaries/bonafide-group-collective</loc>",
    "<loc>https://weedmaps.com/dispensaries/smokane</loc>",
    "<loc>https://weedmaps.com/dispensaries/sticky-ypsi</loc>",
    "<loc>https://weedmaps.com/dispensaries/sacred-garden-santa-fe</loc>",
    "<loc>https://weedmaps.com/dispensaries/alpine-alternative</loc>",
    "<loc>https://weedmaps.com/dispensaries/little-trees-collective</loc>",
    "<loc>https://weedmaps.com/dispensaries/willie-wortel-s-sinsemilla-cannabisshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabisshop-willy-wortel-sativa</loc>",
    "<loc>https://weedmaps.com/dispensaries/willy-wortel-workshop-indica</loc>",
    "<loc>https://weedmaps.com/dispensaries/premium-pacific</loc>",
    "<loc>https://weedmaps.com/dispensaries/healing-house-lakewood</loc>",
    "<loc>https://weedmaps.com/dispensaries/scmobilemeds</loc>",
    "<loc>https://weedmaps.com/dispensaries/lightshade-labs</loc>",
    "<loc>https://weedmaps.com/dispensaries/gradus-by-socc</loc>",
    "<loc>https://weedmaps.com/dispensaries/purple-star-md</loc>",
    "<loc>https://weedmaps.com/dispensaries/have-a-heart-cafe</loc>",
    "<loc>https://weedmaps.com/dispensaries/a-therapeutic-alternative-sacramento</loc>",
    "<loc>https://weedmaps.com/dispensaries/united-growers-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/phoenix-relief-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/los-angeles-kush-east-la</loc>",
    "<loc>https://weedmaps.com/dispensaries/bloom-phoenix-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/noble-herb</loc>",
    "<loc>https://weedmaps.com/dispensaries/earth-s-healing-tucson</loc>",
    "<loc>https://weedmaps.com/dispensaries/foothill-wellness-center-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-mountain-health-flagstaff</loc>",
    "<loc>https://weedmaps.com/dispensaries/arizona-mmj-trading-company-coolidge</loc>",
    "<loc>https://weedmaps.com/dispensaries/desert-bloom-re-leaf-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/swc-tempe</loc>",
    "<loc>https://weedmaps.com/dispensaries/swc-prescott</loc>",
    "<loc>https://weedmaps.com/dispensaries/terrapin-care-station</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-64-store</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-daddys</loc>",
    "<loc>https://weedmaps.com/dispensaries/thomas-c-slater-compassion-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/jars-cannabis-bullhead-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/natures-az-medicines-phoenix</loc>",
    "<loc>https://weedmaps.com/dispensaries/wellness-connection-main-hallowell</loc>",
    "<loc>https://weedmaps.com/dispensaries/wellness-connection-of-main-brewer</loc>",
    "<loc>https://weedmaps.com/dispensaries/cookies-sanfrancisco</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-tree-berthoud</loc>",
    "<loc>https://weedmaps.com/dispensaries/us-bloom</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-nirvana-center-of-prescott-valley</loc>",
    "<loc>https://weedmaps.com/dispensaries/trumed-premier-dispensary-arcadia-phoenix</loc>",
    "<loc>https://weedmaps.com/dispensaries/botanica-tucson</loc>",
    "<loc>https://weedmaps.com/dispensaries/hazel</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-good-dispensary-mesa-az</loc>",
    "<loc>https://weedmaps.com/dispensaries/east-of-eden-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-downtown-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/la-casa-cannabis-ii-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/ponderosa-releaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbal-wellness-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-higher-path-sherman-oaks</loc>",
    "<loc>https://weedmaps.com/dispensaries/apothecary-sherman-oaks</loc>",
    "<loc>https://weedmaps.com/dispensaries/crockpot-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/sylmar-holistic-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/la-wonderland-caregivers</loc>",
    "<loc>https://weedmaps.com/dispensaries/2nd-step-medical-marijuana-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/nature-s-story-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/west-hollywood-healing</loc>",
    "<loc>https://weedmaps.com/dispensaries/health-for-life-cave-creek</loc>",
    "<loc>https://weedmaps.com/dispensaries/highway-420-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/health-for-life-ellsworth</loc>",
    "<loc>https://weedmaps.com/dispensaries/infinity-medical-alliance</loc>",
    "<loc>https://weedmaps.com/dispensaries/arizona-organix</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-gold-cultivators</loc>",
    "<loc>https://weedmaps.com/dispensaries/reseda-discount-caregivers-van-nuys</loc>",
    "<loc>https://weedmaps.com/dispensaries/doctors-orders-rx</loc>",
    "<loc>https://weedmaps.com/dispensaries/bloom-tucson</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-florin-and-wellness-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/kinds-meds</loc>",
    "<loc>https://weedmaps.com/dispensaries/magnolia-road-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/health-for-life-mcdowell</loc>",
    "<loc>https://weedmaps.com/dispensaries/thc-the-healing-center-needles-walk-in</loc>",
    "<loc>https://weedmaps.com/dispensaries/a-greener-today-north</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-cross-of-cherry-creek-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/bloom-sedona</loc>",
    "<loc>https://weedmaps.com/dispensaries/house-of-organics</loc>",
    "<loc>https://weedmaps.com/dispensaries/nw-natural-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/todays-health-care-lll</loc>",
    "<loc>https://weedmaps.com/dispensaries/medical-caregivers-co-op-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/lakeshore-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/denver-kush-club-more-than-a-club-adult-use</loc>",
    "<loc>https://weedmaps.com/dispensaries/dank-colorado-adult-use-denvers-best</loc>",
    "<loc>https://weedmaps.com/dispensaries/motown-meds</loc>",
    "<loc>https://weedmaps.com/dispensaries/denco-adult-use</loc>",
    "<loc>https://weedmaps.com/dispensaries/thedropla</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-country-healing-2-recreational-alma</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunset-herbal-corner-2-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/amch-adult-use</loc>",
    "<loc>https://weedmaps.com/dispensaries/starbuds-adult-use</loc>",
    "<loc>https://weedmaps.com/dispensaries/tlccollective</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-chemistry-adult-use</loc>",
    "<loc>https://weedmaps.com/dispensaries/sacramento</loc>",
    "<loc>https://weedmaps.com/dispensaries/frosted-leaf-federal-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/twelve-hour-care</loc>",
    "<loc>https://weedmaps.com/dispensaries/kindpeoples-collective</loc>",
    "<loc>https://weedmaps.com/dispensaries/urban-farmacy-patient-resource-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/sacred-seed-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/studio-city-caregivers-studio-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/coast-to-coast-collective</loc>",
    "<loc>https://weedmaps.com/dispensaries/lightshade-adult-use-2-holly-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/caregivers-for-life-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/doctor-resin</loc>",
    "<loc>https://weedmaps.com/dispensaries/sibaritas-cannabis-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/weedom</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-age-barcelona</loc>",
    "<loc>https://weedmaps.com/dispensaries/cys-center-2-80916</loc>",
    "<loc>https://weedmaps.com/dispensaries/marina-caregivers-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/mmd</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbs4you-adult-use</loc>",
    "<loc>https://weedmaps.com/dispensaries/sas</loc>",
    "<loc>https://weedmaps.com/dispensaries/livwell-rino</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannacruz</loc>",
    "<loc>https://weedmaps.com/dispensaries/a-better-way-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/alternative-herbal-health-services</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-pharm-clinic</loc>",
    "<loc>https://weedmaps.com/dispensaries/mountain-medicals-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/south-bay-collective</loc>",
    "<loc>https://weedmaps.com/dispensaries/arizona-natural-selections-of-scottsdale</loc>",
    "<loc>https://weedmaps.com/dispensaries/best-colorado-meds-adult-use</loc>",
    "<loc>https://weedmaps.com/dispensaries/prima-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/livwell-adult-use</loc>",
    "<loc>https://weedmaps.com/dispensaries/amc</loc>",
    "<loc>https://weedmaps.com/dispensaries/maggie-s-farm-medical</loc>",
    "<loc>https://weedmaps.com/dispensaries/metro-meds-arizona</loc>",
    "<loc>https://weedmaps.com/dispensaries/strain-30-cap-selective</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-dragon-aspen-adult-use</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-solutions</loc>",
    "<loc>https://weedmaps.com/dispensaries/fox-cannabis-co-adult-use</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenardo</loc>",
    "<loc>https://weedmaps.com/dispensaries/choko-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/golden-meds-adult-use</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannasseur</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-cross-3-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/sweet-relief-natural-medicine</loc>",
    "<loc>https://weedmaps.com/dispensaries/trenchtown-marijuana-center-recreational-mmj-denver</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenwolf-la</loc>",
    "<loc>https://weedmaps.com/dispensaries/breeze-botanicals</loc>",
    "<loc>https://weedmaps.com/dispensaries/natures-herbs-and-wellness-center-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbal-wellness-2-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/west-valley-patients-group-2-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-valley-healing-center-2-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-valley-retail-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/bridge-city-collective-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/delta-health-and-wellness-2-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/maggie-s-farm-pueblo-west-adult</loc>",
    "<loc>https://weedmaps.com/dispensaries/kaleafa</loc>",
    "<loc>https://weedmaps.com/dispensaries/kaleafa-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/unified-patient-alliance</loc>",
    "<loc>https://weedmaps.com/dispensaries/outliers-collective</loc>",
    "<loc>https://weedmaps.com/dispensaries/pure-medical-dispensary-40th-ave-med</loc>",
    "<loc>https://weedmaps.com/dispensaries/satori-cured</loc>",
    "<loc>https://weedmaps.com/dispensaries/freedom-market-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/altitude</loc>",
    "<loc>https://weedmaps.com/dispensaries/2020-solutions-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-door-alternative</loc>",
    "<loc>https://weedmaps.com/dispensaries/today-s-herbal-choice</loc>",
    "<loc>https://weedmaps.com/dispensaries/vhc-van-nuys</loc>",
    "<loc>https://weedmaps.com/dispensaries/verts-medical-marijuana-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/star-buds-garden-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-life-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-lady-marijuana</loc>",
    "<loc>https://weedmaps.com/dispensaries/cg-corrigan-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/project-cannabis-noho</loc>",
    "<loc>https://weedmaps.com/dispensaries/savage-thc</loc>",
    "<loc>https://weedmaps.com/dispensaries/native-roots-apothecary-medical</loc>",
    "<loc>https://weedmaps.com/dispensaries/livwell-fort-collins-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/longview-freedom-market-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/mountain-view-naturals</loc>",
    "<loc>https://weedmaps.com/dispensaries/mary-mart</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-city-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/shango-cannabis-portland</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-hartford</loc>",
    "<loc>https://weedmaps.com/dispensaries/terrapin-care-station-aurora</loc>",
    "<loc>https://weedmaps.com/dispensaries/solace-meds-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/delta-9-thc</loc>",
    "<loc>https://weedmaps.com/dispensaries/sky-dispensaries-ahwatukee</loc>",
    "<loc>https://weedmaps.com/dispensaries/trinidads-higher-calling-u</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-stone-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/terrapin-care-station-aurora-33rd-ave-recreational-adult-use</loc>",
    "<loc>https://weedmaps.com/dispensaries/dockside-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-bakeree</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-glass-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/biomeds</loc>",
    "<loc>https://weedmaps.com/dispensaries/medicine-man-aurora</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-room</loc>",
    "<loc>https://weedmaps.com/dispensaries/rocky-road-vail</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-world</loc>",
    "<loc>https://weedmaps.com/dispensaries/a-bud-leaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/kashmir-lounge</loc>",
    "<loc>https://weedmaps.com/dispensaries/takoma-wellness-center-2-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-youngtown</loc>",
    "<loc>https://weedmaps.com/dispensaries/starbuds-aurora</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-country-store</loc>",
    "<loc>https://weedmaps.com/dispensaries/aurora-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/house-of-dank-313</loc>",
    "<loc>https://weedmaps.com/dispensaries/euphoria-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/mr-doobees-natural-high-store</loc>",
    "<loc>https://weedmaps.com/dispensaries/oregon-house-of-herbs</loc>",
    "<loc>https://weedmaps.com/dispensaries/purple-haze</loc>",
    "<loc>https://weedmaps.com/dispensaries/st-andrews-green-tujunga</loc>",
    "<loc>https://weedmaps.com/dispensaries/h-street-patient-care-2-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/420-friendly</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-cabinet</loc>",
    "<loc>https://weedmaps.com/dispensaries/nature-spot</loc>",
    "<loc>https://weedmaps.com/dispensaries/farma</loc>",
    "<loc>https://weedmaps.com/dispensaries/white-mountain-health-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/rose-collective</loc>",
    "<loc>https://weedmaps.com/dispensaries/issaquah-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenway-marijuana</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-top-shelf-2-9</loc>",
    "<loc>https://weedmaps.com/dispensaries/datura-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-cut</loc>",
    "<loc>https://weedmaps.com/dispensaries/natures-herbs-and-wellness-ii</loc>",
    "<loc>https://weedmaps.com/dispensaries/bangh-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/belveda-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/betty-boop</loc>",
    "<loc>https://weedmaps.com/dispensaries/las-tres-brujas</loc>",
    "<loc>https://weedmaps.com/dispensaries/mazar-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/secret-garden</loc>",
    "<loc>https://weedmaps.com/dispensaries/emerald-fields-denver</loc>",
    "<loc>https://weedmaps.com/dispensaries/lionheart-caregiving-helena</loc>",
    "<loc>https://weedmaps.com/dispensaries/bluepoint-wellness-of-branford</loc>",
    "<loc>https://weedmaps.com/dispensaries/native-roots-south</loc>",
    "<loc>https://weedmaps.com/dispensaries/kush-gardens-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/harvest-hoc-south-mesa</loc>",
    "<loc>https://weedmaps.com/dispensaries/golden-meds-leetsdale</loc>",
    "<loc>https://weedmaps.com/dispensaries/happy-leaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/coffeeshop-vlouw</loc>",
    "<loc>https://weedmaps.com/dispensaries/silver-state-relief</loc>",
    "<loc>https://weedmaps.com/dispensaries/la-bruja-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/aniser-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/cva-central-valley-alternative</loc>",
    "<loc>https://weedmaps.com/dispensaries/la-nube</loc>",
    "<loc>https://weedmaps.com/dispensaries/mj-expresso</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-quality-compassion</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-sugar-2-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-glendale</loc>",
    "<loc>https://weedmaps.com/dispensaries/lucy-sky</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-brothers</loc>",
    "<loc>https://weedmaps.com/dispensaries/elevations</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenpharms-flagstaff</loc>",
    "<loc>https://weedmaps.com/dispensaries/breeze-botanicals-ashland</loc>",
    "<loc>https://weedmaps.com/dispensaries/medical-green</loc>",
    "<loc>https://weedmaps.com/dispensaries/nectar-4125-n-mississippi-ave</loc>",
    "<loc>https://weedmaps.com/dispensaries/seattle-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/coffeeshop-the-stud</loc>",
    "<loc>https://weedmaps.com/dispensaries/ifrane</loc>",
    "<loc>https://weedmaps.com/dispensaries/physician-preferred-products-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-dispensaries-queen-creek</loc>",
    "<loc>https://weedmaps.com/dispensaries/lightshade-sheridan-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/compassionate-care-foundation</loc>",
    "<loc>https://weedmaps.com/dispensaries/trees-dispensary-hampden</loc>",
    "<loc>https://weedmaps.com/dispensaries/nectar-portland-oregon</loc>",
    "<loc>https://weedmaps.com/dispensaries/goodbudstore</loc>",
    "<loc>https://weedmaps.com/dispensaries/jokesupicekream</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabliss-burnside</loc>",
    "<loc>https://weedmaps.com/dispensaries/faragosi-farms</loc>",
    "<loc>https://weedmaps.com/dispensaries/native-roots-edgewater-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/hhc-plus</loc>",
    "<loc>https://weedmaps.com/dispensaries/asoc-green-planet-cannabis-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/euphorium-2-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/ascarfd</loc>",
    "<loc>https://weedmaps.com/dispensaries/natural-selections-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/99-high-tide-collective</loc>",
    "<loc>https://weedmaps.com/dispensaries/emerald-fields-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/native-roots-longmont-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-chemistry-aurora</loc>",
    "<loc>https://weedmaps.com/dispensaries/silver-stem-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/nature-s-alternative-inc-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/so-cal-co-op-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/garden-state-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/rocky-road-aurora</loc>",
    "<loc>https://weedmaps.com/dispensaries/floors</loc>",
    "<loc>https://weedmaps.com/dispensaries/have-a-heart-skyway</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-kushery-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/boulder-wellness-center-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/mesa-organics</loc>",
    "<loc>https://weedmaps.com/dispensaries/growhouse-nederland</loc>",
    "<loc>https://weedmaps.com/dispensaries/nectar-1019-ne-122nd-ave</loc>",
    "<loc>https://weedmaps.com/dispensaries/oc3-collective</loc>",
    "<loc>https://weedmaps.com/dispensaries/kaleafa-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-dispensary-fulton</loc>",
    "<loc>https://weedmaps.com/dispensaries/la-cima</loc>",
    "<loc>https://weedmaps.com/dispensaries/affinity-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-gilbert</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-jazz-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-herbal-cure-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-society-thc-a-division-of-the-high-company-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/organic-alternatives-2-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/organic-greens-collective</loc>",
    "<loc>https://weedmaps.com/dispensaries/little-brown-house</loc>",
    "<loc>https://weedmaps.com/dispensaries/sage-shop</loc>",
    "<loc>https://weedmaps.com/dispensaries/365-recreational-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/starbuds-pueblo</loc>",
    "<loc>https://weedmaps.com/dispensaries/holistic-pain-relief-lax</loc>",
    "<loc>https://weedmaps.com/dispensaries/oz-recreational-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/bolera-besaya-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/have-a-heart-ocean-shores-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/coffeeshop-green-valley</loc>",
    "<loc>https://weedmaps.com/dispensaries/denver-terrapincarestation-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/bud-hut-inc-81089-walsenburg-trinidad</loc>",
    "<loc>https://weedmaps.com/dispensaries/zen-leaf-reno</loc>",
    "<loc>https://weedmaps.com/dispensaries/blue-mountain-collective-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/silver-stem-fine-cannabis-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbalist</loc>",
    "<loc>https://weedmaps.com/dispensaries/cookies-on-the-strip</loc>",
    "<loc>https://weedmaps.com/dispensaries/magnolia-road-cannabis-co-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenindoor</loc>",
    "<loc>https://weedmaps.com/dispensaries/smartcollective-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/hwy-420</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-valley-retail-cannabis-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/hemperor</loc>",
    "<loc>https://weedmaps.com/dispensaries/todays-health-care-pueblo-west</loc>",
    "<loc>https://weedmaps.com/dispensaries/zona-zero</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-herbery-i-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/1841-el-camino</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-dragon-denver</loc>",
    "<loc>https://weedmaps.com/dispensaries/from-the-earth</loc>",
    "<loc>https://weedmaps.com/dispensaries/paper-and-leaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/dockside-cannabis-sodo</loc>",
    "<loc>https://weedmaps.com/dispensaries/southwest-organic-producers-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/emerald-fields-aurora</loc>",
    "<loc>https://weedmaps.com/dispensaries/three-rivers-organics</loc>",
    "<loc>https://weedmaps.com/dispensaries/starbuds-3-6-nw-denver</loc>",
    "<loc>https://weedmaps.com/dispensaries/big-medicine-cannabissary</loc>",
    "<loc>https://weedmaps.com/dispensaries/west-coast-holistic-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/organic-green-treatment-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/native-roots-littleton</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-kushery-at-cathcart-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/i-and-i-newport</loc>",
    "<loc>https://weedmaps.com/dispensaries/stockton-patient-clinic</loc>",
    "<loc>https://weedmaps.com/dispensaries/sweet-relief-portland</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-genie-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/white-rabbit-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/gb-meds</loc>",
    "<loc>https://weedmaps.com/dispensaries/sofa-king-medicinal-wellness-products</loc>",
    "<loc>https://weedmaps.com/dispensaries/blowin-smoke</loc>",
    "<loc>https://weedmaps.com/dispensaries/advanced-wellness-downtown</loc>",
    "<loc>https://weedmaps.com/dispensaries/mindrite-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-medication-station-ii</loc>",
    "<loc>https://weedmaps.com/dispensaries/silver-stem-fine-cannabis-lakewood-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/minnesota-medical-solutions</loc>",
    "<loc>https://weedmaps.com/dispensaries/leafline-labs-willmar</loc>",
    "<loc>https://weedmaps.com/dispensaries/minnesota-medical-solutions-rochester</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbal-alternatives-ii</loc>",
    "<loc>https://weedmaps.com/dispensaries/higher-level-mmj</loc>",
    "<loc>https://weedmaps.com/dispensaries/happy-leaf-la</loc>",
    "<loc>https://weedmaps.com/dispensaries/flowr-of-lyfe</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-holistic-co-op-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/alternative-solutions-2-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/left-coast-connection-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-cdc-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/gorge-greenery</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-care-collective</loc>",
    "<loc>https://weedmaps.com/dispensaries/park-range-recreationals</loc>",
    "<loc>https://weedmaps.com/dispensaries/lucid-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-clinic-detroit</loc>",
    "<loc>https://weedmaps.com/dispensaries/weed-soul</loc>",
    "<loc>https://weedmaps.com/dispensaries/tru-cannabis-portland</loc>",
    "<loc>https://weedmaps.com/dispensaries/inyo-fine-cannabis-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/strawberry-fields-pueblo</loc>",
    "<loc>https://weedmaps.com/dispensaries/kind-love-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/doctor-s-orders-portland</loc>",
    "<loc>https://weedmaps.com/dispensaries/beaver-bowls</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-epic-remedy</loc>",
    "<loc>https://weedmaps.com/dispensaries/super-clinik</loc>",
    "<loc>https://weedmaps.com/dispensaries/420-central</loc>",
    "<loc>https://weedmaps.com/dispensaries/frontier-farms-hood-river</loc>",
    "<loc>https://weedmaps.com/dispensaries/plantae-health-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/mon-cannabic</loc>",
    "<loc>https://weedmaps.com/dispensaries/breakwater-treatment-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/chalice-farms</loc>",
    "<loc>https://weedmaps.com/dispensaries/kings-of-canna</loc>",
    "<loc>https://weedmaps.com/dispensaries/artemisa-manresa</loc>",
    "<loc>https://weedmaps.com/dispensaries/nevada-medical-marijuana</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-zen-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/cougar-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/corktown-collective</loc>",
    "<loc>https://weedmaps.com/dispensaries/t-brothers-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/west-coast-organics</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-canton</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-kushery</loc>",
    "<loc>https://weedmaps.com/dispensaries/earthmed-addison</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-agrestic-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/positive-vibrations-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/bud-bros-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/country-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/galicanna</loc>",
    "<loc>https://weedmaps.com/dispensaries/dispensary-0039</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-mundelein</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-rockford</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabal-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/ace-s-place-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/pharmacannis-ottawa</loc>",
    "<loc>https://weedmaps.com/dispensaries/pharmacannis-north-aurora</loc>",
    "<loc>https://weedmaps.com/dispensaries/pharmacannis-schaumburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/ganja-vita</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-pearl-organics</loc>",
    "<loc>https://weedmaps.com/dispensaries/nevada-medical-marijuana-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-joliet-colorado</loc>",
    "<loc>https://weedmaps.com/dispensaries/simply-pure-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/mapleglen-care-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/new-age-care</loc>",
    "<loc>https://weedmaps.com/dispensaries/sticky-s-pot-shop</loc>",
    "<loc>https://weedmaps.com/dispensaries/sky-dispensaries-mesa</loc>",
    "<loc>https://weedmaps.com/dispensaries/lightshade-6th-ave</loc>",
    "<loc>https://weedmaps.com/dispensaries/bigfoot-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/weden-santa-ana</loc>",
    "<loc>https://weedmaps.com/dispensaries/jayden-s-journey</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-buffalo-grove</loc>",
    "<loc>https://weedmaps.com/dispensaries/2one2-california-street-medical-cannabis-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/chalice-farms-tigard</loc>",
    "<loc>https://weedmaps.com/dispensaries/natural-alternative-for-health-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-stash-box</loc>",
    "<loc>https://weedmaps.com/dispensaries/oasis-dispensaries-chandler</loc>",
    "<loc>https://weedmaps.com/dispensaries/sweet-jane</loc>",
    "<loc>https://weedmaps.com/dispensaries/sweet-relief-tillamook</loc>",
    "<loc>https://weedmaps.com/dispensaries/oregon-medical-grade-rainier</loc>",
    "<loc>https://weedmaps.com/dispensaries/harbor-collective-2-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/nsa-nuestra-senora-del-agua</loc>",
    "<loc>https://weedmaps.com/dispensaries/etain-kingston</loc>",
    "<loc>https://weedmaps.com/dispensaries/columbia-care-manhattan</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-justice</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-hidden-bush</loc>",
    "<loc>https://weedmaps.com/dispensaries/pharmancannis-amherst</loc>",
    "<loc>https://weedmaps.com/dispensaries/pharmancannis-liverpool</loc>",
    "<loc>https://weedmaps.com/dispensaries/verilife-albany</loc>",
    "<loc>https://weedmaps.com/dispensaries/verilife-bronx</loc>",
    "<loc>https://weedmaps.com/dispensaries/yilo-superstore</loc>",
    "<loc>https://weedmaps.com/dispensaries/livwell-cortez</loc>",
    "<loc>https://weedmaps.com/dispensaries/hemprat</loc>",
    "<loc>https://weedmaps.com/dispensaries/mankind-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/white-fire</loc>",
    "<loc>https://weedmaps.com/dispensaries/root-cellar</loc>",
    "<loc>https://weedmaps.com/dispensaries/buds-sos</loc>",
    "<loc>https://weedmaps.com/dispensaries/kanna-reno</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-seed</loc>",
    "<loc>https://weedmaps.com/dispensaries/livwell-tower-road</loc>",
    "<loc>https://weedmaps.com/dispensaries/dr-dou</loc>",
    "<loc>https://weedmaps.com/dispensaries/secret-herb-shop</loc>",
    "<loc>https://weedmaps.com/dispensaries/kaya-cannabis-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/dancing-gypsies</loc>",
    "<loc>https://weedmaps.com/dispensaries/columbia-care-riverhead</loc>",
    "<loc>https://weedmaps.com/dispensaries/columbia-care-rochester</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-worth</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-new-lenox</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-deerfield</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-dispensaries-warwick</loc>",
    "<loc>https://weedmaps.com/dispensaries/etain-syracuse</loc>",
    "<loc>https://weedmaps.com/dispensaries/torrey-holistics</loc>",
    "<loc>https://weedmaps.com/dispensaries/pecos-valley-pharmaceuticals-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/mr-niceguy-portland-se-holgate</loc>",
    "<loc>https://weedmaps.com/dispensaries/essence-west</loc>",
    "<loc>https://weedmaps.com/dispensaries/pecos-valley-pharmaceuticals-2-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabist-lowell</loc>",
    "<loc>https://weedmaps.com/dispensaries/cloud-9-collective-fort-st</loc>",
    "<loc>https://weedmaps.com/dispensaries/house-of-zen-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/natures-care</loc>",
    "<loc>https://weedmaps.com/dispensaries/ultra-health-albuquerque-n-e</loc>",
    "<loc>https://weedmaps.com/dispensaries/albuquerque-nob-hill-birdland</loc>",
    "<loc>https://weedmaps.com/dispensaries/ultra-health-bernalillo</loc>",
    "<loc>https://weedmaps.com/dispensaries/ultra-health-hobbs</loc>",
    "<loc>https://weedmaps.com/dispensaries/ultra-health-santa-fe</loc>",
    "<loc>https://weedmaps.com/dispensaries/garden-of-the-budz</loc>",
    "<loc>https://weedmaps.com/dispensaries/starbuds-glendale</loc>",
    "<loc>https://weedmaps.com/dispensaries/leafline-labs-eagan</loc>",
    "<loc>https://weedmaps.com/dispensaries/ramona-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-city-sevilla</loc>",
    "<loc>https://weedmaps.com/dispensaries/carmate-asociacion</loc>",
    "<loc>https://weedmaps.com/dispensaries/durango-rec-room</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbal-remedies-il</loc>",
    "<loc>https://weedmaps.com/dispensaries/farm-fresh-medical-marijuana-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/silver-sage-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-champaign</loc>",
    "<loc>https://weedmaps.com/dispensaries/capital-city-care</loc>",
    "<loc>https://weedmaps.com/dispensaries/reef-dispensaries</loc>",
    "<loc>https://weedmaps.com/dispensaries/nature-s-treatment-of-illinois-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/cornerstone-research-collective</loc>",
    "<loc>https://weedmaps.com/dispensaries/altitude-organic-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-henderson</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-5-cannabis-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/thc-the-healing-center-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-elmwood-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/life-flower-dispensary-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/mcpg-happy-herbs</loc>",
    "<loc>https://weedmaps.com/dispensaries/vilaherba</loc>",
    "<loc>https://weedmaps.com/dispensaries/medithrive-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/zen-leaf-highland-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/show-grow-las-vegas</loc>",
    "<loc>https://weedmaps.com/dispensaries/thedispensarynv</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-resin-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/mary-jane-s-house-of-grass</loc>",
    "<loc>https://weedmaps.com/dispensaries/stem-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-dragon-aurora</loc>",
    "<loc>https://weedmaps.com/dispensaries/local-product-of-colorado-recreational-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/oasis-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/planet-13-las-vegas-strip</loc>",
    "<loc>https://weedmaps.com/dispensaries/hashstoria</loc>",
    "<loc>https://weedmaps.com/dispensaries/etain-yonkers</loc>",
    "<loc>https://weedmaps.com/dispensaries/strawberry-fields-adult-use-north</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-grove-2-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/purelife-alternative-wellness-center-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/san-juan-strains</loc>",
    "<loc>https://weedmaps.com/dispensaries/choice-organics-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-mart</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbal-choices</loc>",
    "<loc>https://weedmaps.com/dispensaries/nectar-eugene</loc>",
    "<loc>https://weedmaps.com/dispensaries/veronia-s-natural-choice</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-lady-westside-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/nature-s-gifts-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/floyds-cannabis-co-shelton</loc>",
    "<loc>https://weedmaps.com/dispensaries/pot-zone</loc>",
    "<loc>https://weedmaps.com/dispensaries/walla-walla-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-joint-burlington</loc>",
    "<loc>https://weedmaps.com/dispensaries/karma-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-reef</loc>",
    "<loc>https://weedmaps.com/dispensaries/aloha-green</loc>",
    "<loc>https://weedmaps.com/dispensaries/chalice-farms-airport</loc>",
    "<loc>https://weedmaps.com/dispensaries/sanctuary-atc</loc>",
    "<loc>https://weedmaps.com/dispensaries/satori</loc>",
    "<loc>https://weedmaps.com/dispensaries/puget-sound-marijuana-recreational-21</loc>",
    "<loc>https://weedmaps.com/dispensaries/reef-dispensaries-north-las-vegas</loc>",
    "<loc>https://weedmaps.com/dispensaries/acannvi</loc>",
    "<loc>https://weedmaps.com/dispensaries/main-street-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/gold-bar-marijuana</loc>",
    "<loc>https://weedmaps.com/dispensaries/bud-pharmacy</loc>",
    "<loc>https://weedmaps.com/dispensaries/kind-care-of-colorado-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/315north</loc>",
    "<loc>https://weedmaps.com/dispensaries/grant-pharms-mmc</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-las-vegas-boulevard</loc>",
    "<loc>https://weedmaps.com/dispensaries/hatch</loc>",
    "<loc>https://weedmaps.com/dispensaries/full-bloom-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/doctors-orders-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/marleys-dream</loc>",
    "<loc>https://weedmaps.com/dispensaries/evergreen-santa-ana</loc>",
    "<loc>https://weedmaps.com/dispensaries/holistic-health-wayne</loc>",
    "<loc>https://weedmaps.com/dispensaries/doobie-sisters-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/reef-dispensaries-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/village-bloomery</loc>",
    "<loc>https://weedmaps.com/dispensaries/asociacion-dany</loc>",
    "<loc>https://weedmaps.com/dispensaries/leafline-labs-st-cloud</loc>",
    "<loc>https://weedmaps.com/dispensaries/platinum-club-la</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabist-boston</loc>",
    "<loc>https://weedmaps.com/dispensaries/ultra-health-westside</loc>",
    "<loc>https://weedmaps.com/dispensaries/solace-meds-of-wheat-ridge-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/mono-loco</loc>",
    "<loc>https://weedmaps.com/dispensaries/atomic-budz-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/vegas-treehouse</loc>",
    "<loc>https://weedmaps.com/dispensaries/thedispensarynv-henderson</loc>",
    "<loc>https://weedmaps.com/dispensaries/perfect-union-shasta</loc>",
    "<loc>https://weedmaps.com/dispensaries/urban-wellness-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/pisos</loc>",
    "<loc>https://weedmaps.com/dispensaries/uem-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/starbuds-pueblo-west-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/fiveanddimedetroit</loc>",
    "<loc>https://weedmaps.com/dispensaries/empire-health-and-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenhand-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/drgreenrx</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-fire-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/zen-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/malibu-community-collective</loc>",
    "<loc>https://weedmaps.com/dispensaries/leafline-labs-st-paul</loc>",
    "<loc>https://weedmaps.com/dispensaries/leafline-labs-hibbing</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-tallahassee</loc>",
    "<loc>https://weedmaps.com/dispensaries/exhale-nevada</loc>",
    "<loc>https://weedmaps.com/dispensaries/lightshade-iliff</loc>",
    "<loc>https://weedmaps.com/dispensaries/oasis-cannabis-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/posh-green</loc>",
    "<loc>https://weedmaps.com/dispensaries/serra-belmont</loc>",
    "<loc>https://weedmaps.com/dispensaries/everest-apothecary</loc>",
    "<loc>https://weedmaps.com/dispensaries/jenny-s-dispensary-2-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-ridge-apothecary</loc>",
    "<loc>https://weedmaps.com/dispensaries/marley-420-recreational-marijuana</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/have-a-heart-belltown</loc>",
    "<loc>https://weedmaps.com/dispensaries/zen-leaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/thc-of-lacey</loc>",
    "<loc>https://weedmaps.com/dispensaries/caring-nature-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/deep-roots-harvest</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-room-campus</loc>",
    "<loc>https://weedmaps.com/dispensaries/tangent-farmacy</loc>",
    "<loc>https://weedmaps.com/dispensaries/red-run-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/discovery-bay-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/kush-pointe</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-nirvana-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/northwest-cannabis-connection</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannazone</loc>",
    "<loc>https://weedmaps.com/dispensaries/thedispensarynv-reno</loc>",
    "<loc>https://weedmaps.com/dispensaries/reef-dispensaries-sun-valley</loc>",
    "<loc>https://weedmaps.com/dispensaries/starbuds-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/flores-de-canarias</loc>",
    "<loc>https://weedmaps.com/dispensaries/freedom-road-on-main</loc>",
    "<loc>https://weedmaps.com/dispensaries/jardin-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/caniba-naturals</loc>",
    "<loc>https://weedmaps.com/dispensaries/burnside-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-fire-house-2-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-ellsworth-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/top-notch-thc-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-lady-marijuana-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/la-cremme-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/tulare-alternative-relief-assocition</loc>",
    "<loc>https://weedmaps.com/dispensaries/nectar-gresham</loc>",
    "<loc>https://weedmaps.com/dispensaries/magnolia-road-cannabis-co-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/642-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-carson-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/long-beach-collective-2-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/iwaco-freedom-market</loc>",
    "<loc>https://weedmaps.com/dispensaries/cure-colorado-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/mynt-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/mt-hood-rec-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/mt-shasta-patients-collective</loc>",
    "<loc>https://weedmaps.com/dispensaries/pend-oreille-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/den-rec-medical</loc>",
    "<loc>https://weedmaps.com/dispensaries/new-generation</loc>",
    "<loc>https://weedmaps.com/dispensaries/kush21-burien-s-first-pot-shop</loc>",
    "<loc>https://weedmaps.com/dispensaries/15-strain-station</loc>",
    "<loc>https://weedmaps.com/dispensaries/i-90-green-house</loc>",
    "<loc>https://weedmaps.com/dispensaries/weed-dudes</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-herbery-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/bud-bloom</loc>",
    "<loc>https://weedmaps.com/dispensaries/evergreen-cannabis-society</loc>",
    "<loc>https://weedmaps.com/dispensaries/potco-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/enlighten-alaska</loc>",
    "<loc>https://weedmaps.com/dispensaries/alaskabuds</loc>",
    "<loc>https://weedmaps.com/dispensaries/alaskan-leaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/el-chinatown-verde</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbal-outfitters</loc>",
    "<loc>https://weedmaps.com/dispensaries/port-city-alternative</loc>",
    "<loc>https://weedmaps.com/dispensaries/happydayscunit</loc>",
    "<loc>https://weedmaps.com/dispensaries/blum-las-vegas-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-jar-wasilla</loc>",
    "<loc>https://weedmaps.com/dispensaries/empire-twin-palms</loc>",
    "<loc>https://weedmaps.com/dispensaries/shango-las-vegas</loc>",
    "<loc>https://weedmaps.com/dispensaries/nectar-tillamook</loc>",
    "<loc>https://weedmaps.com/dispensaries/ivthc</loc>",
    "<loc>https://weedmaps.com/dispensaries/raspberry-roots</loc>",
    "<loc>https://weedmaps.com/dispensaries/remedy-shoppe</loc>",
    "<loc>https://weedmaps.com/dispensaries/denali-420-recreationals</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-frost-farms</loc>",
    "<loc>https://weedmaps.com/dispensaries/dankorage</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-og-collective-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-roots-midtown</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-haus-kalispell</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-effingham</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-sw-miami</loc>",
    "<loc>https://weedmaps.com/dispensaries/reefer-madness</loc>",
    "<loc>https://weedmaps.com/dispensaries/satori-bellingham</loc>",
    "<loc>https://weedmaps.com/dispensaries/kushman-s</loc>",
    "<loc>https://weedmaps.com/dispensaries/zen-leaf-carson-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/patient-care-first</loc>",
    "<loc>https://weedmaps.com/dispensaries/rocky-farms</loc>",
    "<loc>https://weedmaps.com/dispensaries/champion-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/minnesota-medical-solutions-bloomington</loc>",
    "<loc>https://weedmaps.com/dispensaries/minnesota-medical-solutions-moorhead</loc>",
    "<loc>https://weedmaps.com/dispensaries/zen-leaf-pilsen</loc>",
    "<loc>https://weedmaps.com/dispensaries/rainier-cannabis-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/permafrost-distributors</loc>",
    "<loc>https://weedmaps.com/dispensaries/goodsinse</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-west-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/221-inc-222</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenplacecoffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/kaya-santa-fe</loc>",
    "<loc>https://weedmaps.com/dispensaries/house-of-dankness-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/blum-reno</loc>",
    "<loc>https://weedmaps.com/dispensaries/nextgen-pharma</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-family-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/showgrow-long-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/showgrow-san-diego</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-health-clinic-thc</loc>",
    "<loc>https://weedmaps.com/dispensaries/san-diego-natural</loc>",
    "<loc>https://weedmaps.com/dispensaries/tamarack-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/new-england-reserve</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbal-choices-coos-bay</loc>",
    "<loc>https://weedmaps.com/dispensaries/mo-faya-csc</loc>",
    "<loc>https://weedmaps.com/dispensaries/wccc-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/superskunk-coffeeshop-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-bulldog-palace</loc>",
    "<loc>https://weedmaps.com/dispensaries/mellowyellow-altea-d11ee01d-56d4-4dc1-9208-736e086a4388</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-jamaican-secret</loc>",
    "<loc>https://weedmaps.com/dispensaries/ultra-health-clovis</loc>",
    "<loc>https://weedmaps.com/dispensaries/funky-munkey-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/northern-standard</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-glass</loc>",
    "<loc>https://weedmaps.com/dispensaries/surefyre</loc>",
    "<loc>https://weedmaps.com/dispensaries/elevated-billings</loc>",
    "<loc>https://weedmaps.com/dispensaries/mary-jane-s-2-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-tampa</loc>",
    "<loc>https://weedmaps.com/dispensaries/happy-smile-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/maximillian-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/easy-max-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/maximillian-coffeeshop-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/groene-gaper-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/csc-nube-9-aguilas</loc>",
    "<loc>https://weedmaps.com/dispensaries/purlife-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/kosbor-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/sweet-relief-st-helens</loc>",
    "<loc>https://weedmaps.com/dispensaries/mr-k-and-co-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/coastal-high-ways</loc>",
    "<loc>https://weedmaps.com/dispensaries/faro-wellness-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/starrbuds</loc>",
    "<loc>https://weedmaps.com/dispensaries/culture-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/me-planto</loc>",
    "<loc>https://weedmaps.com/dispensaries/native-roots-denver-highlands-med</loc>",
    "<loc>https://weedmaps.com/dispensaries/garden-mother-herbs-missoula</loc>",
    "<loc>https://weedmaps.com/dispensaries/three-rivers-organics-rec</loc>",
    "<loc>https://weedmaps.com/dispensaries/yakima-weed-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/northern-lights-indoor-gardens</loc>",
    "<loc>https://weedmaps.com/dispensaries/citi-zen</loc>",
    "<loc>https://weedmaps.com/dispensaries/satori-south</loc>",
    "<loc>https://weedmaps.com/dispensaries/dutch-passion-retail</loc>",
    "<loc>https://weedmaps.com/dispensaries/earth-gift-cannabis-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-central</loc>",
    "<loc>https://weedmaps.com/dispensaries/sticky-fingerz</loc>",
    "<loc>https://weedmaps.com/dispensaries/louisiana-medical-marijuana-dispensaries</loc>",
    "<loc>https://weedmaps.com/dispensaries/louisiana-medical-marijuana-dispensaries-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/la-creme-gracia</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbal-healing-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/connected-belmont-shore</loc>",
    "<loc>https://weedmaps.com/dispensaries/desert-kush</loc>",
    "<loc>https://weedmaps.com/dispensaries/shift-new-mexico</loc>",
    "<loc>https://weedmaps.com/dispensaries/catalyst-cherry</loc>",
    "<loc>https://weedmaps.com/dispensaries/empire-gardens</loc>",
    "<loc>https://weedmaps.com/dispensaries/nord-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/hessel-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/europa-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/rathaus-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/rossi-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/dom-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/stadtapotheke-titisee-neustadt</loc>",
    "<loc>https://weedmaps.com/dispensaries/rems-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-am-theater</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-im-illerpark</loc>",
    "<loc>https://weedmaps.com/dispensaries/elvira-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/rosen-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/baren-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/spangenberg-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/forum-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/oasen-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/st-jakobs-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/turm-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-am-moritzplatz</loc>",
    "<loc>https://weedmaps.com/dispensaries/quartier-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-im-niedersachsenhaus</loc>",
    "<loc>https://weedmaps.com/dispensaries/adler-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/markt-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/bock-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/hermes-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/basberg-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-zur-kugel</loc>",
    "<loc>https://weedmaps.com/dispensaries/aegidien-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/stern-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/alternative-releaf-libby</loc>",
    "<loc>https://weedmaps.com/dispensaries/st-rochus-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-hiddesen</loc>",
    "<loc>https://weedmaps.com/dispensaries/konigstor-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/schwanen-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/albert-schweizer-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/sonnen-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/georg-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/patroklus-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/westfalia-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/spitzweg-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-am-stadtweiher</loc>",
    "<loc>https://weedmaps.com/dispensaries/walburga-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/schwanen-apotheke-ludinghausen</loc>",
    "<loc>https://weedmaps.com/dispensaries/baren-apotheke-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/hachtor-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-am-schlossplatz</loc>",
    "<loc>https://weedmaps.com/dispensaries/lowen-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/kristall-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-lux-99</loc>",
    "<loc>https://weedmaps.com/dispensaries/marien-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/alexandria-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/schwanenapotheke-speicher</loc>",
    "<loc>https://weedmaps.com/dispensaries/sankt-georg-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/rathaus-apotheke-hohenwestedt</loc>",
    "<loc>https://weedmaps.com/dispensaries/rats-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/awesome-blossoms</loc>",
    "<loc>https://weedmaps.com/dispensaries/cool-running</loc>",
    "<loc>https://weedmaps.com/dispensaries/dank-of-america-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannamigos-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-mint-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/nirvana-wayne</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-underground-station</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunray-cannabis-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/greener-medical-waterville</loc>",
    "<loc>https://weedmaps.com/dispensaries/remedy-missoula</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-pensacola</loc>",
    "<loc>https://weedmaps.com/dispensaries/pendleton-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/homegrown-oregon-lansing-ave</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-bradenton</loc>",
    "<loc>https://weedmaps.com/dispensaries/mississippi-medical-marijuana-dispensaries</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-joint-burien</loc>",
    "<loc>https://weedmaps.com/dispensaries/medical-cannabis-partners</loc>",
    "<loc>https://weedmaps.com/dispensaries/2020-solutions-pacific-highway</loc>",
    "<loc>https://weedmaps.com/dispensaries/dits-verds</loc>",
    "<loc>https://weedmaps.com/dispensaries/seed-smith-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-hopes</loc>",
    "<loc>https://weedmaps.com/dispensaries/mary-jane-s-cannabis-connection</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-connection-bozeman</loc>",
    "<loc>https://weedmaps.com/dispensaries/deep-roots-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenleaf-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannazone-mount-vernon</loc>",
    "<loc>https://weedmaps.com/dispensaries/alhorn-apotheken-zeppelin</loc>",
    "<loc>https://weedmaps.com/dispensaries/ribbeck-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/konigin-luise-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-im-cottbus-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/sandower-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-im-lausitzpark</loc>",
    "<loc>https://weedmaps.com/dispensaries/city-apotheke-im-blenchen-carre</loc>",
    "<loc>https://weedmaps.com/dispensaries/lowen-apotheke-frankfurt-oder</loc>",
    "<loc>https://weedmaps.com/dispensaries/hansa-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/sonnen-apotheke-frankfurt-oder</loc>",
    "<loc>https://weedmaps.com/dispensaries/lenne-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/baltic-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/korallen-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/hansa-apotheke-stralsund</loc>",
    "<loc>https://weedmaps.com/dispensaries/rats-apotheke-stralsund</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-an-marienplatz</loc>",
    "<loc>https://weedmaps.com/dispensaries/schlosspark-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/weststadt-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/lowenapotheke-am-bahnhof</loc>",
    "<loc>https://weedmaps.com/dispensaries/sudstadt-center-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/steintor-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/greif-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-im-gdz</loc>",
    "<loc>https://weedmaps.com/dispensaries/fetscher-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/bahnhof-apotheke-dresden</loc>",
    "<loc>https://weedmaps.com/dispensaries/city-apotheke-hauptstrasse</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-am-dr-kulz-ring</loc>",
    "<loc>https://weedmaps.com/dispensaries/guten-tag-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-mockern</loc>",
    "<loc>https://weedmaps.com/dispensaries/central-apotheke-leipzig</loc>",
    "<loc>https://weedmaps.com/dispensaries/lowen-apotheke-leipzig</loc>",
    "<loc>https://weedmaps.com/dispensaries/kastanien-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-sachsen-allee</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-an-der-zentralhaltestelle</loc>",
    "<loc>https://weedmaps.com/dispensaries/viktoria-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/stadtapotheke-saarbrucken</loc>",
    "<loc>https://weedmaps.com/dispensaries/gluckauf-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/markt-apotheke-neunkirchen</loc>",
    "<loc>https://weedmaps.com/dispensaries/adler-apotheke-neunkirchen</loc>",
    "<loc>https://weedmaps.com/dispensaries/osterholzer-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/horner-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/elefanten-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-im-neustadt-centrum</loc>",
    "<loc>https://weedmaps.com/dispensaries/linden-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/mohren-apotheke-halle</loc>",
    "<loc>https://weedmaps.com/dispensaries/maxxipharm-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-am-theater-magdeburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/herz-apotheke-reform</loc>",
    "<loc>https://weedmaps.com/dispensaries/park-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/beethoven-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/luther-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/die-apotheke-im-globus-jena-isserstedt</loc>",
    "<loc>https://weedmaps.com/dispensaries/hufeland-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/forst-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/classic-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/adler-apotheke-weimar</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-am-jakobstor</loc>",
    "<loc>https://weedmaps.com/dispensaries/silber-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/centro-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-am-roten-rathaus</loc>",
    "<loc>https://weedmaps.com/dispensaries/bud-bros-cave-junction</loc>",
    "<loc>https://weedmaps.com/dispensaries/hwy-420-2-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-miami</loc>",
    "<loc>https://weedmaps.com/dispensaries/tru-cannabis-se-28th</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-stoney-moose-ketchikan</loc>",
    "<loc>https://weedmaps.com/dispensaries/american-harvest</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-chronic-boutique</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenhouse-morris</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-dispensaries-phoenix</loc>",
    "<loc>https://weedmaps.com/dispensaries/modern-meds-mt</loc>",
    "<loc>https://weedmaps.com/dispensaries/kush-dispensing-company-kushco</loc>",
    "<loc>https://weedmaps.com/dispensaries/romer-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/wasserturm-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-in-der-schwabengalerie</loc>",
    "<loc>https://weedmaps.com/dispensaries/falkert-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/karl-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-berlin-hauptbahnhof</loc>",
    "<loc>https://weedmaps.com/dispensaries/fraulein-beckers-lieblingsapotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-am-hermannplatz</loc>",
    "<loc>https://weedmaps.com/dispensaries/zieten-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/vita-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/europa-apotheke-hamburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/apo-rot-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/arcaden-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/cacilien-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-im-ring-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-im-real-eschborn</loc>",
    "<loc>https://weedmaps.com/dispensaries/flora-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/hirsch-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-im-hauptbahnhof-koln</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-im-hauptbahnhof-dusseldorf</loc>",
    "<loc>https://weedmaps.com/dispensaries/ausbuttels-adler-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/gemarken-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/australia-medicinal-cannabis-pharmacy</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/think-higher-caregiving-bozeman</loc>",
    "<loc>https://weedmaps.com/dispensaries/apothecary-farms-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/2020-solutions-ephrata</loc>",
    "<loc>https://weedmaps.com/dispensaries/rose-bowls</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-camelback</loc>",
    "<loc>https://weedmaps.com/dispensaries/ascend-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-bell</loc>",
    "<loc>https://weedmaps.com/dispensaries/marimed</loc>",
    "<loc>https://weedmaps.com/dispensaries/amberlight-cannabis-house</loc>",
    "<loc>https://weedmaps.com/dispensaries/natural-healing-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/cutie-judy-s</loc>",
    "<loc>https://weedmaps.com/dispensaries/sacred-garden-albuquerque</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-fireweed-factory-juneau</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-las-vegas-western-ave</loc>",
    "<loc>https://weedmaps.com/dispensaries/catalyst-sa</loc>",
    "<loc>https://weedmaps.com/dispensaries/collective-elevation</loc>",
    "<loc>https://weedmaps.com/dispensaries/organic-releaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/st-georg-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/allegany-medical-marijuana-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/bad-gramm3r</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-alternatives-missoula</loc>",
    "<loc>https://weedmaps.com/dispensaries/mommaz-kalispell-and-cut-bank</loc>",
    "<loc>https://weedmaps.com/dispensaries/lorbeer-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/above-the-clouds-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/hofgarten-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/bohlweg-apotheke-im-schlosscarree</loc>",
    "<loc>https://weedmaps.com/dispensaries/flotow-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/malteser-apotheke-meiderich</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-am-kirchplatz</loc>",
    "<loc>https://weedmaps.com/dispensaries/anger-vital-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/ohm-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/k-k-apotheke-im-vauban</loc>",
    "<loc>https://weedmaps.com/dispensaries/jakobinen-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/elster-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/goethe-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-muller</loc>",
    "<loc>https://weedmaps.com/dispensaries/holistic-healing-collective-2-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-am-nollendorfer-hof</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-im-lohr-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/patriot-care-greenfield</loc>",
    "<loc>https://weedmaps.com/dispensaries/linden-apotheke-neunkirchen</loc>",
    "<loc>https://weedmaps.com/dispensaries/city-apotheke-oldenburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/pharmaphant</loc>",
    "<loc>https://weedmaps.com/dispensaries/stengel-apotheke-saarbrucken</loc>",
    "<loc>https://weedmaps.com/dispensaries/diana-apotheke-salzgitter-barum</loc>",
    "<loc>https://weedmaps.com/dispensaries/olympia-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/lavendel-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/strobitzer-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-zur-rose</loc>",
    "<loc>https://weedmaps.com/dispensaries/adler-apotheke-leipzig</loc>",
    "<loc>https://weedmaps.com/dispensaries/adler-apotheke-rostock</loc>",
    "<loc>https://weedmaps.com/dispensaries/bahnhofs-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/glocken-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/schiller-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/curb-stone-exchange</loc>",
    "<loc>https://weedmaps.com/dispensaries/ak-slow-burn</loc>",
    "<loc>https://weedmaps.com/dispensaries/paraiso-javea</loc>",
    "<loc>https://weedmaps.com/dispensaries/hill-bog</loc>",
    "<loc>https://weedmaps.com/dispensaries/lightshade-peoria-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/electric-lettuce-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-edgewater</loc>",
    "<loc>https://weedmaps.com/dispensaries/rocky-mountain-blaze</loc>",
    "<loc>https://weedmaps.com/dispensaries/remedy-columbia</loc>",
    "<loc>https://weedmaps.com/dispensaries/lionheart-caregiving-missoula</loc>",
    "<loc>https://weedmaps.com/dispensaries/mj-expresso-abq</loc>",
    "<loc>https://weedmaps.com/dispensaries/nova-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-lodge-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/starbuds-se-aurora</loc>",
    "<loc>https://weedmaps.com/dispensaries/starbuds-cc-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-solution-water-st</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-solution-commercial-st</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-solution-6</loc>",
    "<loc>https://weedmaps.com/dispensaries/ak-frost</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-lady-lake-the-villages</loc>",
    "<loc>https://weedmaps.com/dispensaries/dispensary-1-11</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-manchester</loc>",
    "<loc>https://weedmaps.com/dispensaries/southern-ct-wellness-and-healing</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-desert-relief-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/r-greenleaf-midtown</loc>",
    "<loc>https://weedmaps.com/dispensaries/r-greenleaf-westside</loc>",
    "<loc>https://weedmaps.com/dispensaries/r-greenleaf-nob-hill</loc>",
    "<loc>https://weedmaps.com/dispensaries/r-greenleaf-ne-heights</loc>",
    "<loc>https://weedmaps.com/dispensaries/r-greenleaf-grants</loc>",
    "<loc>https://weedmaps.com/dispensaries/paul-lincke-apotheke-kreuzberg</loc>",
    "<loc>https://weedmaps.com/dispensaries/star-buds-boulder</loc>",
    "<loc>https://weedmaps.com/dispensaries/fruit-of-the-earth-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/hanftheke-bern</loc>",
    "<loc>https://weedmaps.com/dispensaries/hanftheke-brig</loc>",
    "<loc>https://weedmaps.com/dispensaries/hanftheke-einsiedeln</loc>",
    "<loc>https://weedmaps.com/dispensaries/hanftheke-herisau</loc>",
    "<loc>https://weedmaps.com/dispensaries/hanftheke-liestal</loc>",
    "<loc>https://weedmaps.com/dispensaries/hanftheke-luzern</loc>",
    "<loc>https://weedmaps.com/dispensaries/hanftheke-winterthur</loc>",
    "<loc>https://weedmaps.com/dispensaries/cbd-star</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-passion-zurich</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-passion-winterthur</loc>",
    "<loc>https://weedmaps.com/dispensaries/treasure-tree</loc>",
    "<loc>https://weedmaps.com/dispensaries/420-ibiza</loc>",
    "<loc>https://weedmaps.com/dispensaries/secret-society</loc>",
    "<loc>https://weedmaps.com/dispensaries/kaya-yverdon-les-bains</loc>",
    "<loc>https://weedmaps.com/dispensaries/kaya-freiburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/kaya-vevey</loc>",
    "<loc>https://weedmaps.com/dispensaries/sow-grow</loc>",
    "<loc>https://weedmaps.com/dispensaries/olive-tree-wellness-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/ethnic-world-shop</loc>",
    "<loc>https://weedmaps.com/dispensaries/fourtwenty-trendshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/vision-of-hemp</loc>",
    "<loc>https://weedmaps.com/dispensaries/hh-shop</loc>",
    "<loc>https://weedmaps.com/dispensaries/jungle-shop</loc>",
    "<loc>https://weedmaps.com/dispensaries/la-belle-verte</loc>",
    "<loc>https://weedmaps.com/dispensaries/let-s-grow-lausanne</loc>",
    "<loc>https://weedmaps.com/dispensaries/mindblowers</loc>",
    "<loc>https://weedmaps.com/dispensaries/hempner</loc>",
    "<loc>https://weedmaps.com/dispensaries/paff-paff-zurich</loc>",
    "<loc>https://weedmaps.com/dispensaries/werner-s-head-shop-langstrasse-230</loc>",
    "<loc>https://weedmaps.com/dispensaries/werner-s-head-shop-limmatquai</loc>",
    "<loc>https://weedmaps.com/dispensaries/werner-s-head-shop-chur</loc>",
    "<loc>https://weedmaps.com/dispensaries/werner-s-head-shop-langstrasse-79</loc>",
    "<loc>https://weedmaps.com/dispensaries/holos-wollerau</loc>",
    "<loc>https://weedmaps.com/dispensaries/holos-sirnach</loc>",
    "<loc>https://weedmaps.com/dispensaries/tamar-headshop-winterthur</loc>",
    "<loc>https://weedmaps.com/dispensaries/secret-nature-thun</loc>",
    "<loc>https://weedmaps.com/dispensaries/secret-nature-bern</loc>",
    "<loc>https://weedmaps.com/dispensaries/noa-botanicals</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-jacksonville</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-st-petersburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/kaleafa-cannabis-company-oregon-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/mj-express-o-t-c</loc>",
    "<loc>https://weedmaps.com/dispensaries/golden-meds-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-farm</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-epic-remedy-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/bud-hut-friday-harbor</loc>",
    "<loc>https://weedmaps.com/dispensaries/heart-of-maria</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-silver-spring</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-dragon-telluride</loc>",
    "<loc>https://weedmaps.com/dispensaries/golden-nugget-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/denco-alternative-medicine-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-provisions-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-dragon-mountain-view</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-dragon-colfax</loc>",
    "<loc>https://weedmaps.com/dispensaries/great-northern-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/gushi-culture</loc>",
    "<loc>https://weedmaps.com/dispensaries/alternative-therapy</loc>",
    "<loc>https://weedmaps.com/dispensaries/spacebuds-the-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/one-hit-wonder</loc>",
    "<loc>https://weedmaps.com/dispensaries/exhalence</loc>",
    "<loc>https://weedmaps.com/dispensaries/nuvue-pueblo</loc>",
    "<loc>https://weedmaps.com/dispensaries/smokee-luzern</loc>",
    "<loc>https://weedmaps.com/dispensaries/smokee-aarau</loc>",
    "<loc>https://weedmaps.com/dispensaries/farmaverde</loc>",
    "<loc>https://weedmaps.com/dispensaries/colorado-harvest-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannatheque-nyon</loc>",
    "<loc>https://weedmaps.com/dispensaries/dispensarios-420-caguas</loc>",
    "<loc>https://weedmaps.com/dispensaries/etain-new-york-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/walla-walla-weedery-2-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/hanftheke-basel-clara-shopping</loc>",
    "<loc>https://weedmaps.com/dispensaries/hanftheke-solothurn</loc>",
    "<loc>https://weedmaps.com/dispensaries/hanftheke-kreuzlingen</loc>",
    "<loc>https://weedmaps.com/dispensaries/culture-cannabis-club-moreno-valley-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/planet-13</loc>",
    "<loc>https://weedmaps.com/dispensaries/hunter-s-filiaal</loc>",
    "<loc>https://weedmaps.com/dispensaries/sensemillia-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/ying-yang</loc>",
    "<loc>https://weedmaps.com/dispensaries/best-friends</loc>",
    "<loc>https://weedmaps.com/dispensaries/colorado-harvest-company-yale</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-bulldog-port-26</loc>",
    "<loc>https://weedmaps.com/dispensaries/maxcy-s-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/heaven-69</loc>",
    "<loc>https://weedmaps.com/dispensaries/missouri</loc>",
    "<loc>https://weedmaps.com/dispensaries/de-stoelendans</loc>",
    "<loc>https://weedmaps.com/dispensaries/capricorn-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/uncle-sam</loc>",
    "<loc>https://weedmaps.com/dispensaries/coffeeshop-blikken-deur</loc>",
    "<loc>https://weedmaps.com/dispensaries/joint-venture</loc>",
    "<loc>https://weedmaps.com/dispensaries/cafe-mix</loc>",
    "<loc>https://weedmaps.com/dispensaries/zero-zero</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-grasshopper</loc>",
    "<loc>https://weedmaps.com/dispensaries/kronkel</loc>",
    "<loc>https://weedmaps.com/dispensaries/coffeeshop-the-black-sea</loc>",
    "<loc>https://weedmaps.com/dispensaries/coffeeshop-t-kunsje</loc>",
    "<loc>https://weedmaps.com/dispensaries/dreadlock-jetset-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/coffeeshop-bazar</loc>",
    "<loc>https://weedmaps.com/dispensaries/coffeeshop-relax</loc>",
    "<loc>https://weedmaps.com/dispensaries/de-waterpijp</loc>",
    "<loc>https://weedmaps.com/dispensaries/ankara</loc>",
    "<loc>https://weedmaps.com/dispensaries/cheers</loc>",
    "<loc>https://weedmaps.com/dispensaries/bunkertje</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-happy-corner</loc>",
    "<loc>https://weedmaps.com/dispensaries/coffeeshop-havana</loc>",
    "<loc>https://weedmaps.com/dispensaries/sky-high-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/alternative-therapy-carolina</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-mid-town</loc>",
    "<loc>https://weedmaps.com/dispensaries/d2-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-apothecarium-marina</loc>",
    "<loc>https://weedmaps.com/dispensaries/goldn-bloom</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-am-bahnhof</loc>",
    "<loc>https://weedmaps.com/dispensaries/farmacia-bengoechea</loc>",
    "<loc>https://weedmaps.com/dispensaries/csc-oceano-las-canteras-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/strawberry-fields-trinidad</loc>",
    "<loc>https://weedmaps.com/dispensaries/purlife-dispensary-montgomery</loc>",
    "<loc>https://weedmaps.com/dispensaries/kannavis</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-dadeland</loc>",
    "<loc>https://weedmaps.com/dispensaries/silver-stem-fine-cannabis-fraser-winter-park-area-rec-21</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabar-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/falken-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/five-zero-trees-astoria</loc>",
    "<loc>https://weedmaps.com/dispensaries/southwest-wellness-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/adams</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-montana</loc>",
    "<loc>https://weedmaps.com/dispensaries/farmacia-termal-guaviyu</loc>",
    "<loc>https://weedmaps.com/dispensaries/farmacia-la-cabina</loc>",
    "<loc>https://weedmaps.com/dispensaries/upland-s-finest</loc>",
    "<loc>https://weedmaps.com/dispensaries/gartenfeld-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/cosmos-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/atos-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/magnolien-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/mittelrain-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/karl-olga-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/zoeppritz-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/klosterhof-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/neutor-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/stifter-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-am-pfuhlpark</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-am-rosenberg</loc>",
    "<loc>https://weedmaps.com/dispensaries/drei-linden-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/minerva-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/dr-linz-sche-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/ina-apotheke-lustnau</loc>",
    "<loc>https://weedmaps.com/dispensaries/schonbuch-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-dr-vetter</loc>",
    "<loc>https://weedmaps.com/dispensaries/die-obere-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/farmacia-albisu-termal</loc>",
    "<loc>https://weedmaps.com/dispensaries/farmacia-antartida</loc>",
    "<loc>https://weedmaps.com/dispensaries/yellowstone-cannabis-billings</loc>",
    "<loc>https://weedmaps.com/dispensaries/farmacia-carmelo</loc>",
    "<loc>https://weedmaps.com/dispensaries/farmacia-gortari</loc>",
    "<loc>https://weedmaps.com/dispensaries/farmacia-caceres</loc>",
    "<loc>https://weedmaps.com/dispensaries/farmacia-nueva-brun</loc>",
    "<loc>https://weedmaps.com/dispensaries/farmacia-medicci-iii</loc>",
    "<loc>https://weedmaps.com/dispensaries/kuony-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/schwarzwald-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-haass-schillerplatz</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-am-ebertplatz</loc>",
    "<loc>https://weedmaps.com/dispensaries/herdern-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/frida-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/blasius-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/flora-apotheke-mullheim</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenhouse-basel</loc>",
    "<loc>https://weedmaps.com/dispensaries/cultivate-leicester</loc>",
    "<loc>https://weedmaps.com/dispensaries/pals-collective-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-am-schillerplatz</loc>",
    "<loc>https://weedmaps.com/dispensaries/werder-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/waldhorn-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/bodensee-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/carme-66-cannabis-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/hofener-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/michaelis-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/munster-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/spindel-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/marien-apotheke-bayreuth</loc>",
    "<loc>https://weedmaps.com/dispensaries/brandenburger-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-an-roten-hugel</loc>",
    "<loc>https://weedmaps.com/dispensaries/museo-del-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-marijuana-factory</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-manhattan</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-rochester</loc>",
    "<loc>https://weedmaps.com/dispensaries/ganja-goddess-bozeman</loc>",
    "<loc>https://weedmaps.com/dispensaries/lau-ola</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-aloha-kauai</loc>",
    "<loc>https://weedmaps.com/dispensaries/igel-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-auf-der-wart</loc>",
    "<loc>https://weedmaps.com/dispensaries/daig-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/dreifaltigkeits-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/hof-apotheke-am-markt</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-im-reichsgraf</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-scheuerfeld</loc>",
    "<loc>https://weedmaps.com/dispensaries/kronenapotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/gold-apotheke-am-bergl</loc>",
    "<loc>https://weedmaps.com/dispensaries/nectar-hillsdale</loc>",
    "<loc>https://weedmaps.com/dispensaries/deutschhof-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/mozart-apotheke-am-dom</loc>",
    "<loc>https://weedmaps.com/dispensaries/frauenland-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/tannig-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/noris-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/admira-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-am-stadtpark</loc>",
    "<loc>https://weedmaps.com/dispensaries/zimmermann-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/regental-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/theresien-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/stadt-apotheke-hauzenberg</loc>",
    "<loc>https://weedmaps.com/dispensaries/bayerwald-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/gesundhof-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/florian-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/sage-naturals</loc>",
    "<loc>https://weedmaps.com/dispensaries/revolutionary-clinics</loc>",
    "<loc>https://weedmaps.com/dispensaries/ascend-cannabis-co-medical</loc>",
    "<loc>https://weedmaps.com/dispensaries/healthy-education-society-carlsbad</loc>",
    "<loc>https://weedmaps.com/dispensaries/marigold</loc>",
    "<loc>https://weedmaps.com/dispensaries/cure-oahu</loc>",
    "<loc>https://weedmaps.com/dispensaries/boerejongens-coffeeshop-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/boerejongens-coffeeshop-west</loc>",
    "<loc>https://weedmaps.com/dispensaries/boerejongens-bij</loc>",
    "<loc>https://weedmaps.com/dispensaries/be-brooklyn</loc>",
    "<loc>https://weedmaps.com/dispensaries/good</loc>",
    "<loc>https://weedmaps.com/dispensaries/asam-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/severin-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/pazzawa-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-am-messepark</loc>",
    "<loc>https://weedmaps.com/dispensaries/hofberg-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/rennweg-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/ruffini-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/alpha-apohteke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-im-turm</loc>",
    "<loc>https://weedmaps.com/dispensaries/rathaus-apotheke-pfarrkirche</loc>",
    "<loc>https://weedmaps.com/dispensaries/alz-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/christorphus-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/st-georg-apotheke-traunstein</loc>",
    "<loc>https://weedmaps.com/dispensaries/hubertus-apotheke-traunstein</loc>",
    "<loc>https://weedmaps.com/dispensaries/pauersche-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/hahn-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/mountain-heights</loc>",
    "<loc>https://weedmaps.com/dispensaries/710-montana</loc>",
    "<loc>https://weedmaps.com/dispensaries/glacier-view-providers</loc>",
    "<loc>https://weedmaps.com/dispensaries/keystone-integrated-care-greensburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-im-luitpoldcenter</loc>",
    "<loc>https://weedmaps.com/dispensaries/st-pauls-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-vor-den-toren</loc>",
    "<loc>https://weedmaps.com/dispensaries/theresien-apotheke-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-am-vincentinum</loc>",
    "<loc>https://weedmaps.com/dispensaries/don-bosco-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/easyapotheke-landsberg</loc>",
    "<loc>https://weedmaps.com/dispensaries/bayertor-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/lechrain-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-im-oberosch</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-im-lyzeum</loc>",
    "<loc>https://weedmaps.com/dispensaries/alpin-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/christkonig-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/max-josef-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/enzian-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/purexis</loc>",
    "<loc>https://weedmaps.com/dispensaries/a-cut-above-denver-rec</loc>",
    "<loc>https://weedmaps.com/dispensaries/dreitorspitz-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/repelsteeltje</loc>",
    "<loc>https://weedmaps.com/dispensaries/loungeshop-domino</loc>",
    "<loc>https://weedmaps.com/dispensaries/culture-boat</loc>",
    "<loc>https://weedmaps.com/dispensaries/promenade-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/pluspunkt-apotheke-im-oder-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/urban-wellness-paradise</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-am-waldrand</loc>",
    "<loc>https://weedmaps.com/dispensaries/stadt-apotheke-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/forst-apotheke-eberswalde</loc>",
    "<loc>https://weedmaps.com/dispensaries/neue-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/drgreen</loc>",
    "<loc>https://weedmaps.com/dispensaries/westend-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/neue-apotheke-wittenberge</loc>",
    "<loc>https://weedmaps.com/dispensaries/bismarck-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/elbland-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/nordapotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/la-isla-verde-marbella</loc>",
    "<loc>https://weedmaps.com/dispensaries/flower-power-2-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/coffeeshop-de-eenhoorn</loc>",
    "<loc>https://weedmaps.com/dispensaries/koffieshop-relax</loc>",
    "<loc>https://weedmaps.com/dispensaries/koffie-en-dromen</loc>",
    "<loc>https://weedmaps.com/dispensaries/chillie-kiki</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-low</loc>",
    "<loc>https://weedmaps.com/dispensaries/coffeeshop-andersom</loc>",
    "<loc>https://weedmaps.com/dispensaries/elevate-shasta-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/chesapeake-apothecary</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbafi-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/linden-apotheke-pritzwalk</loc>",
    "<loc>https://weedmaps.com/dispensaries/hirsch-apotheke-pritzwalk</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-am-hain</loc>",
    "<loc>https://weedmaps.com/dispensaries/serturner-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/brucken-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/dammtor-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/mauritius-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/monchen-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/trees-dispensary-englewood</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-devon</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-philadelphia</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-shops-king-of-prussia-henderson</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-cured</loc>",
    "<loc>https://weedmaps.com/dispensaries/strawberry-fields-dumont</loc>",
    "<loc>https://weedmaps.com/dispensaries/storchen-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannaflora</loc>",
    "<loc>https://weedmaps.com/dispensaries/alte-priviligierte-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/sander-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/kurbad-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/enzian-apotheke-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/bahnhof-apotheke-eschwege</loc>",
    "<loc>https://weedmaps.com/dispensaries/schloss-apotheke-eschwege</loc>",
    "<loc>https://weedmaps.com/dispensaries/goldbach-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/st-lioba-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/st-bonifatius-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/florenberg-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-am-riegerplatz</loc>",
    "<loc>https://weedmaps.com/dispensaries/donnersberg-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-an-der-sudanlage</loc>",
    "<loc>https://weedmaps.com/dispensaries/durer-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/liebig-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-am-muhlentor</loc>",
    "<loc>https://weedmaps.com/dispensaries/rosen-apotheke-grimmen</loc>",
    "<loc>https://weedmaps.com/dispensaries/sonnen-apotheke-wismar</loc>",
    "<loc>https://weedmaps.com/dispensaries/mowen-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/friedenshof-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/lowen-apotheke-demmin</loc>",
    "<loc>https://weedmaps.com/dispensaries/muhrbeck-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/pommern-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/nectar-montavilla</loc>",
    "<loc>https://weedmaps.com/dispensaries/hilltop-premium-green</loc>",
    "<loc>https://weedmaps.com/dispensaries/lowen-apotheke-anklam</loc>",
    "<loc>https://weedmaps.com/dispensaries/adler-apotheke-anklam</loc>",
    "<loc>https://weedmaps.com/dispensaries/cothenius-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/wall-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/ost-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/iris-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/fritz-reuter-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/buchholz-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/moltke-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/fasanerie-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-im-technikum</loc>",
    "<loc>https://weedmaps.com/dispensaries/semmelweis-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/kreis-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/eichen-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/storch-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/deich-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/nordwest-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/kaiser-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/st-willehad-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/neptun-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/grune-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-marijuana-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/hochhaus-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/constantia-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/warf-apotheke-larrelt</loc>",
    "<loc>https://weedmaps.com/dispensaries/jade-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/friesen-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-obenstrohe</loc>",
    "<loc>https://weedmaps.com/dispensaries/elisabeth-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/uhlen-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/wunderburg-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/maxi-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/moor-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/hansa-apotheke-friesoythe</loc>",
    "<loc>https://weedmaps.com/dispensaries/neue-apotheke-zeven</loc>",
    "<loc>https://weedmaps.com/dispensaries/kloster-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/die-alte-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/biokonopia</loc>",
    "<loc>https://weedmaps.com/dispensaries/biokonopia-sarl</loc>",
    "<loc>https://weedmaps.com/dispensaries/wasserknecht-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-am-bockelsberg</loc>",
    "<loc>https://weedmaps.com/dispensaries/elch-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/kloster-apotheke-uelzen</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-am-stern</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-am-hambrocker-berg</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-k10</loc>",
    "<loc>https://weedmaps.com/dispensaries/tauben-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/die-alte-stadtapotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/medicus-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/nuss-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-klein-hehlen</loc>",
    "<loc>https://weedmaps.com/dispensaries/neuenhauser-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-garssen</loc>",
    "<loc>https://weedmaps.com/dispensaries/ratsapotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/hindenburg-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-am-pohl</loc>",
    "<loc>https://weedmaps.com/dispensaries/pinguin-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/krapendorfer-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/laings-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/burg-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/wiho-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/ultra-health-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbacann-apothecary</loc>",
    "<loc>https://weedmaps.com/dispensaries/ecocann</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-damaschke</loc>",
    "<loc>https://weedmaps.com/dispensaries/medicus-wesken-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/darmer-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbal-healing-colorado-ave</loc>",
    "<loc>https://weedmaps.com/dispensaries/igadi-central-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/christorphorus-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/silberfund-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/salzer-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/kurpark-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/hubertus-apotheke-seesen</loc>",
    "<loc>https://weedmaps.com/dispensaries/max-moritz-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/kreuzberg-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/hubertus-apotheke-gottingen</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-b27</loc>",
    "<loc>https://weedmaps.com/dispensaries/weser-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/kuhlenkamp-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/teutoburger-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/wittekind-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/vennhof-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-an-der-post</loc>",
    "<loc>https://weedmaps.com/dispensaries/hasselbach-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/freedom-market-cathlamet</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-spanish-springs</loc>",
    "<loc>https://weedmaps.com/dispensaries/bonifatius-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/maspernapotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/st-meinolf-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/magdalenen-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/johannis-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/petri-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/tannenberg-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/zeughaus-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/widder-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/420-capitol</loc>",
    "<loc>https://weedmaps.com/dispensaries/mile-high-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/kushman-s-mukilteo</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-roots-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/phoenix-apotheke-munster</loc>",
    "<loc>https://weedmaps.com/dispensaries/erpho-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/blumen-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/einhorn-apotheke-duisburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/kleeblatt-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/st-anton-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/linner-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/alte-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/hummel-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/aquis-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/abtei-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/artemis-luzern</loc>",
    "<loc>https://weedmaps.com/dispensaries/vennbahn-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/stein-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/schloss-apotheke-montabaur</loc>",
    "<loc>https://weedmaps.com/dispensaries/billamboz-shop</loc>",
    "<loc>https://weedmaps.com/dispensaries/rathaus-apotheke-montabaur</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-am-brand</loc>",
    "<loc>https://weedmaps.com/dispensaries/dijon-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/fackeltor-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/bannjerruck-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/hildegardis-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-am-oberen-markt</loc>",
    "<loc>https://weedmaps.com/dispensaries/grizzly-pine</loc>",
    "<loc>https://weedmaps.com/dispensaries/baldenau-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/eulen-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/servatius-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/festung-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/beda-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/liebfrauen-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/petrus-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-am-jahnstadion</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-am-kirchplatz-hoyerswerda</loc>",
    "<loc>https://weedmaps.com/dispensaries/gesundbrunnen-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/husaren-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/scarabaeus-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/robert-koch-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/humboldt-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/hirsch-apotheke-gorlitz</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-bethesda</loc>",
    "<loc>https://weedmaps.com/dispensaries/missoula-cannabis-caregivers</loc>",
    "<loc>https://weedmaps.com/dispensaries/heirloom-remedies</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-solution-edgewater</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-rockville</loc>",
    "<loc>https://weedmaps.com/dispensaries/verilife-maryland</loc>",
    "<loc>https://weedmaps.com/dispensaries/hahnemann-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/die-bahnhofs-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/mohren-apotheke-torgau</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-im-kaufland</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-am-krankenhaus</loc>",
    "<loc>https://weedmaps.com/dispensaries/lolshop-sagl</loc>",
    "<loc>https://weedmaps.com/dispensaries/adler-apotheke-borna</loc>",
    "<loc>https://weedmaps.com/dispensaries/auer-stadt-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/brunnen-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-zeller-berg</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-in-haselbrunn</loc>",
    "<loc>https://weedmaps.com/dispensaries/concordia-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/kurt-mothes-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/herz-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/arkaden-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-am-luchower-tor</loc>",
    "<loc>https://weedmaps.com/dispensaries/albrecht-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/roland-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/coffee-shop-moon-s-vevey</loc>",
    "<loc>https://weedmaps.com/dispensaries/altmarkt-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/coffee-shop-moon-s-lausanne</loc>",
    "<loc>https://weedmaps.com/dispensaries/winckelmann-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/house-of-flowers-echo-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-am-luisium</loc>",
    "<loc>https://weedmaps.com/dispensaries/elbe-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/kobold-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-am-saigertor</loc>",
    "<loc>https://weedmaps.com/dispensaries/aesculap-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/kahnaqueen</loc>",
    "<loc>https://weedmaps.com/dispensaries/kupferberg-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/harz-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/nathusius-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/flora-friesland-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/godske-hansen-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/st-michael-apotheke-am-ochsenmarkt</loc>",
    "<loc>https://weedmaps.com/dispensaries/friedheim-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/easyapotheke-flensburg-galerie</loc>",
    "<loc>https://weedmaps.com/dispensaries/schwan-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/staggenborg-die-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/schlei-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/adler-apotheke-kappeln</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenhouse-wellness-columbia</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-pittsburgh</loc>",
    "<loc>https://weedmaps.com/dispensaries/lowen-apotheke-kappeln</loc>",
    "<loc>https://weedmaps.com/dispensaries/nectar-salem</loc>",
    "<loc>https://weedmaps.com/dispensaries/surterra-wellness-center-pensacola</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-am-wattenmeer</loc>",
    "<loc>https://weedmaps.com/dispensaries/utholm-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/st-georg-apotheke-heide</loc>",
    "<loc>https://weedmaps.com/dispensaries/esmarch-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/rungholt-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/stadtapotheke-heiligenhafen</loc>",
    "<loc>https://weedmaps.com/dispensaries/anker-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/einhorn-apotheke-kiel</loc>",
    "<loc>https://weedmaps.com/dispensaries/lornsen-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/medica-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/aesculap-apotheke-neumunster</loc>",
    "<loc>https://weedmaps.com/dispensaries/delphin-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/falderapark-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/glashutter-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/tarpenbek-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/hillside-natural-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/medicine-man-5-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-zelienople</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-gainesville</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-vero-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-orlando</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-fort-lauderdale</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-deerfield-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-sarasota</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-dania-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/la-naturacura</loc>",
    "<loc>https://weedmaps.com/dispensaries/columbia-care-pennsylvania-scranton</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-solution-potomac</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-solution-colfax</loc>",
    "<loc>https://weedmaps.com/dispensaries/wallis-roots</loc>",
    "<loc>https://weedmaps.com/dispensaries/cbdplus-snc</loc>",
    "<loc>https://weedmaps.com/dispensaries/columbia-care-pennsylvania-wilkes-barre</loc>",
    "<loc>https://weedmaps.com/dispensaries/mister-nice-basel</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-brookville</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-bradford</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-reading-5th</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-apothecarium-plymouth-meeting-7</loc>",
    "<loc>https://weedmaps.com/dispensaries/maitri-medicinals-pittsburgh</loc>",
    "<loc>https://weedmaps.com/dispensaries/ethos-allentown</loc>",
    "<loc>https://weedmaps.com/dispensaries/organic-remedies-inc-enola</loc>",
    "<loc>https://weedmaps.com/dispensaries/mister-nice-olten</loc>",
    "<loc>https://weedmaps.com/dispensaries/restore-integrative-wellness-center-philadelphia</loc>",
    "<loc>https://weedmaps.com/dispensaries/zen-leaf-sellersville</loc>",
    "<loc>https://weedmaps.com/dispensaries/zen-leaf-abington</loc>",
    "<loc>https://weedmaps.com/dispensaries/zen-leaf-malvern</loc>",
    "<loc>https://weedmaps.com/dispensaries/moorbek-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/galenus-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/hirsch-apotheke-lubeck</loc>",
    "<loc>https://weedmaps.com/dispensaries/neue-laurentius-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/aesculap-apotheke-gera</loc>",
    "<loc>https://weedmaps.com/dispensaries/abakus-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-an-der-brute</loc>",
    "<loc>https://weedmaps.com/dispensaries/markt-apotheke-saalfeld</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-von-hirschhausen</loc>",
    "<loc>https://weedmaps.com/dispensaries/gertruden-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/mohren-apotheke-bad-salzungen</loc>",
    "<loc>https://weedmaps.com/dispensaries/stadion-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-am-goethepark</loc>",
    "<loc>https://weedmaps.com/dispensaries/surterra-wellness-center-deltona</loc>",
    "<loc>https://weedmaps.com/dispensaries/ultra-health-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-apothecarium-2-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/verde-natural-on-colfax-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/top-shelf-botanicals-sheridan</loc>",
    "<loc>https://weedmaps.com/dispensaries/crystal-spring-healing-alternatives</loc>",
    "<loc>https://weedmaps.com/dispensaries/starbuds-federal-heights</loc>",
    "<loc>https://weedmaps.com/dispensaries/stokes</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-hanover</loc>",
    "<loc>https://weedmaps.com/dispensaries/natures-care-and-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/culcanna</loc>",
    "<loc>https://weedmaps.com/dispensaries/starbuds-niwot</loc>",
    "<loc>https://weedmaps.com/dispensaries/speedy-janes</loc>",
    "<loc>https://weedmaps.com/dispensaries/asociacion-cannabis-club-cafe-alcala</loc>",
    "<loc>https://weedmaps.com/dispensaries/tutankhamon</loc>",
    "<loc>https://weedmaps.com/dispensaries/bahama-buds</loc>",
    "<loc>https://weedmaps.com/dispensaries/idris-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/hometree-industry-st</loc>",
    "<loc>https://weedmaps.com/dispensaries/peake-releaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/senses-med-center-san-juan</loc>",
    "<loc>https://weedmaps.com/dispensaries/zen-leaf-elkridge</loc>",
    "<loc>https://weedmaps.com/dispensaries/tomafarm</loc>",
    "<loc>https://weedmaps.com/dispensaries/emerald-court-yard</loc>",
    "<loc>https://weedmaps.com/dispensaries/catalyst-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/pharmkent</loc>",
    "<loc>https://weedmaps.com/dispensaries/kure-cannabis-downtown</loc>",
    "<loc>https://weedmaps.com/dispensaries/vapes-n-dabs</loc>",
    "<loc>https://weedmaps.com/dispensaries/lightshade-federal-heights-medical</loc>",
    "<loc>https://weedmaps.com/dispensaries/sespe-creek-collective-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/rebel-roots</loc>",
    "<loc>https://weedmaps.com/dispensaries/vapor-spirit-zurich</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-coffee-joint</loc>",
    "<loc>https://weedmaps.com/dispensaries/cbd360</loc>",
    "<loc>https://weedmaps.com/dispensaries/organico</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-chemistry-broadway</loc>",
    "<loc>https://weedmaps.com/dispensaries/tru-cannabis-portland-north-east</loc>",
    "<loc>https://weedmaps.com/dispensaries/bali</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-experience-420</loc>",
    "<loc>https://weedmaps.com/dispensaries/nature-amp-co-growshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/cbd-pilatus</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-station</loc>",
    "<loc>https://weedmaps.com/dispensaries/aaa-pharmaceutical-alternatives</loc>",
    "<loc>https://weedmaps.com/dispensaries/sky-dispensaries-phoenix</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-point-wellness-baltimore</loc>",
    "<loc>https://weedmaps.com/dispensaries/rontgen-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/florian-geyer-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-kleinrinderfeld</loc>",
    "<loc>https://weedmaps.com/dispensaries/sira-naturals</loc>",
    "<loc>https://weedmaps.com/dispensaries/bloom-medicinals-cannabis-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/district-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/bilmam-sweed</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-botanicals</loc>",
    "<loc>https://weedmaps.com/dispensaries/holistic-alternative-inc-57-d-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/southern-maryland-relief</loc>",
    "<loc>https://weedmaps.com/dispensaries/smoking-crow</loc>",
    "<loc>https://weedmaps.com/dispensaries/isla-bonita</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-pharmacie-rapahel</loc>",
    "<loc>https://weedmaps.com/dispensaries/farm-grass-table</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-cabinet-rec</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenwave-maryland</loc>",
    "<loc>https://weedmaps.com/dispensaries/klinic-cannabis-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/punto-verde-medical-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/haven</loc>",
    "<loc>https://weedmaps.com/dispensaries/choice-labs</loc>",
    "<loc>https://weedmaps.com/dispensaries/revolution-releaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/chalice-farms-downtown</loc>",
    "<loc>https://weedmaps.com/dispensaries/mindright-llc-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/hanftheke-bremgarten</loc>",
    "<loc>https://weedmaps.com/dispensaries/hanftheke-rapperswil</loc>",
    "<loc>https://weedmaps.com/dispensaries/hanftheke-wiedikon</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbiculture</loc>",
    "<loc>https://weedmaps.com/dispensaries/complete-releaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/elevated-sf-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/positive-energy</loc>",
    "<loc>https://weedmaps.com/dispensaries/ak-joint</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-phoenix</loc>",
    "<loc>https://weedmaps.com/dispensaries/rosebuds-shatter-house</loc>",
    "<loc>https://weedmaps.com/dispensaries/top-shelf-botanicals-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/top-shelf-botanicals-bozeman</loc>",
    "<loc>https://weedmaps.com/dispensaries/king-of-budz</loc>",
    "<loc>https://weedmaps.com/dispensaries/a-greener-today-bothell-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-living-room</loc>",
    "<loc>https://weedmaps.com/dispensaries/rocky-road-thornton</loc>",
    "<loc>https://weedmaps.com/dispensaries/friends-lounge-bcn</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-solution-grape-st</loc>",
    "<loc>https://weedmaps.com/dispensaries/riverside-provisioning</loc>",
    "<loc>https://weedmaps.com/dispensaries/tru-bliss</loc>",
    "<loc>https://weedmaps.com/dispensaries/asociacion-kyo</loc>",
    "<loc>https://weedmaps.com/dispensaries/southwest-wellness-group</loc>",
    "<loc>https://weedmaps.com/dispensaries/valley-pure</loc>",
    "<loc>https://weedmaps.com/dispensaries/new-millennium-eugene</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-lake-worth</loc>",
    "<loc>https://weedmaps.com/dispensaries/releaf-shop</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-dank</loc>",
    "<loc>https://weedmaps.com/dispensaries/ojai-greens</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-hop</loc>",
    "<loc>https://weedmaps.com/dispensaries/amagi</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-bean-pharm-3-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/tamar-headshop-hauptgeschaft-winterthur</loc>",
    "<loc>https://weedmaps.com/dispensaries/roots-dispensaries</loc>",
    "<loc>https://weedmaps.com/dispensaries/house-of-green-anchorage</loc>",
    "<loc>https://weedmaps.com/dispensaries/sacred-garden-las-cruces</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-lutherville</loc>",
    "<loc>https://weedmaps.com/dispensaries/cutthroat-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/trove-cannabis-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/jet-room-high-desert</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-kind-castle</loc>",
    "<loc>https://weedmaps.com/dispensaries/elite-bay-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/cookies-la</loc>",
    "<loc>https://weedmaps.com/dispensaries/colibri-greenshop-montreux</loc>",
    "<loc>https://weedmaps.com/dispensaries/bio-top-zurich</loc>",
    "<loc>https://weedmaps.com/dispensaries/state-line-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/ash-ember-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-reisterstown</loc>",
    "<loc>https://weedmaps.com/dispensaries/colibri-greenshop-neuchatel</loc>",
    "<loc>https://weedmaps.com/dispensaries/marry-jane-niederdorf</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-apothecarium-bethlehem</loc>",
    "<loc>https://weedmaps.com/dispensaries/dispensary-works</loc>",
    "<loc>https://weedmaps.com/dispensaries/triple-m-plymouth</loc>",
    "<loc>https://weedmaps.com/dispensaries/triple-m-mashpee</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannassence</loc>",
    "<loc>https://weedmaps.com/dispensaries/babacool-fashion</loc>",
    "<loc>https://weedmaps.com/dispensaries/trilogy-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/schwanen-apotheke-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/starbuds-baltimore</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-lid</loc>",
    "<loc>https://weedmaps.com/dispensaries/strawberry-fields-apothecary</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/liberty-health-sciences-summerfield</loc>",
    "<loc>https://weedmaps.com/dispensaries/liberty-health-sciences-st-petersburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/liberty-health-sciences-port-st-lucie</loc>",
    "<loc>https://weedmaps.com/dispensaries/liberty-health-sciences-tampa-hyde-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/liberty-health-sciences-fort-myers</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-wrigleyville</loc>",
    "<loc>https://weedmaps.com/dispensaries/cy-dispensaries</loc>",
    "<loc>https://weedmaps.com/dispensaries/skunkmasters</loc>",
    "<loc>https://weedmaps.com/dispensaries/glacier-green-cross</loc>",
    "<loc>https://weedmaps.com/dispensaries/albatros-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/fat-hippie</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenhouse-herbal-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannassence-san-juan</loc>",
    "<loc>https://weedmaps.com/dispensaries/hanflife</loc>",
    "<loc>https://weedmaps.com/dispensaries/365-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-cannabis-dispensary-lancaster</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-cannabis-dispensary-philadelphia-city-ave</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-cannabis-dispensary-phoenixville</loc>",
    "<loc>https://weedmaps.com/dispensaries/cg-corrigan-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/keep-calm-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/culta</loc>",
    "<loc>https://weedmaps.com/dispensaries/seed-of-life-labs</loc>",
    "<loc>https://weedmaps.com/dispensaries/pura-vida-csc</loc>",
    "<loc>https://weedmaps.com/dispensaries/ethos-baltimore</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-solution-kentucky</loc>",
    "<loc>https://weedmaps.com/dispensaries/sky-high-cannabis-ltd</loc>",
    "<loc>https://weedmaps.com/dispensaries/d-o-a-dank-on-arrival</loc>",
    "<loc>https://weedmaps.com/dispensaries/la-florista-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/blair-wellness-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-pottery</loc>",
    "<loc>https://weedmaps.com/dispensaries/smoke-signals-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-hudson-valley</loc>",
    "<loc>https://weedmaps.com/dispensaries/kannabis-works</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-ny-plattsburgh</loc>",
    "<loc>https://weedmaps.com/dispensaries/hi-tide-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/firehaus</loc>",
    "<loc>https://weedmaps.com/dispensaries/uncle-bucks-billings</loc>",
    "<loc>https://weedmaps.com/dispensaries/chalice-farms-happy-valley</loc>",
    "<loc>https://weedmaps.com/dispensaries/liberty-health-sciences-dania-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/dispensario-chg</loc>",
    "<loc>https://weedmaps.com/dispensaries/higher-elevation-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/dispo-bay-city-north</loc>",
    "<loc>https://weedmaps.com/dispensaries/euphoria-wellness-missoula</loc>",
    "<loc>https://weedmaps.com/dispensaries/buda-smoking-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/420-factory</loc>",
    "<loc>https://weedmaps.com/dispensaries/zen-leaf-il-st-charles</loc>",
    "<loc>https://weedmaps.com/dispensaries/homegrown-oregon-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/marry-jane-geneve</loc>",
    "<loc>https://weedmaps.com/dispensaries/flavors</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-columbia</loc>",
    "<loc>https://weedmaps.com/dispensaries/satori-wellness-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-station-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/lighthouse-organics-billings-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/naturalpes</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-oxford-adult-use</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbaceous-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/sensimilla-cbd-shop</loc>",
    "<loc>https://weedmaps.com/dispensaries/chronic-caregivers</loc>",
    "<loc>https://weedmaps.com/dispensaries/jwellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/critical-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-arboretum-florence</loc>",
    "<loc>https://weedmaps.com/dispensaries/storehouse-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/l-b-collective-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/der-hollaender</loc>",
    "<loc>https://weedmaps.com/dispensaries/your-green-thumb</loc>",
    "<loc>https://weedmaps.com/dispensaries/elite-glass</loc>",
    "<loc>https://weedmaps.com/dispensaries/smoking-grasshopper-cannabis-sommelier-coffeeshop-headshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/medical-green-solution</loc>",
    "<loc>https://weedmaps.com/dispensaries/cironishop</loc>",
    "<loc>https://weedmaps.com/dispensaries/family-first</loc>",
    "<loc>https://weedmaps.com/dispensaries/eclipse-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/ac-lanzacana</loc>",
    "<loc>https://weedmaps.com/dispensaries/cookies-eugene</loc>",
    "<loc>https://weedmaps.com/dispensaries/hollyweed-907</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenlabs</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-joppa</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-pearl-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/coachella-lighthouse-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/coffee-shop-moon-s</loc>",
    "<loc>https://weedmaps.com/dispensaries/lessing-apotheke-gesundbrunnen</loc>",
    "<loc>https://weedmaps.com/dispensaries/zieten-apotheke-wedding</loc>",
    "<loc>https://weedmaps.com/dispensaries/kolberger-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/prisma-apotheke-gesundbrunnen</loc>",
    "<loc>https://weedmaps.com/dispensaries/utrechter-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-am-nauener-platz</loc>",
    "<loc>https://weedmaps.com/dispensaries/dr-groplers-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/leo-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/wedding-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/diana-apotheke-mitte</loc>",
    "<loc>https://weedmaps.com/dispensaries/selektum</loc>",
    "<loc>https://weedmaps.com/dispensaries/kind-acres-farm</loc>",
    "<loc>https://weedmaps.com/dispensaries/matanuska-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/markische-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/virchow-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/doc-apotheke-wedding</loc>",
    "<loc>https://weedmaps.com/dispensaries/muller-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-augustenburger-platz</loc>",
    "<loc>https://weedmaps.com/dispensaries/die-herz-apotheke-mitte</loc>",
    "<loc>https://weedmaps.com/dispensaries/oehmichens-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/oasis-dispensaries</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-point-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/die-herz-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/jasmin-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-am-gartenplatz</loc>",
    "<loc>https://weedmaps.com/dispensaries/julicher-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/gesundbrunnen-apotheke-mitte</loc>",
    "<loc>https://weedmaps.com/dispensaries/dafne-apotheke-wedding</loc>",
    "<loc>https://weedmaps.com/dispensaries/nordkreuz-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/schinkel-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/everest-apothecary-north-east-heights</loc>",
    "<loc>https://weedmaps.com/dispensaries/chesacanna</loc>",
    "<loc>https://weedmaps.com/dispensaries/brunnen-apotheke-wedding</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-im-gesundbrunnen-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/bad-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/neue-apotheke-wedding</loc>",
    "<loc>https://weedmaps.com/dispensaries/heinrich-zille-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/wollank-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/kolonie-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/tilia-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/sanimedius-apotheke-pankower-allee</loc>",
    "<loc>https://weedmaps.com/dispensaries/martin-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/residenz-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/berkshire-roots</loc>",
    "<loc>https://weedmaps.com/dispensaries/mayflower-boston</loc>",
    "<loc>https://weedmaps.com/dispensaries/mrs-green-ch-shop</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-popeye</loc>",
    "<loc>https://weedmaps.com/dispensaries/sensimilla-trading</loc>",
    "<loc>https://weedmaps.com/dispensaries/eber-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/growhealthy-bonita-springs</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-pharm-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/earth-s-healing</loc>",
    "<loc>https://weedmaps.com/dispensaries/cross-genetics-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-wellness-cannabis-center-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/traditional</loc>",
    "<loc>https://weedmaps.com/dispensaries/strains</loc>",
    "<loc>https://weedmaps.com/dispensaries/stiiizy-san-francisco</loc>",
    "<loc>https://weedmaps.com/dispensaries/everbloom-on-race</loc>",
    "<loc>https://weedmaps.com/dispensaries/everbloom-on-49th</loc>",
    "<loc>https://weedmaps.com/dispensaries/everbloom-on-jackson</loc>",
    "<loc>https://weedmaps.com/dispensaries/last-best-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/centro-demeter</loc>",
    "<loc>https://weedmaps.com/dispensaries/dixon-wellness-collective</loc>",
    "<loc>https://weedmaps.com/dispensaries/salvera</loc>",
    "<loc>https://weedmaps.com/dispensaries/mary-and-main-capitol-heights</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-amherst</loc>",
    "<loc>https://weedmaps.com/dispensaries/all-about-wellness-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/herban-legends-towson</loc>",
    "<loc>https://weedmaps.com/dispensaries/ascend-cannabis-co-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/headbanger</loc>",
    "<loc>https://weedmaps.com/dispensaries/cy-dispensaries-pittsburgh</loc>",
    "<loc>https://weedmaps.com/dispensaries/natural-mystic-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-grove-la-mesa</loc>",
    "<loc>https://weedmaps.com/dispensaries/medusa-farms</loc>",
    "<loc>https://weedmaps.com/dispensaries/heidi-s-garden</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-fort-myers</loc>",
    "<loc>https://weedmaps.com/dispensaries/temescal-wellness-hudson</loc>",
    "<loc>https://weedmaps.com/dispensaries/temescal-wellness-pittsfield</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-drug-store</loc>",
    "<loc>https://weedmaps.com/dispensaries/electric-lettuce-beaverton</loc>",
    "<loc>https://weedmaps.com/dispensaries/billo</loc>",
    "<loc>https://weedmaps.com/dispensaries/wonderlog</loc>",
    "<loc>https://weedmaps.com/dispensaries/treestar-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-marijuana-company-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-higher-conscious</loc>",
    "<loc>https://weedmaps.com/dispensaries/california-cannabis-crenshaw</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-st-petersburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/originalslashop</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-owl</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-abingdon</loc>",
    "<loc>https://weedmaps.com/dispensaries/kaya-cannabis-on-jewell</loc>",
    "<loc>https://weedmaps.com/dispensaries/magic-flower</loc>",
    "<loc>https://weedmaps.com/dispensaries/nectar-springfield</loc>",
    "<loc>https://weedmaps.com/dispensaries/kaleafa-cannabis-company-gresham</loc>",
    "<loc>https://weedmaps.com/dispensaries/manhattan-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/mmedics</loc>",
    "<loc>https://weedmaps.com/dispensaries/mint-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/apotheke-im-west-erding-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/raices</loc>",
    "<loc>https://weedmaps.com/dispensaries/yanks-indian-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/jars-cannabis-peoria</loc>",
    "<loc>https://weedmaps.com/dispensaries/a-cut-off-the-top</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-halethorpe</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-new-port-richey</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbal-choices-bandon-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/mon-ami</loc>",
    "<loc>https://weedmaps.com/dispensaries/dr-pleasure</loc>",
    "<loc>https://weedmaps.com/dispensaries/headshop-hanfkranz</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-klub</loc>",
    "<loc>https://weedmaps.com/dispensaries/babylon</loc>",
    "<loc>https://weedmaps.com/dispensaries/maple-valley-pharms</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-house-of-dank</loc>",
    "<loc>https://weedmaps.com/dispensaries/rosebud-remedies</loc>",
    "<loc>https://weedmaps.com/dispensaries/hi-fidelity</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenleaf-apothecary-cleveland</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenleaf-apothecary-akron</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenleaf-apothecary-canton</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenleaf-apothecary-wickliffe</loc>",
    "<loc>https://weedmaps.com/dispensaries/leaf-relief</loc>",
    "<loc>https://weedmaps.com/dispensaries/zen-leaf-bowling-green</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-forest-ohio-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenleaf-apothecary-columbus</loc>",
    "<loc>https://weedmaps.com/dispensaries/zen-leaf-newark</loc>",
    "<loc>https://weedmaps.com/dispensaries/ohio-valley-natural-relief-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/pure-ohio-wellness-dayton</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-lorain</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-lakewood-madison</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-cleveland</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-toledo</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-lakewood-detroit</loc>",
    "<loc>https://weedmaps.com/dispensaries/cure-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/positive-growth-gardens</loc>",
    "<loc>https://weedmaps.com/dispensaries/health-for-life-crismon</loc>",
    "<loc>https://weedmaps.com/dispensaries/slamdank</loc>",
    "<loc>https://weedmaps.com/dispensaries/flowers-farmacy-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/kush-klub-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/east-bay-therapeutic</loc>",
    "<loc>https://weedmaps.com/dispensaries/nature-s-medicines-state-college</loc>",
    "<loc>https://weedmaps.com/dispensaries/sweet-relief-gearhart</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-passyunk</loc>",
    "<loc>https://weedmaps.com/dispensaries/1-mill</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbology-gettysburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-sensation</loc>",
    "<loc>https://weedmaps.com/dispensaries/grow-healthy-orlando</loc>",
    "<loc>https://weedmaps.com/dispensaries/grow-healthy-tampa</loc>",
    "<loc>https://weedmaps.com/dispensaries/grow-healthy-deerfield-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/growhealthy-west-palm-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/mynt-cannabis-north</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-palm-harbor</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-palm-bay</loc>",
    "<loc>https://weedmaps.com/dispensaries/terrapin-care-station-manhattan-circle</loc>",
    "<loc>https://weedmaps.com/dispensaries/defyne-premium-cannabis-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/cherry-peak</loc>",
    "<loc>https://weedmaps.com/dispensaries/columbia-care-delaware</loc>",
    "<loc>https://weedmaps.com/dispensaries/mamcuc</loc>",
    "<loc>https://weedmaps.com/dispensaries/port-city-relief</loc>",
    "<loc>https://weedmaps.com/dispensaries/bloom-peoria</loc>",
    "<loc>https://weedmaps.com/dispensaries/starbuds-longmont</loc>",
    "<loc>https://weedmaps.com/dispensaries/liberty-philadelphia</loc>",
    "<loc>https://weedmaps.com/dispensaries/liberty-bensalem</loc>",
    "<loc>https://weedmaps.com/dispensaries/rocky-farms-med</loc>",
    "<loc>https://weedmaps.com/dispensaries/happy-shack-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/satori-fremont</loc>",
    "<loc>https://weedmaps.com/dispensaries/nectar-hall</loc>",
    "<loc>https://weedmaps.com/dispensaries/zen-garden-wellness-stockton</loc>",
    "<loc>https://weedmaps.com/dispensaries/dr-green-spot</loc>",
    "<loc>https://weedmaps.com/dispensaries/hirsch-apotheke-am-wormser-tor</loc>",
    "<loc>https://weedmaps.com/dispensaries/eden-gran-canaria</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-apollo-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/weedis</loc>",
    "<loc>https://weedmaps.com/dispensaries/outlawz-store</loc>",
    "<loc>https://weedmaps.com/dispensaries/mana-supply-co-edgewater</loc>",
    "<loc>https://weedmaps.com/dispensaries/first-medical-cannabis-isabela</loc>",
    "<loc>https://weedmaps.com/dispensaries/verd-fort</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-chemistry-worcester</loc>",
    "<loc>https://weedmaps.com/dispensaries/harvest-goddess</loc>",
    "<loc>https://weedmaps.com/dispensaries/growbox</loc>",
    "<loc>https://weedmaps.com/dispensaries/exotic</loc>",
    "<loc>https://weedmaps.com/dispensaries/columbia-care-logan</loc>",
    "<loc>https://weedmaps.com/dispensaries/columbia-care-marietta</loc>",
    "<loc>https://weedmaps.com/dispensaries/columbia-care-monroe</loc>",
    "<loc>https://weedmaps.com/dispensaries/columbia-care-dayton</loc>",
    "<loc>https://weedmaps.com/dispensaries/la-mala</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenport-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/blunts-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/denali-dispensaries</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-airpark</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-flushing</loc>",
    "<loc>https://weedmaps.com/dispensaries/verilife-cincinnati</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-marijuana-company-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/harvest-of-ohio-columbus</loc>",
    "<loc>https://weedmaps.com/dispensaries/harvest-of-athens</loc>",
    "<loc>https://weedmaps.com/dispensaries/harvest-of-ohio-beavercreek</loc>",
    "<loc>https://weedmaps.com/dispensaries/dockside-cannabis-ballard</loc>",
    "<loc>https://weedmaps.com/dispensaries/dockside-cannabis-85th-aurora</loc>",
    "<loc>https://weedmaps.com/dispensaries/gleaf-warren</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-dispensary-wintersville</loc>",
    "<loc>https://weedmaps.com/dispensaries/zen-leaf-dayton</loc>",
    "<loc>https://weedmaps.com/dispensaries/terrasana-dispensaries-garfield-heights</loc>",
    "<loc>https://weedmaps.com/dispensaries/terrasana-dispensaries-fremont</loc>",
    "<loc>https://weedmaps.com/dispensaries/terrasana-dispensaries-columbus</loc>",
    "<loc>https://weedmaps.com/dispensaries/terrasana-dispensaries-springfield</loc>",
    "<loc>https://weedmaps.com/dispensaries/truth-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/natures-releaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/dragonfly-wellness-center-fort-bragg</loc>",
    "<loc>https://weedmaps.com/dispensaries/lifted-meds-missoula</loc>",
    "<loc>https://weedmaps.com/dispensaries/lifted-meds-kalispell</loc>",
    "<loc>https://weedmaps.com/dispensaries/elevation-2477</loc>",
    "<loc>https://weedmaps.com/dispensaries/west-coast-cannabis-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/royal-greens</loc>",
    "<loc>https://weedmaps.com/dispensaries/liberty-somerville</loc>",
    "<loc>https://weedmaps.com/dispensaries/western-bud-skagit-valley-wa</loc>",
    "<loc>https://weedmaps.com/dispensaries/western-bud-anacortes-wa</loc>",
    "<loc>https://weedmaps.com/dispensaries/health-for-life-bethesda</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-kendall-dadeland</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-spot-420-pueblo</loc>",
    "<loc>https://weedmaps.com/dispensaries/toke-house-thunder-bay-james-st</loc>",
    "<loc>https://weedmaps.com/dispensaries/elite</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-peoples-remedy-patterson</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-lady-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/march-and-ash</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbology-oakdale</loc>",
    "<loc>https://weedmaps.com/dispensaries/detroit-herbal-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-lakeland</loc>",
    "<loc>https://weedmaps.com/dispensaries/marijuanaclub99</loc>",
    "<loc>https://weedmaps.com/dispensaries/memorial-drive-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-gaia-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/sinsemilla-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/oklahoma-roots</loc>",
    "<loc>https://weedmaps.com/dispensaries/sol-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/euphoria-wellness-butte</loc>",
    "<loc>https://weedmaps.com/dispensaries/maggies-farm-pueblo-north</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-palm-coast</loc>",
    "<loc>https://weedmaps.com/dispensaries/mmd-lbc</loc>",
    "<loc>https://weedmaps.com/dispensaries/treeline-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/buda</loc>",
    "<loc>https://weedmaps.com/dispensaries/hempbar-biel</loc>",
    "<loc>https://weedmaps.com/dispensaries/coffee-shop-club-private</loc>",
    "<loc>https://weedmaps.com/dispensaries/garden-remedies-melrose</loc>",
    "<loc>https://weedmaps.com/dispensaries/coltura-botanica</loc>",
    "<loc>https://weedmaps.com/dispensaries/pure-options-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/one-love-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/t-town-medical-marijuana-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/los-gigantes</loc>",
    "<loc>https://weedmaps.com/dispensaries/mass-wellspring-acton</loc>",
    "<loc>https://weedmaps.com/dispensaries/shift-new-mexico-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/naturemeds</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannacruz-salinas</loc>",
    "<loc>https://weedmaps.com/dispensaries/higher-health</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-sarasota</loc>",
    "<loc>https://weedmaps.com/dispensaries/vidacann-jacksonville</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-tampa-dale-mabry</loc>",
    "<loc>https://weedmaps.com/dispensaries/vidacann-fort-lauderdale</loc>",
    "<loc>https://weedmaps.com/dispensaries/vidacann-orlando</loc>",
    "<loc>https://weedmaps.com/dispensaries/vidacann-port-charlotte</loc>",
    "<loc>https://weedmaps.com/dispensaries/vidacann-pensacola</loc>",
    "<loc>https://weedmaps.com/dispensaries/vidacann-st-petersburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/vidacann-west-palm-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/vidacann-tallahassee</loc>",
    "<loc>https://weedmaps.com/dispensaries/vidacann-panama-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/vidacann-miami</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-jacksonville</loc>",
    "<loc>https://weedmaps.com/dispensaries/apothca-eugene</loc>",
    "<loc>https://weedmaps.com/dispensaries/jars-cannabis-north-phoenix</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-boynton-beach-8th</loc>",
    "<loc>https://weedmaps.com/dispensaries/friendly-market</loc>",
    "<loc>https://weedmaps.com/dispensaries/friendly-market-norman</loc>",
    "<loc>https://weedmaps.com/dispensaries/patient-centric</loc>",
    "<loc>https://weedmaps.com/dispensaries/oklahoma-medical-garden</loc>",
    "<loc>https://weedmaps.com/dispensaries/daily-blaze</loc>",
    "<loc>https://weedmaps.com/dispensaries/health-for-life-baltimore</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbology-dubois</loc>",
    "<loc>https://weedmaps.com/dispensaries/health-for-life-white-marsh</loc>",
    "<loc>https://weedmaps.com/dispensaries/farma-verde</loc>",
    "<loc>https://weedmaps.com/dispensaries/small-town-buds-coming-soon</loc>",
    "<loc>https://weedmaps.com/dispensaries/majestic-gardens</loc>",
    "<loc>https://weedmaps.com/dispensaries/f-street-dispensary-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-tree-medicinals-of-berthoud</loc>",
    "<loc>https://weedmaps.com/dispensaries/westside-weed</loc>",
    "<loc>https://weedmaps.com/dispensaries/liberty-health-sciences-boca-raton</loc>",
    "<loc>https://weedmaps.com/dispensaries/mari-med</loc>",
    "<loc>https://weedmaps.com/dispensaries/liberty-health-sciences-merritt-island</loc>",
    "<loc>https://weedmaps.com/dispensaries/liberty-health-sciences-gainesville</loc>",
    "<loc>https://weedmaps.com/dispensaries/jouw-coffee-shop</loc>",
    "<loc>https://weedmaps.com/dispensaries/whole-leaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-solution-sheridan</loc>",
    "<loc>https://weedmaps.com/dispensaries/lush-leaf-cannabis-ltd-comingsoon</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbology-altoona</loc>",
    "<loc>https://weedmaps.com/dispensaries/t-g-c</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-cured-fairfield</loc>",
    "<loc>https://weedmaps.com/dispensaries/great-northern-cannabis-south-anchorage</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-botanist-worcester</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-botanist-leominster</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-botanist-shrewsbury</loc>",
    "<loc>https://weedmaps.com/dispensaries/comfort-care-cannabliss-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-botanist-buffalo</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-botanist-middletown</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-botanist-farmingdale</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-botanist-queens</loc>",
    "<loc>https://weedmaps.com/dispensaries/house-of-terps</loc>",
    "<loc>https://weedmaps.com/dispensaries/davis-cannabis-collective</loc>",
    "<loc>https://weedmaps.com/dispensaries/wicked-weed</loc>",
    "<loc>https://weedmaps.com/dispensaries/sqdc</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-micro-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/craft-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/sqdc-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/sqdc-levis-store</loc>",
    "<loc>https://weedmaps.com/dispensaries/sqdc-trois-rivieres</loc>",
    "<loc>https://weedmaps.com/dispensaries/sqdc-drummondville-store</loc>",
    "<loc>https://weedmaps.com/dispensaries/sqdc-montreal1</loc>",
    "<loc>https://weedmaps.com/dispensaries/leaf-and-lion</loc>",
    "<loc>https://weedmaps.com/dispensaries/sqdc-montreal2</loc>",
    "<loc>https://weedmaps.com/dispensaries/sqdc-montreal3</loc>",
    "<loc>https://weedmaps.com/dispensaries/sqdc-rosemont</loc>",
    "<loc>https://weedmaps.com/dispensaries/sqdc-rimouski</loc>",
    "<loc>https://weedmaps.com/dispensaries/sqdc-mirabel</loc>",
    "<loc>https://weedmaps.com/dispensaries/sqdc-st-jean-sur-richelieu</loc>",
    "<loc>https://weedmaps.com/dispensaries/sqdc-mascouche</loc>",
    "<loc>https://weedmaps.com/dispensaries/sqdc-joliette</loc>",
    "<loc>https://weedmaps.com/dispensaries/sqdc-brossard-coming-soon</loc>",
    "<loc>https://weedmaps.com/dispensaries/liberty-health-sciences-palm-harbor</loc>",
    "<loc>https://weedmaps.com/dispensaries/liberty-health-sciences-miami</loc>",
    "<loc>https://weedmaps.com/dispensaries/liberty-health-sciences-winter-haven</loc>",
    "<loc>https://weedmaps.com/dispensaries/rustys-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/primaweed</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-nassau</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-ft-pierce</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-orlando</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-deerfield-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-daytona</loc>",
    "<loc>https://weedmaps.com/dispensaries/beltline-cannabis-calgary-nov-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/global-leaf-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/new-leaf-cannabis-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/co-op-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/natures-medicines-fall-river</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-bud-ranch</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-greens-of-central-oklahoma</loc>",
    "<loc>https://weedmaps.com/dispensaries/sensicare</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-dankery-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/cbd-plus-usa-newcastle</loc>",
    "<loc>https://weedmaps.com/dispensaries/cbd-plus-usa-oklahoma-city-warwick-dr</loc>",
    "<loc>https://weedmaps.com/dispensaries/cbd-plus-usa-del-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/cbd-plus-usa-yukon</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-health-center-tulsa</loc>",
    "<loc>https://weedmaps.com/dispensaries/plantlife</loc>",
    "<loc>https://weedmaps.com/dispensaries/leaf-better</loc>",
    "<loc>https://weedmaps.com/dispensaries/rocky-mountain-collective</loc>",
    "<loc>https://weedmaps.com/dispensaries/rocky-mountain-collective-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/buds-lawton</loc>",
    "<loc>https://weedmaps.com/dispensaries/no-rhyme-or-reason-medical-marijuana-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/hemprx</loc>",
    "<loc>https://weedmaps.com/dispensaries/farmacia-silleda</loc>",
    "<loc>https://weedmaps.com/dispensaries/compassionate-care-by-design</loc>",
    "<loc>https://weedmaps.com/dispensaries/inhal</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-west-palm-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/710-montana-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/atrium</loc>",
    "<loc>https://weedmaps.com/dispensaries/blunts-moore</loc>",
    "<loc>https://weedmaps.com/dispensaries/surterra-wellness-center-palm-bay</loc>",
    "<loc>https://weedmaps.com/dispensaries/rocky-mountain-cannabis-trinidad</loc>",
    "<loc>https://weedmaps.com/dispensaries/likewise-broadway-edmond-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-east-village-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/newleaf-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/newleaf-cannabis-8</loc>",
    "<loc>https://weedmaps.com/dispensaries/numo-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannawise</loc>",
    "<loc>https://weedmaps.com/dispensaries/kip</loc>",
    "<loc>https://weedmaps.com/dispensaries/nectar-milwaukie</loc>",
    "<loc>https://weedmaps.com/dispensaries/nectar-hazelwood</loc>",
    "<loc>https://weedmaps.com/dispensaries/five-zero-trees-oregon-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/east-coast-gold</loc>",
    "<loc>https://weedmaps.com/dispensaries/get-bak-d-okc</loc>",
    "<loc>https://weedmaps.com/dispensaries/kaleafa-cannabis-company-ashland</loc>",
    "<loc>https://weedmaps.com/dispensaries/apothecary-farms-pueblo-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/thedabandgo-edmond</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-vth</loc>",
    "<loc>https://weedmaps.com/dispensaries/theglasscook-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/greendoctor-420</loc>",
    "<loc>https://weedmaps.com/dispensaries/bongs-and-such-plus</loc>",
    "<loc>https://weedmaps.com/dispensaries/bridge-bud-supply-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/cac-venice-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/sacred-garden-san-mateo-blvd</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-orlando-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/rite-greens-maywood</loc>",
    "<loc>https://weedmaps.com/dispensaries/clarity-cannabis-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/elevate-118</loc>",
    "<loc>https://weedmaps.com/dispensaries/ultra-health-albuquerque-north-valley</loc>",
    "<loc>https://weedmaps.com/dispensaries/ultra-health-espanola</loc>",
    "<loc>https://weedmaps.com/dispensaries/newleaf-cannabis-14</loc>",
    "<loc>https://weedmaps.com/dispensaries/alaska-weeds</loc>",
    "<loc>https://weedmaps.com/dispensaries/silver-therapeutics</loc>",
    "<loc>https://weedmaps.com/dispensaries/cbd-kingz-sous-gare</loc>",
    "<loc>https://weedmaps.com/dispensaries/cbd-kingz-flon</loc>",
    "<loc>https://weedmaps.com/dispensaries/wildwood-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/premium-class</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-tallahassee</loc>",
    "<loc>https://weedmaps.com/dispensaries/growhealthy-brandon</loc>",
    "<loc>https://weedmaps.com/dispensaries/growhealthy-lake-worth</loc>",
    "<loc>https://weedmaps.com/dispensaries/growhealthy-cape-coral</loc>",
    "<loc>https://weedmaps.com/dispensaries/growhealthy-tallahassee</loc>",
    "<loc>https://weedmaps.com/dispensaries/growhealthy-north-miami</loc>",
    "<loc>https://weedmaps.com/dispensaries/growhealthy-lakeland</loc>",
    "<loc>https://weedmaps.com/dispensaries/growhealthy-daytona</loc>",
    "<loc>https://weedmaps.com/dispensaries/growhealthy-gainesville</loc>",
    "<loc>https://weedmaps.com/dispensaries/growhealthy-sarasota</loc>",
    "<loc>https://weedmaps.com/dispensaries/253-organic-montague-farmacy</loc>",
    "<loc>https://weedmaps.com/dispensaries/caroline-s-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-health-switzerland</loc>",
    "<loc>https://weedmaps.com/dispensaries/terrapin-care-station-longmont</loc>",
    "<loc>https://weedmaps.com/dispensaries/field-of-dreams</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-tampa</loc>",
    "<loc>https://weedmaps.com/dispensaries/amnesia-haze-barcelona</loc>",
    "<loc>https://weedmaps.com/dispensaries/nslc-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/nslc-cannabis-amherst</loc>",
    "<loc>https://weedmaps.com/dispensaries/nslc-cannabis-antigonish</loc>",
    "<loc>https://weedmaps.com/dispensaries/nslc-cannabis-bridgewater</loc>",
    "<loc>https://weedmaps.com/dispensaries/nslc-cannabis-dartmouth</loc>",
    "<loc>https://weedmaps.com/dispensaries/nslc-cannabis-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/nslc-cannabis-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/kind-and-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/k-beach-reef-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/rosebud-remedies-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/ohg-oklahoma-home-grown</loc>",
    "<loc>https://weedmaps.com/dispensaries/palm-royale-collective</loc>",
    "<loc>https://weedmaps.com/dispensaries/ok-roots-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/la-granja</loc>",
    "<loc>https://weedmaps.com/dispensaries/monterey-bay-reef</loc>",
    "<loc>https://weedmaps.com/dispensaries/hemp24</loc>",
    "<loc>https://weedmaps.com/dispensaries/magic-city-releaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/liberty-health-sciences-hollywood</loc>",
    "<loc>https://weedmaps.com/dispensaries/liberty-health-sciences-bonita-springs</loc>",
    "<loc>https://weedmaps.com/dispensaries/liberty-health-sciences-orange-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/nslc-new-glasgow</loc>",
    "<loc>https://weedmaps.com/dispensaries/powell-farms-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-north-miami</loc>",
    "<loc>https://weedmaps.com/dispensaries/hohenzollern-apotheke</loc>",
    "<loc>https://weedmaps.com/dispensaries/capital-dank-edmond</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-med-shop</loc>",
    "<loc>https://weedmaps.com/dispensaries/queen-of-bud-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-plus</loc>",
    "<loc>https://weedmaps.com/dispensaries/urbn-roots</loc>",
    "<loc>https://weedmaps.com/dispensaries/med-pharm</loc>",
    "<loc>https://weedmaps.com/dispensaries/tulsa-dispensary-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/lit-la-mesa</loc>",
    "<loc>https://weedmaps.com/dispensaries/cowboy-cannabis-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/emerald-elite-thc</loc>",
    "<loc>https://weedmaps.com/dispensaries/cbd-farm-s</loc>",
    "<loc>https://weedmaps.com/dispensaries/surterra-wellness-lakeland</loc>",
    "<loc>https://weedmaps.com/dispensaries/surterra-wellness-new-port-richey</loc>",
    "<loc>https://weedmaps.com/dispensaries/surterra-wellness-ocala</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-releaf-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-honeypot-shop</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-leaf-el-paseo</loc>",
    "<loc>https://weedmaps.com/dispensaries/peak-green-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/318-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-kushery-at-stanwood</loc>",
    "<loc>https://weedmaps.com/dispensaries/kind-guy-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-ocala</loc>",
    "<loc>https://weedmaps.com/dispensaries/natures-medicines-bay-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/everest-apothecary-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/swiss-free-shop</loc>",
    "<loc>https://weedmaps.com/dispensaries/nslc-cannabis-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/arctic-herbery-anchorage</loc>",
    "<loc>https://weedmaps.com/dispensaries/jova-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/guzman-enterprises-llc-dba-pharm-788</loc>",
    "<loc>https://weedmaps.com/dispensaries/maryjanes-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-remedy</loc>",
    "<loc>https://weedmaps.com/dispensaries/big-island-grown-b-i-g-kona</loc>",
    "<loc>https://weedmaps.com/dispensaries/big-island-grown-b-i-g-waimea</loc>",
    "<loc>https://weedmaps.com/dispensaries/relax-barcelona-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/provisions-med</loc>",
    "<loc>https://weedmaps.com/dispensaries/releaf-resolutions</loc>",
    "<loc>https://weedmaps.com/dispensaries/sticky-fingers-marbella</loc>",
    "<loc>https://weedmaps.com/dispensaries/sticky-tarifa</loc>",
    "<loc>https://weedmaps.com/dispensaries/redeye-420</loc>",
    "<loc>https://weedmaps.com/dispensaries/el-barrio</loc>",
    "<loc>https://weedmaps.com/dispensaries/werner-s-head-shop-zug</loc>",
    "<loc>https://weedmaps.com/dispensaries/hanf-laden-bern</loc>",
    "<loc>https://weedmaps.com/dispensaries/punto-verde-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/happy-valley-boston</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-doctor</loc>",
    "<loc>https://weedmaps.com/dispensaries/root-66-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/clarity-cannabis-4-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/fred-s-farmacopia-emporium</loc>",
    "<loc>https://weedmaps.com/dispensaries/doc-green-s-oklahoma</loc>",
    "<loc>https://weedmaps.com/dispensaries/three-rivers-dispensary-pueblo-west</loc>",
    "<loc>https://weedmaps.com/dispensaries/el-remedio</loc>",
    "<loc>https://weedmaps.com/dispensaries/big-daddy-s-marijuana-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-solution-pueblo</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-lutz</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-tops-powers</loc>",
    "<loc>https://weedmaps.com/dispensaries/grow-west-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/affordable-medical-cannabis-west-siloam-springs</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-bellmawr</loc>",
    "<loc>https://weedmaps.com/dispensaries/care-collective-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/columbia-care-allentown</loc>",
    "<loc>https://weedmaps.com/dispensaries/okie-kush-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/lucky-monkey-buds</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-washington</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-flower-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/firehouse-365</loc>",
    "<loc>https://weedmaps.com/dispensaries/mr-green-the-village</loc>",
    "<loc>https://weedmaps.com/dispensaries/all-kind</loc>",
    "<loc>https://weedmaps.com/dispensaries/verilife-wareham-receational</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-country-bud-inc-tulsa-s-yale-ave</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-dispensary-on-52nd</loc>",
    "<loc>https://weedmaps.com/dispensaries/highbrow-manchester</loc>",
    "<loc>https://weedmaps.com/dispensaries/highbrow-topsham</loc>",
    "<loc>https://weedmaps.com/dispensaries/cushman-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/dtn-retail</loc>",
    "<loc>https://weedmaps.com/dispensaries/blooms-industry</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-canopy-solutions-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-truck-farms-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/thc-med-las-palmas</loc>",
    "<loc>https://weedmaps.com/dispensaries/nirvana-thc-club-555</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-venice</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbal-remedies-caregivers-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/happa-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/nslc-sydney-river</loc>",
    "<loc>https://weedmaps.com/dispensaries/nslc-truro</loc>",
    "<loc>https://weedmaps.com/dispensaries/nslc-yarmouth</loc>",
    "<loc>https://weedmaps.com/dispensaries/guerrilla-green-inspiration</loc>",
    "<loc>https://weedmaps.com/dispensaries/blum-san-leandro</loc>",
    "<loc>https://weedmaps.com/dispensaries/minerva-canna-of-grove</loc>",
    "<loc>https://weedmaps.com/dispensaries/og-nation-south-gate</loc>",
    "<loc>https://weedmaps.com/dispensaries/merry-jane-s-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/oasis-healing-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/mission-grown-medicinals</loc>",
    "<loc>https://weedmaps.com/dispensaries/santa-cruz-naturals</loc>",
    "<loc>https://weedmaps.com/dispensaries/harvest-health-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/tweed-conception-bay-hwy</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-titusville</loc>",
    "<loc>https://weedmaps.com/dispensaries/zen-leaf-rogers-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/mango-cannabis-enid</loc>",
    "<loc>https://weedmaps.com/dispensaries/cube-barcelona</loc>",
    "<loc>https://weedmaps.com/dispensaries/traditional-biologics-company-dba-remedy-tulalip</loc>",
    "<loc>https://weedmaps.com/dispensaries/best-dispensaries</loc>",
    "<loc>https://weedmaps.com/dispensaries/plantlife-canada</loc>",
    "<loc>https://weedmaps.com/dispensaries/hazy-leaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-horizons</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-bonita-springs</loc>",
    "<loc>https://weedmaps.com/dispensaries/cbd-plus-usa-broken-arrow</loc>",
    "<loc>https://weedmaps.com/dispensaries/cbd-plus-usa-bartlesville</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-st-augustine</loc>",
    "<loc>https://weedmaps.com/dispensaries/houston-grass-station</loc>",
    "<loc>https://weedmaps.com/dispensaries/prairie-records</loc>",
    "<loc>https://weedmaps.com/dispensaries/405-cbd</loc>",
    "<loc>https://weedmaps.com/dispensaries/natures-medicines-uxbridge</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-provisions-group</loc>",
    "<loc>https://weedmaps.com/dispensaries/silver-state-relief-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/chill-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/mr-green-12th</loc>",
    "<loc>https://weedmaps.com/dispensaries/midwest-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/elite-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/black-purple-kush-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/keystone-alternative-medicine</loc>",
    "<loc>https://weedmaps.com/dispensaries/terpy</loc>",
    "<loc>https://weedmaps.com/dispensaries/buds-unlimited-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-room-ak</loc>",
    "<loc>https://weedmaps.com/dispensaries/jar-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunburst-pharm</loc>",
    "<loc>https://weedmaps.com/dispensaries/plant-family-therapeutics</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-circle-long-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/jar-cannabis-co-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/fuel-420</loc>",
    "<loc>https://weedmaps.com/dispensaries/elevate-6010</loc>",
    "<loc>https://weedmaps.com/dispensaries/leaf-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/almalibre</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-door-dispensary-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-springs-medical</loc>",
    "<loc>https://weedmaps.com/dispensaries/420-dispensary-russellville</loc>",
    "<loc>https://weedmaps.com/dispensaries/terra-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-deerfield-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-halfmoon-clifton-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/fiddler-s-green</loc>",
    "<loc>https://weedmaps.com/dispensaries/arkansas-natural-products</loc>",
    "<loc>https://weedmaps.com/dispensaries/nea-full-spectrum</loc>",
    "<loc>https://weedmaps.com/dispensaries/suite-443</loc>",
    "<loc>https://weedmaps.com/dispensaries/me-medco</loc>",
    "<loc>https://weedmaps.com/dispensaries/maine-releaf-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-plainz-strains-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/california-cannabis-soto</loc>",
    "<loc>https://weedmaps.com/dispensaries/shatter-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/solar-therapeutics</loc>",
    "<loc>https://weedmaps.com/dispensaries/kahnaqueen-paquis</loc>",
    "<loc>https://weedmaps.com/dispensaries/nectar-house</loc>",
    "<loc>https://weedmaps.com/dispensaries/medicine-man-longmont</loc>",
    "<loc>https://weedmaps.com/dispensaries/pecos-valley-production</loc>",
    "<loc>https://weedmaps.com/dispensaries/verilife-westminster</loc>",
    "<loc>https://weedmaps.com/dispensaries/pecos-valley-production-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/pecos-valley-production-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-morton</loc>",
    "<loc>https://weedmaps.com/dispensaries/tamarack-cannabis-boutique</loc>",
    "<loc>https://weedmaps.com/dispensaries/ridge-cannabis-products</loc>",
    "<loc>https://weedmaps.com/dispensaries/juniper-cannabis-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-king-of-prussia</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbology-lebanon</loc>",
    "<loc>https://weedmaps.com/dispensaries/mystic-shop</loc>",
    "<loc>https://weedmaps.com/dispensaries/evolv-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/apothecary-farms-denver-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/phenos</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-fall-river</loc>",
    "<loc>https://weedmaps.com/dispensaries/dreamweavers-cannabis-products-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-wellington</loc>",
    "<loc>https://weedmaps.com/dispensaries/hanfecke-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/be-hudson-valley</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-dispensary-hudson-valley</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-dispensary-mohawk-valley</loc>",
    "<loc>https://weedmaps.com/dispensaries/bluepoint-wellness-westport</loc>",
    "<loc>https://weedmaps.com/dispensaries/arrow-alternative-care-stamford</loc>",
    "<loc>https://weedmaps.com/dispensaries/zen-leaf-meriden</loc>",
    "<loc>https://weedmaps.com/dispensaries/ultra-health-las-cruces</loc>",
    "<loc>https://weedmaps.com/dispensaries/plantlife-canada-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/noa-botanicals-kaneohe</loc>",
    "<loc>https://weedmaps.com/dispensaries/liberty-health-sciences-north-miami</loc>",
    "<loc>https://weedmaps.com/dispensaries/gleaf-frederick</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-roots-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/gardengirl420</loc>",
    "<loc>https://weedmaps.com/dispensaries/ultra-health-los-lunas</loc>",
    "<loc>https://weedmaps.com/dispensaries/ultra-health-farmington</loc>",
    "<loc>https://weedmaps.com/dispensaries/ultra-health-sunland-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/ultra-health-roswell</loc>",
    "<loc>https://weedmaps.com/dispensaries/essence-provisioning-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-provisions-holyoke</loc>",
    "<loc>https://weedmaps.com/dispensaries/celestialbuds</loc>",
    "<loc>https://weedmaps.com/dispensaries/smokee-olten</loc>",
    "<loc>https://weedmaps.com/dispensaries/smokee-baden</loc>",
    "<loc>https://weedmaps.com/dispensaries/royal-green-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/north-island-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/maryjane-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/urban-canna</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannamed-ok</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-sunshine</loc>",
    "<loc>https://weedmaps.com/dispensaries/cy-dispensaries-new-kensington</loc>",
    "<loc>https://weedmaps.com/dispensaries/urban-leaf-company-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/lucky-lion</loc>",
    "<loc>https://weedmaps.com/dispensaries/perfect-union-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/strains-perris-ca</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-drug-store-bern</loc>",
    "<loc>https://weedmaps.com/dispensaries/natural-green-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/nature-s-fix-dispensary-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/atlantic-farms-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/johnny-appleseed-remedies</loc>",
    "<loc>https://weedmaps.com/dispensaries/big-chief-collective</loc>",
    "<loc>https://weedmaps.com/dispensaries/broken-arrow-cbd-medicinal-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/big-chief-collective-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/cloudberry-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/big-papa-s-okie-toke</loc>",
    "<loc>https://weedmaps.com/dispensaries/fire-on-fore-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/best-leaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/starbuds-97229925-ec06-40f1-9a04-be2df130bae8</loc>",
    "<loc>https://weedmaps.com/dispensaries/california-cannabis-melrose</loc>",
    "<loc>https://weedmaps.com/dispensaries/mana-middle-river</loc>",
    "<loc>https://weedmaps.com/dispensaries/wild-horse-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-gold-group-charlton</loc>",
    "<loc>https://weedmaps.com/dispensaries/tempel-store</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-miami</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-plus-moore</loc>",
    "<loc>https://weedmaps.com/dispensaries/elevated-longmont</loc>",
    "<loc>https://weedmaps.com/dispensaries/minerva-canna-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/northwest-medical-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-day-farms-hensley</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbal-choices-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-miami-gardens</loc>",
    "<loc>https://weedmaps.com/dispensaries/dutch-alchemy</loc>",
    "<loc>https://weedmaps.com/dispensaries/compassionate-alternative-medicine-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/choom-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/dutch-love-centretown</loc>",
    "<loc>https://weedmaps.com/dispensaries/jar-cannabis-co-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/phoenixhanf-gmbh</loc>",
    "<loc>https://weedmaps.com/dispensaries/relax-social-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/med-bag</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-dragon-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-plant</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-p-i-f</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-melbourne</loc>",
    "<loc>https://weedmaps.com/dispensaries/elevate-woodland-hills</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-spunky-skunk</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenlight-therapeutics</loc>",
    "<loc>https://weedmaps.com/dispensaries/420-farmacy</loc>",
    "<loc>https://weedmaps.com/dispensaries/east-coast-cure-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/mood-club-420</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-gainesville</loc>",
    "<loc>https://weedmaps.com/dispensaries/smoke-from-the-bong</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-cannabis-refinery</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-doctor</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-society-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/rosebuds</loc>",
    "<loc>https://weedmaps.com/dispensaries/electric-lettuce-happy-valley</loc>",
    "<loc>https://weedmaps.com/dispensaries/weedmart-355d0b47-c809-4e8e-b795-47914e2d6cc1</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannawave</loc>",
    "<loc>https://weedmaps.com/dispensaries/everest-apothecary-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/ganja-s-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/gravity-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/maikoh-holistics</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-class</loc>",
    "<loc>https://weedmaps.com/dispensaries/trees-waterfront</loc>",
    "<loc>https://weedmaps.com/dispensaries/dabbington-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/doctor-green-glenpool</loc>",
    "<loc>https://weedmaps.com/dispensaries/nuvue-trinidad</loc>",
    "<loc>https://weedmaps.com/dispensaries/remedii</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-cloud</loc>",
    "<loc>https://weedmaps.com/dispensaries/oklahoma-kind-connection</loc>",
    "<loc>https://weedmaps.com/dispensaries/hemp-basement</loc>",
    "<loc>https://weedmaps.com/dispensaries/swissbud</loc>",
    "<loc>https://weedmaps.com/dispensaries/big-b-s-farmacy</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-vault-dispensary-lounge</loc>",
    "<loc>https://weedmaps.com/dispensaries/prime-cannabis-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/hive-medicinal</loc>",
    "<loc>https://weedmaps.com/dispensaries/gold-leaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-source-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/elevate-harbor-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/prairie-records-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/j-london</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-marijuana-dispensary-marion</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-marijuana-dispensary-cincinnati</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-marijuana-dispensary-newark</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-marijuana-dispensary-chillicothe</loc>",
    "<loc>https://weedmaps.com/dispensaries/verdant-creations-columbus</loc>",
    "<loc>https://weedmaps.com/dispensaries/tundra-herb-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/happy-dispensaries</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-bonita-springs</loc>",
    "<loc>https://weedmaps.com/dispensaries/bad-dog</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-farm-stand</loc>",
    "<loc>https://weedmaps.com/dispensaries/la-feuille-d-or</loc>",
    "<loc>https://weedmaps.com/dispensaries/sundial-collective</loc>",
    "<loc>https://weedmaps.com/dispensaries/fluffy-s-apothecary</loc>",
    "<loc>https://weedmaps.com/dispensaries/cookies</loc>",
    "<loc>https://weedmaps.com/dispensaries/kindpeoples-ocean-street</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-tonic</loc>",
    "<loc>https://weedmaps.com/dispensaries/oak-creek-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/livwell-meds-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-pinellas-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/empire-dispensary-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/prairie-records-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/bow-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/pipe-and-leaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/division-street-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/westcan-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/silver-therapeutics-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/stillwater-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/buddies-cannabis-co-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/empire-oakdale</loc>",
    "<loc>https://weedmaps.com/dispensaries/elevate-lompoc</loc>",
    "<loc>https://weedmaps.com/dispensaries/triple-j-s-canna-space-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/nirvanawestphoenix</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-orange-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/boston-bud-factory</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-jacksonville</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-longwood</loc>",
    "<loc>https://weedmaps.com/dispensaries/pure-lapeer</loc>",
    "<loc>https://weedmaps.com/dispensaries/connected-cannabis-bellflower</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-boutique</loc>",
    "<loc>https://weedmaps.com/dispensaries/columbia-care-brooklyn</loc>",
    "<loc>https://weedmaps.com/dispensaries/rino-supply-co-rec</loc>",
    "<loc>https://weedmaps.com/dispensaries/coffeeshop-the-squad</loc>",
    "<loc>https://weedmaps.com/dispensaries/pauls-boutique-nursery-maine</loc>",
    "<loc>https://weedmaps.com/dispensaries/kaya-veronicas-tenerife</loc>",
    "<loc>https://weedmaps.com/dispensaries/urban-wellness-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/happy-isle-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/urban-leaf-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/boyman-acres-cannabis-farm</loc>",
    "<loc>https://weedmaps.com/dispensaries/re-up-cannabis-dispensary-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/zen-leaf-germantown</loc>",
    "<loc>https://weedmaps.com/dispensaries/zen-leaf-canton</loc>",
    "<loc>https://weedmaps.com/dispensaries/kush-house-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/tokelahoma-s-finest</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbology-little-rock</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbology-groton</loc>",
    "<loc>https://weedmaps.com/dispensaries/wild-seed-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbology-devils-lake</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-dickinson</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbology-jamestown</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbology-minot</loc>",
    "<loc>https://weedmaps.com/dispensaries/laughing-grass-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/lefty-s-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/dutch-love-main-st</loc>",
    "<loc>https://weedmaps.com/dispensaries/vertica-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/columbia-care-san-diego</loc>",
    "<loc>https://weedmaps.com/dispensaries/greensprings</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-rx-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-joint-7</loc>",
    "<loc>https://weedmaps.com/dispensaries/fighting-flower</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbology-mchenry</loc>",
    "<loc>https://weedmaps.com/dispensaries/tweed-sask-ave-portage-prairie</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-depot-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/coachella-valley-green-dragon</loc>",
    "<loc>https://weedmaps.com/dispensaries/royal-dream-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-smoke-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/club-cannabico-cotiledon</loc>",
    "<loc>https://weedmaps.com/dispensaries/purple-moon-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-medicine-woman</loc>",
    "<loc>https://weedmaps.com/dispensaries/uptown-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/exclusive-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/highest-releaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/beach-boys-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/kush-pharm</loc>",
    "<loc>https://weedmaps.com/dispensaries/hobo-recreational-cannabis-store-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/affordable-medical-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-temple</loc>",
    "<loc>https://weedmaps.com/dispensaries/gage-usa</loc>",
    "<loc>https://weedmaps.com/dispensaries/gage-usa-ferndale</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-cannection-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/wisely-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/doctors-garden-med</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-valley-fuerteventura</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-bonita-springs</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-hallandale-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-dispensaries-hagerstown</loc>",
    "<loc>https://weedmaps.com/dispensaries/coffeeshop-de-pijp</loc>",
    "<loc>https://weedmaps.com/dispensaries/santa-haze</loc>",
    "<loc>https://weedmaps.com/dispensaries/bacco-farms-provisioning-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/affordable-medical-cannabis-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/flora-verde</loc>",
    "<loc>https://weedmaps.com/dispensaries/blue-ridge-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/organic-meds-llc-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/floyd-s-fine-cannabis-on-whitaker-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/mmd-marina-del-rey</loc>",
    "<loc>https://weedmaps.com/dispensaries/20after4</loc>",
    "<loc>https://weedmaps.com/dispensaries/420-grand-central</loc>",
    "<loc>https://weedmaps.com/dispensaries/a-greener-today-walla-walla</loc>",
    "<loc>https://weedmaps.com/dispensaries/shift-new-mexico-jefferson</loc>",
    "<loc>https://weedmaps.com/dispensaries/kootenay-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/natural-relief-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannazone-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/main-street-meds</loc>",
    "<loc>https://weedmaps.com/dispensaries/area-52-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/bloom-cannabis-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-sanford</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbal-e-scents-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/chaco-valley-dispensaries</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-bilbi</loc>",
    "<loc>https://weedmaps.com/dispensaries/hometown-stash</loc>",
    "<loc>https://weedmaps.com/dispensaries/seed-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/starbuds-ordway</loc>",
    "<loc>https://weedmaps.com/dispensaries/berkeley-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/bobby-brown-best-buds</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-latrobe</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-mechanicsburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/marry-jane-basel</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-gallery-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/nurses-station</loc>",
    "<loc>https://weedmaps.com/dispensaries/fluent-gainesville</loc>",
    "<loc>https://weedmaps.com/dispensaries/fluent-tallahassee</loc>",
    "<loc>https://weedmaps.com/dispensaries/uncle-ando-s-wurld-of-weed</loc>",
    "<loc>https://weedmaps.com/dispensaries/terps-dispensary-pueblo</loc>",
    "<loc>https://weedmaps.com/dispensaries/chiefin-meds</loc>",
    "<loc>https://weedmaps.com/dispensaries/710-montana-helena</loc>",
    "<loc>https://weedmaps.com/dispensaries/710-montana-florence</loc>",
    "<loc>https://weedmaps.com/dispensaries/mango-cannabis-tulsa</loc>",
    "<loc>https://weedmaps.com/dispensaries/great-barrier-reefer-medical-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/frost-harvest-co-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenlight-helena</loc>",
    "<loc>https://weedmaps.com/dispensaries/koli-cannabis-east</loc>",
    "<loc>https://weedmaps.com/dispensaries/greendoctor-420-se</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannasseurs-club-noho</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannaland-cannabis-boutique-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/dolphin-gang</loc>",
    "<loc>https://weedmaps.com/dispensaries/coffee-shop-the-happy-garden</loc>",
    "<loc>https://weedmaps.com/dispensaries/march-and-ash-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/zen-garden-sacramento</loc>",
    "<loc>https://weedmaps.com/dispensaries/earth-gift-cannabis-dispensary-san-miguel</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-herbandstone-club-183</loc>",
    "<loc>https://weedmaps.com/dispensaries/american-cannabis-company-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-bud-shop-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/lokal-source-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/cbd-automat-zitronic-zollstrasse-54</loc>",
    "<loc>https://weedmaps.com/dispensaries/cbd-automat-zitronic-leimenstrasse-9</loc>",
    "<loc>https://weedmaps.com/dispensaries/cbd-automat-zitronic-bernstrasse-19</loc>",
    "<loc>https://weedmaps.com/dispensaries/cbd-automat-zitronic-seestrasse-407</loc>",
    "<loc>https://weedmaps.com/dispensaries/cbd-automat-zitronic-hauptstrasse-23</loc>",
    "<loc>https://weedmaps.com/dispensaries/amigos-de-maria-nervion</loc>",
    "<loc>https://weedmaps.com/dispensaries/stem-on-may</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-oviedo</loc>",
    "<loc>https://weedmaps.com/dispensaries/hempathy</loc>",
    "<loc>https://weedmaps.com/dispensaries/dosist</loc>",
    "<loc>https://weedmaps.com/dispensaries/southern-maine-apothecary</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/eco-cannabis-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-orlando-south</loc>",
    "<loc>https://weedmaps.com/dispensaries/peace-pipe-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/trail-bud-a-bong-shop</loc>",
    "<loc>https://weedmaps.com/dispensaries/buddy-s-place</loc>",
    "<loc>https://weedmaps.com/dispensaries/broadway-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/purlife-dispensary-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/micro-gold-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/vertica-okc</loc>",
    "<loc>https://weedmaps.com/dispensaries/dragon</loc>",
    "<loc>https://weedmaps.com/dispensaries/mountain-fork-mary-jane</loc>",
    "<loc>https://weedmaps.com/dispensaries/compassionate-distributors-carlsbad</loc>",
    "<loc>https://weedmaps.com/dispensaries/harvest-foundation</loc>",
    "<loc>https://weedmaps.com/dispensaries/classen-apothecare</loc>",
    "<loc>https://weedmaps.com/dispensaries/minerva-canna-group-santa-fe</loc>",
    "<loc>https://weedmaps.com/dispensaries/karmaceuticals-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/ultra-health-clayton</loc>",
    "<loc>https://weedmaps.com/dispensaries/pecos-valley-production-ruidoso</loc>",
    "<loc>https://weedmaps.com/dispensaries/stiiizy-davis</loc>",
    "<loc>https://weedmaps.com/dispensaries/dr-greenthumb-humboldt</loc>",
    "<loc>https://weedmaps.com/dispensaries/mango-cannabis-edmond</loc>",
    "<loc>https://weedmaps.com/dispensaries/woke-wellness-nw-okc</loc>",
    "<loc>https://weedmaps.com/dispensaries/choom-lloydminster</loc>",
    "<loc>https://weedmaps.com/dispensaries/utokem</loc>",
    "<loc>https://weedmaps.com/dispensaries/flora-terra</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-port-charlotte</loc>",
    "<loc>https://weedmaps.com/dispensaries/plantlife-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/plantlife-cannabis-grande-prairie</loc>",
    "<loc>https://weedmaps.com/dispensaries/plantlife-cannabis-okotoks</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-jensen-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-sarasota</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-clearwater-19north</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-pineapple-warfield</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannacat</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-bud-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-stash-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-goods-rockville</loc>",
    "<loc>https://weedmaps.com/dispensaries/el-laboratorio-medicinal</loc>",
    "<loc>https://weedmaps.com/dispensaries/euflora-long-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/senses-med-center-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/choom-medicine-hat</loc>",
    "<loc>https://weedmaps.com/dispensaries/bougis-hemp-store</loc>",
    "<loc>https://weedmaps.com/dispensaries/garden-state-dispensary-union</loc>",
    "<loc>https://weedmaps.com/dispensaries/buena-vida-social-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/uwd-norman</loc>",
    "<loc>https://weedmaps.com/dispensaries/sanctuary-atc-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/kush-gardens-6</loc>",
    "<loc>https://weedmaps.com/dispensaries/proper-wellness-center-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/ocean-s-view-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-fernandina-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-gas-station-11</loc>",
    "<loc>https://weedmaps.com/dispensaries/natures-medicines-selinsgrove</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-wellness-bloomsburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-pavilions</loc>",
    "<loc>https://weedmaps.com/dispensaries/strain-lane</loc>",
    "<loc>https://weedmaps.com/dispensaries/socana</loc>",
    "<loc>https://weedmaps.com/dispensaries/tahoe-wellness-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/made-in-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/choom-cold-lake-50th</loc>",
    "<loc>https://weedmaps.com/dispensaries/choom-westlock</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-prime-leaf-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/sweed-quality-labs</loc>",
    "<loc>https://weedmaps.com/dispensaries/evergreen-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/san-peter-green</loc>",
    "<loc>https://weedmaps.com/dispensaries/sweed</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-clearwater</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-tropicanna-life</loc>",
    "<loc>https://weedmaps.com/dispensaries/ethos-watertown</loc>",
    "<loc>https://weedmaps.com/dispensaries/dana-s-green-depot-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-dispensary-williamsburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-dispensary-huntington</loc>",
    "<loc>https://weedmaps.com/dispensaries/newleaf-cannabis-airdrie</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-doctor-420</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-doctor-420-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-society-se-okc</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-cabana-bonnyville</loc>",
    "<loc>https://weedmaps.com/dispensaries/four20-premium-market-beltline</loc>",
    "<loc>https://weedmaps.com/dispensaries/nova-farms-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/emerald-cut-cannabis-care-center-llc-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-budologist</loc>",
    "<loc>https://weedmaps.com/dispensaries/co-op-cannabis-brentwood</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-wanted-leaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/pharmhouse-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/btg</loc>",
    "<loc>https://weedmaps.com/dispensaries/casper</loc>",
    "<loc>https://weedmaps.com/dispensaries/bud-brother-coughy-shop-bartlesville</loc>",
    "<loc>https://weedmaps.com/dispensaries/valhalla-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/fivepoint-cannabis-bridgeland</loc>",
    "<loc>https://weedmaps.com/dispensaries/merry-janes-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/lloberas</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-nation-naturals</loc>",
    "<loc>https://weedmaps.com/dispensaries/haven-long-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/native-harvest-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/stiiizy</loc>",
    "<loc>https://weedmaps.com/dispensaries/newleaf-cannabis-rundlehorn</loc>",
    "<loc>https://weedmaps.com/dispensaries/newleaf-cannabis-westgate</loc>",
    "<loc>https://weedmaps.com/dispensaries/newleaf-cannabis-north-hill</loc>",
    "<loc>https://weedmaps.com/dispensaries/namah-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/project-cannabis-soma</loc>",
    "<loc>https://weedmaps.com/dispensaries/vacationland-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/cloud-420-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/co-op-cannabis-dalhousie</loc>",
    "<loc>https://weedmaps.com/dispensaries/aloha-green-apothecary</loc>",
    "<loc>https://weedmaps.com/dispensaries/amber-horizons</loc>",
    "<loc>https://weedmaps.com/dispensaries/alternative-cures-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/co-op-cannabis-forest-lawn</loc>",
    "<loc>https://weedmaps.com/dispensaries/co-op-cannabis-oakridge</loc>",
    "<loc>https://weedmaps.com/dispensaries/co-op-cannabis-richmond-road</loc>",
    "<loc>https://weedmaps.com/dispensaries/prairie-records-forest-lawn</loc>",
    "<loc>https://weedmaps.com/dispensaries/kushbar-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/stiiizy-kalamazoo-michigan-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/southwest-wellness-san-pedro</loc>",
    "<loc>https://weedmaps.com/dispensaries/urbancanna-killarney</loc>",
    "<loc>https://weedmaps.com/dispensaries/urbancanna-erin-woods</loc>",
    "<loc>https://weedmaps.com/dispensaries/vincanna-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/reseda-discount-caregivers-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/likewise-craft-okc-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/natural-aid-cannabis-dispensary-losangeles</loc>",
    "<loc>https://weedmaps.com/dispensaries/stick-and-stone-cannabis-co-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/divine-herbs-cannabis-clinic</loc>",
    "<loc>https://weedmaps.com/dispensaries/eden-rose-dispensary-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/hey-buddys</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-plus-norman</loc>",
    "<loc>https://weedmaps.com/dispensaries/asociacion-alicantina</loc>",
    "<loc>https://weedmaps.com/dispensaries/cloud-13-rx-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-corp</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-land</loc>",
    "<loc>https://weedmaps.com/dispensaries/cana-cabana-111th-ave</loc>",
    "<loc>https://weedmaps.com/dispensaries/cana-cabana-parsons-rd</loc>",
    "<loc>https://weedmaps.com/dispensaries/sugar-loaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-mountain-cannabis-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/grass-monkey-cannabis-company-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/club-23-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-port-charlotte</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-key-west</loc>",
    "<loc>https://weedmaps.com/dispensaries/re-up-cannabis-dispensary-2-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/leaf-seak</loc>",
    "<loc>https://weedmaps.com/dispensaries/small-batch-maine</loc>",
    "<loc>https://weedmaps.com/dispensaries/rocky-mountain-cannabis-10</loc>",
    "<loc>https://weedmaps.com/dispensaries/tradecraft-farms</loc>",
    "<loc>https://weedmaps.com/dispensaries/house-of-dank-fort-st</loc>",
    "<loc>https://weedmaps.com/dispensaries/king-street-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabist-cape-coral</loc>",
    "<loc>https://weedmaps.com/dispensaries/plantlife-canada-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/plantlife-canada-9a79ea95-39fe-4f03-a0dd-ecde5910a6bd</loc>",
    "<loc>https://weedmaps.com/dispensaries/along-came-mary-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/logistics-demo-pickup-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenhub-cannabis-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-bud-king</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenstone-rosamond-mojave-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/affinity-cannabis-store-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannajacks-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/el-lobo-verde-4-20</loc>",
    "<loc>https://weedmaps.com/dispensaries/https-weedmaps-com-dispensaries-natures-kiss-oklahoma-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/doobies-mustang</loc>",
    "<loc>https://weedmaps.com/dispensaries/solace-meds-colfax</loc>",
    "<loc>https://weedmaps.com/dispensaries/first-medical-cannabis-cabo-rojo</loc>",
    "<loc>https://weedmaps.com/dispensaries/kinship-provisioning</loc>",
    "<loc>https://weedmaps.com/dispensaries/light-n-up-provisioning</loc>",
    "<loc>https://weedmaps.com/dispensaries/grampa-s-finest-cannabis-accessories</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-gainesville</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-farmacy-sb</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-bakeree-north-seattle</loc>",
    "<loc>https://weedmaps.com/dispensaries/dutch-love-kitsilano</loc>",
    "<loc>https://weedmaps.com/dispensaries/dutch-love-robson</loc>",
    "<loc>https://weedmaps.com/dispensaries/liberty-health-sciences-cape-coral</loc>",
    "<loc>https://weedmaps.com/dispensaries/silver-stem-fine-cannabis-northfield</loc>",
    "<loc>https://weedmaps.com/dispensaries/modernleaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/peaceleaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/elevated-apothic-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/master-of-buds</loc>",
    "<loc>https://weedmaps.com/dispensaries/grassroots-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/yys-lloydminster</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-peak-quail-springs-okc-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-voltage-cannabis-co-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/montana-kush</loc>",
    "<loc>https://weedmaps.com/dispensaries/rock-steady-remedies-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/waves-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/full-bloom-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-jacksonville-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-largo</loc>",
    "<loc>https://weedmaps.com/dispensaries/one-eyes-weedery</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-johnstown</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-scranton</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-harrisburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/natures-leaf-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/blue-fire</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbarium-la</loc>",
    "<loc>https://weedmaps.com/dispensaries/columbia-care-wilmington</loc>",
    "<loc>https://weedmaps.com/dispensaries/columbia-care-rehoboth-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabist-bonita-springs</loc>",
    "<loc>https://weedmaps.com/dispensaries/columbia-care-chevy-chase</loc>",
    "<loc>https://weedmaps.com/dispensaries/higher-by-bad-gramm3r</loc>",
    "<loc>https://weedmaps.com/dispensaries/anchorage-bowl-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/aroma</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-room-21</loc>",
    "<loc>https://weedmaps.com/dispensaries/secret-garden-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/wild-wild-weed-las-animas</loc>",
    "<loc>https://weedmaps.com/dispensaries/kush-korner-35c03952-24fb-4ac9-a879-c52b0f87351d</loc>",
    "<loc>https://weedmaps.com/dispensaries/xclusive-cannabis-federal-heights</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-chemistry-15th-st</loc>",
    "<loc>https://weedmaps.com/dispensaries/restore-integrative-wellness-center-elkins-park-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/token-farms-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/proper-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/lucid-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/zzfluxdtla</loc>",
    "<loc>https://weedmaps.com/dispensaries/minerva-canna-of-broken-arrow</loc>",
    "<loc>https://weedmaps.com/dispensaries/bcc-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/selecta</loc>",
    "<loc>https://weedmaps.com/dispensaries/harvest-cannabis-dispensary-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-fort-myers-colonial</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-jacksonville-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/karlinna-elkartea</loc>",
    "<loc>https://weedmaps.com/dispensaries/420-premium-market-8</loc>",
    "<loc>https://weedmaps.com/dispensaries/beyond-the-leaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/rolling-stoned-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/buddha-barn-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/jimmy-s-cannabis-shop</loc>",
    "<loc>https://weedmaps.com/dispensaries/el-jardi-de-graci</loc>",
    "<loc>https://weedmaps.com/dispensaries/asociacion-club-mentalidad-cannabica</loc>",
    "<loc>https://weedmaps.com/dispensaries/city-cannabis-co-fraser-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-apothecarium-allentown</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-mountain-cannabis-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/jimmy-s-cannabis-shop-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/northern-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/orange-bridge-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/native-pure</loc>",
    "<loc>https://weedmaps.com/dispensaries/pacificanna-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/pacificanna-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/summit-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/summit-cannabis-co-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-greenhorn</loc>",
    "<loc>https://weedmaps.com/dispensaries/maple-leaf-greenery</loc>",
    "<loc>https://weedmaps.com/dispensaries/hikei</loc>",
    "<loc>https://weedmaps.com/dispensaries/dank-of-oklahoma-cannabis-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/kushbar-cannabis-co-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/lucid-cannabis-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/lucid-cannabis-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-dispo-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-garden-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/fountain-organics</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-artist-tree</loc>",
    "<loc>https://weedmaps.com/dispensaries/vernon-cannabis-store-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/hometree-hesper-rd</loc>",
    "<loc>https://weedmaps.com/dispensaries/coachella-smoke-co-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/likewise-stillwater-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/elevate-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannatopia-langley</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbal-choices-north-bend</loc>",
    "<loc>https://weedmaps.com/dispensaries/cbd-plus-usa-mcalester</loc>",
    "<loc>https://weedmaps.com/dispensaries/roland-natural-health</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-street-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/thur-s-smoke-shop</loc>",
    "<loc>https://weedmaps.com/dispensaries/euphora</loc>",
    "<loc>https://weedmaps.com/dispensaries/med-leaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/capital-dank-midwest-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/paradox-asociacion</loc>",
    "<loc>https://weedmaps.com/dispensaries/prestige-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/delta-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/bighorn-buds</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-panama-city-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/rosebud-cannabis-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/mesa-organics-las-animas</loc>",
    "<loc>https://weedmaps.com/dispensaries/nova-farms-framingham</loc>",
    "<loc>https://weedmaps.com/dispensaries/mesa-organics-rocky-ford</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannavida-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-quincy</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-king-of-prussia</loc>",
    "<loc>https://weedmaps.com/dispensaries/weedology-ontario</loc>",
    "<loc>https://weedmaps.com/dispensaries/one-plant-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/valley-pure-farmersville</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-deerfield-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-house-sevilla</loc>",
    "<loc>https://weedmaps.com/dispensaries/sicamous-trading-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/nepal-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/march-and-ash-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-reading-lancaster</loc>",
    "<loc>https://weedmaps.com/dispensaries/sweet-flower-arts-district</loc>",
    "<loc>https://weedmaps.com/dispensaries/minerva-canna-group-minerva-west</loc>",
    "<loc>https://weedmaps.com/dispensaries/highley-green-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/plantlife-canada-7783</loc>",
    "<loc>https://weedmaps.com/dispensaries/plantlife-canada-7784</loc>",
    "<loc>https://weedmaps.com/dispensaries/higher-vision</loc>",
    "<loc>https://weedmaps.com/dispensaries/kush-gardens-enid</loc>",
    "<loc>https://weedmaps.com/dispensaries/dr-who</loc>",
    "<loc>https://weedmaps.com/dispensaries/mbc</loc>",
    "<loc>https://weedmaps.com/dispensaries/mile-high-alaska</loc>",
    "<loc>https://weedmaps.com/dispensaries/euphoria-wellness-hamilton</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-altitude-gardens-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-refinery</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-alien-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/cookies-oakland</loc>",
    "<loc>https://weedmaps.com/dispensaries/down-under-industry</loc>",
    "<loc>https://weedmaps.com/dispensaries/double-dogs-cannabis-higgins-ave</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-west-palm-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-healing-community-medco</loc>",
    "<loc>https://weedmaps.com/dispensaries/20-past-4-provisioning-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-day-farm-texarkana</loc>",
    "<loc>https://weedmaps.com/dispensaries/tulsa-house-of-fire</loc>",
    "<loc>https://weedmaps.com/dispensaries/refined-flowers</loc>",
    "<loc>https://weedmaps.com/dispensaries/top-shelf-botanicals-ennis</loc>",
    "<loc>https://weedmaps.com/dispensaries/top-shelf-botanicals-helena</loc>",
    "<loc>https://weedmaps.com/dispensaries/top-shelf-botanicals-kalispell</loc>",
    "<loc>https://weedmaps.com/dispensaries/top-shelf-botanicals-livingston</loc>",
    "<loc>https://weedmaps.com/dispensaries/top-shelf-botanicals-missoula</loc>",
    "<loc>https://weedmaps.com/dispensaries/lone-peak-caregivers-big-sky</loc>",
    "<loc>https://weedmaps.com/dispensaries/lone-peak-caregivers-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/lone-peak-caregivers-ennis</loc>",
    "<loc>https://weedmaps.com/dispensaries/berners-merced</loc>",
    "<loc>https://weedmaps.com/dispensaries/creek-bud-supply</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-cannabis-cure</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-lakeland</loc>",
    "<loc>https://weedmaps.com/dispensaries/hobo-cannabis-company-coming-soon-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/mountain-view-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/nectar-406</loc>",
    "<loc>https://weedmaps.com/dispensaries/rural-leaf-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-bee-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/710-montana-butte</loc>",
    "<loc>https://weedmaps.com/dispensaries/authentic-209</loc>",
    "<loc>https://weedmaps.com/dispensaries/montana-preferred-provider-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-wesley-chapel</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-lakeland</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-destin</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-cape-coral</loc>",
    "<loc>https://weedmaps.com/dispensaries/uluwatu-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/alternativa-natural-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/la-to-ok</loc>",
    "<loc>https://weedmaps.com/dispensaries/lionheart-caregiving-great-falls-herbal-bliss</loc>",
    "<loc>https://weedmaps.com/dispensaries/kind-love-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/ibex-of-bozeman</loc>",
    "<loc>https://weedmaps.com/dispensaries/infinity-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/apogee-gardens</loc>",
    "<loc>https://weedmaps.com/dispensaries/livity</loc>",
    "<loc>https://weedmaps.com/dispensaries/exclusive-ann-arbor-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/gentlemen-smokers-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/stability-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/euphora-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/klover</loc>",
    "<loc>https://weedmaps.com/dispensaries/bluewater-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-pit-stop</loc>",
    "<loc>https://weedmaps.com/dispensaries/vetted-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-dispensary-westminster</loc>",
    "<loc>https://weedmaps.com/dispensaries/growhealthy-ocala</loc>",
    "<loc>https://weedmaps.com/dispensaries/spark1-missoula</loc>",
    "<loc>https://weedmaps.com/dispensaries/hilltop-farms</loc>",
    "<loc>https://weedmaps.com/dispensaries/greengirlsberria</loc>",
    "<loc>https://weedmaps.com/dispensaries/verilife-romeoville</loc>",
    "<loc>https://weedmaps.com/dispensaries/weeds-llc-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/marin-alliance-for-medical-marijuana-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/nature-s-kiss-tulsa</loc>",
    "<loc>https://weedmaps.com/dispensaries/natural-healing-remedies-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-time</loc>",
    "<loc>https://weedmaps.com/dispensaries/montana-kush-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/seed-cannabis-company-sheridan</loc>",
    "<loc>https://weedmaps.com/dispensaries/blaze-on-haight</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-cure-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/pinnacle-emporium</loc>",
    "<loc>https://weedmaps.com/dispensaries/stiiizyalameda</loc>",
    "<loc>https://weedmaps.com/dispensaries/growhealthy-stuart</loc>",
    "<loc>https://weedmaps.com/dispensaries/okie-wonderland</loc>",
    "<loc>https://weedmaps.com/dispensaries/indica-private-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/apothecarium-maplewood</loc>",
    "<loc>https://weedmaps.com/dispensaries/k-for-kush</loc>",
    "<loc>https://weedmaps.com/dispensaries/coffeeshop-club-media</loc>",
    "<loc>https://weedmaps.com/dispensaries/deep-roots-harvest-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/asociacion-green-village</loc>",
    "<loc>https://weedmaps.com/dispensaries/port-420</loc>",
    "<loc>https://weedmaps.com/dispensaries/black-tie-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/t-nugs-world-of-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-skunk-house</loc>",
    "<loc>https://weedmaps.com/dispensaries/mainstage</loc>",
    "<loc>https://weedmaps.com/dispensaries/puff-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/metro-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-provincetown</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-orlando-colonial</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-orange-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/mr-tau</loc>",
    "<loc>https://weedmaps.com/dispensaries/rockstar-cannabis-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-orlando-garland</loc>",
    "<loc>https://weedmaps.com/dispensaries/cookiesdetroit</loc>",
    "<loc>https://weedmaps.com/dispensaries/captain-jacks</loc>",
    "<loc>https://weedmaps.com/dispensaries/nobo</loc>",
    "<loc>https://weedmaps.com/dispensaries/honey-creek-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/bud-bar</loc>",
    "<loc>https://weedmaps.com/dispensaries/bloomerang-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/shango-moreno-valley</loc>",
    "<loc>https://weedmaps.com/dispensaries/ethos</loc>",
    "<loc>https://weedmaps.com/dispensaries/deep-south-greenery</loc>",
    "<loc>https://weedmaps.com/dispensaries/essence-cannabis-dispensary-rainbow-warm-springs</loc>",
    "<loc>https://weedmaps.com/dispensaries/zen-leaf-cincinnati</loc>",
    "<loc>https://weedmaps.com/dispensaries/haven-paramount</loc>",
    "<loc>https://weedmaps.com/dispensaries/vibe-by-california-redding</loc>",
    "<loc>https://weedmaps.com/dispensaries/la-canna-bcn</loc>",
    "<loc>https://weedmaps.com/dispensaries/apothecare-ann-arbor</loc>",
    "<loc>https://weedmaps.com/dispensaries/tranquilo-socialclub</loc>",
    "<loc>https://weedmaps.com/dispensaries/mj-highway</loc>",
    "<loc>https://weedmaps.com/dispensaries/ako-farms-llc-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/big-chief-collective-2-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/coffeeshop-siberie</loc>",
    "<loc>https://weedmaps.com/dispensaries/planta-de-vida</loc>",
    "<loc>https://weedmaps.com/dispensaries/cookies-modesto</loc>",
    "<loc>https://weedmaps.com/dispensaries/kana-leaf-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/vive-y-se-feliz-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/coastal-wellness-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-nirvana-center-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/hobo-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/canyon-creek-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/indico</loc>",
    "<loc>https://weedmaps.com/dispensaries/native-nugs</loc>",
    "<loc>https://weedmaps.com/dispensaries/kind-farms-reserve-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-dank-tavern</loc>",
    "<loc>https://weedmaps.com/dispensaries/san-diego-s-herb</loc>",
    "<loc>https://weedmaps.com/dispensaries/breeze</loc>",
    "<loc>https://weedmaps.com/dispensaries/nevada-made-marijuana-warm-springs</loc>",
    "<loc>https://weedmaps.com/dispensaries/kush-alley-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/fat-nugs</loc>",
    "<loc>https://weedmaps.com/dispensaries/el-medicinal</loc>",
    "<loc>https://weedmaps.com/dispensaries/mr-bombastik-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/firestorm</loc>",
    "<loc>https://weedmaps.com/dispensaries/buddy-s-place-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/bewell-organic-medicine</loc>",
    "<loc>https://weedmaps.com/dispensaries/haven-downtown-long-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/doobiez</loc>",
    "<loc>https://weedmaps.com/dispensaries/bud-bar-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-joint-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/himalaya-ganjika-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/vallejo-relief-center-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-grass-station-in-oakridge</loc>",
    "<loc>https://weedmaps.com/dispensaries/majestic-jane-8</loc>",
    "<loc>https://weedmaps.com/dispensaries/eastwood-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-glass-liberty-lake</loc>",
    "<loc>https://weedmaps.com/dispensaries/one-plant-atwater</loc>",
    "<loc>https://weedmaps.com/dispensaries/ultra-health-south-valley</loc>",
    "<loc>https://weedmaps.com/dispensaries/ultra-health-s-e-heights</loc>",
    "<loc>https://weedmaps.com/dispensaries/ultra-health-rio-rancho</loc>",
    "<loc>https://weedmaps.com/dispensaries/plantlife-canada-7785</loc>",
    "<loc>https://weedmaps.com/dispensaries/plantlife-canada-7786</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-cabana-airdrie</loc>",
    "<loc>https://weedmaps.com/dispensaries/cultivate-framingham</loc>",
    "<loc>https://weedmaps.com/dispensaries/revolutionary-clinics-central-square</loc>",
    "<loc>https://weedmaps.com/dispensaries/coffeeshop-de-supermarkt</loc>",
    "<loc>https://weedmaps.com/dispensaries/coffeeshop-loft</loc>",
    "<loc>https://weedmaps.com/dispensaries/coffeeshop-espresso</loc>",
    "<loc>https://weedmaps.com/dispensaries/ritual-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/king-street-cannabis-co-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/surf-city-original</loc>",
    "<loc>https://weedmaps.com/dispensaries/mrs-mary-jane</loc>",
    "<loc>https://weedmaps.com/dispensaries/tip-top</loc>",
    "<loc>https://weedmaps.com/dispensaries/barcelona-coffeeshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/kush-gardens-okmulgee-coming-soon</loc>",
    "<loc>https://weedmaps.com/dispensaries/west-coast-cannabis-club-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/lady-janes-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/grant-pharms</loc>",
    "<loc>https://weedmaps.com/dispensaries/upnorth-meds</loc>",
    "<loc>https://weedmaps.com/dispensaries/stiiizy-palm-desert</loc>",
    "<loc>https://weedmaps.com/dispensaries/shiny-bud</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-paradise-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/atlas-fumador-barcelona</loc>",
    "<loc>https://weedmaps.com/dispensaries/spiicetherapy</loc>",
    "<loc>https://weedmaps.com/dispensaries/flight-420-west-siloam-springs</loc>",
    "<loc>https://weedmaps.com/dispensaries/information-entropy</loc>",
    "<loc>https://weedmaps.com/dispensaries/delta-boyz-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/natural-state-medical-group-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-peak-fox-lake</loc>",
    "<loc>https://weedmaps.com/dispensaries/freddie-s-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-stuart</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-pensacola-nine-mile</loc>",
    "<loc>https://weedmaps.com/dispensaries/a-better-solution-unlimited</loc>",
    "<loc>https://weedmaps.com/dispensaries/club-la-oficina</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-springs-oklahoma-bethany</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunrise-cannabis-retail-ltd</loc>",
    "<loc>https://weedmaps.com/dispensaries/american-cannabis-dispensary-newcastle</loc>",
    "<loc>https://weedmaps.com/dispensaries/newe-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/bhang-lass</loc>",
    "<loc>https://weedmaps.com/dispensaries/marley-club-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/superfarm-texarkana</loc>",
    "<loc>https://weedmaps.com/dispensaries/asociacion-terapeutica-el-gnomo</loc>",
    "<loc>https://weedmaps.com/dispensaries/fire-ranch-farmacy</loc>",
    "<loc>https://weedmaps.com/dispensaries/daily-stash-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/og-barcelona</loc>",
    "<loc>https://weedmaps.com/dispensaries/barbary-coast-sunset-cannabis-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/hello-cannabis-sault-ste-marie</loc>",
    "<loc>https://weedmaps.com/dispensaries/km420</loc>",
    "<loc>https://weedmaps.com/dispensaries/la-mezquita</loc>",
    "<loc>https://weedmaps.com/dispensaries/cabbage-brothers</loc>",
    "<loc>https://weedmaps.com/dispensaries/gp-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/peaky-blinders</loc>",
    "<loc>https://weedmaps.com/dispensaries/homegrown-cannabis-company-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/reefer-madness-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/new-world-organics</loc>",
    "<loc>https://weedmaps.com/dispensaries/flora</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-life-arona</loc>",
    "<loc>https://weedmaps.com/dispensaries/elevate-maine-yarmouth</loc>",
    "<loc>https://weedmaps.com/dispensaries/weedlemonsevilla</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-cookie</loc>",
    "<loc>https://weedmaps.com/dispensaries/club-67</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbology-cannabis-co-rec</loc>",
    "<loc>https://weedmaps.com/dispensaries/jungleboysoc</loc>",
    "<loc>https://weedmaps.com/dispensaries/natures-kiss-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/montana-kush-bozeman</loc>",
    "<loc>https://weedmaps.com/dispensaries/american-releaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-dispensary-river-north</loc>",
    "<loc>https://weedmaps.com/dispensaries/nectar-merced</loc>",
    "<loc>https://weedmaps.com/dispensaries/hi-roots-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/capital-dank-shawnee</loc>",
    "<loc>https://weedmaps.com/dispensaries/royal-apothecary-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/humanity-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-sadhus</loc>",
    "<loc>https://weedmaps.com/dispensaries/tweed-visitor-centre-smiths-falls</loc>",
    "<loc>https://weedmaps.com/dispensaries/proper-wellness-rio-dell</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-leaf-cannabis-norman</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-marketplace</loc>",
    "<loc>https://weedmaps.com/dispensaries/private-stash</loc>",
    "<loc>https://weedmaps.com/dispensaries/joyology</loc>",
    "<loc>https://weedmaps.com/dispensaries/farmacy-berkeley-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/plenty-of-time</loc>",
    "<loc>https://weedmaps.com/dispensaries/badger-buds-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenst-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannaisseur-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/omkara-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/packs</loc>",
    "<loc>https://weedmaps.com/dispensaries/inspire</loc>",
    "<loc>https://weedmaps.com/dispensaries/bonnefire</loc>",
    "<loc>https://weedmaps.com/dispensaries/tumblweedz-cannabis-calgary</loc>",
    "<loc>https://weedmaps.com/dispensaries/seed-and-stone</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-river-provisioning-recreational-adult-use</loc>",
    "<loc>https://weedmaps.com/dispensaries/nature-s-miracle-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/royal-healing-emporium-lompoc</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-dispensaries-cranberry</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-raven</loc>",
    "<loc>https://weedmaps.com/dispensaries/kush-gardens-oklahoma-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-apothecarium-lancaster-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-apothecarium-thorndale-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/kush-gardens-woodward</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-apothecarium-phillipsburg-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-society-fairgrounds-okc</loc>",
    "<loc>https://weedmaps.com/dispensaries/laughing-salmon-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-chamber</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-spot-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-white-hemp</loc>",
    "<loc>https://weedmaps.com/dispensaries/quality-roots-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/medical-woods-provisioning-center-receational</loc>",
    "<loc>https://weedmaps.com/dispensaries/okie-medicine-flower</loc>",
    "<loc>https://weedmaps.com/dispensaries/nature-s-medicines-grand</loc>",
    "<loc>https://weedmaps.com/dispensaries/zen-leaf-pasadena</loc>",
    "<loc>https://weedmaps.com/dispensaries/pure-roots</loc>",
    "<loc>https://weedmaps.com/dispensaries/kiskanu-cannabis-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/64-dispensary-65</loc>",
    "<loc>https://weedmaps.com/dispensaries/dr-greenthumb-cathedral-city-palmsprings-palmdesert</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-north-cannabis-store-airport-uplands-coming-soon</loc>",
    "<loc>https://weedmaps.com/dispensaries/ashe-society</loc>",
    "<loc>https://weedmaps.com/dispensaries/zen-leaf-elizabeth</loc>",
    "<loc>https://weedmaps.com/dispensaries/american-cannabis-company-10</loc>",
    "<loc>https://weedmaps.com/dispensaries/american-cannabis-company-11</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-culture-dispensary-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/amazing-budz-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/west-memphis-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-pharm-tc</loc>",
    "<loc>https://weedmaps.com/dispensaries/comprehensive-care-group-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/one-plant-stouffville</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-sanctuary-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/southwest-wellness-center-99</loc>",
    "<loc>https://weedmaps.com/dispensaries/liv-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-day-farm-monticello</loc>",
    "<loc>https://weedmaps.com/dispensaries/puff-traverse-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/420-brothers</loc>",
    "<loc>https://weedmaps.com/dispensaries/brain-lickers</loc>",
    "<loc>https://weedmaps.com/dispensaries/choom-cannabis-co-lethbridge</loc>",
    "<loc>https://weedmaps.com/dispensaries/ak-slow-burn-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/gage-lansing</loc>",
    "<loc>https://weedmaps.com/dispensaries/gleaf-rockville</loc>",
    "<loc>https://weedmaps.com/dispensaries/gage-traverse</loc>",
    "<loc>https://weedmaps.com/dispensaries/natures-gift-medical</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-boulevard-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/northstar</loc>",
    "<loc>https://weedmaps.com/dispensaries/gas-house-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/fire-flower-cannabis-co-rice-howard-way-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/okie-wonderland-south-tulsa-coming-soon</loc>",
    "<loc>https://weedmaps.com/dispensaries/fire-flower-cannabis-co-york-street</loc>",
    "<loc>https://weedmaps.com/dispensaries/fire-flower-cannabis-co-estevan-plaza</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-dispensaries-niles</loc>",
    "<loc>https://weedmaps.com/dispensaries/pureleaf-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-buddha-cannabis-co-adult-use</loc>",
    "<loc>https://weedmaps.com/dispensaries/lake-country-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-nirvana-center-traverse-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/straiin-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/remedy-room</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medicals</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-lighthouse</loc>",
    "<loc>https://weedmaps.com/dispensaries/velvet-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/choom-cannabis-co-vancouver-ov</loc>",
    "<loc>https://weedmaps.com/dispensaries/douglasdale-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/best-buds-farm</loc>",
    "<loc>https://weedmaps.com/dispensaries/sensewave</loc>",
    "<loc>https://weedmaps.com/dispensaries/maine-medical-canna-connection</loc>",
    "<loc>https://weedmaps.com/dispensaries/scorpion-grass</loc>",
    "<loc>https://weedmaps.com/dispensaries/spark-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/a-green-alternative-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-underground</loc>",
    "<loc>https://weedmaps.com/dispensaries/wallflower</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-plainz-strains-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/stash-co-recreational-cannabis-merivale</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-cannabis-dispensary-danville</loc>",
    "<loc>https://weedmaps.com/dispensaries/skye-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/pincanna</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-corner-lbc</loc>",
    "<loc>https://weedmaps.com/dispensaries/atrium-valley-village</loc>",
    "<loc>https://weedmaps.com/dispensaries/diamond-smokers-clubs</loc>",
    "<loc>https://weedmaps.com/dispensaries/c1b-true-organics</loc>",
    "<loc>https://weedmaps.com/dispensaries/natura-al-natural</loc>",
    "<loc>https://weedmaps.com/dispensaries/edmond-dispensary-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/select-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/canora-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-shark-s-cave-tenerife</loc>",
    "<loc>https://weedmaps.com/dispensaries/humblebee-products-adult-use</loc>",
    "<loc>https://weedmaps.com/dispensaries/kind-love-admiral</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-klub-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/alchemy</loc>",
    "<loc>https://weedmaps.com/dispensaries/embarc-tahoe</loc>",
    "<loc>https://weedmaps.com/dispensaries/montana-canna-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/cookies-la-mesa</loc>",
    "<loc>https://weedmaps.com/dispensaries/empire-gardens-moreno-valley</loc>",
    "<loc>https://weedmaps.com/dispensaries/essence-cannabis-dispensary-i-south-durango</loc>",
    "<loc>https://weedmaps.com/dispensaries/alaska-cannabis-exchange</loc>",
    "<loc>https://weedmaps.com/dispensaries/xcheerful-charlies-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/calice-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/wayne-releaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/houghton-lake-provisioning-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/10killer-bud</loc>",
    "<loc>https://weedmaps.com/dispensaries/next-elevation</loc>",
    "<loc>https://weedmaps.com/dispensaries/herban-kulture</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-lyfe</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-get-happy</loc>",
    "<loc>https://weedmaps.com/dispensaries/sen-cannabis-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/christiania-vng</loc>",
    "<loc>https://weedmaps.com/dispensaries/cloud-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/elemental-progress</loc>",
    "<loc>https://weedmaps.com/dispensaries/coldsmoke</loc>",
    "<loc>https://weedmaps.com/dispensaries/seven-point-cannabis-toronto</loc>",
    "<loc>https://weedmaps.com/dispensaries/heads-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/dona-juanita</loc>",
    "<loc>https://weedmaps.com/dispensaries/burton-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/craft-cannabis-company-yukon</loc>",
    "<loc>https://weedmaps.com/dispensaries/420-love-at-246-king-st-w</loc>",
    "<loc>https://weedmaps.com/dispensaries/420-love-at-976-main-st-e</loc>",
    "<loc>https://weedmaps.com/dispensaries/native-leaf-dispensary-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-dispensaries-duncansville</loc>",
    "<loc>https://weedmaps.com/dispensaries/catalyst-eastside</loc>",
    "<loc>https://weedmaps.com/dispensaries/napa-cannabis-collective</loc>",
    "<loc>https://weedmaps.com/dispensaries/lotus-gold-choctaw</loc>",
    "<loc>https://weedmaps.com/dispensaries/body-and-spirit-cannabis-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/hallowell-4twenty</loc>",
    "<loc>https://weedmaps.com/dispensaries/budders-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-we-store-chatham</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-sheep</loc>",
    "<loc>https://weedmaps.com/dispensaries/happy-family</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-tree-house-ak</loc>",
    "<loc>https://weedmaps.com/dispensaries/windy-city-cannabis-weed-st</loc>",
    "<loc>https://weedmaps.com/dispensaries/hod-traverse-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/hod-gratiot</loc>",
    "<loc>https://weedmaps.com/dispensaries/peak-mj-sedgwick</loc>",
    "<loc>https://weedmaps.com/dispensaries/american-cannabis-dispensary-midwest-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/pinnacle-emporium-addison</loc>",
    "<loc>https://weedmaps.com/dispensaries/catalyst-cannabis-company-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/rocky-mountain-roots-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-brooksville</loc>",
    "<loc>https://weedmaps.com/dispensaries/oasis-dispensaries-glendale</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannatopia-tulsa</loc>",
    "<loc>https://weedmaps.com/dispensaries/dream-lounge-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/refresh-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/natural-healing-center-lemoore</loc>",
    "<loc>https://weedmaps.com/dispensaries/elite-wellness-jackson</loc>",
    "<loc>https://weedmaps.com/dispensaries/riverside-provisioning-kalkaska</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-ocala</loc>",
    "<loc>https://weedmaps.com/dispensaries/country-cannabis-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-herbery-ne-78th</loc>",
    "<loc>https://weedmaps.com/dispensaries/abide-cannabis-company-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/mystique-of-maine</loc>",
    "<loc>https://weedmaps.com/dispensaries/power-plant-foster</loc>",
    "<loc>https://weedmaps.com/dispensaries/la-cure-marbella</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-rollers-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/jars-lansing</loc>",
    "<loc>https://weedmaps.com/dispensaries/lotus-gold-muskogee</loc>",
    "<loc>https://weedmaps.com/dispensaries/homegrown-oregon-beaverton</loc>",
    "<loc>https://weedmaps.com/dispensaries/electric-lettuce-foster-powell</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-tropic</loc>",
    "<loc>https://weedmaps.com/dispensaries/valley-pure-lemoore</loc>",
    "<loc>https://weedmaps.com/dispensaries/mcc-flight</loc>",
    "<loc>https://weedmaps.com/dispensaries/oz-cannabis-ypsilanti-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/smooth-roots-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-dragon-6</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-dragon-7</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-cannabis-dispensary-south-beloit</loc>",
    "<loc>https://weedmaps.com/dispensaries/nature-s-herbs-wellness-sedgwick</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-dispensaries-kendall</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-kind-caste-nederland</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-point-wellness-laurel</loc>",
    "<loc>https://weedmaps.com/dispensaries/premiere-provisions</loc>",
    "<loc>https://weedmaps.com/dispensaries/cimarron-pharma</loc>",
    "<loc>https://weedmaps.com/dispensaries/canadian-cannabis-connoisseur-club-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/oz-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/dort-hwy-disp</loc>",
    "<loc>https://weedmaps.com/dispensaries/highly-south-portland</loc>",
    "<loc>https://weedmaps.com/dispensaries/top-shelf-herbs-of-alaska-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/lux-smoke-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-sanctuary-north-las-vegas</loc>",
    "<loc>https://weedmaps.com/dispensaries/lemonnade-sacramento</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-joliet-rock-creek</loc>",
    "<loc>https://weedmaps.com/dispensaries/lotus-gold-kingfisher</loc>",
    "<loc>https://weedmaps.com/dispensaries/numo-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/flower-haze</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-oz-store-orleans</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-port-st-lucie-gatlin</loc>",
    "<loc>https://weedmaps.com/dispensaries/sessions-cannabis-6</loc>",
    "<loc>https://weedmaps.com/dispensaries/delta-cannabis-co-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/nug-sundial</loc>",
    "<loc>https://weedmaps.com/dispensaries/berlin</loc>",
    "<loc>https://weedmaps.com/dispensaries/euphora-del-city-coming-soon</loc>",
    "<loc>https://weedmaps.com/dispensaries/araba-fenice</loc>",
    "<loc>https://weedmaps.com/dispensaries/wonderland-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/enlightened-herb-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/jars-owosso</loc>",
    "<loc>https://weedmaps.com/dispensaries/queensborough-cannabis-co-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/white-label-csc</loc>",
    "<loc>https://weedmaps.com/dispensaries/qbud-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/4-20-heka</loc>",
    "<loc>https://weedmaps.com/dispensaries/organtica-truth-or-consequences</loc>",
    "<loc>https://weedmaps.com/dispensaries/whappy-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-light-cannabis-shop-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/verilife-shrewsbury</loc>",
    "<loc>https://weedmaps.com/dispensaries/moderna-cannabis-society</loc>",
    "<loc>https://weedmaps.com/dispensaries/cloud-620</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-den-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/firehouse-366</loc>",
    "<loc>https://weedmaps.com/dispensaries/marsellus-wallace-social-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/86avenue</loc>",
    "<loc>https://weedmaps.com/dispensaries/mary-jane-rigs-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/off-the-charts</loc>",
    "<loc>https://weedmaps.com/dispensaries/megan-s-organic-market-slo</loc>",
    "<loc>https://weedmaps.com/dispensaries/verilife-shamokin-medical</loc>",
    "<loc>https://weedmaps.com/dispensaries/montana-kush-gateway</loc>",
    "<loc>https://weedmaps.com/dispensaries/highlife-social-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-cannabis-dispensary-schaumburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/cloud-cannabis-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/cloud-cannabis-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/beach-boys-cannabis-company-oob</loc>",
    "<loc>https://weedmaps.com/dispensaries/oklahoma-green-stop-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/trinity-tree-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/sip-koi</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-englewood</loc>",
    "<loc>https://weedmaps.com/dispensaries/la-canapa-boutique-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/phenofinders</loc>",
    "<loc>https://weedmaps.com/dispensaries/pufftastic-cannabis-co-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-apothecarium-berkeley</loc>",
    "<loc>https://weedmaps.com/dispensaries/zen-leaf-aurora</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-reefer-shack</loc>",
    "<loc>https://weedmaps.com/dispensaries/perfect-union-seaside</loc>",
    "<loc>https://weedmaps.com/dispensaries/420-pharma</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabission-cannabis-ltd</loc>",
    "<loc>https://weedmaps.com/dispensaries/clouds-barcelona</loc>",
    "<loc>https://weedmaps.com/dispensaries/mary-j-s-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-botanist-atlantic-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-pharm-kalkaska</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-tampa-busch</loc>",
    "<loc>https://weedmaps.com/dispensaries/7-bello-social-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/welcome-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/welcome-cannabis-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/wheeler-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/mango-cannabis-norman</loc>",
    "<loc>https://weedmaps.com/dispensaries/stiiizy-909-san-bernardino</loc>",
    "<loc>https://weedmaps.com/dispensaries/michigan-s-finest-mmd-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/capturing-eden-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/premier-provisioning</loc>",
    "<loc>https://weedmaps.com/dispensaries/te-verde</loc>",
    "<loc>https://weedmaps.com/dispensaries/haven-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/lucky-lion-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/csc-green-brothers</loc>",
    "<loc>https://weedmaps.com/dispensaries/pura-vida-san-juan-playa</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-pot-hole-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/t-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/deepgreen-420</loc>",
    "<loc>https://weedmaps.com/dispensaries/cookies-hayward</loc>",
    "<loc>https://weedmaps.com/dispensaries/dutch-love</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-power-plant-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/bean-cannabis-shop</loc>",
    "<loc>https://weedmaps.com/dispensaries/exclusive-kalamazoo-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/okie-genetics</loc>",
    "<loc>https://weedmaps.com/dispensaries/207thc</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-neighbourhood-joint</loc>",
    "<loc>https://weedmaps.com/dispensaries/blunt-cherry-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/hamilton-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-peak-36th-st</loc>",
    "<loc>https://weedmaps.com/dispensaries/dr-terpz</loc>",
    "<loc>https://weedmaps.com/dispensaries/freedom-cannabis-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/dank-bank-27</loc>",
    "<loc>https://weedmaps.com/dispensaries/spdr-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-paseo-cannabis-collective</loc>",
    "<loc>https://weedmaps.com/dispensaries/seed-cannabis-company-riverside</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-we-store-belle-river</loc>",
    "<loc>https://weedmaps.com/dispensaries/baybud-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenhouse-skokie</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-hook-outlet</loc>",
    "<loc>https://weedmaps.com/dispensaries/alaska-harvest-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/jars-mt-pleasant</loc>",
    "<loc>https://weedmaps.com/dispensaries/asomante-mountain-green</loc>",
    "<loc>https://weedmaps.com/dispensaries/capturing-eden</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-hempire-collective-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/next-elevation-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/amnesia-marbella</loc>",
    "<loc>https://weedmaps.com/dispensaries/family-green</loc>",
    "<loc>https://weedmaps.com/dispensaries/smoker-class</loc>",
    "<loc>https://weedmaps.com/dispensaries/dawg-house</loc>",
    "<loc>https://weedmaps.com/dispensaries/mint-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/cdac-cherry-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/lemonnade-melrose</loc>",
    "<loc>https://weedmaps.com/dispensaries/pure-aloha-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/caribbean-oxygen</loc>",
    "<loc>https://weedmaps.com/dispensaries/hempasy</loc>",
    "<loc>https://weedmaps.com/dispensaries/blue-sky-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-vitality-adult-use</loc>",
    "<loc>https://weedmaps.com/dispensaries/nuvue-pueblo-east</loc>",
    "<loc>https://weedmaps.com/dispensaries/la-dispensa</loc>",
    "<loc>https://weedmaps.com/dispensaries/jars-river-rouge</loc>",
    "<loc>https://weedmaps.com/dispensaries/asociacion-de-amigos-para-el-estudio-de-la-botanica-the-dawghouse-mojacar</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-tampa-hillsborough</loc>",
    "<loc>https://weedmaps.com/dispensaries/bob-s-bud-emporium</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-merchant-bloor-st</loc>",
    "<loc>https://weedmaps.com/dispensaries/weedadvisor</loc>",
    "<loc>https://weedmaps.com/dispensaries/fresh-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/vulcan-bud-supply</loc>",
    "<loc>https://weedmaps.com/dispensaries/yerba-buena-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/cali-kosher</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-gorilla-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-naperville-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/raw-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/thebudcompany</loc>",
    "<loc>https://weedmaps.com/dispensaries/jane-s-cannabis-shop</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-bank-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-market-ak</loc>",
    "<loc>https://weedmaps.com/dispensaries/speakeasy-cannabis-cambridge</loc>",
    "<loc>https://weedmaps.com/dispensaries/cookies-san-bernardino</loc>",
    "<loc>https://weedmaps.com/dispensaries/associacio-cannabica-greenday</loc>",
    "<loc>https://weedmaps.com/dispensaries/spiritleaf-morningside</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-hopes-north</loc>",
    "<loc>https://weedmaps.com/dispensaries/diem-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/fire-creek</loc>",
    "<loc>https://weedmaps.com/dispensaries/releaf-clinic-west</loc>",
    "<loc>https://weedmaps.com/dispensaries/planta-madre</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-glass-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenwolf-bellflower</loc>",
    "<loc>https://weedmaps.com/dispensaries/speakeasy-cannabis-bowmanville</loc>",
    "<loc>https://weedmaps.com/dispensaries/nature-s-care-west-loop</loc>",
    "<loc>https://weedmaps.com/dispensaries/speakeasy-cannabis-midland</loc>",
    "<loc>https://weedmaps.com/dispensaries/tradecraft-farms-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenhouse-northbrook</loc>",
    "<loc>https://weedmaps.com/dispensaries/kalamalka-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-merchant-yonge-st</loc>",
    "<loc>https://weedmaps.com/dispensaries/stiiizy-union-square</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-merchant-danforth-ave</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbiculture-ontario-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/executive-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-doctor420</loc>",
    "<loc>https://weedmaps.com/dispensaries/kanopya-springs-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/navigator-life-medical-marijuana-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/besame-wellness-warrensburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/besame-wellness-smithville</loc>",
    "<loc>https://weedmaps.com/dispensaries/besame-wellness-liberty</loc>",
    "<loc>https://weedmaps.com/dispensaries/besame-wellness-nkansascity</loc>",
    "<loc>https://weedmaps.com/dispensaries/from-the-earth-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/from-the-earth-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/from-the-earth-6</loc>",
    "<loc>https://weedmaps.com/dispensaries/rdc-huntington-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-provo</loc>",
    "<loc>https://weedmaps.com/dispensaries/dank-vault</loc>",
    "<loc>https://weedmaps.com/dispensaries/puffy</loc>",
    "<loc>https://weedmaps.com/dispensaries/enlightened-cannabis-for-people-arkadelphia</loc>",
    "<loc>https://weedmaps.com/dispensaries/enlightened-cannabis-for-people-heber-springs</loc>",
    "<loc>https://weedmaps.com/dispensaries/enlightened-cannabis-for-people-morrilton</loc>",
    "<loc>https://weedmaps.com/dispensaries/enlightened-cannabis-for-people-clarksville</loc>",
    "<loc>https://weedmaps.com/dispensaries/bazonzoes-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/lemonnade-arleta</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-healing-tree-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/house-of-hash</loc>",
    "<loc>https://weedmaps.com/dispensaries/lucky-monkey-buds-ii-coming-soon</loc>",
    "<loc>https://weedmaps.com/dispensaries/sparq-retail-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/maine-only</loc>",
    "<loc>https://weedmaps.com/dispensaries/haze-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/joyology-center-line</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenhouse-melrose-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/superblend</loc>",
    "<loc>https://weedmaps.com/dispensaries/eufloria-dispensary-mcalester</loc>",
    "<loc>https://weedmaps.com/dispensaries/ketama-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/stem-haverhill</loc>",
    "<loc>https://weedmaps.com/dispensaries/amp-atlantic-medicinal-partners-medical</loc>",
    "<loc>https://weedmaps.com/dispensaries/alternative-essence-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/pure-ohio-wellness-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/island-smoke</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-dispensaries-dracut</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-cloud-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-dispensaries-monroeville</loc>",
    "<loc>https://weedmaps.com/dispensaries/grasshopper-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/greentown-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/j-supply-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-underground-bancroft</loc>",
    "<loc>https://weedmaps.com/dispensaries/j-supply-co-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/shinybud-windsor</loc>",
    "<loc>https://weedmaps.com/dispensaries/star-buds-cannabis-co-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/star-buds-cannabis-co-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/insalata-cannabis-market</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-underground-barrie</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-merchant-jefferson-ave</loc>",
    "<loc>https://weedmaps.com/dispensaries/ameri-yorkville</loc>",
    "<loc>https://weedmaps.com/dispensaries/higher-limits-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/ameri-forest-hill</loc>",
    "<loc>https://weedmaps.com/dispensaries/ameri-eglinton</loc>",
    "<loc>https://weedmaps.com/dispensaries/dolly-s-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-wellness-new-castle</loc>",
    "<loc>https://weedmaps.com/dispensaries/greencare-medical</loc>",
    "<loc>https://weedmaps.com/dispensaries/bud-s-cannabis-store</loc>",
    "<loc>https://weedmaps.com/dispensaries/qbud</loc>",
    "<loc>https://weedmaps.com/dispensaries/shiny-budy-cornwall</loc>",
    "<loc>https://weedmaps.com/dispensaries/shiny-bud-hawkesbury</loc>",
    "<loc>https://weedmaps.com/dispensaries/shiny-bud-orleans-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/calyx-trichomes</loc>",
    "<loc>https://weedmaps.com/dispensaries/ovcs</loc>",
    "<loc>https://weedmaps.com/dispensaries/info-cannabisjacks-ca</loc>",
    "<loc>https://weedmaps.com/dispensaries/local-cannabis-scarborough</loc>",
    "<loc>https://weedmaps.com/dispensaries/royal-cannabis-supply-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/1922-cannabis-retail-toronto</loc>",
    "<loc>https://weedmaps.com/dispensaries/ethos-montgomeryville</loc>",
    "<loc>https://weedmaps.com/dispensaries/ethos-wilkes-barre</loc>",
    "<loc>https://weedmaps.com/dispensaries/ethos-pittsburgh</loc>",
    "<loc>https://weedmaps.com/dispensaries/ethos-hazelton</loc>",
    "<loc>https://weedmaps.com/dispensaries/ipot</loc>",
    "<loc>https://weedmaps.com/dispensaries/nature-s-herbs-wellness-milliken</loc>",
    "<loc>https://weedmaps.com/dispensaries/710-kingston</loc>",
    "<loc>https://weedmaps.com/dispensaries/dulce-nectar</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-leaf-dispo</loc>",
    "<loc>https://weedmaps.com/dispensaries/creekside-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/happy-dayz-orillia</loc>",
    "<loc>https://weedmaps.com/dispensaries/above-and-beyond-cannabis-llc-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/pleasantrees-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-herbery-fourth-plain</loc>",
    "<loc>https://weedmaps.com/dispensaries/olswell-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/mammoth-holistic</loc>",
    "<loc>https://weedmaps.com/dispensaries/spark-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-solution-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/proper-cannabis-festus-west</loc>",
    "<loc>https://weedmaps.com/dispensaries/proper-cannabis-manchester</loc>",
    "<loc>https://weedmaps.com/dispensaries/proper-cannabis-ellisville</loc>",
    "<loc>https://weedmaps.com/dispensaries/proper-cannabis-house-springs</loc>",
    "<loc>https://weedmaps.com/dispensaries/three-greens-collective-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/original-goods-cannabis-downtown</loc>",
    "<loc>https://weedmaps.com/dispensaries/four20-premium-market-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/accupure-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/crowsnest-s-classy-joint</loc>",
    "<loc>https://weedmaps.com/dispensaries/bongs-such-plus-16-ave</loc>",
    "<loc>https://weedmaps.com/dispensaries/bridgeland-cannabis-store</loc>",
    "<loc>https://weedmaps.com/dispensaries/plantlife-canada-7787</loc>",
    "<loc>https://weedmaps.com/dispensaries/icon</loc>",
    "<loc>https://weedmaps.com/dispensaries/co-op-cannabis-crowfoot</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenspot-shop</loc>",
    "<loc>https://weedmaps.com/dispensaries/plantlife-canada-edmonton-windermere-crossing</loc>",
    "<loc>https://weedmaps.com/dispensaries/loft-cannabis-market</loc>",
    "<loc>https://weedmaps.com/dispensaries/magic-buds</loc>",
    "<loc>https://weedmaps.com/dispensaries/plantlife-canada-south-edmonton-common</loc>",
    "<loc>https://weedmaps.com/dispensaries/quantum-1-cannabis-grand-forks</loc>",
    "<loc>https://weedmaps.com/dispensaries/plantlife-canada-edmonton-windermere-currents</loc>",
    "<loc>https://weedmaps.com/dispensaries/quantum-1-cannabis-keremeos</loc>",
    "<loc>https://weedmaps.com/dispensaries/quantum-1-cannabis-vernon</loc>",
    "<loc>https://weedmaps.com/dispensaries/quantum-1-cannabis-creston</loc>",
    "<loc>https://weedmaps.com/dispensaries/plantlife-cannabis-fort-mcmurray-eagle-ridge</loc>",
    "<loc>https://weedmaps.com/dispensaries/mt-kushmore-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/upstairs-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/kushbar-cannabis-co-medicine-hat</loc>",
    "<loc>https://weedmaps.com/dispensaries/seaweed-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/prairie-records-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-corp-crossfield</loc>",
    "<loc>https://weedmaps.com/dispensaries/prairie-records-edmonton</loc>",
    "<loc>https://weedmaps.com/dispensaries/glenora-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/didsbury-cannabis-mart-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/f-l-cannabis-inc-drayton-valley</loc>",
    "<loc>https://weedmaps.com/dispensaries/f-l-cannabis-inc-entwistle</loc>",
    "<loc>https://weedmaps.com/dispensaries/127-st-cannabis-queen</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-aurora-cannabis-store</loc>",
    "<loc>https://weedmaps.com/dispensaries/kushi-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/ek-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/mountain-standard-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/mountain-standard-cannabis-82-street</loc>",
    "<loc>https://weedmaps.com/dispensaries/mountain-standard-cannabis-118-avenue</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunshine-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/inhale-detroit</loc>",
    "<loc>https://weedmaps.com/dispensaries/budding-creations-grande-prarie</loc>",
    "<loc>https://weedmaps.com/dispensaries/fire-flower-cannabis-co-calgary-applewood-circle-k</loc>",
    "<loc>https://weedmaps.com/dispensaries/tweed-whyte-ave</loc>",
    "<loc>https://weedmaps.com/dispensaries/la-milla-verde-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/wild-flower-medical-marijuana-dispensary-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-street-cannabis-retail</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-punch</loc>",
    "<loc>https://weedmaps.com/dispensaries/pakalolo-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/prairie-and-luna</loc>",
    "<loc>https://weedmaps.com/dispensaries/ethos-fitchburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/stash-co-recreational-cannabis-stittsville</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-bakery-by-e7-fort-bragg</loc>",
    "<loc>https://weedmaps.com/dispensaries/cookies-denver</loc>",
    "<loc>https://weedmaps.com/dispensaries/purple-circle-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/westcan-cannabis-aquitania-blvd</loc>",
    "<loc>https://weedmaps.com/dispensaries/four20-premium-market-pembina</loc>",
    "<loc>https://weedmaps.com/dispensaries/headquarters-medical-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/silver-stem-fine-cannabis-nederland-boulder</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-doctor-420-nichols-hills</loc>",
    "<loc>https://weedmaps.com/dispensaries/sweet-dirt-waterville</loc>",
    "<loc>https://weedmaps.com/dispensaries/stash-co-rockland</loc>",
    "<loc>https://weedmaps.com/dispensaries/releaf-organix</loc>",
    "<loc>https://weedmaps.com/dispensaries/seaweed-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/berkshire-roots-boston</loc>",
    "<loc>https://weedmaps.com/dispensaries/wetaskiwin-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-factory</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-factory-18th-street</loc>",
    "<loc>https://weedmaps.com/dispensaries/bud-mart-carstairs</loc>",
    "<loc>https://weedmaps.com/dispensaries/mary-jane-on-penny-lane</loc>",
    "<loc>https://weedmaps.com/dispensaries/american-marijuana-cultivators</loc>",
    "<loc>https://weedmaps.com/dispensaries/ziggyz-plus</loc>",
    "<loc>https://weedmaps.com/dispensaries/dutch-love-lake-country-winfield</loc>",
    "<loc>https://weedmaps.com/dispensaries/shiny-bud-rockland</loc>",
    "<loc>https://weedmaps.com/dispensaries/sun-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/t-cannabis-smith-falls</loc>",
    "<loc>https://weedmaps.com/dispensaries/zen-leaf-york</loc>",
    "<loc>https://weedmaps.com/dispensaries/zen-leaf-harrisburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/t-cannabis-acton</loc>",
    "<loc>https://weedmaps.com/dispensaries/t-cannabis-rockland</loc>",
    "<loc>https://weedmaps.com/dispensaries/haven-6</loc>",
    "<loc>https://weedmaps.com/dispensaries/berkshire-roots-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/t-cannabis-fort-frances</loc>",
    "<loc>https://weedmaps.com/dispensaries/t-cannabis-renfrew</loc>",
    "<loc>https://weedmaps.com/dispensaries/mr-nice-guy-portland-se-woodstock</loc>",
    "<loc>https://weedmaps.com/dispensaries/stiiizy-jurupa-valley</loc>",
    "<loc>https://weedmaps.com/dispensaries/asociacion-cannabica-kosy</loc>",
    "<loc>https://weedmaps.com/dispensaries/monte-verde</loc>",
    "<loc>https://weedmaps.com/dispensaries/prime-cannabis-cranbrook</loc>",
    "<loc>https://weedmaps.com/dispensaries/this-is-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/releaf-resources</loc>",
    "<loc>https://weedmaps.com/dispensaries/inspired-cannabis-co-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/jimmy-s-cannabis-shop-castlegar</loc>",
    "<loc>https://weedmaps.com/dispensaries/jimmy-s-cannabis-shop-creston</loc>",
    "<loc>https://weedmaps.com/dispensaries/dawson-creek-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-cannabis-dispensary-boynton-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-cannabis-dispensary-port-st-lucie</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbal-remedies-dispensary-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-day-farm-independence</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenlight-bannister-coming-soon</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenlight-harrisonville-coming-soon</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenlight-hayti-coming-soon</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenlight-sikeston-coming-soon</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenlight-poplar-bluff-coming-soon</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenlight-cape-girardeau-coming-soon</loc>",
    "<loc>https://weedmaps.com/dispensaries/mani-s-toke</loc>",
    "<loc>https://weedmaps.com/dispensaries/mind-full-the-cannabis-store-118-ave-nw</loc>",
    "<loc>https://weedmaps.com/dispensaries/yaletown-cannabis-store</loc>",
    "<loc>https://weedmaps.com/dispensaries/d9-social-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/from-the-earth-mcgee</loc>",
    "<loc>https://weedmaps.com/dispensaries/from-the-earth-raytown</loc>",
    "<loc>https://weedmaps.com/dispensaries/misty-mountain-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/earth-s-own-naturals-ltd-fernie</loc>",
    "<loc>https://weedmaps.com/dispensaries/earth-s-own-naturals-ltd-kimberley</loc>",
    "<loc>https://weedmaps.com/dispensaries/canuvo-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/bam-lounge-420</loc>",
    "<loc>https://weedmaps.com/dispensaries/swade-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/one-plant-jacksonville</loc>",
    "<loc>https://weedmaps.com/dispensaries/earth-keeper-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/one-plant-st-petersburg-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/one-plant-ocala-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/one-plant-avon-park-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/tulsa-treehouse</loc>",
    "<loc>https://weedmaps.com/dispensaries/fiore-fresco</loc>",
    "<loc>https://weedmaps.com/dispensaries/xplore</loc>",
    "<loc>https://weedmaps.com/dispensaries/treasure-valley-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/erbn-green-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/planet-420</loc>",
    "<loc>https://weedmaps.com/dispensaries/hi-lo-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-charleston</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-bloom-room-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/maple-leaf-greenery-ltd</loc>",
    "<loc>https://weedmaps.com/dispensaries/san-fernando-valley-medical-supply-inc-dba-elevate-on-3rd</loc>",
    "<loc>https://weedmaps.com/dispensaries/toke-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/kaya-connection-parksville</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-honey-pot-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/highway-cannabis-port-alberni</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-gaia-cannabis-co-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/alberni-cannabis-store</loc>",
    "<loc>https://weedmaps.com/dispensaries/bzam</loc>",
    "<loc>https://weedmaps.com/dispensaries/pacificanna-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/organic-goods-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-valley-pineapple-kelowna</loc>",
    "<loc>https://weedmaps.com/dispensaries/american-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/shango-bay-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/common-citizen-hazel-park-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/grand-ave-haze</loc>",
    "<loc>https://weedmaps.com/dispensaries/maryjane-s-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/lolly</loc>",
    "<loc>https://weedmaps.com/dispensaries/swade-dispensary-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-brandon</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-delray-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-longwood</loc>",
    "<loc>https://weedmaps.com/dispensaries/buds-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/canapa-sky-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/bud-barn-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/nirvana-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/weed-paradise</loc>",
    "<loc>https://weedmaps.com/dispensaries/vernon-cannabis-store-31st-st</loc>",
    "<loc>https://weedmaps.com/dispensaries/blended-buds-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/highlander-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/a-little-bud-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/oso-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/payson-s-joint</loc>",
    "<loc>https://weedmaps.com/dispensaries/cottage-country-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/oso-cannabis-company-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/pied-cow</loc>",
    "<loc>https://weedmaps.com/dispensaries/farmer-jane-cannabis-co-winnipeg</loc>",
    "<loc>https://weedmaps.com/dispensaries/farmer-jane-cannabis-co-regina</loc>",
    "<loc>https://weedmaps.com/dispensaries/flint-embers-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/up-in-smoke-henderson</loc>",
    "<loc>https://weedmaps.com/dispensaries/forbidden-cannabis-club-carson</loc>",
    "<loc>https://weedmaps.com/dispensaries/royal-hotel</loc>",
    "<loc>https://weedmaps.com/dispensaries/little-leaf-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/marks-organix-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/moonlit-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/toke-cannabis-toronto</loc>",
    "<loc>https://weedmaps.com/dispensaries/toke-cannabis-st-catherines</loc>",
    "<loc>https://weedmaps.com/dispensaries/muse-cannabis-stores-deep-cove</loc>",
    "<loc>https://weedmaps.com/dispensaries/one-eye-s-weedery</loc>",
    "<loc>https://weedmaps.com/dispensaries/hiyah-green-medicinals</loc>",
    "<loc>https://weedmaps.com/dispensaries/preeceville-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-lake-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/firebreak-associates-incorporated-dba-cannablue</loc>",
    "<loc>https://weedmaps.com/dispensaries/flower-pot-toronto</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-day-farm-university-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/pure-canna</loc>",
    "<loc>https://weedmaps.com/dispensaries/bud-s-goods-provisions-corp</loc>",
    "<loc>https://weedmaps.com/dispensaries/bluebird-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/higher-limits-cannabis-company-amherstburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-woods-cheboygan</loc>",
    "<loc>https://weedmaps.com/dispensaries/ethos-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/bahama-buds-cannon-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-southwest-bake-shoppe</loc>",
    "<loc>https://weedmaps.com/dispensaries/ddm-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-stop-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/kush-gardens-8</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-stop-clarenville</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-stop-paradise</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-herbal-centre-thc</loc>",
    "<loc>https://weedmaps.com/dispensaries/interior-cannabis-company-inc-osoyoos</loc>",
    "<loc>https://weedmaps.com/dispensaries/east-coast-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/spring-river-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/coffeeshop-ibiza-amsterdam</loc>",
    "<loc>https://weedmaps.com/dispensaries/east-coast-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-joint-cannabis-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-joint-cannabis-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-joint-cannabis-6</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-joint-cannabis-7</loc>",
    "<loc>https://weedmaps.com/dispensaries/spiritleaf-little-italy-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/maine-cannabis-exchange-rec</loc>",
    "<loc>https://weedmaps.com/dispensaries/zen-leaf-naperville</loc>",
    "<loc>https://weedmaps.com/dispensaries/star-buds-chickasha-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/star-buds-altus-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/star-buds-lawton-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-herbary-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/pei-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/pei-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/pei-cannabis-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/pei-cannabis-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-cannabis-retail-store-leamington</loc>",
    "<loc>https://weedmaps.com/dispensaries/dispensary-7884-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-doctor-420-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/fire-house-421</loc>",
    "<loc>https://weedmaps.com/dispensaries/highly-auburn-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-xxi</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-cupboard-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/captains-cannabis-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/ashario</loc>",
    "<loc>https://weedmaps.com/dispensaries/native-harvest-dispensary-moore</loc>",
    "<loc>https://weedmaps.com/dispensaries/3fifteen-primo</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-amherstburg-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/gormley-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/sweet-seven-cannabis-co-waterloo</loc>",
    "<loc>https://weedmaps.com/dispensaries/sweet-seven-cannabis-co-cambridge</loc>",
    "<loc>https://weedmaps.com/dispensaries/puffy-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-420-store-lindsay</loc>",
    "<loc>https://weedmaps.com/dispensaries/star-buds-cannabis-co-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/canja</loc>",
    "<loc>https://weedmaps.com/dispensaries/tony-s-joint</loc>",
    "<loc>https://weedmaps.com/dispensaries/bud-bank-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/blazed-utopia</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-potery</loc>",
    "<loc>https://weedmaps.com/dispensaries/zen-leaf-north-las-vegas</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-health-clinic-punta-las-maria</loc>",
    "<loc>https://weedmaps.com/dispensaries/zen-lead-prospect-heights</loc>",
    "<loc>https://weedmaps.com/dispensaries/star-buds-cannabis-co-huronia</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-forest-ohio</loc>",
    "<loc>https://weedmaps.com/dispensaries/botanical-co-lansing</loc>",
    "<loc>https://weedmaps.com/dispensaries/original-goods-cannabis-strathmore</loc>",
    "<loc>https://weedmaps.com/dispensaries/utopia-cannabis-north-york</loc>",
    "<loc>https://weedmaps.com/dispensaries/13th-floor-cannabis-airdrie</loc>",
    "<loc>https://weedmaps.com/dispensaries/eufloria-12</loc>",
    "<loc>https://weedmaps.com/dispensaries/spiritleaf-parkdale-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/spiritleaf-75</loc>",
    "<loc>https://weedmaps.com/dispensaries/elevated-roots</loc>",
    "<loc>https://weedmaps.com/dispensaries/r-greenleaf-roswell</loc>",
    "<loc>https://weedmaps.com/dispensaries/r-greenleaf-las-cruces</loc>",
    "<loc>https://weedmaps.com/dispensaries/dr-a-s-releaf-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/tree-tops-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/thehoneycomb-farm</loc>",
    "<loc>https://weedmaps.com/dispensaries/wonder-buds-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenfoot-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/farmer-jane-cannabis-co-saskatoon</loc>",
    "<loc>https://weedmaps.com/dispensaries/westside-cllctv-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/lakeview-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-grass-hut</loc>",
    "<loc>https://weedmaps.com/dispensaries/turf-cannabis-collection</loc>",
    "<loc>https://weedmaps.com/dispensaries/sessions-cannabis-windsor</loc>",
    "<loc>https://weedmaps.com/dispensaries/head-bangers-bangers-only</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-dispensaries-erie-peach</loc>",
    "<loc>https://weedmaps.com/dispensaries/timber-cannabis-co-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/quality-roots-hamtramck</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-gym-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/verilife-chester</loc>",
    "<loc>https://weedmaps.com/dispensaries/sweetbuds-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/west-paris-provisions</loc>",
    "<loc>https://weedmaps.com/dispensaries/mota-toke-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/405budz-south</loc>",
    "<loc>https://weedmaps.com/dispensaries/kush-gardens-duncan</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-higher-cannabis-windsor</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabay</loc>",
    "<loc>https://weedmaps.com/dispensaries/dacanna</loc>",
    "<loc>https://weedmaps.com/dispensaries/seagrass</loc>",
    "<loc>https://weedmaps.com/dispensaries/united-cultivation</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-cannabis-dispensary-naperville</loc>",
    "<loc>https://weedmaps.com/dispensaries/level-up-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/floyd-s-fine-cannabis-slab-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/stiiizy-moreno-valley</loc>",
    "<loc>https://weedmaps.com/dispensaries/shivaas-rose-toronto</loc>",
    "<loc>https://weedmaps.com/dispensaries/abide-napa-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/stonedepot</loc>",
    "<loc>https://weedmaps.com/dispensaries/central-plains-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/premier-provisioning-centerllc</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-goods-blaine</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-goods-duluth</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-goods-burnsville</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-goods-woodbury</loc>",
    "<loc>https://weedmaps.com/dispensaries/peaceful-releaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/sessions-cannabis-welland</loc>",
    "<loc>https://weedmaps.com/dispensaries/authentic-415</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-21-hoquiam</loc>",
    "<loc>https://weedmaps.com/dispensaries/miss-jones-orillia</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-day-farm-belton</loc>",
    "<loc>https://weedmaps.com/dispensaries/gage-grand-rapids</loc>",
    "<loc>https://weedmaps.com/dispensaries/sessions-cannabis-windsor-lakeshore</loc>",
    "<loc>https://weedmaps.com/dispensaries/zen-leaf-lombard</loc>",
    "<loc>https://weedmaps.com/dispensaries/march-and-ash-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/earthmed-rosemont</loc>",
    "<loc>https://weedmaps.com/dispensaries/lit-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/marijuana-mercantile-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/jars-detroit-west</loc>",
    "<loc>https://weedmaps.com/dispensaries/canntinas</loc>",
    "<loc>https://weedmaps.com/dispensaries/puffin-s-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/neku-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/neku-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/erbn-green-cannabis-co-picton</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-bend-cannabis-co-grand-bend</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-den-cannabis-store</loc>",
    "<loc>https://weedmaps.com/dispensaries/rocky-s-pot-shop</loc>",
    "<loc>https://weedmaps.com/dispensaries/peach-cannabis-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-grove-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/ridgeburn-cannabis-shop</loc>",
    "<loc>https://weedmaps.com/dispensaries/budzz-pot-shoppe</loc>",
    "<loc>https://weedmaps.com/dispensaries/406-farmacy-407</loc>",
    "<loc>https://weedmaps.com/dispensaries/ez-medz</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-nation-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/hello-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabliss-429</loc>",
    "<loc>https://weedmaps.com/dispensaries/buds-cannabis-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-we-store-wyandotte-st</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-we-store-windsor-walker-rd</loc>",
    "<loc>https://weedmaps.com/dispensaries/little-house-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-pot-spot-toronto</loc>",
    "<loc>https://weedmaps.com/dispensaries/miawpukek-cannabis-boutique</loc>",
    "<loc>https://weedmaps.com/dispensaries/miss-jones-kozlov-outpost</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-den-cannabis-barrie</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-pot-spot-dovercourt</loc>",
    "<loc>https://weedmaps.com/dispensaries/hibuzz-cannabis-rivermont-road</loc>",
    "<loc>https://weedmaps.com/dispensaries/lotw-cannabis-kenora</loc>",
    "<loc>https://weedmaps.com/dispensaries/sessions-cannabis-brockville</loc>",
    "<loc>https://weedmaps.com/dispensaries/sessions-cannabis-aurora</loc>",
    "<loc>https://weedmaps.com/dispensaries/one-plant-delivery-antioch</loc>",
    "<loc>https://weedmaps.com/dispensaries/smokin-savage</loc>",
    "<loc>https://weedmaps.com/dispensaries/missouri-health-and-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/missouri-health-and-wellness-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/missouri-health-and-wellness-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/missouri-health-and-wellness-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/big-chief-collective-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/hibuzz-cannabis-financial-dr</loc>",
    "<loc>https://weedmaps.com/dispensaries/sessions-cannabis-dundurn-st</loc>",
    "<loc>https://weedmaps.com/dispensaries/sessions-cannabis-rymal-rd</loc>",
    "<loc>https://weedmaps.com/dispensaries/brothers-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/sweet-releafs-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/sessions-cannabis-brantford</loc>",
    "<loc>https://weedmaps.com/dispensaries/urb-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/sessions-cannabis-waterloo</loc>",
    "<loc>https://weedmaps.com/dispensaries/sativa-bliss-cannabis-kitchener</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-house-of-cannabis-toronto-king-st</loc>",
    "<loc>https://weedmaps.com/dispensaries/dank-cannabis-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/euphora-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/shango-lapeer-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/ph-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-great-atlantic-puffin-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/nectar-hillsboro</loc>",
    "<loc>https://weedmaps.com/dispensaries/natures-haze-aurora</loc>",
    "<loc>https://weedmaps.com/dispensaries/today-s-herbal-choice-tillamook-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/thermodynamix-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/nectar-122nd-division</loc>",
    "<loc>https://weedmaps.com/dispensaries/sessions-cannabis-timmins</loc>",
    "<loc>https://weedmaps.com/dispensaries/great-lakes-provision-center-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/timber-cannabis-co-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-bake-shoppe</loc>",
    "<loc>https://weedmaps.com/dispensaries/highland-cannabis-kitchener</loc>",
    "<loc>https://weedmaps.com/dispensaries/budside-danforth</loc>",
    "<loc>https://weedmaps.com/dispensaries/urbnbud-windsor</loc>",
    "<loc>https://weedmaps.com/dispensaries/spot420-toronto</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-cabana-banff</loc>",
    "<loc>https://weedmaps.com/dispensaries/elevate-maine-south-portland</loc>",
    "<loc>https://weedmaps.com/dispensaries/embarc-alameda</loc>",
    "<loc>https://weedmaps.com/dispensaries/10-seventeen-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/trease</loc>",
    "<loc>https://weedmaps.com/dispensaries/flowertown-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/deep-river-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/kiosk-cannabis-to</loc>",
    "<loc>https://weedmaps.com/dispensaries/kiosk-cannabis-scarborough</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-ties-cannabis-store-tenth-line</loc>",
    "<loc>https://weedmaps.com/dispensaries/woodstock-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/jars-cannabis-sky-harbor</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-cannabist-shop-king-w</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-den-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/pinnacle-emporium-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/golden-meds-quebec-st-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-releaf-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/denco-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/somewhere</loc>",
    "<loc>https://weedmaps.com/dispensaries/danks-wellness-emporium</loc>",
    "<loc>https://weedmaps.com/dispensaries/miss-jones-bell-city-outpost</loc>",
    "<loc>https://weedmaps.com/dispensaries/miss-jones-hespeler-outpost</loc>",
    "<loc>https://weedmaps.com/dispensaries/oregrown-cannon-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/campfire-cannabis-ma</loc>",
    "<loc>https://weedmaps.com/dispensaries/choom-cannabis-co-vancouver-yaletown</loc>",
    "<loc>https://weedmaps.com/dispensaries/holland-daze</loc>",
    "<loc>https://weedmaps.com/dispensaries/edition-x-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/alpha-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/mayflower-worcester</loc>",
    "<loc>https://weedmaps.com/dispensaries/garden-city-cannabis-co-st-catharines</loc>",
    "<loc>https://weedmaps.com/dispensaries/highbrow-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-dude-abides-provisioning-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/banana-leaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-bud-bar</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-xpress-brampton</loc>",
    "<loc>https://weedmaps.com/dispensaries/dab-cannabis-brampton</loc>",
    "<loc>https://weedmaps.com/dispensaries/yellow-dream-barcelona</loc>",
    "<loc>https://weedmaps.com/dispensaries/garden-city-cannabis-co-welland</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-dispensary-11</loc>",
    "<loc>https://weedmaps.com/dispensaries/holyoke-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-house-dispensary-in-midwest-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/dispo-bay-city-south</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-bud-house-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/due-north-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/danks-wellness-emporium-perkins</loc>",
    "<loc>https://weedmaps.com/dispensaries/olswell-cannabis-co-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/inspired-cannabis-co-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/inspired-cannabis-co-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/inspired-cannabis-co-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/my-buddies-spruce-grove</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-cannabis-shop-victoria</loc>",
    "<loc>https://weedmaps.com/dispensaries/house-of-hash-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-tea-cannabis-co-kennedy-rd</loc>",
    "<loc>https://weedmaps.com/dispensaries/harbor-house-collective</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-joint-cannabis-shop-leila-avenue</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-daytona-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/dank-s-wellness-emporium-norman</loc>",
    "<loc>https://weedmaps.com/dispensaries/ganja-guys-of-alaska-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/sessions-albion</loc>",
    "<loc>https://weedmaps.com/dispensaries/420-asociacion-terapeutica</loc>",
    "<loc>https://weedmaps.com/dispensaries/village-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/sessions-cannabis-dufferin-creek-north-york</loc>",
    "<loc>https://weedmaps.com/dispensaries/sessions-cannabis-heritage-on-lansdowne-peterborough</loc>",
    "<loc>https://weedmaps.com/dispensaries/burb-cannabis-broadway</loc>",
    "<loc>https://weedmaps.com/dispensaries/jaderoom-oc</loc>",
    "<loc>https://weedmaps.com/dispensaries/zen-humboldt</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-cannabis-guys</loc>",
    "<loc>https://weedmaps.com/dispensaries/salty-farmers</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-cannabis-guys-brampton</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-belleville</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-tillsonburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-quinte-west-trenton</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-leamington</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-strathroy</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-wallaceburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-fort-erie</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-cambridge</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-port-colborne</loc>",
    "<loc>https://weedmaps.com/dispensaries/solucion-natural</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-chatham</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-main-st-cambridge</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-huntsville</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-sarnia</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-gravenhurst</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-stratford</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-hamilton</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-windsor</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-brockville</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-st-catherines</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-simcoe</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-welland</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-listowel</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-brantford</loc>",
    "<loc>https://weedmaps.com/dispensaries/inspired-cannabis-co-6</loc>",
    "<loc>https://weedmaps.com/dispensaries/inspired-cannabis-co-7</loc>",
    "<loc>https://weedmaps.com/dispensaries/harvest-corner</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-depot</loc>",
    "<loc>https://weedmaps.com/dispensaries/farmer-jane-cannabis-co-aurora-regina</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-good-cannabis-company-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/oceanic-releaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/lyons-roar</loc>",
    "<loc>https://weedmaps.com/dispensaries/st-jean-grocery-j-s-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-at-the-green-brier</loc>",
    "<loc>https://weedmaps.com/dispensaries/burb-cannabis-poco-place</loc>",
    "<loc>https://weedmaps.com/dispensaries/burb-cannabis-port-moody</loc>",
    "<loc>https://weedmaps.com/dispensaries/shinybud-brockville</loc>",
    "<loc>https://weedmaps.com/dispensaries/global-greenz-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/ak-tasty-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/puff-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/420-sunshine-dispensary-llc-421</loc>",
    "<loc>https://weedmaps.com/dispensaries/mindflower</loc>",
    "<loc>https://weedmaps.com/dispensaries/affx-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-bakery-cannabis-shop</loc>",
    "<loc>https://weedmaps.com/dispensaries/4-cs-shop-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/dank-factory</loc>",
    "<loc>https://weedmaps.com/dispensaries/black-crow-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/mindflower-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/ripple-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/prestige-cannabis-co-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/dona-flor-social-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/blue-sage-cannabis-company-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/blue-sage-cannabis-company-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/canaculture-cannabis-store</loc>",
    "<loc>https://weedmaps.com/dispensaries/buzzed-leaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/cloud-cannabis-utica-rec</loc>",
    "<loc>https://weedmaps.com/dispensaries/inside-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/biogrow-cannabis-corp</loc>",
    "<loc>https://weedmaps.com/dispensaries/sun-provisions</loc>",
    "<loc>https://weedmaps.com/dispensaries/urban-cream-collective</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-cannabist-shop-guelph</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-cannabist-shop-london</loc>",
    "<loc>https://weedmaps.com/dispensaries/happy-life-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/ybh-green</loc>",
    "<loc>https://weedmaps.com/dispensaries/stok-d-cannabis-pharmacy-ave</loc>",
    "<loc>https://weedmaps.com/dispensaries/stok-d-cannabis-kingston-road</loc>",
    "<loc>https://weedmaps.com/dispensaries/roll-d-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/wavy-buds</loc>",
    "<loc>https://weedmaps.com/dispensaries/zipz-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-day-farm-st-clair</loc>",
    "<loc>https://weedmaps.com/dispensaries/cookies-kalamazoo</loc>",
    "<loc>https://weedmaps.com/dispensaries/exclusive-grand-rapids-rec-marijuana</loc>",
    "<loc>https://weedmaps.com/dispensaries/nature-s-remedy-41</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-bradenton</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-brandon</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-clearwater</loc>",
    "<loc>https://weedmaps.com/dispensaries/forest-city-marijuana</loc>",
    "<loc>https://weedmaps.com/dispensaries/dundas-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/sweet-dirt-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-north-st-petersburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-pensacola</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-west-palm-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/verilife-hillsboro</loc>",
    "<loc>https://weedmaps.com/dispensaries/dr-greenthumbs-san-diego</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-releaf-dispensary-mexico</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-releaf-dispensary-troy</loc>",
    "<loc>https://weedmaps.com/dispensaries/verilife-wapakoneta</loc>",
    "<loc>https://weedmaps.com/dispensaries/zaza-dispensary-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/apothecary-farms-okc</loc>",
    "<loc>https://weedmaps.com/dispensaries/blackstar-cannabis-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-square-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/subtropical</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-6ix-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/sage-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/shango-hazel-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/cultivate-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/cultivate-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/spiritleaf-76</loc>",
    "<loc>https://weedmaps.com/dispensaries/el-cochino-chico</loc>",
    "<loc>https://weedmaps.com/dispensaries/puffs-haven</loc>",
    "<loc>https://weedmaps.com/dispensaries/voodoo-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/embarc-martinez</loc>",
    "<loc>https://weedmaps.com/dispensaries/fine-fettle</loc>",
    "<loc>https://weedmaps.com/dispensaries/verilife-rosemont</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-haven</loc>",
    "<loc>https://weedmaps.com/dispensaries/push-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/collective-elevation-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/arnprior-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/this-is-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/pot-of-gold-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/paradise-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/silverleaf-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/hurbn</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-niagara-falls</loc>",
    "<loc>https://weedmaps.com/dispensaries/lucky-toke-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-day-farm-sf-west</loc>",
    "<loc>https://weedmaps.com/dispensaries/rose-gold-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-cannabis-connoisseur</loc>",
    "<loc>https://weedmaps.com/dispensaries/dynasty-pot-shop</loc>",
    "<loc>https://weedmaps.com/dispensaries/forever-buds</loc>",
    "<loc>https://weedmaps.com/dispensaries/corner-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-bend-cannabis-co-kensington</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-woods-cannabis-brampton</loc>",
    "<loc>https://weedmaps.com/dispensaries/verilife-galena</loc>",
    "<loc>https://weedmaps.com/dispensaries/first-class-grass-farm-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannasavvy-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/culture-rising-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-wellflower-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-ware-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-harrisburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/root-66-south-grand</loc>",
    "<loc>https://weedmaps.com/dispensaries/nirvana-budz-stoney-creek</loc>",
    "<loc>https://weedmaps.com/dispensaries/naturalpes-sion</loc>",
    "<loc>https://weedmaps.com/dispensaries/mkl-industries-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/remedy-313</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-greeneasy-ltd-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-dispensaries-paramus</loc>",
    "<loc>https://weedmaps.com/dispensaries/essence-dispensaries-pasadena</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-dispensaries-lake-in-the-hills</loc>",
    "<loc>https://weedmaps.com/dispensaries/crazy-duck-sevilla</loc>",
    "<loc>https://weedmaps.com/dispensaries/trifecta-farmacy</loc>",
    "<loc>https://weedmaps.com/dispensaries/moraira-420</loc>",
    "<loc>https://weedmaps.com/dispensaries/above-all-greenery-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/village-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/barttowa-tree</loc>",
    "<loc>https://weedmaps.com/dispensaries/indian-trail-farms</loc>",
    "<loc>https://weedmaps.com/dispensaries/friendly-stranger-cannabis-culture-shop-midland</loc>",
    "<loc>https://weedmaps.com/dispensaries/spark-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/friendly-stranger-cannabis-culture-shop</loc>",
    "<loc>https://weedmaps.com/dispensaries/friendly-stranger-cannabis-culture-shop-danforth-ave</loc>",
    "<loc>https://weedmaps.com/dispensaries/friendly-stranger-cannabis-culture-shop-queen-st</loc>",
    "<loc>https://weedmaps.com/dispensaries/smoky-trails-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/my-buddies-calgary</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannab3</loc>",
    "<loc>https://weedmaps.com/dispensaries/kasa-kana-huntsville</loc>",
    "<loc>https://weedmaps.com/dispensaries/mood-jackson</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-day</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-cabana-waterloo</loc>",
    "<loc>https://weedmaps.com/dispensaries/humboldt-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/first-class-canabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-dispensary-eastern-express</loc>",
    "<loc>https://weedmaps.com/dispensaries/rare-earth-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-dab-east-louisville</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-mint-cannabis-cave-creek</loc>",
    "<loc>https://weedmaps.com/dispensaries/la-canapa-boutique-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/cape-cod-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/red-dirt-experience</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-wellness-plymouth-meeting</loc>",
    "<loc>https://weedmaps.com/dispensaries/mountain-annie-s-durango</loc>",
    "<loc>https://weedmaps.com/dispensaries/gage-battle-creek</loc>",
    "<loc>https://weedmaps.com/dispensaries/toke-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/verilife-river-north</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-bangor-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-dab-west-louisville-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/sticky-battle-creek</loc>",
    "<loc>https://weedmaps.com/dispensaries/sugar-leaf-co-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/flower-factory-fine-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-goods-frederick</loc>",
    "<loc>https://weedmaps.com/dispensaries/san-diego-recreational-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/yellowstone-buds-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/spot420-the-cannabis-store</loc>",
    "<loc>https://weedmaps.com/dispensaries/endless-heights</loc>",
    "<loc>https://weedmaps.com/dispensaries/kush-gardens-lawton</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-day-farm-st-louis-cwe</loc>",
    "<loc>https://weedmaps.com/dispensaries/budd-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-hex-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/mary-jane-s-cannabis-weston</loc>",
    "<loc>https://weedmaps.com/dispensaries/north-80-cannabis-merchants</loc>",
    "<loc>https://weedmaps.com/dispensaries/backpack-boyz</loc>",
    "<loc>https://weedmaps.com/dispensaries/rob-sons-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-health-7</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-nation-dispensary-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/stiiizy-san-bernardino</loc>",
    "<loc>https://weedmaps.com/dispensaries/club-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/meta-cannabis-supply-co-guelph</loc>",
    "<loc>https://weedmaps.com/dispensaries/grasshopper-42</loc>",
    "<loc>https://weedmaps.com/dispensaries/grassstation-hamilton</loc>",
    "<loc>https://weedmaps.com/dispensaries/kushbar-cannabis-co-morinville</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-cabana-bowness</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-cabana-116</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-cabana-district</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-cabana-east-village</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-cabana-haysboro</loc>",
    "<loc>https://weedmaps.com/dispensaries/nobo-muskegon</loc>",
    "<loc>https://weedmaps.com/dispensaries/neat-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-botanist-williamstown</loc>",
    "<loc>https://weedmaps.com/dispensaries/kanab</loc>",
    "<loc>https://weedmaps.com/dispensaries/miss-jones-cannabis-white-oaks-outpost</loc>",
    "<loc>https://weedmaps.com/dispensaries/miss-jones-cannabis-mckeown-outpost</loc>",
    "<loc>https://weedmaps.com/dispensaries/otc-dispensary-lounge</loc>",
    "<loc>https://weedmaps.com/dispensaries/kellys-cannabis-huntsville</loc>",
    "<loc>https://weedmaps.com/dispensaries/zen-leaf-west-loop</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbal-remedies-of-maine</loc>",
    "<loc>https://weedmaps.com/dispensaries/dispensary-90909090911</loc>",
    "<loc>https://weedmaps.com/dispensaries/pure-roots-centerline</loc>",
    "<loc>https://weedmaps.com/dispensaries/pure-roots-lansing</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-dispensaries-maynard</loc>",
    "<loc>https://weedmaps.com/dispensaries/hatch-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-cross-caregivers-rec</loc>",
    "<loc>https://weedmaps.com/dispensaries/a-high-class-joint</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-day-farm-imperial</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-day-farm-lindbergh</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-kensary-toronto</loc>",
    "<loc>https://weedmaps.com/dispensaries/aloha-green-herb-house</loc>",
    "<loc>https://weedmaps.com/dispensaries/vip-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/4twenty-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/coastal-green</loc>",
    "<loc>https://weedmaps.com/dispensaries/mary-j-s-cannabis-cobourg</loc>",
    "<loc>https://weedmaps.com/dispensaries/shinybud-alliston</loc>",
    "<loc>https://weedmaps.com/dispensaries/one-plant-north-york</loc>",
    "<loc>https://weedmaps.com/dispensaries/weed-art-cannabis-club-malaga</loc>",
    "<loc>https://weedmaps.com/dispensaries/mood420</loc>",
    "<loc>https://weedmaps.com/dispensaries/missouri-made-marijuana</loc>",
    "<loc>https://weedmaps.com/dispensaries/embr</loc>",
    "<loc>https://weedmaps.com/dispensaries/higher-grounds-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/dr-green-huelva</loc>",
    "<loc>https://weedmaps.com/dispensaries/capital-dank-tulsa</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbal-wellness-center-north</loc>",
    "<loc>https://weedmaps.com/dispensaries/nirvana-center-center-line-rec</loc>",
    "<loc>https://weedmaps.com/dispensaries/edition-x-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/seaweed-co-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/cookies-halsey</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-ties-cannabis-store-embrun</loc>",
    "<loc>https://weedmaps.com/dispensaries/7engines-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/fresh-karma-dispensaries</loc>",
    "<loc>https://weedmaps.com/dispensaries/spiritleaf-stratford</loc>",
    "<loc>https://weedmaps.com/dispensaries/shinybud-blenheim</loc>",
    "<loc>https://weedmaps.com/dispensaries/shinybud-pitt-st</loc>",
    "<loc>https://weedmaps.com/dispensaries/leaf-and-bud</loc>",
    "<loc>https://weedmaps.com/dispensaries/nevada-made-marijuana</loc>",
    "<loc>https://weedmaps.com/dispensaries/small-town-buds-kapuskasing</loc>",
    "<loc>https://weedmaps.com/dispensaries/natural-healing-center-8</loc>",
    "<loc>https://weedmaps.com/dispensaries/emjay-la-mesa</loc>",
    "<loc>https://weedmaps.com/dispensaries/yerba-buena-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/broadway-cannabis-market-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-artist-tree-marijuana-dispensary-and-weed-delivery-los-angeles</loc>",
    "<loc>https://weedmaps.com/dispensaries/verilife-schaumburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/beyond-the-pines</loc>",
    "<loc>https://weedmaps.com/dispensaries/dispo-portage</loc>",
    "<loc>https://weedmaps.com/dispensaries/house-of-dank-center-line-med</loc>",
    "<loc>https://weedmaps.com/dispensaries/ivy-hill-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/montrose</loc>",
    "<loc>https://weedmaps.com/dispensaries/supergas-cannabis-hamilton</loc>",
    "<loc>https://weedmaps.com/dispensaries/lemonnade-van-nuys</loc>",
    "<loc>https://weedmaps.com/dispensaries/gv530-cannabis-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenlight-independence-south</loc>",
    "<loc>https://weedmaps.com/dispensaries/stairway-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenlight-branson</loc>",
    "<loc>https://weedmaps.com/dispensaries/apollo-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenlight-springfield</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenlight-joplin</loc>",
    "<loc>https://weedmaps.com/dispensaries/dragon-lily-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannamazoo-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/landrace-cannabis-co-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/liberty-madison-heights</loc>",
    "<loc>https://weedmaps.com/dispensaries/flower-power-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-park-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/besame-wellness-kansascity</loc>",
    "<loc>https://weedmaps.com/dispensaries/strawberry-fields-denver</loc>",
    "<loc>https://weedmaps.com/dispensaries/trees-cannabis-6</loc>",
    "<loc>https://weedmaps.com/dispensaries/trees-cannabis-7</loc>",
    "<loc>https://weedmaps.com/dispensaries/trees-cannabis-8</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-we-store-tecumseh-rd</loc>",
    "<loc>https://weedmaps.com/dispensaries/spiritleaf-owen-sound</loc>",
    "<loc>https://weedmaps.com/dispensaries/full-harvest-moonz</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-door-8</loc>",
    "<loc>https://weedmaps.com/dispensaries/farmer-jane-cannabis-co-mcormond-dr-saskatoon</loc>",
    "<loc>https://weedmaps.com/dispensaries/spiritleaf-stittsville</loc>",
    "<loc>https://weedmaps.com/dispensaries/tweed-38672920</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabuds</loc>",
    "<loc>https://weedmaps.com/dispensaries/capital-dank-rockwell</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-profile-44th-street-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/puff-madison-heights-recreational-coming-soon</loc>",
    "<loc>https://weedmaps.com/dispensaries/bazonzoes-7</loc>",
    "<loc>https://weedmaps.com/dispensaries/lucky-buds</loc>",
    "<loc>https://weedmaps.com/dispensaries/gatsby-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/t-cannabis-128</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-outpost</loc>",
    "<loc>https://weedmaps.com/dispensaries/take-me-home-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/budding-creations-peachland</loc>",
    "<loc>https://weedmaps.com/dispensaries/nature-s-fix</loc>",
    "<loc>https://weedmaps.com/dispensaries/sillicann-valley</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-south-portland</loc>",
    "<loc>https://weedmaps.com/dispensaries/3fifteen-primo-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/take-off-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/proper-cannabis-warrenton</loc>",
    "<loc>https://weedmaps.com/dispensaries/proper-cannabis-south-county</loc>",
    "<loc>https://weedmaps.com/dispensaries/alpine-smoke-coboconk</loc>",
    "<loc>https://weedmaps.com/dispensaries/nova-cannabis-of-marietta</loc>",
    "<loc>https://weedmaps.com/dispensaries/verde-mou</loc>",
    "<loc>https://weedmaps.com/dispensaries/ashario-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/bluebird-cannabis-co-march-rd</loc>",
    "<loc>https://weedmaps.com/dispensaries/bluebird-cannabis-co-manotick</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-cookie-malvern</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-roll</loc>",
    "<loc>https://weedmaps.com/dispensaries/reup</loc>",
    "<loc>https://weedmaps.com/dispensaries/em-jay-s-smart-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-day-farms-cantrell</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunfish-cannabis-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/malibu-road</loc>",
    "<loc>https://weedmaps.com/dispensaries/tweed-38672921</loc>",
    "<loc>https://weedmaps.com/dispensaries/yield-cannabis-co-belleville</loc>",
    "<loc>https://weedmaps.com/dispensaries/goodsinse-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/jupiter-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/borderland-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/toke-cannabis-midland</loc>",
    "<loc>https://weedmaps.com/dispensaries/bloomers-dispensary-and-sundries</loc>",
    "<loc>https://weedmaps.com/dispensaries/mosaic-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/puff-utica-recreational-coming-soon</loc>",
    "<loc>https://weedmaps.com/dispensaries/white-fire-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/reserved-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-ties-cannabis-store-downtown</loc>",
    "<loc>https://weedmaps.com/dispensaries/ronin-cannabis-guelph</loc>",
    "<loc>https://weedmaps.com/dispensaries/enlightened</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-higher-cannabis-company-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/exclusive-muskegon</loc>",
    "<loc>https://weedmaps.com/dispensaries/dr-a-s-releaf-center-reading</loc>",
    "<loc>https://weedmaps.com/dispensaries/old-west-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/noa-botanicals-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/hidden-leaf-cannabis-co-brampton</loc>",
    "<loc>https://weedmaps.com/dispensaries/one-plant-lompoc</loc>",
    "<loc>https://weedmaps.com/dispensaries/healing-zone-humacao</loc>",
    "<loc>https://weedmaps.com/dispensaries/fritz-fire-house</loc>",
    "<loc>https://weedmaps.com/dispensaries/v-d-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/meta-cannabis-supply-co-walker-place</loc>",
    "<loc>https://weedmaps.com/dispensaries/tack-d-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/liberty-cranberry</loc>",
    "<loc>https://weedmaps.com/dispensaries/originals-san-diego</loc>",
    "<loc>https://weedmaps.com/dispensaries/earth-gift</loc>",
    "<loc>https://weedmaps.com/dispensaries/earth-gift-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/pinnacle-emporium-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-onion-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/root-66-dogtown</loc>",
    "<loc>https://weedmaps.com/dispensaries/root-66-st-peters</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-cookie-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/budssmoke-burlington</loc>",
    "<loc>https://weedmaps.com/dispensaries/athena-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/inspired-cannabis-co-8</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-cannabist-shop-manitou-dr</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-planet-ibiza</loc>",
    "<loc>https://weedmaps.com/dispensaries/farmer-jane-cannabis-co-preston-ave</loc>",
    "<loc>https://weedmaps.com/dispensaries/farmer-jane-cannabis-co-charleswood</loc>",
    "<loc>https://weedmaps.com/dispensaries/farmer-jane-cannabis-co-henderson</loc>",
    "<loc>https://weedmaps.com/dispensaries/premium-cannabis-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/farmer-jane-cannabis-co-normanview</loc>",
    "<loc>https://weedmaps.com/dispensaries/casa-verde-healing-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannavibes-emporium</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-cabana-shawnessy</loc>",
    "<loc>https://weedmaps.com/dispensaries/zen-leaf-neptune</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbnjoy-hanford</loc>",
    "<loc>https://weedmaps.com/dispensaries/hidden-leaf-cannabis-co-lawrence-ave</loc>",
    "<loc>https://weedmaps.com/dispensaries/hiddenn-leaf-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/pharmabis-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-joint-cannabis-shop-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/shinybud-centre-rd</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-haven-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/shinybud-kemptville</loc>",
    "<loc>https://weedmaps.com/dispensaries/shinybud-prescott</loc>",
    "<loc>https://weedmaps.com/dispensaries/shinybud-tilbury</loc>",
    "<loc>https://weedmaps.com/dispensaries/shinybud-orillia</loc>",
    "<loc>https://weedmaps.com/dispensaries/native-harvest-chickasha</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-tampa-gandy</loc>",
    "<loc>https://weedmaps.com/dispensaries/tonik-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/phoenix-cannabis-kitchener</loc>",
    "<loc>https://weedmaps.com/dispensaries/everbloom-rifle</loc>",
    "<loc>https://weedmaps.com/dispensaries/tree-of-echo-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/joyology-quincy</loc>",
    "<loc>https://weedmaps.com/dispensaries/hive-medicinal-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/medible-delights</loc>",
    "<loc>https://weedmaps.com/dispensaries/terrabis-o-fallon</loc>",
    "<loc>https://weedmaps.com/dispensaries/element-7-rio-dell</loc>",
    "<loc>https://weedmaps.com/dispensaries/rama-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/cottontail-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/buzzed-buds</loc>",
    "<loc>https://weedmaps.com/dispensaries/unplug-cannabis-co-orleans</loc>",
    "<loc>https://weedmaps.com/dispensaries/native-therapeutics-ltd-co-dba-big-chief-collective-duncan</loc>",
    "<loc>https://weedmaps.com/dispensaries/shinybud-arnprior</loc>",
    "<loc>https://weedmaps.com/dispensaries/herb-n-bud</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-wavy-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/trees-cannabis-10</loc>",
    "<loc>https://weedmaps.com/dispensaries/trees-cannabis-11</loc>",
    "<loc>https://weedmaps.com/dispensaries/trees-cannabis-fifth-street-nanaimo</loc>",
    "<loc>https://weedmaps.com/dispensaries/caliyield</loc>",
    "<loc>https://weedmaps.com/dispensaries/borincanna</loc>",
    "<loc>https://weedmaps.com/dispensaries/stiiizy-vista</loc>",
    "<loc>https://weedmaps.com/dispensaries/vip-cannabis-co-woodstock</loc>",
    "<loc>https://weedmaps.com/dispensaries/kind-goods-manchester</loc>",
    "<loc>https://weedmaps.com/dispensaries/kind-goods-st-peters</loc>",
    "<loc>https://weedmaps.com/dispensaries/kind-goods-fenton</loc>",
    "<loc>https://weedmaps.com/dispensaries/mystique-of-maine-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-farm-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/howhi</loc>",
    "<loc>https://weedmaps.com/dispensaries/stone-age-bcn</loc>",
    "<loc>https://weedmaps.com/dispensaries/desrochers-hotel-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/dutch-love-parry-sound-pine</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-xpress-uxbridge</loc>",
    "<loc>https://weedmaps.com/dispensaries/pottymouth-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-hit-pit</loc>",
    "<loc>https://weedmaps.com/dispensaries/weighburn-weed-supply-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/mango-cannabis-lawton</loc>",
    "<loc>https://weedmaps.com/dispensaries/treetz</loc>",
    "<loc>https://weedmaps.com/dispensaries/brown-square-assets-llc-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/eufloria-13</loc>",
    "<loc>https://weedmaps.com/dispensaries/gas-pros-edmond</loc>",
    "<loc>https://weedmaps.com/dispensaries/emerald-grove-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/prairie-cannabis-8th-street</loc>",
    "<loc>https://weedmaps.com/dispensaries/marvin-s-mary-j</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-day-farm-berners</loc>",
    "<loc>https://weedmaps.com/dispensaries/southwest-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/zen-leaf-lawrence</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-doctor-420-hefner</loc>",
    "<loc>https://weedmaps.com/dispensaries/leisure-for-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/hash-and-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/ethos-pleasant-hills</loc>",
    "<loc>https://weedmaps.com/dispensaries/one-plant-goleta-opening-5-28</loc>",
    "<loc>https://weedmaps.com/dispensaries/health-express-6t</loc>",
    "<loc>https://weedmaps.com/dispensaries/prairie-cannabis-22nd-st</loc>",
    "<loc>https://weedmaps.com/dispensaries/prairie-cannabis-prince-albert</loc>",
    "<loc>https://weedmaps.com/dispensaries/lucky-lion-sandy</loc>",
    "<loc>https://weedmaps.com/dispensaries/highbrow-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/marlees-den-williams-lake</loc>",
    "<loc>https://weedmaps.com/dispensaries/chuck-wellness-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/sessions-cannabis-binbrook-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/sessions-cannabis-rosedale</loc>",
    "<loc>https://weedmaps.com/dispensaries/sessions-cannabis-sault-ste-marie</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-cured-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-cured-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-day-farm-joplin</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-day-farm-sf-south</loc>",
    "<loc>https://weedmaps.com/dispensaries/budssmoke-pickering-south</loc>",
    "<loc>https://weedmaps.com/dispensaries/ace-of-spades-weed-limited</loc>",
    "<loc>https://weedmaps.com/dispensaries/spiritleaf-imperial-centre</loc>",
    "<loc>https://weedmaps.com/dispensaries/spiritleaf-brickyard</loc>",
    "<loc>https://weedmaps.com/dispensaries/hardmoore-green</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-unlimited-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenhouse-cannabis-hamilton</loc>",
    "<loc>https://weedmaps.com/dispensaries/grafters</loc>",
    "<loc>https://weedmaps.com/dispensaries/inspired-cannabis-co-9</loc>",
    "<loc>https://weedmaps.com/dispensaries/inspired-cannabis-co-10</loc>",
    "<loc>https://weedmaps.com/dispensaries/uphi-cannabis-emporium</loc>",
    "<loc>https://weedmaps.com/dispensaries/bluebird-cannabis-co-almonte</loc>",
    "<loc>https://weedmaps.com/dispensaries/redden-s-junction-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/shinybud-alexandria</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-treehouse-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/stiiizy-wildomar-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/cultivate-6</loc>",
    "<loc>https://weedmaps.com/dispensaries/okie-og-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-treatment-cannabis-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/red-rock-cannabis-co-toronto</loc>",
    "<loc>https://weedmaps.com/dispensaries/stiiizy-pomona</loc>",
    "<loc>https://weedmaps.com/dispensaries/lindsay-cannabis-store</loc>",
    "<loc>https://weedmaps.com/dispensaries/okc-10-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/vibe-consulting-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/origins-cannabis-co-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/above-the-clouds-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/kind-farms-reserve-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/zagas-hemp-shop-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/golden-tree-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/28grahams-cannabis-29</loc>",
    "<loc>https://weedmaps.com/dispensaries/paradise-air</loc>",
    "<loc>https://weedmaps.com/dispensaries/catalyst-pine-downtownlongbeach</loc>",
    "<loc>https://weedmaps.com/dispensaries/cosmic-charlies</loc>",
    "<loc>https://weedmaps.com/dispensaries/lake-city-cannabis-lincoln-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/highway-80-dispensary-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-fire-station-cannabis-co-houghton</loc>",
    "<loc>https://weedmaps.com/dispensaries/kryptonite-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/euphoric-pot</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-228</loc>",
    "<loc>https://weedmaps.com/dispensaries/lolly-cannabis-pickering</loc>",
    "<loc>https://weedmaps.com/dispensaries/la-catedral-de-madrid</loc>",
    "<loc>https://weedmaps.com/dispensaries/smokeys-101-avenue</loc>",
    "<loc>https://weedmaps.com/dispensaries/primo-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/dutch-love-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/deep-roots-harvest-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/callie-s-cannabis-shoppe-denver</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-fire-station-cannabis-co-hannahville</loc>",
    "<loc>https://weedmaps.com/dispensaries/catalyst-florence-southla</loc>",
    "<loc>https://weedmaps.com/dispensaries/ivy-league-cannabis-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/evermore-cannabis-dispensary-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/spdr-pickering</loc>",
    "<loc>https://weedmaps.com/dispensaries/flor</loc>",
    "<loc>https://weedmaps.com/dispensaries/dreamer-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-outpost-crystal-falls</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabliss-431</loc>",
    "<loc>https://weedmaps.com/dispensaries/glenmore-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/cookies-santa-ana</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-kure-cannabis-society-deroche</loc>",
    "<loc>https://weedmaps.com/dispensaries/black-bear-bud</loc>",
    "<loc>https://weedmaps.com/dispensaries/revo-group</loc>",
    "<loc>https://weedmaps.com/dispensaries/davenport-leaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/cookies-denver-commerce-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/windswept-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-canadian-cannabis-store</loc>",
    "<loc>https://weedmaps.com/dispensaries/select-cannabis-97-street</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-smoke-leaf-ltd</loc>",
    "<loc>https://weedmaps.com/dispensaries/cabin-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/southwest-wellness-center-guadalupe</loc>",
    "<loc>https://weedmaps.com/dispensaries/gage-burton</loc>",
    "<loc>https://weedmaps.com/dispensaries/northern-helm-cannabis-danforth</loc>",
    "<loc>https://weedmaps.com/dispensaries/anathallo-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/fresh-karma-dispensaries-saint-joseph</loc>",
    "<loc>https://weedmaps.com/dispensaries/flamingo-cannabis-portage-ave</loc>",
    "<loc>https://weedmaps.com/dispensaries/sinsemilla-portland</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-piping-plover</loc>",
    "<loc>https://weedmaps.com/dispensaries/heya-wellness-st-ann-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/heya-wellness-st-charles</loc>",
    "<loc>https://weedmaps.com/dispensaries/heya-wellness-park-hills</loc>",
    "<loc>https://weedmaps.com/dispensaries/codes-kirksville</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-apothecarium-dispensary-stroudsburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/smokeys-walker-lake</loc>",
    "<loc>https://weedmaps.com/dispensaries/smokeys-mill-creek</loc>",
    "<loc>https://weedmaps.com/dispensaries/story-of-maryland</loc>",
    "<loc>https://weedmaps.com/dispensaries/sessions-cannabis-carlisle</loc>",
    "<loc>https://weedmaps.com/dispensaries/pecos-valley-production-mermod</loc>",
    "<loc>https://weedmaps.com/dispensaries/pecos-valley-production-portales</loc>",
    "<loc>https://weedmaps.com/dispensaries/pecos-valley-production-alamogordo</loc>",
    "<loc>https://weedmaps.com/dispensaries/pecos-valley-production-hobbs</loc>",
    "<loc>https://weedmaps.com/dispensaries/pecos-valley-production-sunland-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/pecos-valley-production-roadrunner</loc>",
    "<loc>https://weedmaps.com/dispensaries/plateau</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-cannabis-dispensary-philadelphia-chestnut-street</loc>",
    "<loc>https://weedmaps.com/dispensaries/mend-cannabis-co-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/sessions-cannabis-port-colborne</loc>",
    "<loc>https://weedmaps.com/dispensaries/meta-cannabis-supply-co-otineka-mall</loc>",
    "<loc>https://weedmaps.com/dispensaries/sessions-cannabis-somerset</loc>",
    "<loc>https://weedmaps.com/dispensaries/stiiizy-ferndale</loc>",
    "<loc>https://weedmaps.com/dispensaries/tweed-portage-la-prairie</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-earth-collective-granada-hills-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/wizard-cannabis-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/gather-and-grow-recreational-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/puertoricann</loc>",
    "<loc>https://weedmaps.com/dispensaries/north-hills-society-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-joint-cannabis-circle-dr</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-joint-cannabis-victoria-ave</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-joint-cannabis-regent-avenues</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-joint-cannabis-corydon-ave</loc>",
    "<loc>https://weedmaps.com/dispensaries/shinybud-elmvale</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-edgewater-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/paradise-organics-belgrade</loc>",
    "<loc>https://weedmaps.com/dispensaries/brightleaf-cannabis-st-albert</loc>",
    "<loc>https://weedmaps.com/dispensaries/wellgreens-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/dutch-love-hampton-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/magnolia-city-apothecary</loc>",
    "<loc>https://weedmaps.com/dispensaries/dispensary-587-216-3209</loc>",
    "<loc>https://weedmaps.com/dispensaries/village-cannabis-hamilton</loc>",
    "<loc>https://weedmaps.com/dispensaries/apothecary-farms-tulsa</loc>",
    "<loc>https://weedmaps.com/dispensaries/canntina-725-nelson-st</loc>",
    "<loc>https://weedmaps.com/dispensaries/himalaya-cannabis-ltd</loc>",
    "<loc>https://weedmaps.com/dispensaries/pop-s-cannabis-co-pickering</loc>",
    "<loc>https://weedmaps.com/dispensaries/turtle-pie-co-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/kush-gardens-usa</loc>",
    "<loc>https://weedmaps.com/dispensaries/kush-gardens-usa-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-aylmer</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-grand-bend</loc>",
    "<loc>https://weedmaps.com/dispensaries/diem-cannabis-lynn</loc>",
    "<loc>https://weedmaps.com/dispensaries/apogee-gardens-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/honeypot-smoke-shop</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-xpress-wasaga-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-xpress-hillsdale</loc>",
    "<loc>https://weedmaps.com/dispensaries/planet-13-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/try-redi</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabliss-cannabis-co-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/shinybud-smithfalls</loc>",
    "<loc>https://weedmaps.com/dispensaries/crown-leaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/riverside-wellness-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/trading-post-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/up-in-sky</loc>",
    "<loc>https://weedmaps.com/dispensaries/wonderbrett</loc>",
    "<loc>https://weedmaps.com/dispensaries/cost-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/encanto-tree</loc>",
    "<loc>https://weedmaps.com/dispensaries/pops-cannabis-kirkland-lake</loc>",
    "<loc>https://weedmaps.com/dispensaries/pops-cannabis-sturgeon-falls</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-cake-house</loc>",
    "<loc>https://weedmaps.com/dispensaries/elevated-cannabis-north-york</loc>",
    "<loc>https://weedmaps.com/dispensaries/moksha-cannabis-jane-st</loc>",
    "<loc>https://weedmaps.com/dispensaries/squamish-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/aaa-health-center-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/terrabis-hazelwood</loc>",
    "<loc>https://weedmaps.com/dispensaries/ambrosia-cannabis-emporium</loc>",
    "<loc>https://weedmaps.com/dispensaries/buzzed-buds-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-bouquet-cannabis-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/stiiizy-parkmerced</loc>",
    "<loc>https://weedmaps.com/dispensaries/due-north-cannabis-co-pine-st</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenery-cannabis-boutique-armstrong</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-bodega-boyz-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/flight-club-norman</loc>",
    "<loc>https://weedmaps.com/dispensaries/ultra-health-sf-cerillos</loc>",
    "<loc>https://weedmaps.com/dispensaries/ultra-health-gallup</loc>",
    "<loc>https://weedmaps.com/dispensaries/ultra-health-las-vegas</loc>",
    "<loc>https://weedmaps.com/dispensaries/bogarts-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/emerald-fire-meds-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/origins-north-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/lucky-dog-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-hunny-pot-queen-west</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-hunny-pot-cannabis-co-2103-yonge</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-hunny-pot-burlington</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-hunny-pot-cannabis-co-40-centennial-pkwy-n</loc>",
    "<loc>https://weedmaps.com/dispensaries/stok-d-cannabis-niagara-falls</loc>",
    "<loc>https://weedmaps.com/dispensaries/higher-living-llc-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/khatsahlano-kannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/foresta-cannabis-collection</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-day-farm-ofallon</loc>",
    "<loc>https://weedmaps.com/dispensaries/cbw-labs-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/deep-roots-harvest-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-cannabis-dispensary-bonita-springs</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-cannabis-dispensary-orlando-fern-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/harvest-cannabis-co-southey</loc>",
    "<loc>https://weedmaps.com/dispensaries/levels-provisioning</loc>",
    "<loc>https://weedmaps.com/dispensaries/kaleafa-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/blossom-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-hunny-pot-cannabis-co-cambridge</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-king</loc>",
    "<loc>https://weedmaps.com/dispensaries/hightend</loc>",
    "<loc>https://weedmaps.com/dispensaries/inspired-cannabis-co-11</loc>",
    "<loc>https://weedmaps.com/dispensaries/your-local-cannabis-stouffville</loc>",
    "<loc>https://weedmaps.com/dispensaries/dispensary-exit-243-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/miss-jones-cannabis-gravenhurst-outpost</loc>",
    "<loc>https://weedmaps.com/dispensaries/re-up-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/alphabud-cannabis-company-abc-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/happy-hippy-cannabis-store</loc>",
    "<loc>https://weedmaps.com/dispensaries/ganja-goddess-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/lemon-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/groovy-s</loc>",
    "<loc>https://weedmaps.com/dispensaries/okie-wonderland-west-tulsa-sapulpa</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-xpress-port-hope</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-xpress-ridgetown</loc>",
    "<loc>https://weedmaps.com/dispensaries/blazing-jane-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/vivid-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/river-driver-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/zen-leaf-philadelphia</loc>",
    "<loc>https://weedmaps.com/dispensaries/kind-castle</loc>",
    "<loc>https://weedmaps.com/dispensaries/stiiizy-suisun</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-other-guys</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenworldcann-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-healing-community-medco-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/oveeo</loc>",
    "<loc>https://weedmaps.com/dispensaries/wonderland-cannabis-reserve</loc>",
    "<loc>https://weedmaps.com/dispensaries/inspired-cannabis-co-12</loc>",
    "<loc>https://weedmaps.com/dispensaries/weedjar</loc>",
    "<loc>https://weedmaps.com/dispensaries/weedjar-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/northern-helm-cannabis-gore-road</loc>",
    "<loc>https://weedmaps.com/dispensaries/northern-helm-cannabis-bowmanville</loc>",
    "<loc>https://weedmaps.com/dispensaries/gorilla-rx</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-21-ocean-shores</loc>",
    "<loc>https://weedmaps.com/dispensaries/stinky-leaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-we-store-sarnia</loc>",
    "<loc>https://weedmaps.com/dispensaries/shinybud-ottawa</loc>",
    "<loc>https://weedmaps.com/dispensaries/circa-1818-cobourg</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-cannabis-shop</loc>",
    "<loc>https://weedmaps.com/dispensaries/terrabis-creve-coeur</loc>",
    "<loc>https://weedmaps.com/dispensaries/lucky-monkey-buds-pueblo</loc>",
    "<loc>https://weedmaps.com/dispensaries/magic-city-cannabis-colorado</loc>",
    "<loc>https://weedmaps.com/dispensaries/kush-house-24-hours-never-closed-moore</loc>",
    "<loc>https://weedmaps.com/dispensaries/sessions-cannabis-port-perry</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-door-pleasant-plains</loc>",
    "<loc>https://weedmaps.com/dispensaries/4th-n-twenty-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/speakeasy-cannabis-penetanguishene</loc>",
    "<loc>https://weedmaps.com/dispensaries/timber-cannabis-co-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/encanto-tree-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/village-bud-landsdowne</loc>",
    "<loc>https://weedmaps.com/dispensaries/village-buds-milton</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-connoisseur-lounge-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/flower-leaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-society-8</loc>",
    "<loc>https://weedmaps.com/dispensaries/t-cannabis-cochrane</loc>",
    "<loc>https://weedmaps.com/dispensaries/soul-rebel-cannabis-co-kingston-rd</loc>",
    "<loc>https://weedmaps.com/dispensaries/coastside</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-wells</loc>",
    "<loc>https://weedmaps.com/dispensaries/terps</loc>",
    "<loc>https://weedmaps.com/dispensaries/cookies-coalinga</loc>",
    "<loc>https://weedmaps.com/dispensaries/cookies-worcester</loc>",
    "<loc>https://weedmaps.com/dispensaries/omg-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/shannon-s-best-buds</loc>",
    "<loc>https://weedmaps.com/dispensaries/bud-brothers-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/cake-cannabis-cherry-street</loc>",
    "<loc>https://weedmaps.com/dispensaries/thc-centre-cannabis-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/seed-smith</loc>",
    "<loc>https://weedmaps.com/dispensaries/budget-bud-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/terps-rec-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/dank-plug</loc>",
    "<loc>https://weedmaps.com/dispensaries/uptown-herb</loc>",
    "<loc>https://weedmaps.com/dispensaries/bud-brothers-coughy-shop-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/bluebird-cannabis-packenham</loc>",
    "<loc>https://weedmaps.com/dispensaries/t-cannabis-kenora</loc>",
    "<loc>https://weedmaps.com/dispensaries/evergreen-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/sacramento-care-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/danks-wellness-emporium-warwick-6909-w-hefner-rd-b-18</loc>",
    "<loc>https://weedmaps.com/dispensaries/sacred-roots</loc>",
    "<loc>https://weedmaps.com/dispensaries/josephine-billie-s</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-profile-51</loc>",
    "<loc>https://weedmaps.com/dispensaries/t-cannabis-kirkland-lake</loc>",
    "<loc>https://weedmaps.com/dispensaries/inspired-cannabis-co-komoka</loc>",
    "<loc>https://weedmaps.com/dispensaries/farmaverde-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/farmaverde-islaverde</loc>",
    "<loc>https://weedmaps.com/dispensaries/farmaverde-65-infanteria</loc>",
    "<loc>https://weedmaps.com/dispensaries/farmaverde-cupey</loc>",
    "<loc>https://weedmaps.com/dispensaries/yakima-weed-co-south</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-day-farm-columbia</loc>",
    "<loc>https://weedmaps.com/dispensaries/primitiv-niles</loc>",
    "<loc>https://weedmaps.com/dispensaries/fika-supply</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-day-east-sunshine</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-hut-ltd</loc>",
    "<loc>https://weedmaps.com/dispensaries/3fifteen-primo-saint-louis</loc>",
    "<loc>https://weedmaps.com/dispensaries/pincanna-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/fluent-mechanicsburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/element-7-marina</loc>",
    "<loc>https://weedmaps.com/dispensaries/norquay-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/northern-helm-courtice</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-coast-6</loc>",
    "<loc>https://weedmaps.com/dispensaries/in-n-out-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/verilife-plymouth-meeting</loc>",
    "<loc>https://weedmaps.com/dispensaries/verilife-pottstown</loc>",
    "<loc>https://weedmaps.com/dispensaries/verilife-quakertown</loc>",
    "<loc>https://weedmaps.com/dispensaries/verilife-williamsport-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/verilife-lancaster</loc>",
    "<loc>https://weedmaps.com/dispensaries/craft-cannabis-andresen</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-cake-house-clinton-keith</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-cannabis-dispensary-fort-lauderdale</loc>",
    "<loc>https://weedmaps.com/dispensaries/lux-leaf-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-decatur-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/ack-natural-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/stiiizy-marina</loc>",
    "<loc>https://weedmaps.com/dispensaries/humble-family-farms-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/pure-options-east-lansing</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-cannabis-retail-store</loc>",
    "<loc>https://weedmaps.com/dispensaries/tops-farm</loc>",
    "<loc>https://weedmaps.com/dispensaries/mainstage-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-cabana-falconridge</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-cabana-westgate</loc>",
    "<loc>https://weedmaps.com/dispensaries/dazed-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/farmaverde-canovanas</loc>",
    "<loc>https://weedmaps.com/dispensaries/jars-grand-rapids</loc>",
    "<loc>https://weedmaps.com/dispensaries/jars-flint-mt-morris</loc>",
    "<loc>https://weedmaps.com/dispensaries/pot-spot</loc>",
    "<loc>https://weedmaps.com/dispensaries/garden-state-dispensary-eatontown</loc>",
    "<loc>https://weedmaps.com/dispensaries/rightpuff-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/encanto-tree-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/spot420-the-cannabis-store-orangeville</loc>",
    "<loc>https://weedmaps.com/dispensaries/miss-jones-cannabis-chemong-outpost</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-frost-factory-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/apothecary-farms-durant</loc>",
    "<loc>https://weedmaps.com/dispensaries/r-b-dispensary-dba-mercy-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/reserved-cannabis-waterloo</loc>",
    "<loc>https://weedmaps.com/dispensaries/bud-bar-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/choom-cannabis-co-liberty-village</loc>",
    "<loc>https://weedmaps.com/dispensaries/shinybud-308-north-front</loc>",
    "<loc>https://weedmaps.com/dispensaries/shinybud-48-dundas-west</loc>",
    "<loc>https://weedmaps.com/dispensaries/shinybud-444-dundas-east</loc>",
    "<loc>https://weedmaps.com/dispensaries/shelter-market</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbana-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/purple-moose-cannabis-oshawa</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-mint-cannabis-coldwater</loc>",
    "<loc>https://weedmaps.com/dispensaries/winewood-organics</loc>",
    "<loc>https://weedmaps.com/dispensaries/rocky-mountain-roots-barrhead</loc>",
    "<loc>https://weedmaps.com/dispensaries/nobo-benton-harbor</loc>",
    "<loc>https://weedmaps.com/dispensaries/sixty-four-hope-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-cloud-cannabis-orangeville</loc>",
    "<loc>https://weedmaps.com/dispensaries/fika-herbal-goods-aurora</loc>",
    "<loc>https://weedmaps.com/dispensaries/fika-herbal-goods-barrie</loc>",
    "<loc>https://weedmaps.com/dispensaries/fika-herbal-goods-london-east</loc>",
    "<loc>https://weedmaps.com/dispensaries/fika-herbal-goods-milton</loc>",
    "<loc>https://weedmaps.com/dispensaries/fika-herbal-goods-oshawa</loc>",
    "<loc>https://weedmaps.com/dispensaries/fika-herbal-goods-scarborough</loc>",
    "<loc>https://weedmaps.com/dispensaries/fika-herbal-goods-st-thomas</loc>",
    "<loc>https://weedmaps.com/dispensaries/fika-herbal-goods-sudbury</loc>",
    "<loc>https://weedmaps.com/dispensaries/fika-herbal-goods-canary</loc>",
    "<loc>https://weedmaps.com/dispensaries/fika-herbal-goods-windsor</loc>",
    "<loc>https://weedmaps.com/dispensaries/fika-herbal-goods-golden-mile</loc>",
    "<loc>https://weedmaps.com/dispensaries/bloom-city-club-sturgis</loc>",
    "<loc>https://weedmaps.com/dispensaries/jars-ann-arbor-packard</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-grove-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-ottawa</loc>",
    "<loc>https://weedmaps.com/dispensaries/crop-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/star-budz-provisioning-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/terrabis-springfield-coming-soon</loc>",
    "<loc>https://weedmaps.com/dispensaries/highly-boothbay-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/catalyst-morenovalley</loc>",
    "<loc>https://weedmaps.com/dispensaries/highway-cannabis-stop-chamberlain</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-tea-cannabis-co-york-west</loc>",
    "<loc>https://weedmaps.com/dispensaries/yerba-buena-cannabis-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/elevation-station</loc>",
    "<loc>https://weedmaps.com/dispensaries/pineapple-express-26</loc>",
    "<loc>https://weedmaps.com/dispensaries/shinybud-college-square</loc>",
    "<loc>https://weedmaps.com/dispensaries/toke-house-thunder-bay-arundel-st</loc>",
    "<loc>https://weedmaps.com/dispensaries/toke-house-dryden</loc>",
    "<loc>https://weedmaps.com/dispensaries/dab-cannabis-aurora</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunrise-cannabis-dispensary-st-louis</loc>",
    "<loc>https://weedmaps.com/dispensaries/balagan-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/firestorm-orono</loc>",
    "<loc>https://weedmaps.com/dispensaries/freedom-road-okc</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabist-deptford</loc>",
    "<loc>https://weedmaps.com/dispensaries/house-of-ganja-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/rainbow-cannabis-place-toronto</loc>",
    "<loc>https://weedmaps.com/dispensaries/a-bud-above-craft-boutique</loc>",
    "<loc>https://weedmaps.com/dispensaries/zen-cannabis-kitchener</loc>",
    "<loc>https://weedmaps.com/dispensaries/red-rock-cannabis-co-ajax</loc>",
    "<loc>https://weedmaps.com/dispensaries/stiiizy-benicia</loc>",
    "<loc>https://weedmaps.com/dispensaries/plantshop</loc>",
    "<loc>https://weedmaps.com/dispensaries/your-highness-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/kansas-city-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/zen-leaf-flamingo</loc>",
    "<loc>https://weedmaps.com/dispensaries/budders-cannabis-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/oso-cannabis-company-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/valley-verde</loc>",
    "<loc>https://weedmaps.com/dispensaries/pacificanna-fort-st-victoria</loc>",
    "<loc>https://weedmaps.com/dispensaries/haight-ashbury-cannabis-store</loc>",
    "<loc>https://weedmaps.com/dispensaries/bud-brothers-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/bdz-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/sky-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/sativa-bliss-cannabis-etobicoke</loc>",
    "<loc>https://weedmaps.com/dispensaries/codes-west-plains</loc>",
    "<loc>https://weedmaps.com/dispensaries/west-coast-meds-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/planted-provisioning</loc>",
    "<loc>https://weedmaps.com/dispensaries/codes-raymore</loc>",
    "<loc>https://weedmaps.com/dispensaries/scalable-farms</loc>",
    "<loc>https://weedmaps.com/dispensaries/kelo-retail-corp</loc>",
    "<loc>https://weedmaps.com/dispensaries/capturing-eden-tillsonburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/moonfire-cannabis-owen-sound</loc>",
    "<loc>https://weedmaps.com/dispensaries/capturing-eden-burford</loc>",
    "<loc>https://weedmaps.com/dispensaries/westcanna</loc>",
    "<loc>https://weedmaps.com/dispensaries/march-and-ash-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/moonfire-cannabis-sauble-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/catalyst-el-monte</loc>",
    "<loc>https://weedmaps.com/dispensaries/catalyst-pomona</loc>",
    "<loc>https://weedmaps.com/dispensaries/one-plant-brampton</loc>",
    "<loc>https://weedmaps.com/dispensaries/one-plant-masonville</loc>",
    "<loc>https://weedmaps.com/dispensaries/one-plant-cundles-road</loc>",
    "<loc>https://weedmaps.com/dispensaries/one-plant-north-bay</loc>",
    "<loc>https://weedmaps.com/dispensaries/one-plant-pickering</loc>",
    "<loc>https://weedmaps.com/dispensaries/one-plant-st-catharines</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-plus</loc>",
    "<loc>https://weedmaps.com/dispensaries/sessions-cannabis-8</loc>",
    "<loc>https://weedmaps.com/dispensaries/one-plant-thunder-bay</loc>",
    "<loc>https://weedmaps.com/dispensaries/one-plant-tecumseh</loc>",
    "<loc>https://weedmaps.com/dispensaries/one-plant-walkerville</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-hook-outlet-watsonville</loc>",
    "<loc>https://weedmaps.com/dispensaries/happy-green-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-budstop-cannabis-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/discounted-cannabis-127-st</loc>",
    "<loc>https://weedmaps.com/dispensaries/levels-cannabis-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-cabana-brampton</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-cabana-ajax</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-cabana-roxboro</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-cabana-airdrie-main-street</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-cabana-varsity</loc>",
    "<loc>https://weedmaps.com/dispensaries/top-kush-only-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-dude-abides-provisioning-center-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/sticky-bud-farms</loc>",
    "<loc>https://weedmaps.com/dispensaries/nirvana-center-coldwater</loc>",
    "<loc>https://weedmaps.com/dispensaries/treasure-tree-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/higher-level-of-care-greenfield</loc>",
    "<loc>https://weedmaps.com/dispensaries/cookies-flamingo</loc>",
    "<loc>https://weedmaps.com/dispensaries/selective-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-cannabis-depot-boulder</loc>",
    "<loc>https://weedmaps.com/dispensaries/plug-canna6is-dundas</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-owen-sound</loc>",
    "<loc>https://weedmaps.com/dispensaries/minerva-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-xpress-wingham</loc>",
    "<loc>https://weedmaps.com/dispensaries/deepwell</loc>",
    "<loc>https://weedmaps.com/dispensaries/miss-jones-cannabis-goderich-outpost</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-reef-muskegon</loc>",
    "<loc>https://weedmaps.com/dispensaries/packs-sgv-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/northern-helm-cannabis-oshawa</loc>",
    "<loc>https://weedmaps.com/dispensaries/oki-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/burlington-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/purple-moon-dispensary-coweta</loc>",
    "<loc>https://weedmaps.com/dispensaries/northern-helm</loc>",
    "<loc>https://weedmaps.com/dispensaries/outpost-marquette</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbology-south-modesto</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-oz-store-centretown</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-potery-guelph</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-artist-tree-riverside</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-day-farm-cape-girardeau</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-higher-standard-helena</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-vault-by-grow-happy-farms-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-cabana-kanata</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-cabana-gloucester</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-apothecarium-lodi</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-cabana-innisfil</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-cabana-fort-erie</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-cabana-silvercreek</loc>",
    "<loc>https://weedmaps.com/dispensaries/meta-cannabis-selkirk</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-leaf-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/wake-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/smokey-s-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/relm-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/jars-payson</loc>",
    "<loc>https://weedmaps.com/dispensaries/3fifteen-primo-branson-west</loc>",
    "<loc>https://weedmaps.com/dispensaries/raft-peak-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/stiiizy-pacheco</loc>",
    "<loc>https://weedmaps.com/dispensaries/plant-galaxy</loc>",
    "<loc>https://weedmaps.com/dispensaries/relm-cannabis-hamilton</loc>",
    "<loc>https://weedmaps.com/dispensaries/bluebird-cannabis-gladstone-ave</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-paradise</loc>",
    "<loc>https://weedmaps.com/dispensaries/greentown-cannabis-lasalle</loc>",
    "<loc>https://weedmaps.com/dispensaries/epik-products</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-doctor-420-moore</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-syndicate-desert-hot-springs</loc>",
    "<loc>https://weedmaps.com/dispensaries/choom-hamilton</loc>",
    "<loc>https://weedmaps.com/dispensaries/gas-gas-medical</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-cured-bangor-rec</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-kure-cannabis-society-chilliwack</loc>",
    "<loc>https://weedmaps.com/dispensaries/terpene-journey-llc-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/discounted-cannabis-stony-plain</loc>",
    "<loc>https://weedmaps.com/dispensaries/discounted-cannabis-manning-crossing</loc>",
    "<loc>https://weedmaps.com/dispensaries/tree-of-life-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/tree-factory-oxnard</loc>",
    "<loc>https://weedmaps.com/dispensaries/red-label-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/valley-high-llc-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbal-pathways-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/erb-arbor</loc>",
    "<loc>https://weedmaps.com/dispensaries/cg</loc>",
    "<loc>https://weedmaps.com/dispensaries/heka-inc-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/meduana</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenlight-district-windsor</loc>",
    "<loc>https://weedmaps.com/dispensaries/sapphire-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/n8v-grown</loc>",
    "<loc>https://weedmaps.com/dispensaries/emerald-fields-wash-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/encanto-tree-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/neat-cannabis-company-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-supreme-dispensary-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/fresh-karma-midtown</loc>",
    "<loc>https://weedmaps.com/dispensaries/packs-oc</loc>",
    "<loc>https://weedmaps.com/dispensaries/bananas-cannabis-store</loc>",
    "<loc>https://weedmaps.com/dispensaries/bananas-cannabis-store-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/premier-provisioning-saginaw</loc>",
    "<loc>https://weedmaps.com/dispensaries/pinnacle-emporium-kamden-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/bodega-boyz-georgetown</loc>",
    "<loc>https://weedmaps.com/dispensaries/levicann-toronto</loc>",
    "<loc>https://weedmaps.com/dispensaries/levicann-keswick</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-cannabis-dispensary-wyomissing</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-cannabis-dispensary-clearwater</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-cannabis-dispensary-sarasota</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-cannabis-dispensary-tallahassee</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-cannabis-dispensary-north-miami</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-cannabis-dispensary-oakland-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-bakery-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/totally-baked-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/brainporium-toronto</loc>",
    "<loc>https://weedmaps.com/dispensaries/fire-depot-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-campbellford</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-cornwall</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-goderich</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-hanover</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-sudbury</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-napanee</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-main-st-north-bay</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-algonquin-ave-north-bay</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-sault-ste-marie</loc>",
    "<loc>https://weedmaps.com/dispensaries/red-dirt-experience-oklahoma-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/feel-state-florissant</loc>",
    "<loc>https://weedmaps.com/dispensaries/modern-day-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/rainbow-thunder-bay-cannabis-accessories</loc>",
    "<loc>https://weedmaps.com/dispensaries/rainbow-thunder-bay-cannabis-accessories-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/bloom-medicinals-o-fallon</loc>",
    "<loc>https://weedmaps.com/dispensaries/speakeasy-cannabis-colborne</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-hootenanny</loc>",
    "<loc>https://weedmaps.com/dispensaries/groovybudz</loc>",
    "<loc>https://weedmaps.com/dispensaries/oz-cannabis-owosso</loc>",
    "<loc>https://weedmaps.com/dispensaries/secret-garden-cannabis-ottawa</loc>",
    "<loc>https://weedmaps.com/dispensaries/on-the-ridge-retail-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-ties-cannabis-store-alexandria</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-ties-cannabis-store-blackburn</loc>",
    "<loc>https://weedmaps.com/dispensaries/phenotopia</loc>",
    "<loc>https://weedmaps.com/dispensaries/violet-wild-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-door-allegan</loc>",
    "<loc>https://weedmaps.com/dispensaries/star-buds-dispensary-ardmore</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-labs-provisions</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-north-cannabis-store-baseline-road</loc>",
    "<loc>https://weedmaps.com/dispensaries/mango-cannabis-okc</loc>",
    "<loc>https://weedmaps.com/dispensaries/bath-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/elixir-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/apogee-gardens-butte</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-flower-shop-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/bud-bar-cannabis-crowfoot</loc>",
    "<loc>https://weedmaps.com/dispensaries/kansas-city-cannabis-company-excelsior</loc>",
    "<loc>https://weedmaps.com/dispensaries/kansas-city-cannabis-company-lotawana</loc>",
    "<loc>https://weedmaps.com/dispensaries/kansas-city-cannabis-company-kearney</loc>",
    "<loc>https://weedmaps.com/dispensaries/rgreenleaf-santa-fe</loc>",
    "<loc>https://weedmaps.com/dispensaries/spiritleaf-cross-roads</loc>",
    "<loc>https://weedmaps.com/dispensaries/sallisaw-natural-health</loc>",
    "<loc>https://weedmaps.com/dispensaries/pure-new-baltimore-recreational-medical</loc>",
    "<loc>https://weedmaps.com/dispensaries/flavor-chasers-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/gingers-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/plug-canna6is</loc>",
    "<loc>https://weedmaps.com/dispensaries/lux-leaf-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/fika-local-etobicoke</loc>",
    "<loc>https://weedmaps.com/dispensaries/great-northern-cannabis-tudor</loc>",
    "<loc>https://weedmaps.com/dispensaries/traditional-pico</loc>",
    "<loc>https://weedmaps.com/dispensaries/wild-willys-bud-pharmacy-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/bloom-medicinals-cape-girardeau</loc>",
    "<loc>https://weedmaps.com/dispensaries/plug-canna6is-bloor</loc>",
    "<loc>https://weedmaps.com/dispensaries/hazy-hill-farm</loc>",
    "<loc>https://weedmaps.com/dispensaries/frontier-farms-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/white-flower-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/stones-throw-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/unity-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/vatic-cannabis-co-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-haze-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-society-9</loc>",
    "<loc>https://weedmaps.com/dispensaries/koli-cannabis-miami</loc>",
    "<loc>https://weedmaps.com/dispensaries/lemonnade-denver-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/offside-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/bluebird-cannabis-roydon-pl</loc>",
    "<loc>https://weedmaps.com/dispensaries/easy-going</loc>",
    "<loc>https://weedmaps.com/dispensaries/marquis-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/smileys-cannabis-nepean</loc>",
    "<loc>https://weedmaps.com/dispensaries/local-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/natural-mystic-cannabis-cbd</loc>",
    "<loc>https://weedmaps.com/dispensaries/zen-leaf-west-chester</loc>",
    "<loc>https://weedmaps.com/dispensaries/runway-pot-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/enroute-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-rise-wellness-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/207-hydro-farm</loc>",
    "<loc>https://weedmaps.com/dispensaries/boxcar-bud-supply</loc>",
    "<loc>https://weedmaps.com/dispensaries/backpack-boyz-san-diego</loc>",
    "<loc>https://weedmaps.com/dispensaries/spirtleaf-sandalwood</loc>",
    "<loc>https://weedmaps.com/dispensaries/elevate-downtown</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/piffingtons-cannabis-co-kitchener</loc>",
    "<loc>https://weedmaps.com/dispensaries/miss-jones-cannabis-val-caron-outpost</loc>",
    "<loc>https://weedmaps.com/dispensaries/elevated-minds-stoney-creek</loc>",
    "<loc>https://weedmaps.com/dispensaries/craft-cannabis-company-moore</loc>",
    "<loc>https://weedmaps.com/dispensaries/canuvo-recreational-retail</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-wayne</loc>",
    "<loc>https://weedmaps.com/dispensaries/greener-grass-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-cabana-castleridge</loc>",
    "<loc>https://weedmaps.com/dispensaries/uplift-cannabis-angus</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-cabana-prince-of-whales</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-cabana-cornwall</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-cabana-windsor</loc>",
    "<loc>https://weedmaps.com/dispensaries/cloud-cannabis-6</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-cabana-victoria-ave</loc>",
    "<loc>https://weedmaps.com/dispensaries/bellebud</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-cabana-ancaster</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-cabana-east-hills</loc>",
    "<loc>https://weedmaps.com/dispensaries/square-grouper</loc>",
    "<loc>https://weedmaps.com/dispensaries/best-buds-hatillo</loc>",
    "<loc>https://weedmaps.com/dispensaries/big-mountain-botanicals-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/hempnotize-llc-dba-glass-jar-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/vip-cannabis-company-union-st-new-hamburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-cannabis-cabin</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-sticky-palace-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/clouded-on-locke</loc>",
    "<loc>https://weedmaps.com/dispensaries/smokey-s-107-st-nw</loc>",
    "<loc>https://weedmaps.com/dispensaries/social-cannabliss-inc-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/smokey-s-cartmell-place</loc>",
    "<loc>https://weedmaps.com/dispensaries/march-and-ash-6</loc>",
    "<loc>https://weedmaps.com/dispensaries/your-local-cannabis-trenton</loc>",
    "<loc>https://weedmaps.com/dispensaries/pacific-sea-weed-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/freedom-cannabis-dispensary-sulphur</loc>",
    "<loc>https://weedmaps.com/dispensaries/catalyst-marina</loc>",
    "<loc>https://weedmaps.com/dispensaries/budology</loc>",
    "<loc>https://weedmaps.com/dispensaries/euphoros</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-day-farm-kansas-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/everest-cannabis-co-unser</loc>",
    "<loc>https://weedmaps.com/dispensaries/everest-cannabis-co-las-cruces</loc>",
    "<loc>https://weedmaps.com/dispensaries/deep-roots-harvest-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/vibes-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/sessions-cannabis-kitchener-highland</loc>",
    "<loc>https://weedmaps.com/dispensaries/atlas-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/twist-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/hippos-marijuana-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/piffingtons-cannabis-co-milton</loc>",
    "<loc>https://weedmaps.com/dispensaries/purlife-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/purlife-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/purlife-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-cupboard-hamilton</loc>",
    "<loc>https://weedmaps.com/dispensaries/purple-moon-dispensary-sapulpa</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-we-store-huron-church-rd</loc>",
    "<loc>https://weedmaps.com/dispensaries/fika-local-herbal-goods-fairview-mall</loc>",
    "<loc>https://weedmaps.com/dispensaries/fika-local-herbal-goods-fairview-park-mall</loc>",
    "<loc>https://weedmaps.com/dispensaries/jailhouse-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/boondom-essex</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenwork-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-market-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/dank-s-wellness-emporium-robinson-ave</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-state-college</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-erie</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabist-williamstown</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-dispensaries-reno</loc>",
    "<loc>https://weedmaps.com/dispensaries/elevation-2150-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/exotix</loc>",
    "<loc>https://weedmaps.com/dispensaries/royal-leafs-waterdown</loc>",
    "<loc>https://weedmaps.com/dispensaries/leaf-life-cannabis-32-street</loc>",
    "<loc>https://weedmaps.com/dispensaries/urb-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/gage-kalamazoo</loc>",
    "<loc>https://weedmaps.com/dispensaries/kannabuena</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-cannabis-dispensary-pensacola</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannalabs-usa-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-oz-store-morrisburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/morgan-road-rx</loc>",
    "<loc>https://weedmaps.com/dispensaries/information-entropy-downtown</loc>",
    "<loc>https://weedmaps.com/dispensaries/hierba-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/puff-los-angeles</loc>",
    "<loc>https://weedmaps.com/dispensaries/groove</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-weed-store-regina</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-gateway-plaza</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-strachan-corner</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-dispensaries-chelsea</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-at-deerfoot-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-southbank-centre</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-edson</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-the-meadows</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-at-summerwood</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-summerside</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-nw-landing</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-country-hills</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-17th-avenue</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-sunridge</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-downtown-south-red-deer</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-griesbach</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-lloydminster</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-vegreville</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-vermilion</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-woodbine</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-hamptons</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-riverbend</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-forest-lawn</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-4th-ave</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-high-river</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-southridge</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-village-landing</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-village-mall</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-grande-prairie</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-beacon-hill</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-stony-plain-east</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-meyokumin</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-railtown</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-stony-plain-west</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-thickwood-mall</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-willow-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-grove-landing</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-southpointe</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-herbary-ottawa</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-namao</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-off-whyte</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-rabbit-hill-road</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-herbary-kingston</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-glenmore-landing</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-herbary-london</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-drumheller</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-deerfoot-meadows</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-riverbend-square</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-nisku</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-herbary-nepean</loc>",
    "<loc>https://weedmaps.com/dispensaries/on-the-cannabis-side</loc>",
    "<loc>https://weedmaps.com/dispensaries/gras-cannabis-collection</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-goods-baltimore</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-lansdowne</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-queen-street-east-riverside</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-appleby</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-king-street-hamilton</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-bloor-street</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-maple-mews</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-queen-street-west</loc>",
    "<loc>https://weedmaps.com/dispensaries/wellgreens-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-katz-korner</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-wasaga-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-meaford</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-angus</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-niagara-portage</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-bramalea-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-danforth</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-p410-steeles</loc>",
    "<loc>https://weedmaps.com/dispensaries/sweet-tree-cannabis-co-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/dispo-camden</loc>",
    "<loc>https://weedmaps.com/dispensaries/american-cannabis-company-nw-122nd-st</loc>",
    "<loc>https://weedmaps.com/dispensaries/happy-leaf-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/method-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-dreams-6</loc>",
    "<loc>https://weedmaps.com/dispensaries/bud-s-cannabis-store-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/urbn-roots-south</loc>",
    "<loc>https://weedmaps.com/dispensaries/legacy-kush</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbaceous-inc-big-sky</loc>",
    "<loc>https://weedmaps.com/dispensaries/spark-cannabis-aurora</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-reef-georgetown</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabist-virginia-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-chemistry-lynn</loc>",
    "<loc>https://weedmaps.com/dispensaries/bloom-medicinals-springfield</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-king-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-cured-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-leaf-calgary</loc>",
    "<loc>https://weedmaps.com/dispensaries/flowertopia-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/oleans</loc>",
    "<loc>https://weedmaps.com/dispensaries/legendary-organics</loc>",
    "<loc>https://weedmaps.com/dispensaries/emerald-fire-meds-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/tulsa-treehouse-broken-arrow</loc>",
    "<loc>https://weedmaps.com/dispensaries/groove-cannabis-co-south-lancaster</loc>",
    "<loc>https://weedmaps.com/dispensaries/quest-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/omkara-cannabis-centre-st</loc>",
    "<loc>https://weedmaps.com/dispensaries/humboldt-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-cannabis-depot-pueblo-west</loc>",
    "<loc>https://weedmaps.com/dispensaries/nuleef-cannabis-toronto</loc>",
    "<loc>https://weedmaps.com/dispensaries/doc-s-craft-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/crossroads-cannabis-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/famous-flower-hamilton-rd</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-happy-spliff-toronto</loc>",
    "<loc>https://weedmaps.com/dispensaries/tierra-pura</loc>",
    "<loc>https://weedmaps.com/dispensaries/botiquin</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-xpress-hurontario</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-stoned-moose</loc>",
    "<loc>https://weedmaps.com/dispensaries/famous-flower-southdale-rd</loc>",
    "<loc>https://weedmaps.com/dispensaries/olympus-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/lemonnade</loc>",
    "<loc>https://weedmaps.com/dispensaries/off-the-charts-hollywood</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-true-dope-cannabis-shop</loc>",
    "<loc>https://weedmaps.com/dispensaries/pegasus-green-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/munchies-cannabis-ottawa</loc>",
    "<loc>https://weedmaps.com/dispensaries/union-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/alpine-essentials</loc>",
    "<loc>https://weedmaps.com/dispensaries/ultra-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/xclusives-taste-retail</loc>",
    "<loc>https://weedmaps.com/dispensaries/go-green-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-fire-station-cannabis-co-sault-ste-marie</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-refinery-detroit</loc>",
    "<loc>https://weedmaps.com/dispensaries/cookies-toronto</loc>",
    "<loc>https://weedmaps.com/dispensaries/moonrock-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/roots-2-harvest</loc>",
    "<loc>https://weedmaps.com/dispensaries/bridge-bud-supply-north</loc>",
    "<loc>https://weedmaps.com/dispensaries/apothecary-farms-moore</loc>",
    "<loc>https://weedmaps.com/dispensaries/forbidden-cannabis-club-mount-vernon</loc>",
    "<loc>https://weedmaps.com/dispensaries/big-bamboo-cannabis-ottawa</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-wellness-bryn-mawr</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-wellness-montgomeryville</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-wellness-gibsonia</loc>",
    "<loc>https://weedmaps.com/dispensaries/tribal-nation-flower-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/edmonton-cannabis-edibles</loc>",
    "<loc>https://weedmaps.com/dispensaries/haze-buds-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/bodega-boyz</loc>",
    "<loc>https://weedmaps.com/dispensaries/cloud-nine-london</loc>",
    "<loc>https://weedmaps.com/dispensaries/sparkle-cannabis-burlington</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-purple-dragon-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/smoke-society-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/happy-little-trees</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-syndicate-long-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-dispensary-colorado-springs-north</loc>",
    "<loc>https://weedmaps.com/dispensaries/tree-house-craft-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/purple-tree-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/apothecary-farms-downtown</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-we-store</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-miami-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/roll-n-rock-cannabis-brampton</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-cannabis-dispensary-ambler</loc>",
    "<loc>https://weedmaps.com/dispensaries/remedy-917</loc>",
    "<loc>https://weedmaps.com/dispensaries/bluebird-cannabis-winchester</loc>",
    "<loc>https://weedmaps.com/dispensaries/codes-jackson</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-source-cannabis-lethbridge</loc>",
    "<loc>https://weedmaps.com/dispensaries/dubway-dispensary-llc-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/heat-herb-cannabis-nanaimo</loc>",
    "<loc>https://weedmaps.com/dispensaries/salmo-cannabis-corporation</loc>",
    "<loc>https://weedmaps.com/dispensaries/welcome-cannabis-morriston</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-greensburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/urban-wellness-98th-street</loc>",
    "<loc>https://weedmaps.com/dispensaries/urban-wellness-academy</loc>",
    "<loc>https://weedmaps.com/dispensaries/sessions-cannabis-keswick</loc>",
    "<loc>https://weedmaps.com/dispensaries/sessions-cannabis-spadina-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/dispensary-90909090913</loc>",
    "<loc>https://weedmaps.com/dispensaries/element-7-port-hueneme</loc>",
    "<loc>https://weedmaps.com/dispensaries/chronicles-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/ogden-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/catalyst-indio</loc>",
    "<loc>https://weedmaps.com/dispensaries/codes-lake-saint-louis</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-apple-cannabis-burks-falls</loc>",
    "<loc>https://weedmaps.com/dispensaries/faded-flamingo-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/leaf-890</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-eden</loc>",
    "<loc>https://weedmaps.com/dispensaries/local-products-the-cannabis-store</loc>",
    "<loc>https://weedmaps.com/dispensaries/full-power</loc>",
    "<loc>https://weedmaps.com/dispensaries/sessions-cannabis-nickel-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/roll-n-rock-cannabis-timmins</loc>",
    "<loc>https://weedmaps.com/dispensaries/bud-brothers-coughy-shop-6</loc>",
    "<loc>https://weedmaps.com/dispensaries/sessions-cannabis-london</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-bad-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/puff-oscoada-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/black-cat-cannabis-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/solar-cannabis-seekonk</loc>",
    "<loc>https://weedmaps.com/dispensaries/love-shack-by-sparc-cannabis-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/power-plant-montana</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-hippies-next-door</loc>",
    "<loc>https://weedmaps.com/dispensaries/embarc-fairfield</loc>",
    "<loc>https://weedmaps.com/dispensaries/slocal-roots</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-jointme</loc>",
    "<loc>https://weedmaps.com/dispensaries/valley-health-center-collective</loc>",
    "<loc>https://weedmaps.com/dispensaries/canada-buds-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/canada-buds-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-horsham</loc>",
    "<loc>https://weedmaps.com/dispensaries/mountain-medicine-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-48th-street</loc>",
    "<loc>https://weedmaps.com/dispensaries/ultra-health-deming</loc>",
    "<loc>https://weedmaps.com/dispensaries/liberty-health-sciences-pinellas-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/liberty-health-sciences-eustis</loc>",
    "<loc>https://weedmaps.com/dispensaries/sessions-stoney-creek</loc>",
    "<loc>https://weedmaps.com/dispensaries/imperial-connection</loc>",
    "<loc>https://weedmaps.com/dispensaries/zen-leaf-evanston</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-bordentown</loc>",
    "<loc>https://weedmaps.com/dispensaries/silver-therapeutics-portland-me</loc>",
    "<loc>https://weedmaps.com/dispensaries/pjs-adventures-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/nature-s-medicines-emmett-township</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-westmont</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-olympia-heights</loc>",
    "<loc>https://weedmaps.com/dispensaries/calyx-trichomes-highway-15</loc>",
    "<loc>https://weedmaps.com/dispensaries/quality-roots-monroe</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenlight-district-london</loc>",
    "<loc>https://weedmaps.com/dispensaries/spot420-the-cannabis-store-erin</loc>",
    "<loc>https://weedmaps.com/dispensaries/spot420-the-cannabis-store-bowmanville</loc>",
    "<loc>https://weedmaps.com/dispensaries/harvest-hoc-of-phoenix-alhambra</loc>",
    "<loc>https://weedmaps.com/dispensaries/harvest-hoc-of-phoenix-north-mountain</loc>",
    "<loc>https://weedmaps.com/dispensaries/harvest-hoc-of-scottsdale</loc>",
    "<loc>https://weedmaps.com/dispensaries/buzzed-buds-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/lotus-gold-waterloo</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-philadelphia-center-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/mystic-s-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/rural-buds</loc>",
    "<loc>https://weedmaps.com/dispensaries/rural-buds-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/thc-canada-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-medical-marijuana-lynchburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/jar-cannabis-co-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/3fifteen-primo-florissant</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-supply-co-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/mid-west-grass-station-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-medical-marijuana-christiansburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/buzzed-buds-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/landrace-cannabis-co-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-supply-co-brantford</loc>",
    "<loc>https://weedmaps.com/dispensaries/tlc-s-the-hideaway</loc>",
    "<loc>https://weedmaps.com/dispensaries/everest-cannabis-co-santa-fe</loc>",
    "<loc>https://weedmaps.com/dispensaries/everest-cannabis-co-montano-plaza</loc>",
    "<loc>https://weedmaps.com/dispensaries/msc3-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-cannabis-dispensary-montgomeryville</loc>",
    "<loc>https://weedmaps.com/dispensaries/mint-cannabis-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/liberty-oxon-hill</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-day-farm-rolla</loc>",
    "<loc>https://weedmaps.com/dispensaries/police-and-thieves-harrison</loc>",
    "<loc>https://weedmaps.com/dispensaries/police-and-thieves-colfax</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-coachella-releaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/sweet-releafs-inc-brantford-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/highway-dtla</loc>",
    "<loc>https://weedmaps.com/dispensaries/alaskan-originals</loc>",
    "<loc>https://weedmaps.com/dispensaries/four20-outlet-lethbridge</loc>",
    "<loc>https://weedmaps.com/dispensaries/four20-premium-market-ponoka</loc>",
    "<loc>https://weedmaps.com/dispensaries/hippos-cannabis-springfield</loc>",
    "<loc>https://weedmaps.com/dispensaries/crafted-nugs-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/northwest-cannabis-company-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-day-farm-kennett</loc>",
    "<loc>https://weedmaps.com/dispensaries/joyology-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/ninetails-cannabis-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/timber-cannabis-co-surgis</loc>",
    "<loc>https://weedmaps.com/dispensaries/story-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/star-budz-provisioning-center-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/planted-provisioning-flint</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-rollers-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/cloud-cannabis-co-gaylord</loc>",
    "<loc>https://weedmaps.com/dispensaries/chronicles-cannabis-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/bluebird-cannabis-hawthorne</loc>",
    "<loc>https://weedmaps.com/dispensaries/bloom-medicinals-cameron</loc>",
    "<loc>https://weedmaps.com/dispensaries/element-7-firebaugh</loc>",
    "<loc>https://weedmaps.com/dispensaries/pura-at-exit-one</loc>",
    "<loc>https://weedmaps.com/dispensaries/tree-hill-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/emerald-fire-provisioning-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/zahara-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-syndicate-lake-elsinore-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/eden-rose-dispensary-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/union-station</loc>",
    "<loc>https://weedmaps.com/dispensaries/liv-lansing</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbnjoy-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/vibe-by-california</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-bright-spot</loc>",
    "<loc>https://weedmaps.com/dispensaries/purlife-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/purlife-6</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-xpress-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/atomic-chronic-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-grass-hut-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/medicated-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-pharm</loc>",
    "<loc>https://weedmaps.com/dispensaries/purple-moose-cannabis-1383-lawrence-ave-w-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/purple-moose-cannabis-5984-bathurst-st</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-dispensary-12</loc>",
    "<loc>https://weedmaps.com/dispensaries/hey-bud</loc>",
    "<loc>https://weedmaps.com/dispensaries/dispensario-farmaverde-rexville</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-garden-cannabis-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/crave-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/danks-2-u</loc>",
    "<loc>https://weedmaps.com/dispensaries/canvas-organics</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-day-farm-van-buren</loc>",
    "<loc>https://weedmaps.com/dispensaries/lalaland</loc>",
    "<loc>https://weedmaps.com/dispensaries/ideal-craft-cannabis-worcester</loc>",
    "<loc>https://weedmaps.com/dispensaries/floyds-fine-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/qunubu-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/crown-leaf-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/goodies-ganja-express</loc>",
    "<loc>https://weedmaps.com/dispensaries/rush-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/vintage-smoke-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/mary-j-s-cannabis-russel</loc>",
    "<loc>https://weedmaps.com/dispensaries/mary-j-s-cannabis-campbellford</loc>",
    "<loc>https://weedmaps.com/dispensaries/mary-j-s-cannabis-orleans</loc>",
    "<loc>https://weedmaps.com/dispensaries/mary-j-s-cannabis-gloucester</loc>",
    "<loc>https://weedmaps.com/dispensaries/doc-s-med-grow</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-nisku-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-gordon-street</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-london</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-wanless-centre</loc>",
    "<loc>https://weedmaps.com/dispensaries/value-buds-waterloo</loc>",
    "<loc>https://weedmaps.com/dispensaries/codes-st-louis</loc>",
    "<loc>https://weedmaps.com/dispensaries/codes-sedalia</loc>",
    "<loc>https://weedmaps.com/dispensaries/cape-girardeau</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-highest-choice-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/erth-naturals</loc>",
    "<loc>https://weedmaps.com/dispensaries/jars-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/verilife-state-college</loc>",
    "<loc>https://weedmaps.com/dispensaries/buku-loud-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/olswell-cannabis-co-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/sb-new-heights-llc-dba-the-row-house</loc>",
    "<loc>https://weedmaps.com/dispensaries/collective-billerica</loc>",
    "<loc>https://weedmaps.com/dispensaries/collective-littleton</loc>",
    "<loc>https://weedmaps.com/dispensaries/march-and-ash-7</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-cove-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/fort-cannabis-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-cannabis-dispensary-lady-lake</loc>",
    "<loc>https://weedmaps.com/dispensaries/c3-industries-high-profile</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-boca-raton</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-co-dispensary-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/velvet-cannabis-dispensary-eagle-rock</loc>",
    "<loc>https://weedmaps.com/dispensaries/marketplace</loc>",
    "<loc>https://weedmaps.com/dispensaries/dulce-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/indo-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/flock-flower-farms-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/dreamz</loc>",
    "<loc>https://weedmaps.com/dispensaries/lost-lake</loc>",
    "<loc>https://weedmaps.com/dispensaries/stoned-goat-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/cookies-ann-arbor</loc>",
    "<loc>https://weedmaps.com/dispensaries/proper-cannabis-crestwood</loc>",
    "<loc>https://weedmaps.com/dispensaries/courtside-gardens</loc>",
    "<loc>https://weedmaps.com/dispensaries/cold-city-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-hopes-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/neighborhood-provisions</loc>",
    "<loc>https://weedmaps.com/dispensaries/cloud-cannabis-kalamazoo</loc>",
    "<loc>https://weedmaps.com/dispensaries/cloud-cannabis-new-baltimore</loc>",
    "<loc>https://weedmaps.com/dispensaries/terps-attleboro</loc>",
    "<loc>https://weedmaps.com/dispensaries/jars-centerline</loc>",
    "<loc>https://weedmaps.com/dispensaries/urban-wellness-99</loc>",
    "<loc>https://weedmaps.com/dispensaries/nature-s-medicines-wareham</loc>",
    "<loc>https://weedmaps.com/dispensaries/just-jane</loc>",
    "<loc>https://weedmaps.com/dispensaries/just-jane-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/toronto-cannabis-culture</loc>",
    "<loc>https://weedmaps.com/dispensaries/matchbox-cannabis-rogers-rd</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-end-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/buds-n-more-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/ultra-health-portales-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/broadway-cannabis-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/grasshopper-dispensary-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/native-harvest-guthrie</loc>",
    "<loc>https://weedmaps.com/dispensaries/jungle-boys-san-diego</loc>",
    "<loc>https://weedmaps.com/dispensaries/cryptids</loc>",
    "<loc>https://weedmaps.com/dispensaries/grams-smokey-dreams</loc>",
    "<loc>https://weedmaps.com/dispensaries/cauldron-cannabis-glencoe</loc>",
    "<loc>https://weedmaps.com/dispensaries/endovibe</loc>",
    "<loc>https://weedmaps.com/dispensaries/bloc-dispensary-belton</loc>",
    "<loc>https://weedmaps.com/dispensaries/bloc-dispensary-farmington</loc>",
    "<loc>https://weedmaps.com/dispensaries/bloc-dispensary-kirksville</loc>",
    "<loc>https://weedmaps.com/dispensaries/bloc-michigan</loc>",
    "<loc>https://weedmaps.com/dispensaries/moosehead-farms-llc-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/sun-valley-management-llc-dba-chouteau-house-of-fire</loc>",
    "<loc>https://weedmaps.com/dispensaries/kush-house-24hr-dispensary-drive-thru-moore</loc>",
    "<loc>https://weedmaps.com/dispensaries/king-green-industries-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/purlife-bernalillo</loc>",
    "<loc>https://weedmaps.com/dispensaries/great-lakes-exotics-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/harvest-hoc-of-chandler</loc>",
    "<loc>https://weedmaps.com/dispensaries/harvest-hoc-of-glendale</loc>",
    "<loc>https://weedmaps.com/dispensaries/harvest-hoc-of-cottonwood</loc>",
    "<loc>https://weedmaps.com/dispensaries/harvest-hoc-of-tucson-menlo-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/harvest-hoc-of-tempe</loc>",
    "<loc>https://weedmaps.com/dispensaries/harvest-hoc-of-avondale</loc>",
    "<loc>https://weedmaps.com/dispensaries/harvest-hoc-of-baseline</loc>",
    "<loc>https://weedmaps.com/dispensaries/harvest-hoc-of-casa-grande</loc>",
    "<loc>https://weedmaps.com/dispensaries/harvest-hoc-of-north-mesa</loc>",
    "<loc>https://weedmaps.com/dispensaries/harvest-hoc-of-peoria</loc>",
    "<loc>https://weedmaps.com/dispensaries/harvest-hoc-of-tucson-blenman-elm</loc>",
    "<loc>https://weedmaps.com/dispensaries/bloc-dispensary-richmond-heights</loc>",
    "<loc>https://weedmaps.com/dispensaries/sapura-coldwater</loc>",
    "<loc>https://weedmaps.com/dispensaries/d-seofdope</loc>",
    "<loc>https://weedmaps.com/dispensaries/vana-society</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-dime-on-wortley-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/connoisseur-culture</loc>",
    "<loc>https://weedmaps.com/dispensaries/pakalolo-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-grove-27</loc>",
    "<loc>https://weedmaps.com/dispensaries/mango-cannabis-okc-nw-expy</loc>",
    "<loc>https://weedmaps.com/dispensaries/honey-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/spiritleaf-kingston-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/liv-cannabis-lake-orion</loc>",
    "<loc>https://weedmaps.com/dispensaries/verve-health-shop</loc>",
    "<loc>https://weedmaps.com/dispensaries/plants-over-pills-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/earth-keeper-cannabis-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/vana-society-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/hi-roots-cannabis-co-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/urbn-leaf-grossmont</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-dispensary-on-66</loc>",
    "<loc>https://weedmaps.com/dispensaries/jars-saugatuck</loc>",
    "<loc>https://weedmaps.com/dispensaries/rustic-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/vip-cannabis-company-mitchell</loc>",
    "<loc>https://weedmaps.com/dispensaries/kana-leaf-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/leaf-embers</loc>",
    "<loc>https://weedmaps.com/dispensaries/sweet-flower-pasadena</loc>",
    "<loc>https://weedmaps.com/dispensaries/oso-cannabis-company-6</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-orlando-millenia-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/chasing-wolves-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-north-miami-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-wellflower-ypsilanti</loc>",
    "<loc>https://weedmaps.com/dispensaries/brothers-cannabis-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-grass-medical-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/tradecraft-farms-el-monte</loc>",
    "<loc>https://weedmaps.com/dispensaries/holy-cannabis-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-pharm-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/top-shelf-stock-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/colfax-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/breaking-buds-cannabis-markham-rd</loc>",
    "<loc>https://weedmaps.com/dispensaries/breaking-buds-cannabis-eglinton-e</loc>",
    "<loc>https://weedmaps.com/dispensaries/orange-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/better-science-calle-loiza-san-juan</loc>",
    "<loc>https://weedmaps.com/dispensaries/skunked</loc>",
    "<loc>https://weedmaps.com/dispensaries/stiiizy-santa-rosa</loc>",
    "<loc>https://weedmaps.com/dispensaries/goat-global-dtla</loc>",
    "<loc>https://weedmaps.com/dispensaries/avenue</loc>",
    "<loc>https://weedmaps.com/dispensaries/dr-greenthumb-lax</loc>",
    "<loc>https://weedmaps.com/dispensaries/coachella-greenery-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/matchbox-cannabis-queen-st</loc>",
    "<loc>https://weedmaps.com/dispensaries/matchbox-cannabis-steeles-ave</loc>",
    "<loc>https://weedmaps.com/dispensaries/matchbox-cannabis-second-line-w</loc>",
    "<loc>https://weedmaps.com/dispensaries/vip-cannabis-company-cambridge</loc>",
    "<loc>https://weedmaps.com/dispensaries/vip-cannabis-company-harriston</loc>",
    "<loc>https://weedmaps.com/dispensaries/cloud-621</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-great-atlantic-puffin-company-adult-use</loc>",
    "<loc>https://weedmaps.com/dispensaries/mystic-s-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/sage-and-fire</loc>",
    "<loc>https://weedmaps.com/dispensaries/higher-health-of-miami</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-dispensaries-mankato</loc>",
    "<loc>https://weedmaps.com/dispensaries/spiritleaf-sherway</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-fire-station-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/tree-of-life-dispensary-las-vegas</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-morgantown-granville</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-weston</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-fort-myers-summerlin</loc>",
    "<loc>https://weedmaps.com/dispensaries/deep-roots-harvest-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/blaze1-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/kamsack-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/stellar-bud-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-morgantown</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-zephyrhills</loc>",
    "<loc>https://weedmaps.com/dispensaries/mayflower-lowell</loc>",
    "<loc>https://weedmaps.com/dispensaries/pirate-town</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-syndicate-northridge</loc>",
    "<loc>https://weedmaps.com/dispensaries/worner-btothers-outpost</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-xpress-6</loc>",
    "<loc>https://weedmaps.com/dispensaries/planet-jane</loc>",
    "<loc>https://weedmaps.com/dispensaries/holland-daze-pickering</loc>",
    "<loc>https://weedmaps.com/dispensaries/fabulous-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/holland-daze-wasaga</loc>",
    "<loc>https://weedmaps.com/dispensaries/bract-house</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-door-watervliet</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-set-reseda</loc>",
    "<loc>https://weedmaps.com/dispensaries/electric-lettuce-sellwood</loc>",
    "<loc>https://weedmaps.com/dispensaries/lemon-cannabis-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/rosebuds-cannabis-co-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/devine</loc>",
    "<loc>https://weedmaps.com/dispensaries/catalyst-hemet</loc>",
    "<loc>https://weedmaps.com/dispensaries/city-cannabis-co-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/just-baked</loc>",
    "<loc>https://weedmaps.com/dispensaries/big-daddy-s-gas-company-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/kind-love-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/everest-cannabis-co-sunland-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-jive-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/earl-and-tom-s</loc>",
    "<loc>https://weedmaps.com/dispensaries/quality-roots-corunna</loc>",
    "<loc>https://weedmaps.com/dispensaries/delta-9-exotics</loc>",
    "<loc>https://weedmaps.com/dispensaries/blue-sky-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/downtown-dispensary-tucumcari</loc>",
    "<loc>https://weedmaps.com/dispensaries/dank-bank-28</loc>",
    "<loc>https://weedmaps.com/dispensaries/higherground-san-bernardino</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-back-bay</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-watertown</loc>",
    "<loc>https://weedmaps.com/dispensaries/miss-jones-cannabis-outposts-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/dna-cannabis-solutions</loc>",
    "<loc>https://weedmaps.com/dispensaries/pecos-valley-production-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-pharm-ossineke</loc>",
    "<loc>https://weedmaps.com/dispensaries/mpx-nj</loc>",
    "<loc>https://weedmaps.com/dispensaries/dank-banq-owasso</loc>",
    "<loc>https://weedmaps.com/dispensaries/social-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/burns-cannabis-experience</loc>",
    "<loc>https://weedmaps.com/dispensaries/old-g-kush</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-dragon-west-palm-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/northern-leaf-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/alta-vida</loc>",
    "<loc>https://weedmaps.com/dispensaries/elevate-dispensary-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/hazy-moose</loc>",
    "<loc>https://weedmaps.com/dispensaries/lake-life-farms-inc-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/chronzz-cannabis-shop</loc>",
    "<loc>https://weedmaps.com/dispensaries/southwest-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/rootd-in-the-510</loc>",
    "<loc>https://weedmaps.com/dispensaries/hashtag-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/valley-purveyors</loc>",
    "<loc>https://weedmaps.com/dispensaries/icanna</loc>",
    "<loc>https://weedmaps.com/dispensaries/nela</loc>",
    "<loc>https://weedmaps.com/dispensaries/harbor-farmz-provisioning-llc-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/queen-george-cannabis-caroline-st</loc>",
    "<loc>https://weedmaps.com/dispensaries/puffers-place</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenways-provisioning-center-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-allentown</loc>",
    "<loc>https://weedmaps.com/dispensaries/score-420-albuquerque</loc>",
    "<loc>https://weedmaps.com/dispensaries/score-420-alamagordo</loc>",
    "<loc>https://weedmaps.com/dispensaries/xclusive-smith</loc>",
    "<loc>https://weedmaps.com/dispensaries/california-holistics</loc>",
    "<loc>https://weedmaps.com/dispensaries/big-river-cannabis-laurier-st</loc>",
    "<loc>https://weedmaps.com/dispensaries/big-river-cannabis-calabogie</loc>",
    "<loc>https://weedmaps.com/dispensaries/big-river-cannabis-bourget</loc>",
    "<loc>https://weedmaps.com/dispensaries/lemonnade-union-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/maine-only-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/element-7-cathedral-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/amplify-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-door-11</loc>",
    "<loc>https://weedmaps.com/dispensaries/north-country-cannabis-me</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-third-ave-timmins</loc>",
    "<loc>https://weedmaps.com/dispensaries/harbor-country-collective</loc>",
    "<loc>https://weedmaps.com/dispensaries/white-rabbit-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-grass-station-12</loc>",
    "<loc>https://weedmaps.com/dispensaries/kushology-101</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenlight-lewisburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenlight-princeton</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenlight-beckley</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-profile-webberville</loc>",
    "<loc>https://weedmaps.com/dispensaries/enchanted-botanicals-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/just-jane-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/jungle-boys</loc>",
    "<loc>https://weedmaps.com/dispensaries/jungle-boys-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/chillax-glass-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/endo-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-community-store-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabuena</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-farm-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/undertow-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/n8v-grown-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-craig-st-timmins</loc>",
    "<loc>https://weedmaps.com/dispensaries/vertical-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/off-the-charts-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-remedy-express</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenlight-chippewa</loc>",
    "<loc>https://weedmaps.com/dispensaries/alto-medical-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/dispo-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbana-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/endo-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/bongos-lifestyles</loc>",
    "<loc>https://weedmaps.com/dispensaries/dreams-canna</loc>",
    "<loc>https://weedmaps.com/dispensaries/origins-east</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-park-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-payson</loc>",
    "<loc>https://weedmaps.com/dispensaries/a-little-bud-duncan</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-5-cannabis-stevenson</loc>",
    "<loc>https://weedmaps.com/dispensaries/bulldog-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/element-7-willits</loc>",
    "<loc>https://weedmaps.com/dispensaries/element-7-mendota</loc>",
    "<loc>https://weedmaps.com/dispensaries/c-shop-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/one-stop-smoke-shop</loc>",
    "<loc>https://weedmaps.com/dispensaries/galileo-cannabis-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/northern-tokes</loc>",
    "<loc>https://weedmaps.com/dispensaries/stiiizy-western</loc>",
    "<loc>https://weedmaps.com/dispensaries/cure-co-crenshaw</loc>",
    "<loc>https://weedmaps.com/dispensaries/garden-mother</loc>",
    "<loc>https://weedmaps.com/dispensaries/leaf-flower</loc>",
    "<loc>https://weedmaps.com/dispensaries/leaf-flower-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/leaf-flower-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/american-cannabis-company-12</loc>",
    "<loc>https://weedmaps.com/dispensaries/fort-twenty-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/ashario-aurora</loc>",
    "<loc>https://weedmaps.com/dispensaries/1-connection-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/dedos-verdes-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/rockwood-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/lightsky-farms</loc>",
    "<loc>https://weedmaps.com/dispensaries/mint-cannabis-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/cowboy-kush-dispensary-2-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/reefer-madness-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/black-tie-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/planet-earth-cannabis-bank-st</loc>",
    "<loc>https://weedmaps.com/dispensaries/element-7-chula-vista</loc>",
    "<loc>https://weedmaps.com/dispensaries/mad-leaf-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/cookies-napa</loc>",
    "<loc>https://weedmaps.com/dispensaries/north-beach-pipeline</loc>",
    "<loc>https://weedmaps.com/dispensaries/oso-cannabis-company-8</loc>",
    "<loc>https://weedmaps.com/dispensaries/okie-wonderland-broken-arrow-coweta</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenlight-stateline</loc>",
    "<loc>https://weedmaps.com/dispensaries/mr-nice-guy-9929355</loc>",
    "<loc>https://weedmaps.com/dispensaries/organ-mountain-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/jars-monroe</loc>",
    "<loc>https://weedmaps.com/dispensaries/blaze-n-bake-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/dowatiwant-industries</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-mcnaughton-ave-chatham</loc>",
    "<loc>https://weedmaps.com/dispensaries/elevate-northwest</loc>",
    "<loc>https://weedmaps.com/dispensaries/goldn-bloom-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/piffingtons-cannabis-co-brampton-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-apopka</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-coral-springs</loc>",
    "<loc>https://weedmaps.com/dispensaries/heads-monroe-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-hollywood</loc>",
    "<loc>https://weedmaps.com/dispensaries/sinsemilla-gardiner</loc>",
    "<loc>https://weedmaps.com/dispensaries/goat-house-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/puff-on-66</loc>",
    "<loc>https://weedmaps.com/dispensaries/edmond-green-escaoe</loc>",
    "<loc>https://weedmaps.com/dispensaries/skybox-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-curious-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/northern-tokes-scarborough</loc>",
    "<loc>https://weedmaps.com/dispensaries/elite-61</loc>",
    "<loc>https://weedmaps.com/dispensaries/embarc-fresno</loc>",
    "<loc>https://weedmaps.com/dispensaries/cerenibis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/river-driver-cannabis-co-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/sticky-muskegon</loc>",
    "<loc>https://weedmaps.com/dispensaries/embr-lake-elsinore</loc>",
    "<loc>https://weedmaps.com/dispensaries/puff-cannabis-company-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/garden-of-eden-6</loc>",
    "<loc>https://weedmaps.com/dispensaries/pop-s-cannabis-kapuskasing</loc>",
    "<loc>https://weedmaps.com/dispensaries/flamingo-plus</loc>",
    "<loc>https://weedmaps.com/dispensaries/flamingo-cannabis-st-james-street</loc>",
    "<loc>https://weedmaps.com/dispensaries/flamingo-cannabis-pembina-hwy</loc>",
    "<loc>https://weedmaps.com/dispensaries/flamingo-cannabis-corydon-ave</loc>",
    "<loc>https://weedmaps.com/dispensaries/deep-sky-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-well</loc>",
    "<loc>https://weedmaps.com/dispensaries/lolly-brantford</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-leaf-kalamazoo</loc>",
    "<loc>https://weedmaps.com/dispensaries/pop-s-cannabis-penetanguishene</loc>",
    "<loc>https://weedmaps.com/dispensaries/pop-s-cannabis-hanmer</loc>",
    "<loc>https://weedmaps.com/dispensaries/higher-level-clovis</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-artist-tree-fresno</loc>",
    "<loc>https://weedmaps.com/dispensaries/far-and-dotter</loc>",
    "<loc>https://weedmaps.com/dispensaries/bonnefire-brighton</loc>",
    "<loc>https://weedmaps.com/dispensaries/bonnefire-munro-st</loc>",
    "<loc>https://weedmaps.com/dispensaries/bonnefire-orillia</loc>",
    "<loc>https://weedmaps.com/dispensaries/vip-cannabis-company-elmira</loc>",
    "<loc>https://weedmaps.com/dispensaries/vip-cannabis-company-chesley</loc>",
    "<loc>https://weedmaps.com/dispensaries/vip-cannabis-company-shakespeare</loc>",
    "<loc>https://weedmaps.com/dispensaries/local-cannabis-co-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/local-cannabis-co-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/local-cannabis-co-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-xpress-8</loc>",
    "<loc>https://weedmaps.com/dispensaries/rollin-love</loc>",
    "<loc>https://weedmaps.com/dispensaries/pop-s-cannabis-brights-grove</loc>",
    "<loc>https://weedmaps.com/dispensaries/pop-s-cannabis-mount-forest</loc>",
    "<loc>https://weedmaps.com/dispensaries/pop-s-cannabis-almonte</loc>",
    "<loc>https://weedmaps.com/dispensaries/pop-s-cannabis-smiths-falls</loc>",
    "<loc>https://weedmaps.com/dispensaries/ganja-garden</loc>",
    "<loc>https://weedmaps.com/dispensaries/pop-s-cannabis-new-liskeard</loc>",
    "<loc>https://weedmaps.com/dispensaries/pop-s-cannabis-hawkesbury</loc>",
    "<loc>https://weedmaps.com/dispensaries/pop-s-cannabis-1-tottenham</loc>",
    "<loc>https://weedmaps.com/dispensaries/pop-s-cannabis-big-bay-point</loc>",
    "<loc>https://weedmaps.com/dispensaries/munchiez-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/pop-s-cannabis-courtice</loc>",
    "<loc>https://weedmaps.com/dispensaries/nirvana-center-monroe</loc>",
    "<loc>https://weedmaps.com/dispensaries/oasis-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/oasis-cannabis-co-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/oasis-cannabis-co-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/mango-cannabis-tulsa-harvard-ave</loc>",
    "<loc>https://weedmaps.com/dispensaries/bonsai-cannabis-collection</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-joint-cannabis-kenaston-blvd</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-joint-cannabis-sage-creek-blvd</loc>",
    "<loc>https://weedmaps.com/dispensaries/honey-grove-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/confluence-cannabis-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/clear-cut-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/juniper-cannabis-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/dank-cannabis-ogden</loc>",
    "<loc>https://weedmaps.com/dispensaries/dank-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/dank-cannabis-parkdale</loc>",
    "<loc>https://weedmaps.com/dispensaries/dab-cannabis-garson</loc>",
    "<loc>https://weedmaps.com/dispensaries/elevated-exotics-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/nobody-s-home-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/burb-uvic</loc>",
    "<loc>https://weedmaps.com/dispensaries/highq-thamesford</loc>",
    "<loc>https://weedmaps.com/dispensaries/highq-dorchester</loc>",
    "<loc>https://weedmaps.com/dispensaries/alternative-essence-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/kure-eldorado</loc>",
    "<loc>https://weedmaps.com/dispensaries/new-leaf-wv</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenlight-stollings</loc>",
    "<loc>https://weedmaps.com/dispensaries/everest-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/jars-muskegon</loc>",
    "<loc>https://weedmaps.com/dispensaries/budder-store</loc>",
    "<loc>https://weedmaps.com/dispensaries/mystic-s-cannabis-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/weedys</loc>",
    "<loc>https://weedmaps.com/dispensaries/pacabol</loc>",
    "<loc>https://weedmaps.com/dispensaries/oklahoma-s-best-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/phresh-otd</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunny-daze-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/outlaw-herbs-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-rollers-7</loc>",
    "<loc>https://weedmaps.com/dispensaries/downtown-420</loc>",
    "<loc>https://weedmaps.com/dispensaries/spiritleaf-78</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-wellness-watertown-adult-use</loc>",
    "<loc>https://weedmaps.com/dispensaries/fireflower</loc>",
    "<loc>https://weedmaps.com/dispensaries/bkind-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/sky-club-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/mirage-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/ganjco</loc>",
    "<loc>https://weedmaps.com/dispensaries/your-highness-cannabis-london</loc>",
    "<loc>https://weedmaps.com/dispensaries/stoney-high-cannabis-corp</loc>",
    "<loc>https://weedmaps.com/dispensaries/botanik-cannaboutique</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannavida-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/stiiizy-wilmington</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-syndicate-moreno-valley</loc>",
    "<loc>https://weedmaps.com/dispensaries/pecos-valley-production-6</loc>",
    "<loc>https://weedmaps.com/dispensaries/emerald-fields-south-boulder</loc>",
    "<loc>https://weedmaps.com/dispensaries/encanto-tree-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-clinik-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/oasis-420-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/rgl-dispen-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/nature-s-canopy-house-oshawa</loc>",
    "<loc>https://weedmaps.com/dispensaries/nature-s-canopy-house-weston-south</loc>",
    "<loc>https://weedmaps.com/dispensaries/nature-s-canopy-house-mcleod</loc>",
    "<loc>https://weedmaps.com/dispensaries/nature-s-canopy-weston-north</loc>",
    "<loc>https://weedmaps.com/dispensaries/nature-s-canopy-robinson</loc>",
    "<loc>https://weedmaps.com/dispensaries/mood-provisioning-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-420-place</loc>",
    "<loc>https://weedmaps.com/dispensaries/erbn-green-sundre</loc>",
    "<loc>https://weedmaps.com/dispensaries/brixx</loc>",
    "<loc>https://weedmaps.com/dispensaries/yuma-way</loc>",
    "<loc>https://weedmaps.com/dispensaries/t-cannabis-hearst</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-olde-barber-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-society-norman-12th-ave</loc>",
    "<loc>https://weedmaps.com/dispensaries/lonely-bones</loc>",
    "<loc>https://weedmaps.com/dispensaries/pincanna-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/sugar-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/grand-cannabis-tillsonburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/grand-cannabis-fonthill</loc>",
    "<loc>https://weedmaps.com/dispensaries/grand-cannabis-st-catharines-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-cannabis-dispensary-tampa</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-cannabis-dispensary-cape-coral</loc>",
    "<loc>https://weedmaps.com/dispensaries/weedhub-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/stiiizy-pacoima</loc>",
    "<loc>https://weedmaps.com/dispensaries/cookies-miami</loc>",
    "<loc>https://weedmaps.com/dispensaries/remedy-room-bay-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/nature-s-canopy-house-finch</loc>",
    "<loc>https://weedmaps.com/dispensaries/miss-jones-st-marys-outpost</loc>",
    "<loc>https://weedmaps.com/dispensaries/gnomad</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-cured-eliot</loc>",
    "<loc>https://weedmaps.com/dispensaries/core-empowerment-me-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/bazonzoes-8</loc>",
    "<loc>https://weedmaps.com/dispensaries/big-mountain-botanicals-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/american-cannabis-29th-street</loc>",
    "<loc>https://weedmaps.com/dispensaries/star-buds-cannabis-co-6</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-maven-store</loc>",
    "<loc>https://weedmaps.com/dispensaries/drip-md</loc>",
    "<loc>https://weedmaps.com/dispensaries/n8v-grown-cut-bank</loc>",
    "<loc>https://weedmaps.com/dispensaries/pleasantrees-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/elite-cannabis-meridian</loc>",
    "<loc>https://weedmaps.com/dispensaries/hi-cannabis-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/lakewood-cannabis-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-cure-company-melrose</loc>",
    "<loc>https://weedmaps.com/dispensaries/cunningham-s-craft-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/pecos-valley-production-7</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-herbsman</loc>",
    "<loc>https://weedmaps.com/dispensaries/coastal-green-vancouver</loc>",
    "<loc>https://weedmaps.com/dispensaries/missouri-wild-alchemy-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-hopes-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/redbilly-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/northern-budzz</loc>",
    "<loc>https://weedmaps.com/dispensaries/hippos-marijuana-dispensary-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/capeway-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/timber-cannabis-co-three-rivers</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-dude-abides-provisioning-center-sturgis</loc>",
    "<loc>https://weedmaps.com/dispensaries/c-shop-cannabis-embrun</loc>",
    "<loc>https://weedmaps.com/dispensaries/cmc-cannabis-dispensary-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/cookies-brentwood</loc>",
    "<loc>https://weedmaps.com/dispensaries/house-of-evolution</loc>",
    "<loc>https://weedmaps.com/dispensaries/elevate-south-los-angeles</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-health-clinic</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-valley-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/gelato-retail</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-grove-cannabis-erin</loc>",
    "<loc>https://weedmaps.com/dispensaries/treehouse-605</loc>",
    "<loc>https://weedmaps.com/dispensaries/black-hills-weed-shop</loc>",
    "<loc>https://weedmaps.com/dispensaries/weedjar-eglinton-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/bill-s-herbal-care</loc>",
    "<loc>https://weedmaps.com/dispensaries/sky-leaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/caliwood-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/rose-mary-jane-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/brothers-cannabis-broadway</loc>",
    "<loc>https://weedmaps.com/dispensaries/colorado-cannabis-exchange-ii</loc>",
    "<loc>https://weedmaps.com/dispensaries/lucky-pharms</loc>",
    "<loc>https://weedmaps.com/dispensaries/kijani-corp-dba-hemp-heaven</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-land-tulsa</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabyss-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/off-the-charts-now-open</loc>",
    "<loc>https://weedmaps.com/dispensaries/magnolia-road-cannabis-co-broomfield</loc>",
    "<loc>https://weedmaps.com/dispensaries/dispensary-of-watertown</loc>",
    "<loc>https://weedmaps.com/dispensaries/hybrid-relief-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/ganja-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-dragon-ocala</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-bradenton-hwy-64</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-englewood</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-hallandale</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-103rd-st</loc>",
    "<loc>https://weedmaps.com/dispensaries/catalyst-normandie</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-largo</loc>",
    "<loc>https://weedmaps.com/dispensaries/flathead-med</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-miami-152nd</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-cake-house-battle-creek</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-orlando-university</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-panama-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/above-all-greenery-north-coming-soon</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-wellflower-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/mint-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/bodega-boyz-91st</loc>",
    "<loc>https://weedmaps.com/dispensaries/cal-green-medical-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-south-tampa</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-spring-hill</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-stuart</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-tamarac</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-midtown-dale-mabry</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-tampa-dale-mabry</loc>",
    "<loc>https://weedmaps.com/dispensaries/country-grown-cannabis-martinsburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-wesley-chapel</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-city-avenue</loc>",
    "<loc>https://weedmaps.com/dispensaries/ultra-health-truth-or-consequences</loc>",
    "<loc>https://weedmaps.com/dispensaries/ultra-health-carlsbad</loc>",
    "<loc>https://weedmaps.com/dispensaries/ultra-health-paseo-del-norte</loc>",
    "<loc>https://weedmaps.com/dispensaries/ultra-health-ruidoso</loc>",
    "<loc>https://weedmaps.com/dispensaries/ultra-health-edgewood</loc>",
    "<loc>https://weedmaps.com/dispensaries/ultra-health-tucumcari</loc>",
    "<loc>https://weedmaps.com/dispensaries/ultra-health-eagle-nest</loc>",
    "<loc>https://weedmaps.com/dispensaries/ultra-health-las-cruces-bataan</loc>",
    "<loc>https://weedmaps.com/dispensaries/ultra-health-sunland-park-express</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-ely</loc>",
    "<loc>https://weedmaps.com/dispensaries/round-house-reserve</loc>",
    "<loc>https://weedmaps.com/dispensaries/ganja-garden-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/fika-local-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/simply-pure-trenton-nj</loc>",
    "<loc>https://weedmaps.com/dispensaries/noxx-28th-street</loc>",
    "<loc>https://weedmaps.com/dispensaries/med-canms</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabist-vineland</loc>",
    "<loc>https://weedmaps.com/dispensaries/everest-cannabis-co-texico</loc>",
    "<loc>https://weedmaps.com/dispensaries/livwell-meds-sulphur</loc>",
    "<loc>https://weedmaps.com/dispensaries/hybrid-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/smoking-crow-cannabis-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-heritage-club-cambridge</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-phoenix-roosevelt-row</loc>",
    "<loc>https://weedmaps.com/dispensaries/sky-leaf-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/best-buds-vega-baja</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-flowery-miami</loc>",
    "<loc>https://weedmaps.com/dispensaries/hometown-provisions-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-cannabis-dispensary-panama-city-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/laughing-leaf-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/pure-remedies</loc>",
    "<loc>https://weedmaps.com/dispensaries/best-buds-pr-caguas</loc>",
    "<loc>https://weedmaps.com/dispensaries/best-buds-pr-ponce</loc>",
    "<loc>https://weedmaps.com/dispensaries/purlife-old-town</loc>",
    "<loc>https://weedmaps.com/dispensaries/trail-s-head-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/cookies-mendocino</loc>",
    "<loc>https://weedmaps.com/dispensaries/fire-island</loc>",
    "<loc>https://weedmaps.com/dispensaries/got-greens</loc>",
    "<loc>https://weedmaps.com/dispensaries/love-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/jade-cannabis-co-sky-pointe</loc>",
    "<loc>https://weedmaps.com/dispensaries/oso-cannabis-company-anthony</loc>",
    "<loc>https://weedmaps.com/dispensaries/mother-earth-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/monster-house-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-cured-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/weeds-11</loc>",
    "<loc>https://weedmaps.com/dispensaries/health-for-life-charleston</loc>",
    "<loc>https://weedmaps.com/dispensaries/420-teatime</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-day-farm-buffalo</loc>",
    "<loc>https://weedmaps.com/dispensaries/atlas-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/jersey-shore-extracts-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/blossom</loc>",
    "<loc>https://weedmaps.com/dispensaries/nomade-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/cleveland-cannabis-comapnay</loc>",
    "<loc>https://weedmaps.com/dispensaries/deming-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/best-buds-dispensary-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/cowboy-verde-clovis</loc>",
    "<loc>https://weedmaps.com/dispensaries/pecos-valley-production-edgewood</loc>",
    "<loc>https://weedmaps.com/dispensaries/flwr-co-corona</loc>",
    "<loc>https://weedmaps.com/dispensaries/deo-s-garden</loc>",
    "<loc>https://weedmaps.com/dispensaries/lucky-s-cannabis-co-muskegon</loc>",
    "<loc>https://weedmaps.com/dispensaries/breakwater-treatment-wellness-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/taboo-cannabis-co-sw</loc>",
    "<loc>https://weedmaps.com/dispensaries/kush-groove</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-fuego-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-magnolia-dispensaries</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-tucson-on-swan</loc>",
    "<loc>https://weedmaps.com/dispensaries/herrera-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/super-fire-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/planet-59</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-cannabis-dispensary-pensacola-nine-mile-road</loc>",
    "<loc>https://weedmaps.com/dispensaries/olympia-weed-company-pacific-ave</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-magnolia-canton</loc>",
    "<loc>https://weedmaps.com/dispensaries/experience-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-brighton-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/phoenix-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/merchant-logo-true-north-cannabis-co-midland-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/smoke-n-body</loc>",
    "<loc>https://weedmaps.com/dispensaries/smoke-n-body-2-0</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-oshawa</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-parry-sound-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/ohm-theory-premium-cannabis-retailer</loc>",
    "<loc>https://weedmaps.com/dispensaries/seed-of-life-labs-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/cmh-holdings-2-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/seed-of-life-labs-7</loc>",
    "<loc>https://weedmaps.com/dispensaries/bluewater-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-belle</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-milton</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-hurricane</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-south-charleston</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-parkersburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/vault-66-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/leaf-lab</loc>",
    "<loc>https://weedmaps.com/dispensaries/caliyield-del-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/sea-to-sky-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/jar-cannabis-co-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/farma-verde-cidra</loc>",
    "<loc>https://weedmaps.com/dispensaries/buzzed-buds-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/stone-age-farmacy</loc>",
    "<loc>https://weedmaps.com/dispensaries/ponte-verde-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/stiiizy-barstow</loc>",
    "<loc>https://weedmaps.com/dispensaries/sweetgrass-ms-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/levels-cannabis-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/salty-cultivation</loc>",
    "<loc>https://weedmaps.com/dispensaries/45-north-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-supply</loc>",
    "<loc>https://weedmaps.com/dispensaries/b-leaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/wildflower-medical-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-scottsdale</loc>",
    "<loc>https://weedmaps.com/dispensaries/tnm-holdings</loc>",
    "<loc>https://weedmaps.com/dispensaries/wacky-jackz-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/spot420-the-cannabis-store-oshawa</loc>",
    "<loc>https://weedmaps.com/dispensaries/starrcanna-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/cardiel-botanica-mcnutt</loc>",
    "<loc>https://weedmaps.com/dispensaries/nirvana-center-marquette</loc>",
    "<loc>https://weedmaps.com/dispensaries/main-street-meds-tulsa</loc>",
    "<loc>https://weedmaps.com/dispensaries/confidential-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/hierba-buena-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/kika-kush</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-goat-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/toker-king</loc>",
    "<loc>https://weedmaps.com/dispensaries/flavors-santa-cruz</loc>",
    "<loc>https://weedmaps.com/dispensaries/southern-cannabis-crew</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-cultures</loc>",
    "<loc>https://weedmaps.com/dispensaries/anicedream</loc>",
    "<loc>https://weedmaps.com/dispensaries/gram-station</loc>",
    "<loc>https://weedmaps.com/dispensaries/happy-harry-s-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/embarc-sacramento</loc>",
    "<loc>https://weedmaps.com/dispensaries/bodega-boyz-71st</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-1159713</loc>",
    "<loc>https://weedmaps.com/dispensaries/pipe-leaf-old-steese</loc>",
    "<loc>https://weedmaps.com/dispensaries/enchanted-strains</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-therapy-dispensary-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/cinder-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-lady-lake</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-ocala-maricamp</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-tampa-citrus-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-auburn-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-lancaster-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/theory-wellness-kittery</loc>",
    "<loc>https://weedmaps.com/dispensaries/noire-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/sovereign-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/roll-models</loc>",
    "<loc>https://weedmaps.com/dispensaries/cowboy-verde-belen</loc>",
    "<loc>https://weedmaps.com/dispensaries/foreign-la</loc>",
    "<loc>https://weedmaps.com/dispensaries/origins-cannabis-company-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/s-h-green-life</loc>",
    "<loc>https://weedmaps.com/dispensaries/purlife-roswell</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-dragon-boynton-beach-east</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-dragon-pensacola</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-dragon-tallahassee-gaines-st</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-dragon-tallahassee-monroe-st</loc>",
    "<loc>https://weedmaps.com/dispensaries/la-flor-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/nuve</loc>",
    "<loc>https://weedmaps.com/dispensaries/altitude-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/sticky-river-rouge</loc>",
    "<loc>https://weedmaps.com/dispensaries/thrive-wellness-annapolis</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-sierra-vista</loc>",
    "<loc>https://weedmaps.com/dispensaries/harvest-hoc-chula-vista</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-dragon-orange-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/puffshaven-yonge-finch</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-society-wellness-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/herban-oasis</loc>",
    "<loc>https://weedmaps.com/dispensaries/king-of-budz-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/bluebird-cannabis-hawkesbury</loc>",
    "<loc>https://weedmaps.com/dispensaries/weedjar-yorkville</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-haven-20-union-adult-use</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-budstop-cannabis-inc-st-mary-s-rd</loc>",
    "<loc>https://weedmaps.com/dispensaries/wacky-backy-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenfields-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/viola-missouri-coming-soon</loc>",
    "<loc>https://weedmaps.com/dispensaries/releaf-890</loc>",
    "<loc>https://weedmaps.com/dispensaries/local-cannabis-company-swampscott</loc>",
    "<loc>https://weedmaps.com/dispensaries/magic-mann</loc>",
    "<loc>https://weedmaps.com/dispensaries/union-chill-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-cured-6</loc>",
    "<loc>https://weedmaps.com/dispensaries/purple-pineapple-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/budz-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/puffshaven-yonge-empress</loc>",
    "<loc>https://weedmaps.com/dispensaries/friendly-fire-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-land-o-lakes</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/tamarack-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/red-gorilla-greens-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-dragon-fort-collins-college-ave</loc>",
    "<loc>https://weedmaps.com/dispensaries/pecos-valley-production-tularosa</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-frederick</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-society-south-may</loc>",
    "<loc>https://weedmaps.com/dispensaries/lawton-organics</loc>",
    "<loc>https://weedmaps.com/dispensaries/legally-rooted-cannabis-dispensary-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/spark-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-flowery-inverness</loc>",
    "<loc>https://weedmaps.com/dispensaries/holistika</loc>",
    "<loc>https://weedmaps.com/dispensaries/hikers-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/empire-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/apogee-gardens-bozeman</loc>",
    "<loc>https://weedmaps.com/dispensaries/central-maine-flower-7</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-bonita-springs</loc>",
    "<loc>https://weedmaps.com/dispensaries/nova-farms-dracut-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-farmers-market</loc>",
    "<loc>https://weedmaps.com/dispensaries/nubes-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-health-care</loc>",
    "<loc>https://weedmaps.com/dispensaries/skunked-retail-marijuana</loc>",
    "<loc>https://weedmaps.com/dispensaries/codes-st-louis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-boynton-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-bradenton-75th-west</loc>",
    "<loc>https://weedmaps.com/dispensaries/rebel-roots-farmer-loop</loc>",
    "<loc>https://weedmaps.com/dispensaries/nm-green-door</loc>",
    "<loc>https://weedmaps.com/dispensaries/aura-of-rhode-island</loc>",
    "<loc>https://weedmaps.com/dispensaries/happy-leaf-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/arcane</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-mint-cannabis-northern-ave</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-bradenton-heritage-green</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-brandon</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-cape-coral</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-clearwater-roosevelt</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-fort-myers-cypress</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-fort-myers-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/sovereign-cannabis-co-ft-covington</loc>",
    "<loc>https://weedmaps.com/dispensaries/stiiizy-battle-creek</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-hobe-sound</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-hollywood</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-key-west</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-lady-lake</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-lutz</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-marco-island</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-merritt-island</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-new-tampa</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-north-port</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-ocala</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-orange-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-orange-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-orlando-vineland</loc>",
    "<loc>https://weedmaps.com/dispensaries/jungle-boys-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-ormond-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/higher-elevation-6</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-scene-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-reserve-620</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-palatka</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-pensacola</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-pinellas-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-port-charlotte</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-port-st-lucie</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-sarasota-main-street</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-sebastian</loc>",
    "<loc>https://weedmaps.com/dispensaries/dacut</loc>",
    "<loc>https://weedmaps.com/dispensaries/top-crop-cannabis-co-las-cruces</loc>",
    "<loc>https://weedmaps.com/dispensaries/exclusive-brands</loc>",
    "<loc>https://weedmaps.com/dispensaries/moses-roses</loc>",
    "<loc>https://weedmaps.com/dispensaries/bud-brothers-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/encanto-tree-cupey</loc>",
    "<loc>https://weedmaps.com/dispensaries/encanto-tree-6</loc>",
    "<loc>https://weedmaps.com/dispensaries/encanto-tree-7</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-highest-care</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-shalimar</loc>",
    "<loc>https://weedmaps.com/dispensaries/archer-farms</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-spring-hill</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannaverse-nepean</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannaverse-scarborough</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-st-augustine</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannaverse-port-perry</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannaverse-lundy-s-lane</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-st-petersburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-stuart</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-tallahassee</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-tamarac</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-tampa-2-w-kennedy</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-tampa-himes</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-west-melbourne</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-winter-haven</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenfields-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/pamolab</loc>",
    "<loc>https://weedmaps.com/dispensaries/releaf-85624</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-herbalist-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-vibes-cannabis-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/grizzly-pine-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-xpress-north-gower</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannaverse-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannaverse-picton</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannaverse-ancaster</loc>",
    "<loc>https://weedmaps.com/dispensaries/top-crop-nm-albuquerque-menaul-blvd-ne</loc>",
    "<loc>https://weedmaps.com/dispensaries/top-crop-nm-dark-matter-albuquerque-juan-tabo-blvd-ne</loc>",
    "<loc>https://weedmaps.com/dispensaries/top-crop-nm-sunland-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-genie-warren-ave</loc>",
    "<loc>https://weedmaps.com/dispensaries/offside-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-essentia</loc>",
    "<loc>https://weedmaps.com/dispensaries/joint-venture-carmel-valley</loc>",
    "<loc>https://weedmaps.com/dispensaries/proper-cannabis-bridgeton</loc>",
    "<loc>https://weedmaps.com/dispensaries/brute-s-roots</loc>",
    "<loc>https://weedmaps.com/dispensaries/flora-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/westwood-world-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/dark-matter-sunland-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/highbrow-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/house-of-dank-grand-rapids-rec</loc>",
    "<loc>https://weedmaps.com/dispensaries/house-of-dank-saginaw-rec</loc>",
    "<loc>https://weedmaps.com/dispensaries/house-of-dank-lapeer-rec</loc>",
    "<loc>https://weedmaps.com/dispensaries/uptop-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-state-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-day-farm-hattiesburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-day-farm-laurel</loc>",
    "<loc>https://weedmaps.com/dispensaries/enchanted-botanicals-central</loc>",
    "<loc>https://weedmaps.com/dispensaries/nirvana-center-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-day-farm-tupelo</loc>",
    "<loc>https://weedmaps.com/dispensaries/haven-cannabis-dispensary-porterville-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-day-farm-corinth</loc>",
    "<loc>https://weedmaps.com/dispensaries/leaf-lab-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/dispo-jackson</loc>",
    "<loc>https://weedmaps.com/dispensaries/dispo-ann-arbor</loc>",
    "<loc>https://weedmaps.com/dispensaries/noxx-plainfield</loc>",
    "<loc>https://weedmaps.com/dispensaries/in-the-weeds</loc>",
    "<loc>https://weedmaps.com/dispensaries/cookies-jackson</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-miami-south-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-orlando-international-drive</loc>",
    "<loc>https://weedmaps.com/dispensaries/due-north-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/hello-cannabis-london</loc>",
    "<loc>https://weedmaps.com/dispensaries/off-the-charts-costa-mesa</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-buzz</loc>",
    "<loc>https://weedmaps.com/dispensaries/420-central-421</loc>",
    "<loc>https://weedmaps.com/dispensaries/valley-pure-tulare</loc>",
    "<loc>https://weedmaps.com/dispensaries/botera-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbaceous</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-remedy-6</loc>",
    "<loc>https://weedmaps.com/dispensaries/catalyst-oxnard</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-klub-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/jars-cannabis-globe</loc>",
    "<loc>https://weedmaps.com/dispensaries/jars-cannabis-yuma</loc>",
    "<loc>https://weedmaps.com/dispensaries/jars-cannabis-safford</loc>",
    "<loc>https://weedmaps.com/dispensaries/full-bloom-cannabis-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/purlife-dispensary-santa-fe</loc>",
    "<loc>https://weedmaps.com/dispensaries/boss-buds-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/pure-lotus-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/spot420-the-cannabis-store-north-york</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-vault-8</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-pharm-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/leaf-and-bud-center-line</loc>",
    "<loc>https://weedmaps.com/dispensaries/gramz-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/full-harvest-moonz-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/7-leafs</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-cannabis-dispensary-orlando-west</loc>",
    "<loc>https://weedmaps.com/dispensaries/cloud-cannabis-cedar-springs</loc>",
    "<loc>https://weedmaps.com/dispensaries/catalyst-cannabis-company-spenard</loc>",
    "<loc>https://weedmaps.com/dispensaries/terps-charlton</loc>",
    "<loc>https://weedmaps.com/dispensaries/studio-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/jars-mt-clemens</loc>",
    "<loc>https://weedmaps.com/dispensaries/seed-of-life-labs-8</loc>",
    "<loc>https://weedmaps.com/dispensaries/210-cananbis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/outpost</loc>",
    "<loc>https://weedmaps.com/dispensaries/tropic-canna-west</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-artist-tree-oxnard-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/redi-dispensary-natick</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-dispensary-by-rev-hattiesburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/nature-s-embrace-inc-brockton</loc>",
    "<loc>https://weedmaps.com/dispensaries/420-bay-st-louis</loc>",
    "<loc>https://weedmaps.com/dispensaries/bee-s-buds</loc>",
    "<loc>https://weedmaps.com/dispensaries/big-river</loc>",
    "<loc>https://weedmaps.com/dispensaries/bluegreen-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/coast-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/coast-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/diamondhead-root</loc>",
    "<loc>https://weedmaps.com/dispensaries/dispensary-plus</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-star-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-therapy</loc>",
    "<loc>https://weedmaps.com/dispensaries/kush-vibez</loc>",
    "<loc>https://weedmaps.com/dispensaries/magnolia-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/magnolia-cannabis-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/mississippi-magnolia-canna</loc>",
    "<loc>https://weedmaps.com/dispensaries/nature-med-ms</loc>",
    "<loc>https://weedmaps.com/dispensaries/noble-labs-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/roots-remedy</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-coastal-capital-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-standard-grenada</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-standard-vicksburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-standard-starkville</loc>",
    "<loc>https://weedmaps.com/dispensaries/tru-source</loc>",
    "<loc>https://weedmaps.com/dispensaries/waveland-marijuana</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-clermont</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-tallahassee-tn-st</loc>",
    "<loc>https://weedmaps.com/dispensaries/social-cannabis-denver</loc>",
    "<loc>https://weedmaps.com/dispensaries/tried-and-true</loc>",
    "<loc>https://weedmaps.com/dispensaries/moe-weed-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/natures-solution</loc>",
    "<loc>https://weedmaps.com/dispensaries/proper-cannabis-downtown-kc</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-boca-raton-glades</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-palm-beach-gardens</loc>",
    "<loc>https://weedmaps.com/dispensaries/sweet-flower</loc>",
    "<loc>https://weedmaps.com/dispensaries/joyology-allegan</loc>",
    "<loc>https://weedmaps.com/dispensaries/harvest-bud-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannaverse-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/levelup-infusions</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-dab-lounge</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-grove-pahrump-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-cannabis-dispensary-west-palm-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-cannabis-dispensary-lutz</loc>",
    "<loc>https://weedmaps.com/dispensaries/people-s-remedy</loc>",
    "<loc>https://weedmaps.com/dispensaries/premium-organics</loc>",
    "<loc>https://weedmaps.com/dispensaries/factory-direct-pipes-recreational-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-tide-edibles-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/star-buds-medical</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-set-hollywood</loc>",
    "<loc>https://weedmaps.com/dispensaries/urb-n</loc>",
    "<loc>https://weedmaps.com/dispensaries/power-plant-central-ave</loc>",
    "<loc>https://weedmaps.com/dispensaries/silver-stem-rockrimmon</loc>",
    "<loc>https://weedmaps.com/dispensaries/silver-stem-broadmoor</loc>",
    "<loc>https://weedmaps.com/dispensaries/farmacy-isla-vista</loc>",
    "<loc>https://weedmaps.com/dispensaries/nuera-pekin-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/nuera-aurora</loc>",
    "<loc>https://weedmaps.com/dispensaries/nuera-champaign</loc>",
    "<loc>https://weedmaps.com/dispensaries/bigfoots-cannabis-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannaverse-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/mojo</loc>",
    "<loc>https://weedmaps.com/dispensaries/leaf-labs-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/columbia-recreational-marijuana</loc>",
    "<loc>https://weedmaps.com/dispensaries/nextier</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-fire-station-cannabis-co-ishpeming</loc>",
    "<loc>https://weedmaps.com/dispensaries/planted-provisioning-bay-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/ripple-wellness-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/haqs1000</loc>",
    "<loc>https://weedmaps.com/dispensaries/gas-station-santas-rosa</loc>",
    "<loc>https://weedmaps.com/dispensaries/natural-wellness-lyndale-helena</loc>",
    "<loc>https://weedmaps.com/dispensaries/natural-wellness-belgrade</loc>",
    "<loc>https://weedmaps.com/dispensaries/natural-wellness-montana-helena</loc>",
    "<loc>https://weedmaps.com/dispensaries/natural-wellness-butte</loc>",
    "<loc>https://weedmaps.com/dispensaries/natural-wellness-lewistown</loc>",
    "<loc>https://weedmaps.com/dispensaries/southern-sky-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/collective-elevation-bozeman-7th-ave</loc>",
    "<loc>https://weedmaps.com/dispensaries/yuma-way-ca</loc>",
    "<loc>https://weedmaps.com/dispensaries/kush-trips</loc>",
    "<loc>https://weedmaps.com/dispensaries/river-valley-natural-health-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/nj-leaf-freehold</loc>",
    "<loc>https://weedmaps.com/dispensaries/firelands-scientific-dispensary-huron</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-cannabis-dispensary-orlando-south</loc>",
    "<loc>https://weedmaps.com/dispensaries/culture-cannabis-club-porterville</loc>",
    "<loc>https://weedmaps.com/dispensaries/token-farms-tulare-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/firelands-dispensary-cleveland</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbal-wellness-center-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/edmonton-cannabis-and-edibles-116-ave</loc>",
    "<loc>https://weedmaps.com/dispensaries/edmonton-cannabis-and-edibles-82-ave</loc>",
    "<loc>https://weedmaps.com/dispensaries/sundial-red-bluff</loc>",
    "<loc>https://weedmaps.com/dispensaries/landrace-cannabis-co-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/amplify-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/lawless-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/liv-cannabis-grand-rapids</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbal-rhyno</loc>",
    "<loc>https://weedmaps.com/dispensaries/dispo-hazel-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/lakewood-cannabis-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/stash-cannabis-company-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-citizen-by-klutch-lorain</loc>",
    "<loc>https://weedmaps.com/dispensaries/apothecary-farms</loc>",
    "<loc>https://weedmaps.com/dispensaries/everest-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/its-420-time</loc>",
    "<loc>https://weedmaps.com/dispensaries/coming-soon-diamond-leaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/brothers-cannabis-newport</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannaverse-kitchener</loc>",
    "<loc>https://weedmaps.com/dispensaries/trichomez</loc>",
    "<loc>https://weedmaps.com/dispensaries/kind-castle-ridgway</loc>",
    "<loc>https://weedmaps.com/dispensaries/upstate-gardens</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannaverse-sault-ste-marie</loc>",
    "<loc>https://weedmaps.com/dispensaries/zanzibar-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/treez-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/midnight-show-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/kush-gardens-shawnee</loc>",
    "<loc>https://weedmaps.com/dispensaries/kush-gardens-muskogee</loc>",
    "<loc>https://weedmaps.com/dispensaries/dank-tank-el-reno</loc>",
    "<loc>https://weedmaps.com/dispensaries/long-beach-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/seaweed-co-auburn</loc>",
    "<loc>https://weedmaps.com/dispensaries/firelight-and-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/oasis-cannabis-co-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/tree-house-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/southwest-dispensary-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/exclusive-lowell</loc>",
    "<loc>https://weedmaps.com/dispensaries/stiiizy-soma</loc>",
    "<loc>https://weedmaps.com/dispensaries/richy-rich-hempire</loc>",
    "<loc>https://weedmaps.com/dispensaries/bloom-7</loc>",
    "<loc>https://weedmaps.com/dispensaries/pure-roots-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/evergreen-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbal-alchemy</loc>",
    "<loc>https://weedmaps.com/dispensaries/nm-cannabis-cowboy</loc>",
    "<loc>https://weedmaps.com/dispensaries/star-buds-medical-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/corrales-hemporium</loc>",
    "<loc>https://weedmaps.com/dispensaries/voodoo-buds</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-bristol</loc>",
    "<loc>https://weedmaps.com/dispensaries/ultraleaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/bosque-buds</loc>",
    "<loc>https://weedmaps.com/dispensaries/floyds-mt-vernon</loc>",
    "<loc>https://weedmaps.com/dispensaries/toker-king-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/mint-house-maine</loc>",
    "<loc>https://weedmaps.com/dispensaries/embarc-redwood-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/noxx-cookies</loc>",
    "<loc>https://weedmaps.com/dispensaries/quality-roots-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/erba-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-dragon-boynton-beach-west</loc>",
    "<loc>https://weedmaps.com/dispensaries/sparrow-jane</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-dragon-fort-pierce</loc>",
    "<loc>https://weedmaps.com/dispensaries/roadrunner-cannabis-co-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-dragon-east-jacksonville</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-dragon-lake-worth</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-dragon-merritt-island</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-dragon-stuart</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-dragon-tampa</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-dragon-titusville</loc>",
    "<loc>https://weedmaps.com/dispensaries/mama-and-the-girls</loc>",
    "<loc>https://weedmaps.com/dispensaries/ooh-la-la-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/ferr-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/dazed-on-route-66</loc>",
    "<loc>https://weedmaps.com/dispensaries/carolines-cannabic-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/score-420</loc>",
    "<loc>https://weedmaps.com/dispensaries/east-coast-cannabis-waterville</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-jive-dispensary-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/total-green-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-society-i-35-main</loc>",
    "<loc>https://weedmaps.com/dispensaries/delta-dispensary-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/one-plant-keswick</loc>",
    "<loc>https://weedmaps.com/dispensaries/best-invest-llc-dba-best-high</loc>",
    "<loc>https://weedmaps.com/dispensaries/lucent-abq</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-midori-wellness-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/exclusive-gaylord</loc>",
    "<loc>https://weedmaps.com/dispensaries/omg-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-cannabis-dispensary-erie</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-cannabis-dispensary-daytona-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-cannabis-dispensary-palm-harbor</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-cannabis-dispensary-lake-worth</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-cannabis-dispensary-winter-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-kana-company-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/livwell-meds-of-calera-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/astroterf</loc>",
    "<loc>https://weedmaps.com/dispensaries/pecos-valley-production-8</loc>",
    "<loc>https://weedmaps.com/dispensaries/liv-cannabis-detroit</loc>",
    "<loc>https://weedmaps.com/dispensaries/puff-river-rouge</loc>",
    "<loc>https://weedmaps.com/dispensaries/shake-shack</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannalabs-san-juan</loc>",
    "<loc>https://weedmaps.com/dispensaries/old-route-66-wellness-springfield-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-chronic-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/apoteka-pr-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/flor-de-las-flores</loc>",
    "<loc>https://weedmaps.com/dispensaries/weed-nation</loc>",
    "<loc>https://weedmaps.com/dispensaries/hasheesh-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/blue-river-terps</loc>",
    "<loc>https://weedmaps.com/dispensaries/b-k-buds</loc>",
    "<loc>https://weedmaps.com/dispensaries/lunchboxdispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/tetra-66</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-dayz</loc>",
    "<loc>https://weedmaps.com/dispensaries/lemonnade-centerline</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-xpress-9</loc>",
    "<loc>https://weedmaps.com/dispensaries/cavi-land</loc>",
    "<loc>https://weedmaps.com/dispensaries/serenity-of-lathrop-manteca</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-half-oz-starkville</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-half-oz-olive-branch</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-half-oz-oxford</loc>",
    "<loc>https://weedmaps.com/dispensaries/soar-meridian</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-columbus</loc>",
    "<loc>https://weedmaps.com/dispensaries/okay-cannabis-dispensary-chicago</loc>",
    "<loc>https://weedmaps.com/dispensaries/rocking-y-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/yellowstoned-dispensaries-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-cross-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/ao-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/kush-21-jacksonville</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-spot-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/love-s-harvest</loc>",
    "<loc>https://weedmaps.com/dispensaries/solar-cannabis-co-dartmouth</loc>",
    "<loc>https://weedmaps.com/dispensaries/kind-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/leafology-cannabis-dispensary-santurce</loc>",
    "<loc>https://weedmaps.com/dispensaries/pure-cannabis-outlet</loc>",
    "<loc>https://weedmaps.com/dispensaries/farmacy-santa-ynez</loc>",
    "<loc>https://weedmaps.com/dispensaries/oh-hi</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannaverse-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/american-cannabis-company-lawton</loc>",
    "<loc>https://weedmaps.com/dispensaries/euphoria-wellness-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/alto-dispensary-1-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/theounce</loc>",
    "<loc>https://weedmaps.com/dispensaries/mr-goodbud</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-dragon-pensacola-navy-blvd</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-dragon-panama-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/wise-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/asociacion-golden-leaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/jungle-boys-deerfield-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/jungle-boys-north-miami-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-day-farm-biloxi</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-boca-raton-west</loc>",
    "<loc>https://weedmaps.com/dispensaries/dancing-turtle-greenery</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-dragon-north-ft-myers</loc>",
    "<loc>https://weedmaps.com/dispensaries/remedii-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenwise-cannabis-company-gulfport</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenwise-cannabis-company-starkville</loc>",
    "<loc>https://weedmaps.com/dispensaries/coming-soon-greenwise-cannabis-company-tupelo</loc>",
    "<loc>https://weedmaps.com/dispensaries/catalyst-silver-lake</loc>",
    "<loc>https://weedmaps.com/dispensaries/depotdispo</loc>",
    "<loc>https://weedmaps.com/dispensaries/seed-of-life-labs-9</loc>",
    "<loc>https://weedmaps.com/dispensaries/puffin-store-nj</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-cure-dispensary-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/silverleaf-snyelmn</loc>",
    "<loc>https://weedmaps.com/dispensaries/coming-soon-swish-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/deep-sky-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-haven-augusta</loc>",
    "<loc>https://weedmaps.com/dispensaries/pink-rhino</loc>",
    "<loc>https://weedmaps.com/dispensaries/haylow-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/delirium-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/pack-south-la</loc>",
    "<loc>https://weedmaps.com/dispensaries/homestate-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/homestate-dispensary-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-magnolia-healing</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-releaf-dispensary-columbia</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-releaf-dispensary-lee-s-summit</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-releaf-dispensary-liberty</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/union-square-travel-agency</loc>",
    "<loc>https://weedmaps.com/dispensaries/hundred-leafs</loc>",
    "<loc>https://weedmaps.com/dispensaries/silverbow-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/country-grown-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/nirvana-center-albuquerque</loc>",
    "<loc>https://weedmaps.com/dispensaries/tumbling-tumbleweed</loc>",
    "<loc>https://weedmaps.com/dispensaries/wyld-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/housing-works-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/lucky-seven-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/burnin-barrels</loc>",
    "<loc>https://weedmaps.com/dispensaries/broadway-cannabis-market-sw-downtown</loc>",
    "<loc>https://weedmaps.com/dispensaries/burnin-barrels-farmington</loc>",
    "<loc>https://weedmaps.com/dispensaries/country-grown-huntington</loc>",
    "<loc>https://weedmaps.com/dispensaries/kanacare-health</loc>",
    "<loc>https://weedmaps.com/dispensaries/uplift-mount-orab</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenlight-park-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/from-the-hills</loc>",
    "<loc>https://weedmaps.com/dispensaries/from-the-hills-deadwood</loc>",
    "<loc>https://weedmaps.com/dispensaries/culture-cannabis-club-wildomar</loc>",
    "<loc>https://weedmaps.com/dispensaries/vice-cannabis-by-appointment-only</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-grass-pro-shop</loc>",
    "<loc>https://weedmaps.com/dispensaries/capital-dank-norman</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-bud-2-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/nexleaf-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/big-dub-pharms-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/embarc-chico</loc>",
    "<loc>https://weedmaps.com/dispensaries/cedar-greens-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunrise-cannabis-dispensary-kansas-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunrise-cannabis-dispensary-clinton</loc>",
    "<loc>https://weedmaps.com/dispensaries/codes-macon</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunrise-cannabis-dispensary-maryville</loc>",
    "<loc>https://weedmaps.com/dispensaries/daylite-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/coachella-canna-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/sky-club-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/exclusive-oxford</loc>",
    "<loc>https://weedmaps.com/dispensaries/silver-stem-fine-cannabis-portland-hayden-island</loc>",
    "<loc>https://weedmaps.com/dispensaries/catalyst-mid-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/super-fire</loc>",
    "<loc>https://weedmaps.com/dispensaries/silver-therapeutics-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/backstage-grass</loc>",
    "<loc>https://weedmaps.com/dispensaries/galaxy21</loc>",
    "<loc>https://weedmaps.com/dispensaries/somewhere-on-the-mountain</loc>",
    "<loc>https://weedmaps.com/dispensaries/american-cannabis-company-13</loc>",
    "<loc>https://weedmaps.com/dispensaries/rush-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-buzz-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/buena-suerte-company-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/jars-saginaw</loc>",
    "<loc>https://weedmaps.com/dispensaries/park-social-alameda</loc>",
    "<loc>https://weedmaps.com/dispensaries/botera-taunton</loc>",
    "<loc>https://weedmaps.com/dispensaries/bear-river-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/natural-high-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/budding-genius-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/earth-keeper-cannabis-dixfield</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannalabs-isla-verde</loc>",
    "<loc>https://weedmaps.com/dispensaries/jade-room-moval</loc>",
    "<loc>https://weedmaps.com/dispensaries/bud-barn-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-society-saline</loc>",
    "<loc>https://weedmaps.com/dispensaries/cloud-cannabis-grand-rapids-28th-street-now-open</loc>",
    "<loc>https://weedmaps.com/dispensaries/culture-biloxi-shop</loc>",
    "<loc>https://weedmaps.com/dispensaries/3ft-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/dancing-turtle-greenery-east</loc>",
    "<loc>https://weedmaps.com/dispensaries/smokers-depot</loc>",
    "<loc>https://weedmaps.com/dispensaries/sweet-relief-shop-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/pecos-valley-production-main-st-roswell</loc>",
    "<loc>https://weedmaps.com/dispensaries/quality-roots-berkley</loc>",
    "<loc>https://weedmaps.com/dispensaries/bud-brothers-cannabis-co-bay-city-coming-soon</loc>",
    "<loc>https://weedmaps.com/dispensaries/gotham-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/710-montana-711</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-leaf-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/nativa-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/cookies-orlando</loc>",
    "<loc>https://weedmaps.com/dispensaries/emerald-sage</loc>",
    "<loc>https://weedmaps.com/dispensaries/everest-cannabis-co-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/indigo-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/smokers-outlet</loc>",
    "<loc>https://weedmaps.com/dispensaries/poospatuck-tobacco-and-herb-center</loc>",
    "<loc>https://weedmaps.com/dispensaries/chkn-n-wafflez</loc>",
    "<loc>https://weedmaps.com/dispensaries/back-road-buds</loc>",
    "<loc>https://weedmaps.com/dispensaries/levels-modesto</loc>",
    "<loc>https://weedmaps.com/dispensaries/remaine-humble-farms</loc>",
    "<loc>https://weedmaps.com/dispensaries/neon-heights</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-champ</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannavine-santa-rosa</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbnjoy-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/ohio-cannabis-company-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/highway90</loc>",
    "<loc>https://weedmaps.com/dispensaries/gnomad-canna-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabist-huntington</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabist-norfolk-coming-soon</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-rose-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/los-angeles-kush-hollywood</loc>",
    "<loc>https://weedmaps.com/dispensaries/jars-cannabis-cave-creek</loc>",
    "<loc>https://weedmaps.com/dispensaries/house-of-evolution-bay-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/danknolia-ellis-ave</loc>",
    "<loc>https://weedmaps.com/dispensaries/danknolia-cannabis-dispensary-hwy-18-w</loc>",
    "<loc>https://weedmaps.com/dispensaries/danknolia-cannabis-dispensary-w-northside-dr</loc>",
    "<loc>https://weedmaps.com/dispensaries/local-cannabis-company-santa-monica</loc>",
    "<loc>https://weedmaps.com/dispensaries/swade-cannabis-the-grove</loc>",
    "<loc>https://weedmaps.com/dispensaries/swade-cannabis-delmar</loc>",
    "<loc>https://weedmaps.com/dispensaries/swade-cannabis-cherokee</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-flowery-clearwater</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-flowery-jacksonville</loc>",
    "<loc>https://weedmaps.com/dispensaries/organic-farms</loc>",
    "<loc>https://weedmaps.com/dispensaries/rollin-embers-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/pure-genus</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-grades</loc>",
    "<loc>https://weedmaps.com/dispensaries/black-ridge-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/uphi-cannabis-emporium-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/shaggy-s-baggy</loc>",
    "<loc>https://weedmaps.com/dispensaries/kind-castle-commerce</loc>",
    "<loc>https://weedmaps.com/dispensaries/redwater-cannabis-care</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-tide-nw</loc>",
    "<loc>https://weedmaps.com/dispensaries/spiritleaf-rexdale</loc>",
    "<loc>https://weedmaps.com/dispensaries/parc-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-hive-at-bdt-hazel-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/solar-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/luxuryleafstl</loc>",
    "<loc>https://weedmaps.com/dispensaries/nova-farms-woodbury</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-cannabis-dispensary-miami-west</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-releaf-dispensary-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/off-the-charts-san-francisco</loc>",
    "<loc>https://weedmaps.com/dispensaries/elevation-3789</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-leaf-gowanda</loc>",
    "<loc>https://weedmaps.com/dispensaries/cream-cannabis-company-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/megan-s-organic-market-corona-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabliss-marbella</loc>",
    "<loc>https://weedmaps.com/dispensaries/beach-fire-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/nice-guys-cannabliss-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/noble-cannabis-co-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/march-and-ash-sabre-springs-opening-soon</loc>",
    "<loc>https://weedmaps.com/dispensaries/cloud-9-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/discern-d-cannabis-purveyors</loc>",
    "<loc>https://weedmaps.com/dispensaries/mary-alice</loc>",
    "<loc>https://weedmaps.com/dispensaries/luna-leaf-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/redbird-birdhouse-enid</loc>",
    "<loc>https://weedmaps.com/dispensaries/elevated-herb</loc>",
    "<loc>https://weedmaps.com/dispensaries/highly-cannaco-traverse-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-tea-house</loc>",
    "<loc>https://weedmaps.com/dispensaries/great-lakes-exotics-cannabis-company-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/tafoya-farms</loc>",
    "<loc>https://weedmaps.com/dispensaries/penn-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/joyology-of-monroe</loc>",
    "<loc>https://weedmaps.com/dispensaries/budside</loc>",
    "<loc>https://weedmaps.com/dispensaries/stiiizy-costa-mesa</loc>",
    "<loc>https://weedmaps.com/dispensaries/empire-gardens-lake-elsinore</loc>",
    "<loc>https://weedmaps.com/dispensaries/tree-house-craft-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/stairway-cannabis-branson</loc>",
    "<loc>https://weedmaps.com/dispensaries/long-island-cannabis-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-gold-group-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-matchbox</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-dispensary-by-rev</loc>",
    "<loc>https://weedmaps.com/dispensaries/uplift-milford</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-802-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-gas-station-760</loc>",
    "<loc>https://weedmaps.com/dispensaries/moonwlkr-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/tanana-herb-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/sapura-lansing</loc>",
    "<loc>https://weedmaps.com/dispensaries/capital-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/forage</loc>",
    "<loc>https://weedmaps.com/dispensaries/bubblegum-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/culture-cannabis-stanton</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-west-palm-beach-clematis</loc>",
    "<loc>https://weedmaps.com/dispensaries/dazed</loc>",
    "<loc>https://weedmaps.com/dispensaries/soulflora</loc>",
    "<loc>https://weedmaps.com/dispensaries/natural-healing-center-turlock</loc>",
    "<loc>https://weedmaps.com/dispensaries/wrk-northridge</loc>",
    "<loc>https://weedmaps.com/dispensaries/bam-body-and-mind-dispensary-markham</loc>",
    "<loc>https://weedmaps.com/dispensaries/la-botica-de-medgrow</loc>",
    "<loc>https://weedmaps.com/dispensaries/dispensary-7884-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-cannabis-dispensary-st-petersburg-north</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-three-sisters-collective-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/lone-peak-caregivers-west-yellowstone</loc>",
    "<loc>https://weedmaps.com/dispensaries/jars-cannabis-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/ashe-pomona</loc>",
    "<loc>https://weedmaps.com/dispensaries/northeast-kannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/cookies-oxford</loc>",
    "<loc>https://weedmaps.com/dispensaries/hive-adult-use</loc>",
    "<loc>https://weedmaps.com/dispensaries/american-cannabis-dispensary-stillwater</loc>",
    "<loc>https://weedmaps.com/dispensaries/off-the-charts-ramona</loc>",
    "<loc>https://weedmaps.com/dispensaries/space-flyt</loc>",
    "<loc>https://weedmaps.com/dispensaries/ripple-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/dakota-dispensaries</loc>",
    "<loc>https://weedmaps.com/dispensaries/vermont-bud-barn</loc>",
    "<loc>https://weedmaps.com/dispensaries/superior-buds-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/enchantment-growth-co-jal</loc>",
    "<loc>https://weedmaps.com/dispensaries/enchantment-growth-co-hobbs</loc>",
    "<loc>https://weedmaps.com/dispensaries/groove-solventless</loc>",
    "<loc>https://weedmaps.com/dispensaries/cloud-cannabis-7</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-society-malaga</loc>",
    "<loc>https://weedmaps.com/dispensaries/kanyakpr</loc>",
    "<loc>https://weedmaps.com/dispensaries/newleaf-express</loc>",
    "<loc>https://weedmaps.com/dispensaries/vermont-trap-house</loc>",
    "<loc>https://weedmaps.com/dispensaries/stash-house-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/canvas-organics-enterprises-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/silver-therapeutics-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/backroad-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/backroad-wellness-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/jailhouse-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/tough-bud-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-flowery-ocala</loc>",
    "<loc>https://weedmaps.com/dispensaries/luxury-gas</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-rollers-8</loc>",
    "<loc>https://weedmaps.com/dispensaries/kind-care-of-south-dakota</loc>",
    "<loc>https://weedmaps.com/dispensaries/jars-cannabis-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/jars-cannabis-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/stoned-ape-canna-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/jars-cannabis-6</loc>",
    "<loc>https://weedmaps.com/dispensaries/jars-cannabis-7</loc>",
    "<loc>https://weedmaps.com/dispensaries/grams-dispo</loc>",
    "<loc>https://weedmaps.com/dispensaries/cambridge-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-velvet-tiger</loc>",
    "<loc>https://weedmaps.com/dispensaries/blazed-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/youngs-farm-medical-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-palm-springs-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/off-the-charts-weho</loc>",
    "<loc>https://weedmaps.com/dispensaries/cream-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/hydro-budz</loc>",
    "<loc>https://weedmaps.com/dispensaries/wonderbrett-store</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbal-meds-dispensary-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/coastal-healing</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-epic-remedy-platte</loc>",
    "<loc>https://weedmaps.com/dispensaries/bloomhill-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/catalyst-watsonville</loc>",
    "<loc>https://weedmaps.com/dispensaries/ratio-cannabis-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-cannabis-dispensary-destin</loc>",
    "<loc>https://weedmaps.com/dispensaries/guaranteed-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/okie-wonderland-12th</loc>",
    "<loc>https://weedmaps.com/dispensaries/bern-gallery-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/cookies-harrison-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/bewell-rec</loc>",
    "<loc>https://weedmaps.com/dispensaries/stiiizy-oxnard</loc>",
    "<loc>https://weedmaps.com/dispensaries/stiiizy-el-cerrito</loc>",
    "<loc>https://weedmaps.com/dispensaries/shiva-social-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/falcanna-x-moulin-rouge</loc>",
    "<loc>https://weedmaps.com/dispensaries/pure-iconic</loc>",
    "<loc>https://weedmaps.com/dispensaries/unity-rd</loc>",
    "<loc>https://weedmaps.com/dispensaries/pecos-valley-production-9</loc>",
    "<loc>https://weedmaps.com/dispensaries/cookies-baltimore</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-breeze-banus</loc>",
    "<loc>https://weedmaps.com/dispensaries/dispensary-near-me-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-bristol</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-paterson-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/white-barn-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/betterbuds-longview</loc>",
    "<loc>https://weedmaps.com/dispensaries/three-rivers-dispensary-broadway-rec</loc>",
    "<loc>https://weedmaps.com/dispensaries/three-rivers-dispensary-broadway-med</loc>",
    "<loc>https://weedmaps.com/dispensaries/earth-and-ivy-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-healing-community-medco-lewiston-au</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-supply-saloon</loc>",
    "<loc>https://weedmaps.com/dispensaries/kusty</loc>",
    "<loc>https://weedmaps.com/dispensaries/elevated-bay-city-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-lighthouse-dispensary-palm-springs</loc>",
    "<loc>https://weedmaps.com/dispensaries/highway-dispo-battle-creek</loc>",
    "<loc>https://weedmaps.com/dispensaries/nirvana-center-escanaba</loc>",
    "<loc>https://weedmaps.com/dispensaries/nirvana-center-lowell</loc>",
    "<loc>https://weedmaps.com/dispensaries/story-of-coshocton</loc>",
    "<loc>https://weedmaps.com/dispensaries/euphoria-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/pecos-valley-production-clovis-north-prince-st</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-cannabis-co-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbology-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/la-mota-9759</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-dispensary-limerick</loc>",
    "<loc>https://weedmaps.com/dispensaries/off-the-charts-sacramento</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-loud-flower-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/verdemar</loc>",
    "<loc>https://weedmaps.com/dispensaries/primo-provisioning</loc>",
    "<loc>https://weedmaps.com/dispensaries/bored-n-stone</loc>",
    "<loc>https://weedmaps.com/dispensaries/renew-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/casco-botanical-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-tea-tree-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-boynton-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-bradenton</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-clearwater</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-clermont</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-daytona-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/crescent-city-cannabis-company-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/social-club-smoky-ape</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-florida-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-fort-pierce</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-jacksonville-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-jacksonville-philips</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-key-west</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-lake-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-lake-mary</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-lake-worth</loc>",
    "<loc>https://weedmaps.com/dispensaries/favorite-elements-cannabis-medicinal</loc>",
    "<loc>https://weedmaps.com/dispensaries/healthy-buds-condado</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-lakeland</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-land-o-lakes</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-longwood</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-mary-esther</loc>",
    "<loc>https://weedmaps.com/dispensaries/blue-diamond</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-ocala</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-macon</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-orlando-landstar</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-orlando-south-obt</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-orlando-turkey-lake</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-orlando-ucf</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-palatka</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-palm-bay</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-palmetto-bay</loc>",
    "<loc>https://weedmaps.com/dispensaries/garfield-gardens-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-panama-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-pensacola</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-port-charlotte</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-port-richey</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-sarasota</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-sebring</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-spring-hill</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-st-augustine</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-stuart</loc>",
    "<loc>https://weedmaps.com/dispensaries/leafmed</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-tallahassee</loc>",
    "<loc>https://weedmaps.com/dispensaries/leafmed-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-tampa-s-dale-mabry</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-tampa-tampa-heights</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-tampa-kennedy</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-tarpon-springs</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-west-palm-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-winter-haven-cypress-gardens-blvd</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-winter-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/daft-republic</loc>",
    "<loc>https://weedmaps.com/dispensaries/harvest-of-apache-junction</loc>",
    "<loc>https://weedmaps.com/dispensaries/t-nug-s-dispensary-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/oasis-pharma</loc>",
    "<loc>https://weedmaps.com/dispensaries/rise-danville</loc>",
    "<loc>https://weedmaps.com/dispensaries/el-grand-kush</loc>",
    "<loc>https://weedmaps.com/dispensaries/hoja-loca</loc>",
    "<loc>https://weedmaps.com/dispensaries/new-mill-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-temple-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-of-tatum</loc>",
    "<loc>https://weedmaps.com/dispensaries/atlantic-farms-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/king-of-budz-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/highway-dispo-flint-rec</loc>",
    "<loc>https://weedmaps.com/dispensaries/tracy-cannabis-collective-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/1706-cannabis-tattoo-and-barbershop</loc>",
    "<loc>https://weedmaps.com/dispensaries/galaxy-cannabis-lynnwood</loc>",
    "<loc>https://weedmaps.com/dispensaries/bad-river-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/dank-poet-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/secret-garden-7</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-sacred-herb</loc>",
    "<loc>https://weedmaps.com/dispensaries/off-the-charts-corona</loc>",
    "<loc>https://weedmaps.com/dispensaries/sociale-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/silver-stem-fine-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/lucky-me-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/ridankulous</loc>",
    "<loc>https://weedmaps.com/dispensaries/statis-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/nuggets-canna-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/soma-leaf-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/puff-kalamazoo</loc>",
    "<loc>https://weedmaps.com/dispensaries/catalyst-south-figueroa</loc>",
    "<loc>https://weedmaps.com/dispensaries/rock-steady-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-landing-monroe</loc>",
    "<loc>https://weedmaps.com/dispensaries/ck-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/mr-nice-guy-costa-mesa</loc>",
    "<loc>https://weedmaps.com/dispensaries/cookies-gainesville</loc>",
    "<loc>https://weedmaps.com/dispensaries/mosaic-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/magnolia-road-cannabis-co-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-dragon-north-miami</loc>",
    "<loc>https://weedmaps.com/dispensaries/east-river-farms-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/flynnstoned</loc>",
    "<loc>https://weedmaps.com/dispensaries/hrbn</loc>",
    "<loc>https://weedmaps.com/dispensaries/spillway-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/catalyst-costa-mesa</loc>",
    "<loc>https://weedmaps.com/dispensaries/nar-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/round-house-reserve-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/hash-house</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-mint-cannabis-st-peters</loc>",
    "<loc>https://weedmaps.com/dispensaries/legacy-dispensers</loc>",
    "<loc>https://weedmaps.com/dispensaries/nug-7</loc>",
    "<loc>https://weedmaps.com/dispensaries/stiiizy-oakland</loc>",
    "<loc>https://weedmaps.com/dispensaries/saphyre-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/smoking-house-csc</loc>",
    "<loc>https://weedmaps.com/dispensaries/sugar-leaves-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/big-mountain-botanicals-6</loc>",
    "<loc>https://weedmaps.com/dispensaries/wyld-cannabis-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/juva-cannabis-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-hi-line-company-kalispell</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-hi-line-company-ronan</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-hi-line-company-arlee</loc>",
    "<loc>https://weedmaps.com/dispensaries/y-z-grow-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/nar-cannabis-columbus</loc>",
    "<loc>https://weedmaps.com/dispensaries/nuggets-detroit</loc>",
    "<loc>https://weedmaps.com/dispensaries/culture-south-los-angeles</loc>",
    "<loc>https://weedmaps.com/dispensaries/nexleaf-dispensary-edmond</loc>",
    "<loc>https://weedmaps.com/dispensaries/desert-boyz</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-healing-clinic-lomas-verdes</loc>",
    "<loc>https://weedmaps.com/dispensaries/pecos-valley-production-albuquerque-montgomery</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-citizen-by-klutch-canton</loc>",
    "<loc>https://weedmaps.com/dispensaries/urban-wellness-100</loc>",
    "<loc>https://weedmaps.com/dispensaries/reefer-shop-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-holy-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/mystic-greenz</loc>",
    "<loc>https://weedmaps.com/dispensaries/alton-dispensary-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/terrace-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-bee-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-cannabis-dispensary-washington</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-cannabis-dispensary-somerset</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-releaf-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/bliss-ohio</loc>",
    "<loc>https://weedmaps.com/dispensaries/boutiq-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-point-wellness-millersville</loc>",
    "<loc>https://weedmaps.com/dispensaries/nice-yield</loc>",
    "<loc>https://weedmaps.com/dispensaries/williamjane</loc>",
    "<loc>https://weedmaps.com/dispensaries/dispensary-near-me-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/ohana-cannabis-san-francisco</loc>",
    "<loc>https://weedmaps.com/dispensaries/vibe-the-ultimate-cannabis-experience</loc>",
    "<loc>https://weedmaps.com/dispensaries/jungle-house</loc>",
    "<loc>https://weedmaps.com/dispensaries/catalyst-stanton</loc>",
    "<loc>https://weedmaps.com/dispensaries/mr-nice-guy-on-harbor-costa-mesa</loc>",
    "<loc>https://weedmaps.com/dispensaries/top-cannabis-outlet</loc>",
    "<loc>https://weedmaps.com/dispensaries/pei</loc>",
    "<loc>https://weedmaps.com/dispensaries/2934-cesar-chavez-jv-llc-dba-poncho-brotherz</loc>",
    "<loc>https://weedmaps.com/dispensaries/nectar-costa-mesa</loc>",
    "<loc>https://weedmaps.com/dispensaries/dabbs-cannabis-dispensary-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/better-buds-7</loc>",
    "<loc>https://weedmaps.com/dispensaries/leaf-and-bud-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/off-the-charts-south-los-angeles</loc>",
    "<loc>https://weedmaps.com/dispensaries/standing-akimbo-colorado-springs</loc>",
    "<loc>https://weedmaps.com/dispensaries/power-plant-mt-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/gas-gas-medical-by-appointment-only</loc>",
    "<loc>https://weedmaps.com/dispensaries/elevated-by-the-cannabosslady</loc>",
    "<loc>https://weedmaps.com/dispensaries/uniq-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/royal-m</loc>",
    "<loc>https://weedmaps.com/dispensaries/lume-southfield</loc>",
    "<loc>https://weedmaps.com/dispensaries/lume-portage</loc>",
    "<loc>https://weedmaps.com/dispensaries/ignite-cannabis-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-cure-company-south-la</loc>",
    "<loc>https://weedmaps.com/dispensaries/united-pomona</loc>",
    "<loc>https://weedmaps.com/dispensaries/bluedream-barcelona</loc>",
    "<loc>https://weedmaps.com/dispensaries/stache</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-xpress-11</loc>",
    "<loc>https://weedmaps.com/dispensaries/taos-420-cannabis-coffee</loc>",
    "<loc>https://weedmaps.com/dispensaries/northern-hills-alternative-health</loc>",
    "<loc>https://weedmaps.com/dispensaries/rugged-roots-trading-post</loc>",
    "<loc>https://weedmaps.com/dispensaries/sacred-bloom</loc>",
    "<loc>https://weedmaps.com/dispensaries/leaf-flower-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/kootenay-cannabis-kingsway</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-warehouse-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/reefer-joe-s-midwest-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/cc-health-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/fuel-420-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/blyss-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/coastal-green-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/quality-roots-ypsilanti</loc>",
    "<loc>https://weedmaps.com/dispensaries/planet-buds-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/beehive-farmacy-brigham-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/namaste</loc>",
    "<loc>https://weedmaps.com/dispensaries/livwell-enlightened-health-dtc</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-cannabis-dispensary-orlando-north-obt</loc>",
    "<loc>https://weedmaps.com/dispensaries/megan-s-organic-market-el-centro</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-loaded-toad-medicinal-garden</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannatural</loc>",
    "<loc>https://weedmaps.com/dispensaries/fresh-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-epic-remedy-colorado-ave</loc>",
    "<loc>https://weedmaps.com/dispensaries/sessions-cannabis-goulais-river</loc>",
    "<loc>https://weedmaps.com/dispensaries/nug-bishop</loc>",
    "<loc>https://weedmaps.com/dispensaries/smooth-roots-veneta</loc>",
    "<loc>https://weedmaps.com/dispensaries/smooth-roots-lincoln-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/primitiv-group-boston</loc>",
    "<loc>https://weedmaps.com/dispensaries/sessions-cannabis-waterloo-laurelwood</loc>",
    "<loc>https://weedmaps.com/dispensaries/sessions-cannabis-tottenham</loc>",
    "<loc>https://weedmaps.com/dispensaries/cloudwalker-farm</loc>",
    "<loc>https://weedmaps.com/dispensaries/lion-s-den-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/monster-house-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/crown-leaf-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/rsd-of-michigan</loc>",
    "<loc>https://weedmaps.com/dispensaries/potters-cannabis-boutique</loc>",
    "<loc>https://weedmaps.com/dispensaries/budbar-express-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/rootdown</loc>",
    "<loc>https://weedmaps.com/dispensaries/evermore-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/thrive-craft-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-realm-of-new-york</loc>",
    "<loc>https://weedmaps.com/dispensaries/genesis-farms-rapid-city-west</loc>",
    "<loc>https://weedmaps.com/dispensaries/genesis-farms-aberdeen</loc>",
    "<loc>https://weedmaps.com/dispensaries/genesis-farms-rapid-city-east</loc>",
    "<loc>https://weedmaps.com/dispensaries/shanti-cafe</loc>",
    "<loc>https://weedmaps.com/dispensaries/genesis-farms-sioux-falls</loc>",
    "<loc>https://weedmaps.com/dispensaries/genesis-farms-yankton</loc>",
    "<loc>https://weedmaps.com/dispensaries/bwell-condado</loc>",
    "<loc>https://weedmaps.com/dispensaries/bwell-bayamon-callecorrea</loc>",
    "<loc>https://weedmaps.com/dispensaries/bwell-torrimar</loc>",
    "<loc>https://weedmaps.com/dispensaries/bwell-ocean-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/cj-s-place</loc>",
    "<loc>https://weedmaps.com/dispensaries/bwell-caguas-6</loc>",
    "<loc>https://weedmaps.com/dispensaries/bwell-viejo-san-juan</loc>",
    "<loc>https://weedmaps.com/dispensaries/bwell-carolina</loc>",
    "<loc>https://weedmaps.com/dispensaries/stage-one-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/little-beach-harvest-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-landing-columbus</loc>",
    "<loc>https://weedmaps.com/dispensaries/wobbly-moose</loc>",
    "<loc>https://weedmaps.com/dispensaries/lume-big-rapids</loc>",
    "<loc>https://weedmaps.com/dispensaries/lume-portage-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/rolling-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-society-11</loc>",
    "<loc>https://weedmaps.com/dispensaries/evolve-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/blue-oak</loc>",
    "<loc>https://weedmaps.com/dispensaries/score-420-farmington</loc>",
    "<loc>https://weedmaps.com/dispensaries/a21-wellness-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/score-420-hobbs</loc>",
    "<loc>https://weedmaps.com/dispensaries/dosha</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-cannabis-dispensary-palm-bay</loc>",
    "<loc>https://weedmaps.com/dispensaries/nectar-4126</loc>",
    "<loc>https://weedmaps.com/dispensaries/nectar-4127</loc>",
    "<loc>https://weedmaps.com/dispensaries/nectar-4128</loc>",
    "<loc>https://weedmaps.com/dispensaries/dr-green-organics-las-cruces</loc>",
    "<loc>https://weedmaps.com/dispensaries/fire-pharms-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/new-dia-fenway-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/smoke-smart</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-spark</loc>",
    "<loc>https://weedmaps.com/dispensaries/blooming-world-cannabis-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/dank-716</loc>",
    "<loc>https://weedmaps.com/dispensaries/catalyst-san-bernardino-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/frapp-house-dispensary-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/bwell-santa-maria-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-black-market-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/el-botanico-21</loc>",
    "<loc>https://weedmaps.com/dispensaries/duende-nervio</loc>",
    "<loc>https://weedmaps.com/dispensaries/starbuds-lakewood</loc>",
    "<loc>https://weedmaps.com/dispensaries/ohio-cannabis-company-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/house-of-buds-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/dabbs</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-leaf-central</loc>",
    "<loc>https://weedmaps.com/dispensaries/firefly-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/mesa-flowers-canna-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/yellowstone-cannabis-care-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/new-era-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/ethos-pittsburgh-north-at-harmarville</loc>",
    "<loc>https://weedmaps.com/dispensaries/ethos-dispensary-northeast-philadelphia</loc>",
    "<loc>https://weedmaps.com/dispensaries/ethos-dispensary-lebanon-formerly-about-wellness-ohio</loc>",
    "<loc>https://weedmaps.com/dispensaries/culture-cannabis-canyon-lake</loc>",
    "<loc>https://weedmaps.com/dispensaries/moses-roses-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/cloudi-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabist-suffolk</loc>",
    "<loc>https://weedmaps.com/dispensaries/honeygrove-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-co-mississauga</loc>",
    "<loc>https://weedmaps.com/dispensaries/trees-hollywood</loc>",
    "<loc>https://weedmaps.com/dispensaries/select-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-healthy-cloud-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/farmers-market-703</loc>",
    "<loc>https://weedmaps.com/dispensaries/tmm-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-dank-closet-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/thrive-cottleville</loc>",
    "<loc>https://weedmaps.com/dispensaries/balboa-bud</loc>",
    "<loc>https://weedmaps.com/dispensaries/embarc-ventura</loc>",
    "<loc>https://weedmaps.com/dispensaries/haven-lakewood</loc>",
    "<loc>https://weedmaps.com/dispensaries/urban-wellness-101</loc>",
    "<loc>https://weedmaps.com/dispensaries/thrive-branson-west</loc>",
    "<loc>https://weedmaps.com/dispensaries/backroad-wellness-new-boston</loc>",
    "<loc>https://weedmaps.com/dispensaries/apex-noire</loc>",
    "<loc>https://weedmaps.com/dispensaries/t-s-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/t-s-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/upstate-canna-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/gatsby-cannabis-co-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-west-palm-beach-okeechobee-blvd</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-boynton-beach-dispensary-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-palm-beach-gardens-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-cutler-bay</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-lake-worth</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-summerfield-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-port-orange</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-leesburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-titusville-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-inverness</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-tamarac-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/asociacion-cannabica-uhuru</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-fort-myers</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-lehigh-acres</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-eustis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-clermont-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-fort-walton-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-tampa-fairgrounds</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-gainesville-archer</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-tampa-citrus-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-seffner</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-riverview</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-apollo-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-jacksonville-arrowhead</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-tallahassee-tennessee</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-st-petersburg-tyrone</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-society-birch-run</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-palm-harbor-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-dunedin-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-jacksonville-baymeadows-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/jacksonville-southside-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-clearwater-gulf-to-bay-medical-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-tavernier</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-palatka</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulive-auburndale</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-west-bradenton</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-bartow</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-osprey</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-sebring-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-winter-haven</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-sebastian-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-winter-haven-cypress-gardens</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-port-st-lucie-se-port-st-lucie</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-okeechobee</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-port-richey</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-hobe-sound</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-fort-pierce</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-tarpon-springs-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-crystal-river</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-punta-gorda-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-north-port</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-pace</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-davenport-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-merritt-island-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-kissimmee-e-irlo-bronson</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-kissimmee-w-irlo-bronson-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-kissimmee-south</loc>",
    "<loc>https://weedmaps.com/dispensaries/joyology-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-winter-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-longwood-downtown</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-sanford</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-oviedo</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-orange-city-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/alternative-relief-dba-local-leaf-medical</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbal-wellness-solutions</loc>",
    "<loc>https://weedmaps.com/dispensaries/old-gods-dispensary-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/presidio</loc>",
    "<loc>https://weedmaps.com/dispensaries/olswell-cannabis-co-6</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenery-spot</loc>",
    "<loc>https://weedmaps.com/dispensaries/cultivarx</loc>",
    "<loc>https://weedmaps.com/dispensaries/roadrunner-greens</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-marietta</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-pooler</loc>",
    "<loc>https://weedmaps.com/dispensaries/wolf-creek-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-newnan</loc>",
    "<loc>https://weedmaps.com/dispensaries/loving-bud-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/culture-cannabis-club-marijuana-and-weed-dispensary-corona</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-limerick</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-camp-hill-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-york</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieves-coatesville</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-whitehall</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-pittsburgh-north-shore</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-cranberry-township</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-philadelphia-washington-square</loc>",
    "<loc>https://weedmaps.com/dispensaries/terrabis-grayville</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-circle-okc</loc>",
    "<loc>https://weedmaps.com/dispensaries/bright-bloom-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/star-buds-ocean-springs</loc>",
    "<loc>https://weedmaps.com/dispensaries/r-greenleaf-sunland-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/r-greenleaf-hobbs</loc>",
    "<loc>https://weedmaps.com/dispensaries/r-greenleaf-carlsbad</loc>",
    "<loc>https://weedmaps.com/dispensaries/r-greenleaf-alamogordo</loc>",
    "<loc>https://weedmaps.com/dispensaries/r-greenleaf-ruidoso</loc>",
    "<loc>https://weedmaps.com/dispensaries/apothecarium-green</loc>",
    "<loc>https://weedmaps.com/dispensaries/legendary-organics-oxnard-camarillo-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-beckley</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-huntington</loc>",
    "<loc>https://weedmaps.com/dispensaries/all-american-bud-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/happy-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/spot420-weston-road</loc>",
    "<loc>https://weedmaps.com/dispensaries/flower-haus</loc>",
    "<loc>https://weedmaps.com/dispensaries/phamily-trees</loc>",
    "<loc>https://weedmaps.com/dispensaries/joyology-of-memphis</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-corner-farmacy</loc>",
    "<loc>https://weedmaps.com/dispensaries/morrison-gardens</loc>",
    "<loc>https://weedmaps.com/dispensaries/queen-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/la-2-ok-dispensary-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/dispensary-plus-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-pharm-allegan</loc>",
    "<loc>https://weedmaps.com/dispensaries/gladiator-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/pure-wellness-medical-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/broccoli-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-tides-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-herbal-club</loc>",
    "<loc>https://weedmaps.com/dispensaries/warpath-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/rocky-mountain-cannabis-12</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-cannabis-dispensary-jacksonville-riverside</loc>",
    "<loc>https://weedmaps.com/dispensaries/hazy-moose-by-appointment-only</loc>",
    "<loc>https://weedmaps.com/dispensaries/dr-greenthumb-s-west-la</loc>",
    "<loc>https://weedmaps.com/dispensaries/wildfire-retail-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/stash-dash-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/parkway-dispensary-tilton</loc>",
    "<loc>https://weedmaps.com/dispensaries/liv-cannabis-westland</loc>",
    "<loc>https://weedmaps.com/dispensaries/camden-apothecary-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/royzzz-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/safeport-oxnard</loc>",
    "<loc>https://weedmaps.com/dispensaries/d-s-best-friends-dispensary-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/jars-cannabis-wayland</loc>",
    "<loc>https://weedmaps.com/dispensaries/heaters-edmond</loc>",
    "<loc>https://weedmaps.com/dispensaries/off-the-charts-albuquerque</loc>",
    "<loc>https://weedmaps.com/dispensaries/bleu-diamond</loc>",
    "<loc>https://weedmaps.com/dispensaries/soma-leaf-company-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/46-releaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbal-iq</loc>",
    "<loc>https://weedmaps.com/dispensaries/puff-monroe</loc>",
    "<loc>https://weedmaps.com/dispensaries/strain-stars</loc>",
    "<loc>https://weedmaps.com/dispensaries/nuggies-cannabis-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/st-canapa-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/blue-whale-enterprises-dba-cookies-new-mexico</loc>",
    "<loc>https://weedmaps.com/dispensaries/kushland</loc>",
    "<loc>https://weedmaps.com/dispensaries/bloom-8</loc>",
    "<loc>https://weedmaps.com/dispensaries/smacked-village</loc>",
    "<loc>https://weedmaps.com/dispensaries/main-street-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/nirvana-center-menominee</loc>",
    "<loc>https://weedmaps.com/dispensaries/nirvana-center-florence</loc>",
    "<loc>https://weedmaps.com/dispensaries/down-to-the-roots</loc>",
    "<loc>https://weedmaps.com/dispensaries/indigo-farms</loc>",
    "<loc>https://weedmaps.com/dispensaries/phoenix-cannabis-co-fountain-hills</loc>",
    "<loc>https://weedmaps.com/dispensaries/okie-wonderland-vera</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-flowery-port-orange</loc>",
    "<loc>https://weedmaps.com/dispensaries/dabbs-cannabis-dispensary-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/story-of-cincinnati</loc>",
    "<loc>https://weedmaps.com/dispensaries/kingdom-kind</loc>",
    "<loc>https://weedmaps.com/dispensaries/mothaplant-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-buffalo-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/amg-gold-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/flora-terra-4th-street</loc>",
    "<loc>https://weedmaps.com/dispensaries/standing-akimbo-fort-collins</loc>",
    "<loc>https://weedmaps.com/dispensaries/flora-terra-sonoma-highway</loc>",
    "<loc>https://weedmaps.com/dispensaries/debbie-s-dispensary-6</loc>",
    "<loc>https://weedmaps.com/dispensaries/debbie-s-dispensary-7</loc>",
    "<loc>https://weedmaps.com/dispensaries/lift-medical-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-day-farm-boonville</loc>",
    "<loc>https://weedmaps.com/dispensaries/starbuds-fort-collins</loc>",
    "<loc>https://weedmaps.com/dispensaries/timber-cannabis-co-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/royal-budz</loc>",
    "<loc>https://weedmaps.com/dispensaries/plantlife-canada-university-district</loc>",
    "<loc>https://weedmaps.com/dispensaries/plantlife-cannabis-albany</loc>",
    "<loc>https://weedmaps.com/dispensaries/plantlife-cannabis-airdrie-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/plantlife-cannabis-cobblestone</loc>",
    "<loc>https://weedmaps.com/dispensaries/jars-kingman</loc>",
    "<loc>https://weedmaps.com/dispensaries/jars-mesa</loc>",
    "<loc>https://weedmaps.com/dispensaries/jars-el-mirage</loc>",
    "<loc>https://weedmaps.com/dispensaries/plantlife-cannabis-dalhousie</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-lakes</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-lakes-cannabis-company-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-pocket-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/market-96-neighborhood-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-day-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-cannabis-dispensary-altoona</loc>",
    "<loc>https://weedmaps.com/dispensaries/ognahs-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/traditional-boyle-heights</loc>",
    "<loc>https://weedmaps.com/dispensaries/haven-cannabis-marijuana-and-weed-dispensary-stanton</loc>",
    "<loc>https://weedmaps.com/dispensaries/highly-brunswick-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/seed-of-life-labs-10</loc>",
    "<loc>https://weedmaps.com/dispensaries/original-cannabis-cafe</loc>",
    "<loc>https://weedmaps.com/dispensaries/cakehouse-san-bernardino</loc>",
    "<loc>https://weedmaps.com/dispensaries/ganja-city-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/sundial-banning</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-herbalist-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbology-cannabis-co-west-branch</loc>",
    "<loc>https://weedmaps.com/dispensaries/leaf-haus</loc>",
    "<loc>https://weedmaps.com/dispensaries/wild-legacy-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/gram-central-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/stiiizy-el-centro</loc>",
    "<loc>https://weedmaps.com/dispensaries/packs-san-bernardino</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-bud-stop-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-sacred-blend-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-artist-tree-el-sobrante</loc>",
    "<loc>https://weedmaps.com/dispensaries/aroma-winterhaven</loc>",
    "<loc>https://weedmaps.com/dispensaries/kika-kush-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenport-cannabis-co-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/sabia-collective-guaynabo</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-high-country</loc>",
    "<loc>https://weedmaps.com/dispensaries/kelly-s-green</loc>",
    "<loc>https://weedmaps.com/dispensaries/kelly-s-green-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/kelly-s-green-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-union</loc>",
    "<loc>https://weedmaps.com/dispensaries/rebud-highland-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/top-shelf-botanicals-10</loc>",
    "<loc>https://weedmaps.com/dispensaries/emeraldrose</loc>",
    "<loc>https://weedmaps.com/dispensaries/nubes-dispensary-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/buzz-mission-valley</loc>",
    "<loc>https://weedmaps.com/dispensaries/catalyst-van-nuys-now-open</loc>",
    "<loc>https://weedmaps.com/dispensaries/stiiizy-corona</loc>",
    "<loc>https://weedmaps.com/dispensaries/zen-leaf-norwich</loc>",
    "<loc>https://weedmaps.com/dispensaries/gotham-buds-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/scorpion-grass-south-anchorage</loc>",
    "<loc>https://weedmaps.com/dispensaries/loe-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/a-little-bud-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/mind-yo-budnizz-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/winooski-organics</loc>",
    "<loc>https://weedmaps.com/dispensaries/31-north</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-cruz</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-wellness-plymouth-meeting-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/con-bud</loc>",
    "<loc>https://weedmaps.com/dispensaries/catalyst-daly-city-now-open</loc>",
    "<loc>https://weedmaps.com/dispensaries/dispo-dispensary-benton-harbor</loc>",
    "<loc>https://weedmaps.com/dispensaries/alphabud-cannabis-company-sherbourne</loc>",
    "<loc>https://weedmaps.com/dispensaries/terp-bros</loc>",
    "<loc>https://weedmaps.com/dispensaries/little-city-greens</loc>",
    "<loc>https://weedmaps.com/dispensaries/mary-jane-junction</loc>",
    "<loc>https://weedmaps.com/dispensaries/double-dogs-cannabis-big-sky-mt</loc>",
    "<loc>https://weedmaps.com/dispensaries/double-dogs-cannabis-plentywood-mt</loc>",
    "<loc>https://weedmaps.com/dispensaries/vermont-goodfire-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/double-dogs-cannabis-bozeman-mt-west-main</loc>",
    "<loc>https://weedmaps.com/dispensaries/double-dogs-cannabis-sidney-mt</loc>",
    "<loc>https://weedmaps.com/dispensaries/plantlife-cannabis-harvest-pointe</loc>",
    "<loc>https://weedmaps.com/dispensaries/plantlife-cannabis-hollick-kenyon</loc>",
    "<loc>https://weedmaps.com/dispensaries/pure-options-cannabis-dispensary-detroit</loc>",
    "<loc>https://weedmaps.com/dispensaries/nuvue-pharma</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-cakehouse-lansing</loc>",
    "<loc>https://weedmaps.com/dispensaries/bwell-ponce</loc>",
    "<loc>https://weedmaps.com/dispensaries/cookies-north-miami-coming-soon</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenport-cannabis-damariscotta</loc>",
    "<loc>https://weedmaps.com/dispensaries/oklahoma-s-finest-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-mountain-cannabis-works</loc>",
    "<loc>https://weedmaps.com/dispensaries/insa-willoughby-hills</loc>",
    "<loc>https://weedmaps.com/dispensaries/valley-meade-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/mpx-nj-gloucester</loc>",
    "<loc>https://weedmaps.com/dispensaries/mpx-nj-pennsauken</loc>",
    "<loc>https://weedmaps.com/dispensaries/be-staten-island</loc>",
    "<loc>https://weedmaps.com/dispensaries/growhealthy-clearwater-largo</loc>",
    "<loc>https://weedmaps.com/dispensaries/growhealthy-pensacola</loc>",
    "<loc>https://weedmaps.com/dispensaries/superior-buds-battle-creek</loc>",
    "<loc>https://weedmaps.com/dispensaries/county-meds</loc>",
    "<loc>https://weedmaps.com/dispensaries/s-s-vibes</loc>",
    "<loc>https://weedmaps.com/dispensaries/flower-of-life-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/revolution-dispensary-sunrise-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/enlightened-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/score-420-clovis</loc>",
    "<loc>https://weedmaps.com/dispensaries/leafology-cannabis-dispensary-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/headies-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/united-cultivation-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/levels-cannabis-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-dragon-orlando</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-dragon-madison</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-dragon-palatka</loc>",
    "<loc>https://weedmaps.com/dispensaries/plantlife-cannabis-ice-district</loc>",
    "<loc>https://weedmaps.com/dispensaries/plantlife-cannabis-jagare-ridge</loc>",
    "<loc>https://weedmaps.com/dispensaries/plantlife-cannnabis-jasper</loc>",
    "<loc>https://weedmaps.com/dispensaries/plantlife-cannabis-leduc</loc>",
    "<loc>https://weedmaps.com/dispensaries/plantlife-cannabis-mahogany</loc>",
    "<loc>https://weedmaps.com/dispensaries/plantlife-cannabis-mahogany-market</loc>",
    "<loc>https://weedmaps.com/dispensaries/plantlife-cannabis-orchards</loc>",
    "<loc>https://weedmaps.com/dispensaries/ganjika-gardens</loc>",
    "<loc>https://weedmaps.com/dispensaries/plantlife-cannabis-shawnessy</loc>",
    "<loc>https://weedmaps.com/dispensaries/plantllife-cannabis-shepard</loc>",
    "<loc>https://weedmaps.com/dispensaries/blue-sage</loc>",
    "<loc>https://weedmaps.com/dispensaries/native-sun</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-day-farm-eagleville</loc>",
    "<loc>https://weedmaps.com/dispensaries/plantlife-cannabis-sherwood-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/native-nations-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/verilife-silver-spring</loc>",
    "<loc>https://weedmaps.com/dispensaries/airfield-supply-company-redwood-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/plantlife-cannabis-erin-ridge</loc>",
    "<loc>https://weedmaps.com/dispensaries/plantlife-cannabis-wainwright</loc>",
    "<loc>https://weedmaps.com/dispensaries/benzeen-la-brea</loc>",
    "<loc>https://weedmaps.com/dispensaries/munchiez-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/devil-s-den-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/matterhorn-apothecary-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/juniper-lane</loc>",
    "<loc>https://weedmaps.com/dispensaries/top-shelf-botanicals-11</loc>",
    "<loc>https://weedmaps.com/dispensaries/top-shelf-botanicals-12</loc>",
    "<loc>https://weedmaps.com/dispensaries/top-shelf-botanicals-13</loc>",
    "<loc>https://weedmaps.com/dispensaries/top-shelf-botanicals-14</loc>",
    "<loc>https://weedmaps.com/dispensaries/marina-greens</loc>",
    "<loc>https://weedmaps.com/dispensaries/montana-kush-butte</loc>",
    "<loc>https://weedmaps.com/dispensaries/higher-ground-moreno-valley</loc>",
    "<loc>https://weedmaps.com/dispensaries/spiritleaf-beaches-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/herbaceous-inc-bozeman</loc>",
    "<loc>https://weedmaps.com/dispensaries/megan-s-organic-market-goleta</loc>",
    "<loc>https://weedmaps.com/dispensaries/elixir-extracts</loc>",
    "<loc>https://weedmaps.com/dispensaries/stone-leaf-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannected</loc>",
    "<loc>https://weedmaps.com/dispensaries/kushies</loc>",
    "<loc>https://weedmaps.com/dispensaries/hydro-kush-house</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-delight</loc>",
    "<loc>https://weedmaps.com/dispensaries/kinship-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-landing-dispensary-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/indigenous-budz</loc>",
    "<loc>https://weedmaps.com/dispensaries/san-juan-strains-gunnison</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-season-moreno-valley</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-station-fresno</loc>",
    "<loc>https://weedmaps.com/dispensaries/grizzly-pine-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/zen-leaf-newington</loc>",
    "<loc>https://weedmaps.com/dispensaries/bleezies</loc>",
    "<loc>https://weedmaps.com/dispensaries/house-of-trees</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-snowtree-farms-hash-hut</loc>",
    "<loc>https://weedmaps.com/dispensaries/glenrio-smoke-stop</loc>",
    "<loc>https://weedmaps.com/dispensaries/mj-dispensary-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-cannabis-place</loc>",
    "<loc>https://weedmaps.com/dispensaries/better-science</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-xchange-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/a-golden-state-redding</loc>",
    "<loc>https://weedmaps.com/dispensaries/rootdown-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/rootdown-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/rootdown-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/backpack-boyz-hollywood-sunset</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-leaf-solutions-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/cherry-industries</loc>",
    "<loc>https://weedmaps.com/dispensaries/kush-kabana</loc>",
    "<loc>https://weedmaps.com/dispensaries/fog-products</loc>",
    "<loc>https://weedmaps.com/dispensaries/story-of-silver-spring-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/apogee-gardens-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/element-7-montclair-village</loc>",
    "<loc>https://weedmaps.com/dispensaries/higher-collective-killingly</loc>",
    "<loc>https://weedmaps.com/dispensaries/higher-collective-willington</loc>",
    "<loc>https://weedmaps.com/dispensaries/mmd-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/cousins-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/kannabis-klub</loc>",
    "<loc>https://weedmaps.com/dispensaries/worner-brothers-outpost-solotna</loc>",
    "<loc>https://weedmaps.com/dispensaries/premier-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/freeway-ricks</loc>",
    "<loc>https://weedmaps.com/dispensaries/natural-wellness-harve</loc>",
    "<loc>https://weedmaps.com/dispensaries/lake-life-farms-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/orange-county-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/royal-weed-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/phasal</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-society-12</loc>",
    "<loc>https://weedmaps.com/dispensaries/elevate-6011</loc>",
    "<loc>https://weedmaps.com/dispensaries/off-the-charts-harbor-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/higher-collective-new-london</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-life-maine</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-acres-ecorse</loc>",
    "<loc>https://weedmaps.com/dispensaries/rootdown-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/houz</loc>",
    "<loc>https://weedmaps.com/dispensaries/zenleaf-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/downtown-flwr</loc>",
    "<loc>https://weedmaps.com/dispensaries/crave-ann-arbor-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/share-springfield</loc>",
    "<loc>https://weedmaps.com/dispensaries/misty-mountain-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/ezay-leaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/northern-enlightenment</loc>",
    "<loc>https://weedmaps.com/dispensaries/pot-of-gold-6</loc>",
    "<loc>https://weedmaps.com/dispensaries/ganja-by-sarah</loc>",
    "<loc>https://weedmaps.com/dispensaries/little-buddy-farm-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/del-mundo-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/mystic-greenz-lincoln</loc>",
    "<loc>https://weedmaps.com/dispensaries/joyleaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/levels-cannabis-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/higher-collective-hartford</loc>",
    "<loc>https://weedmaps.com/dispensaries/space-flyt-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-mint-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/happy-place-cannabis-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/stiiizy-noho</loc>",
    "<loc>https://weedmaps.com/dispensaries/tokyo-smoke-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/hello-high-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/kushup</loc>",
    "<loc>https://weedmaps.com/dispensaries/root-66-wentzville-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/marimint-32nd-st</loc>",
    "<loc>https://weedmaps.com/dispensaries/marimint-lockwood</loc>",
    "<loc>https://weedmaps.com/dispensaries/marimint-butte</loc>",
    "<loc>https://weedmaps.com/dispensaries/marimint-great-falls</loc>",
    "<loc>https://weedmaps.com/dispensaries/marimint-missoula</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-day-farm-dispensary-shreveport</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-day-farm-dispensary-lake-charles</loc>",
    "<loc>https://weedmaps.com/dispensaries/la-santa-boadilla</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-road</loc>",
    "<loc>https://weedmaps.com/dispensaries/berkshire-roots-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/revolution-dispensary-moline</loc>",
    "<loc>https://weedmaps.com/dispensaries/codes-osage-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-flowery-tallahassee</loc>",
    "<loc>https://weedmaps.com/dispensaries/lux-leaf-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/sensi-retail</loc>",
    "<loc>https://weedmaps.com/dispensaries/ganja-town-okc</loc>",
    "<loc>https://weedmaps.com/dispensaries/pure-retail</loc>",
    "<loc>https://weedmaps.com/dispensaries/twisted-bee</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-n-fort-myers</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-marianna</loc>",
    "<loc>https://weedmaps.com/dispensaries/trulieve-gainesville-college-park-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/kannabis-works-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-river-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/nirvana-center-houghton</loc>",
    "<loc>https://weedmaps.com/dispensaries/curaleaf-sarasota-university-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/cookies-peoria-heights</loc>",
    "<loc>https://weedmaps.com/dispensaries/capital-district-cannabis-wellness-inc</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-highest-peak</loc>",
    "<loc>https://weedmaps.com/dispensaries/oregrown-chico</loc>",
    "<loc>https://weedmaps.com/dispensaries/lenox-hill-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/keeper-of-the-green</loc>",
    "<loc>https://weedmaps.com/dispensaries/culture-cannabis-club-marijuana-and-weed-dispensary-fresno-bullard-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/culture-cannabis-club-marijuana-and-weed-dispensary-fresno-maple-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/blkbrn-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-road</loc>",
    "<loc>https://weedmaps.com/dispensaries/camo-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/cookies-fresno</loc>",
    "<loc>https://weedmaps.com/dispensaries/oasis-cannabis-coors</loc>",
    "<loc>https://weedmaps.com/dispensaries/verdant-creations-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/phoenix-cannabis-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-wellness-dayton</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-wellness-goshen</loc>",
    "<loc>https://weedmaps.com/dispensaries/ayr-wellness-woodmere</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-cannabis-place-queens</loc>",
    "<loc>https://weedmaps.com/dispensaries/next-elevation-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/mass-grown</loc>",
    "<loc>https://weedmaps.com/dispensaries/oasis-cannabis-osuna</loc>",
    "<loc>https://weedmaps.com/dispensaries/elevate-cannbis-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/codes-chillicothe</loc>",
    "<loc>https://weedmaps.com/dispensaries/codes-hannibal</loc>",
    "<loc>https://weedmaps.com/dispensaries/codes-moberly</loc>",
    "<loc>https://weedmaps.com/dispensaries/far-dotter-olive-branch</loc>",
    "<loc>https://weedmaps.com/dispensaries/wilshire-madison</loc>",
    "<loc>https://weedmaps.com/dispensaries/potherb-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/light-em-up-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/social-cannabis-golden</loc>",
    "<loc>https://weedmaps.com/dispensaries/pure-oasis-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/embarc-tracy</loc>",
    "<loc>https://weedmaps.com/dispensaries/kapn-chronic</loc>",
    "<loc>https://weedmaps.com/dispensaries/oddfellow-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/melo-cannabis-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/super-dank</loc>",
    "<loc>https://weedmaps.com/dispensaries/mission-mountain-organics</loc>",
    "<loc>https://weedmaps.com/dispensaries/culture-cannabis-club-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-21-mission-valley</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-society-13</loc>",
    "<loc>https://weedmaps.com/dispensaries/bwell-vega-alta</loc>",
    "<loc>https://weedmaps.com/dispensaries/advanced-cannabis-care-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/fivepoint-cannabis-1st-street-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/amplify-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/ashe-society-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/holy-cannabis-oxford</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-herbal-care-thc-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-firehaus-ny</loc>",
    "<loc>https://weedmaps.com/dispensaries/hush-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/grow-together-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/fourtwenty</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-society-lenox</loc>",
    "<loc>https://weedmaps.com/dispensaries/ganja-depot</loc>",
    "<loc>https://weedmaps.com/dispensaries/vibes-at-waterville</loc>",
    "<loc>https://weedmaps.com/dispensaries/plantabis-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/earthmed-mchenry</loc>",
    "<loc>https://weedmaps.com/dispensaries/tradecraft-lancaster</loc>",
    "<loc>https://weedmaps.com/dispensaries/golden-valley-honey-laurel</loc>",
    "<loc>https://weedmaps.com/dispensaries/dispensary-near-me-portales</loc>",
    "<loc>https://weedmaps.com/dispensaries/nm-green-door-clovis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/pure-blossom</loc>",
    "<loc>https://weedmaps.com/dispensaries/716-cannabis-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/big-mountain-botanicals-7</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-fruit-factory-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-fruit-factory-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-fruit-factory-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/nbt-group-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/evolve</loc>",
    "<loc>https://weedmaps.com/dispensaries/premier-earth</loc>",
    "<loc>https://weedmaps.com/dispensaries/tj-s-cannabis-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/electric-lettuce-damascus</loc>",
    "<loc>https://weedmaps.com/dispensaries/electric-lettuce-holgate</loc>",
    "<loc>https://weedmaps.com/dispensaries/bloomery-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-dragon-avon-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-dragon-englewood</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-dragon-lake-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-green-standard-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/blackjack-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/flight-420</loc>",
    "<loc>https://weedmaps.com/dispensaries/delta-dispensary-magnolia</loc>",
    "<loc>https://weedmaps.com/dispensaries/dabbs-cannabis-dispensary-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/showcase</loc>",
    "<loc>https://weedmaps.com/dispensaries/pleasure-med-lounge</loc>",
    "<loc>https://weedmaps.com/dispensaries/blue-river-terps-somerville</loc>",
    "<loc>https://weedmaps.com/dispensaries/cookies-tampa</loc>",
    "<loc>https://weedmaps.com/dispensaries/ether-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/elite-cannabis-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/best-buds-514</loc>",
    "<loc>https://weedmaps.com/dispensaries/north-bay-marijuana-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-mint-cannabis-75th-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-flowery-staten-island</loc>",
    "<loc>https://weedmaps.com/dispensaries/20-past-4-sturgis</loc>",
    "<loc>https://weedmaps.com/dispensaries/loveco-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/hightide-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/dagmar-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/love-cannabis-dispensary-isleta-blvd</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-street-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/one-green-leaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/medicated-living-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/verona-collective</loc>",
    "<loc>https://weedmaps.com/dispensaries/dabbs-cannabis-dispensary-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/happy-days-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/get-sum</loc>",
    "<loc>https://weedmaps.com/dispensaries/cookies-chicago</loc>",
    "<loc>https://weedmaps.com/dispensaries/ript-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/gnomad-ruidoso</loc>",
    "<loc>https://weedmaps.com/dispensaries/phula</loc>",
    "<loc>https://weedmaps.com/dispensaries/pecos-valley-production-central-ave</loc>",
    "<loc>https://weedmaps.com/dispensaries/bowz-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannabis-house-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/story-of-lake-havasu</loc>",
    "<loc>https://weedmaps.com/dispensaries/embarc-indio</loc>",
    "<loc>https://weedmaps.com/dispensaries/story-of-kingman</loc>",
    "<loc>https://weedmaps.com/dispensaries/embarc-meyers</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-mint-cannabis-scottsdale</loc>",
    "<loc>https://weedmaps.com/dispensaries/star-buds-lakeside</loc>",
    "<loc>https://weedmaps.com/dispensaries/noxx-peoria</loc>",
    "<loc>https://weedmaps.com/dispensaries/buddies</loc>",
    "<loc>https://weedmaps.com/dispensaries/ponderosa-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/ponderosa-dispensary-florence</loc>",
    "<loc>https://weedmaps.com/dispensaries/elevate-adk</loc>",
    "<loc>https://weedmaps.com/dispensaries/jars-port-huron</loc>",
    "<loc>https://weedmaps.com/dispensaries/jars-chesterfield</loc>",
    "<loc>https://weedmaps.com/dispensaries/genesis-farms-huron</loc>",
    "<loc>https://weedmaps.com/dispensaries/genesis-farms-mitchell</loc>",
    "<loc>https://weedmaps.com/dispensaries/genesis-farms-pierre</loc>",
    "<loc>https://weedmaps.com/dispensaries/verdi</loc>",
    "<loc>https://weedmaps.com/dispensaries/urban-smoke-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/bk-exotics</loc>",
    "<loc>https://weedmaps.com/dispensaries/cookies-melrose</loc>",
    "<loc>https://weedmaps.com/dispensaries/fabulous-southside</loc>",
    "<loc>https://weedmaps.com/dispensaries/healing-side</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-cake-house-ann-arbor</loc>",
    "<loc>https://weedmaps.com/dispensaries/callie-s-cannabis-shoppe-broomfield</loc>",
    "<loc>https://weedmaps.com/dispensaries/kannabis-king</loc>",
    "<loc>https://weedmaps.com/dispensaries/space-camp</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannavista-wellness-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-travel-agency</loc>",
    "<loc>https://weedmaps.com/dispensaries/r-greenleaf-clovis</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-family-first-powered-by-cali-x</loc>",
    "<loc>https://weedmaps.com/dispensaries/garden-state-botanicals</loc>",
    "<loc>https://weedmaps.com/dispensaries/r-greenleaf-cottonwood</loc>",
    "<loc>https://weedmaps.com/dispensaries/r-greenleaf-las-vegas</loc>",
    "<loc>https://weedmaps.com/dispensaries/r-greenleaf-los-lunas</loc>",
    "<loc>https://weedmaps.com/dispensaries/r-greenleaf-paseo</loc>",
    "<loc>https://weedmaps.com/dispensaries/r-greenleaf-yale</loc>",
    "<loc>https://weedmaps.com/dispensaries/elite-cannabis-saginaw</loc>",
    "<loc>https://weedmaps.com/dispensaries/top-shelf-medicine-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-degree-clapp</loc>",
    "<loc>https://weedmaps.com/dispensaries/ragged-wizard</loc>",
    "<loc>https://weedmaps.com/dispensaries/eagle-i-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/420-bliss-humble-county</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-degree-tl-oh-suu</loc>",
    "<loc>https://weedmaps.com/dispensaries/basecamp-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/theory-wellness-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/culture-house</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-paradise-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/leaf-lab-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/affx-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/og-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/royzzz-of-sioux-falls</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-cake-house-encinitas</loc>",
    "<loc>https://weedmaps.com/dispensaries/west-orange-wellness</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-cake-house-corona</loc>",
    "<loc>https://weedmaps.com/dispensaries/hog-cannabis-co-provisioning</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-fire-garden</loc>",
    "<loc>https://weedmaps.com/dispensaries/theo-a-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/new-york-city-cannabis-exchange</loc>",
    "<loc>https://weedmaps.com/dispensaries/fleur-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/backpack-boyz-monroe</loc>",
    "<loc>https://weedmaps.com/dispensaries/fig-trees</loc>",
    "<loc>https://weedmaps.com/dispensaries/highly-cannaco-harrisville</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-mint-cannabis-oxford</loc>",
    "<loc>https://weedmaps.com/dispensaries/revolution-dispensary-maryville</loc>",
    "<loc>https://weedmaps.com/dispensaries/urban-wellness-102</loc>",
    "<loc>https://weedmaps.com/dispensaries/cookies-809</loc>",
    "<loc>https://weedmaps.com/dispensaries/codes-cardwell</loc>",
    "<loc>https://weedmaps.com/dispensaries/best-buds-cabo-rojo</loc>",
    "<loc>https://weedmaps.com/dispensaries/friendly-fire-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/fuego-2</loc>",
    "<loc>https://weedmaps.com/dispensaries/5th-and-green</loc>",
    "<loc>https://weedmaps.com/dispensaries/vip-cannabis-co-cambridge-304</loc>",
    "<loc>https://weedmaps.com/dispensaries/vip-cannabis-co-london</loc>",
    "<loc>https://weedmaps.com/dispensaries/oxnard-holistics</loc>",
    "<loc>https://weedmaps.com/dispensaries/vip-cannabis-co-petersburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/happy-herb-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/northern-lights</loc>",
    "<loc>https://weedmaps.com/dispensaries/unity-rd-cannabis-shop</loc>",
    "<loc>https://weedmaps.com/dispensaries/uncle-willie-s-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/pincanna-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/element-7-laurel-village</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-land-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/okie-wonderland-riverside</loc>",
    "<loc>https://weedmaps.com/dispensaries/all-love</loc>",
    "<loc>https://weedmaps.com/dispensaries/cmcc-llc-dba-california-michigan-cannabis-company-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/lotus-garden-apothecary</loc>",
    "<loc>https://weedmaps.com/dispensaries/jungle-boys-dtla</loc>",
    "<loc>https://weedmaps.com/dispensaries/okie-wonderland-okc</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-arrow-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/ohio-cannabis-company-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/kushie-albuquerque</loc>",
    "<loc>https://weedmaps.com/dispensaries/yerba-santa</loc>",
    "<loc>https://weedmaps.com/dispensaries/jars-cannabis-hazel-park</loc>",
    "<loc>https://weedmaps.com/dispensaries/f-na-buds-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/1634-funk</loc>",
    "<loc>https://weedmaps.com/dispensaries/royale-flower-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/barbary-coast-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/grass-roots-6</loc>",
    "<loc>https://weedmaps.com/dispensaries/silk-road-nyc</loc>",
    "<loc>https://weedmaps.com/dispensaries/jackpot-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/alaskan-grown-cannabis-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/alaskan-grown-cannabis-4</loc>",
    "<loc>https://weedmaps.com/dispensaries/cleo</loc>",
    "<loc>https://weedmaps.com/dispensaries/stoner-co-recreational</loc>",
    "<loc>https://weedmaps.com/dispensaries/miss-mary-jane-s</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-day-farm-caruthersville</loc>",
    "<loc>https://weedmaps.com/dispensaries/wax-house-by-round-house-reserve</loc>",
    "<loc>https://weedmaps.com/dispensaries/star-buds-greenwood</loc>",
    "<loc>https://weedmaps.com/dispensaries/taos-420-cannabis-coffee-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/verilife</loc>",
    "<loc>https://weedmaps.com/dispensaries/puffalo-dreams</loc>",
    "<loc>https://weedmaps.com/dispensaries/sunnyside-medical-cannabis-dispensary-gettysburg</loc>",
    "<loc>https://weedmaps.com/dispensaries/cultivate207</loc>",
    "<loc>https://weedmaps.com/dispensaries/trinity-naturals</loc>",
    "<loc>https://weedmaps.com/dispensaries/dabbs-cannabis-dispensary-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/iostestautodisp</loc>",
    "<loc>https://weedmaps.com/dispensaries/tinity-naturals-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/golden-door-dispensary-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/true-north-cannabis-ak-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/atlas-organics-ok</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannavita</loc>",
    "<loc>https://weedmaps.com/dispensaries/streamside-remedies</loc>",
    "<loc>https://weedmaps.com/dispensaries/jdm-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/high-cannabis</loc>",
    "<loc>https://weedmaps.com/dispensaries/cloud-nine-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/lacannaville</loc>",
    "<loc>https://weedmaps.com/dispensaries/revo-s-finest</loc>",
    "<loc>https://weedmaps.com/dispensaries/cwd-alliance</loc>",
    "<loc>https://weedmaps.com/dispensaries/kush-kween</loc>",
    "<loc>https://weedmaps.com/dispensaries/star-buds-tupelo</loc>",
    "<loc>https://weedmaps.com/dispensaries/kudzu-cannabis-company</loc>",
    "<loc>https://weedmaps.com/dispensaries/alans-test-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/shinybud-cannabis-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/keeper-of-the-green-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/cannamiss-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/matawana</loc>",
    "<loc>https://weedmaps.com/dispensaries/colorado-cannabis-connection-3</loc>",
    "<loc>https://weedmaps.com/dispensaries/bloc-dispensary-palatine</loc>",
    "<loc>https://weedmaps.com/dispensaries/highly-green-dispensary-edmond</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-hippo-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-truck-the-garage</loc>",
    "<loc>https://weedmaps.com/dispensaries/off-the-charts-las-cruces</loc>",
    "<loc>https://weedmaps.com/dispensaries/catalyst-grover-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/canna-remedies-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/rootdown-5</loc>",
    "<loc>https://weedmaps.com/dispensaries/dreamz-farmington</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-emerald-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/dreamz-gallup</loc>",
    "<loc>https://weedmaps.com/dispensaries/dreamz-carlsbad</loc>",
    "<loc>https://weedmaps.com/dispensaries/dreamz-hobbs</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-apopka</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-auburndale</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-fort-pierce</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-jacksonville-skymarks</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-lake-city</loc>",
    "<loc>https://weedmaps.com/dispensaries/cheech-and-chong-dispensoria</loc>",
    "<loc>https://weedmaps.com/dispensaries/dreamz-taos</loc>",
    "<loc>https://weedmaps.com/dispensaries/basa-collective-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/cream-cannabis-remington-park-adventure-district</loc>",
    "<loc>https://weedmaps.com/dispensaries/greenlight-downtown-las-vegas</loc>",
    "<loc>https://weedmaps.com/dispensaries/sweet-flower-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-fire-pit-no-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/hackettstown-dispensary</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-miami-kendall</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-station-hoboken</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-navarre</loc>",
    "<loc>https://weedmaps.com/dispensaries/the-cake-house-pleasant-ridge</loc>",
    "<loc>https://weedmaps.com/dispensaries/exit-31-exotic</loc>",
    "<loc>https://weedmaps.com/dispensaries/market-96-neighborhood-dispensary-chicago</loc>",
    "<loc>https://weedmaps.com/dispensaries/off-the-charts-perris</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-north-miami-biscayne</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-orlando-colonial</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-panama-city-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-port-orange</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-satellite-beach</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-sebring-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-titusville</loc>",
    "<loc>https://weedmaps.com/dispensaries/sacred-grass</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-venice</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-west-palm-beach-datura</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-west-palm-beach-okeechobee</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-winter-spring</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-yulee</loc>",
    "<loc>https://weedmaps.com/dispensaries/muv-dispensary-zephyrhills</loc>",
    "<loc>https://weedmaps.com/dispensaries/jars-cannabis-riverview</loc>",
    "<loc>https://weedmaps.com/dispensaries/off-the-charts-goleta</loc>",
    "<loc>https://weedmaps.com/dispensaries/fire-and-oak</loc>",
    "<loc>https://weedmaps.com/dispensaries/off-the-charts-north-hollywood-noho</loc>",
    "<loc>https://weedmaps.com/dispensaries/dr-greenthumb-s-detroitt</loc>",
    "<loc>https://weedmaps.com/dispensaries/cookies-tampa-on-fletcher</loc>",
    "<loc>https://weedmaps.com/dispensaries/la-vida-garden-coming-soon</loc>",
    "<loc>https://weedmaps.com/dispensaries/brownies-dispensary-llc</loc>",
    "<loc>https://weedmaps.com/dispensaries/rugged-roots-trading-post-1</loc>",
    "<loc>https://weedmaps.com/dispensaries/dunegrassc-co</loc>",
    "<loc>https://weedmaps.com/dispensaries/vibe-river-rouge</loc>",
    "<loc>https://weedmaps.com/dispensaries/holistic-re-leaf</loc>",
    "<loc>https://weedmaps.com/dispensaries/cookies-jacksonville</loc>",
    "<loc>https://weedmaps.com/dispensaries/good-day-farm-jennings</loc>",
    "<loc>https://weedmaps.com/dispensaries/botanical-sciences</loc>",
    "<loc>https://weedmaps.com/dispensaries/cloud9-champaign</loc>",
    "<loc>https://weedmaps.com/dispensaries/cloud9-cannabis-edwardsville</loc>",
    "<loc>https://weedmaps.com/dispensaries/green-acres-421</loc>",
    "<loc>https://weedmaps.com/dispensaries/march-and-ash-palm-desert</loc>"
]
weedmaps_urls = [s[5:-6] for s in weedmaps_urls]

In [83]:
print(weedmaps_urls[:10])

['https://weedmaps.com/dispensaries', 'https://weedmaps.com/dispensaries/dc-collective', 'https://weedmaps.com/dispensaries/santa-barbara-care-center', 'https://weedmaps.com/dispensaries/barccollective', 'https://weedmaps.com/dispensaries/citycompassionatecaregivers', 'https://weedmaps.com/dispensaries/downtown-patients-group-dtpg', 'https://weedmaps.com/dispensaries/greenlight-discount-pharmacy', 'https://weedmaps.com/dispensaries/northhollywood', 'https://weedmaps.com/dispensaries/compassionate-patient-resources', 'https://weedmaps.com/dispensaries/mnrcoop']


In [84]:
import urllib.parse
from lxml import html

def get_wm_products(url):
    products = []
        
    url_str = f'{url}?filter%5BanyClientCategories%5D%5B%5D=flower-category-pages&page=1'
    print(urllib.parse.unquote(url_str))
    
    response = requests.get(url_str)
    if response.status_code != 200:
        return []
        
    root = html.fromstring(response.content)

    page_btns = root.find_class('PaginationButton-sc-4e0a1e84-2')
    p_max = 1 if len(page_btns) <= 1 else int(page_btns[-1].text)
    
    for page in range(1, p_max+1):
        list_elems = root.find_class('ListingItemContainer-sc-bc60ce4c-2')
        
        for item in list_elems:
            name = item.find_class('BaseCardTitle-sc-bc60ce4c-21')[0].text
            category = item.find_class('Category-sc-841af06d-16')[0].text
            link = item.find_class('StretchedLink-sc-bc60ce4c-18')[0].get('href')
            prices = item.find_class('PriceText-sc-b03e0af1-1')
            if len(prices) == 0:
                prices = item.find_class("eNZYOW")[0].text_content()
            else:
                prices = prices[0].text_content()
                
            products.append([name, category, prices, link])
        
        url_str = f'{url}?filter%5BanyClientCategories%5D%5B%5D=flower-category-pages&page={page+1}'
        print(urllib.parse.unquote(url_str))
        
        response = requests.get(url_str)
        root = html.fromstring(response.content)
        
        if root.find_class('NoItems-sc-d70ab7ed-7') or response.status_code != 200:
            print('done', response.status_code)
            break

    return products

In [85]:
santa_b_url = 'https://weedmaps.com/dispensaries/santa-barbara-care-center'
santa_b_products = get_wm_products(santa_b_url)
print(santa_b_products)

https://weedmaps.com/dispensaries/santa-barbara-care-center?filter[anyClientCategories][]=flower-category-pages&page=1
[]


In [86]:
wm_products = pd.DataFrame(columns=['Strain', 'Type', 'TAC', 'THC', 'Amount', 'Price', 'URL', 'Dispensary', 'Value (mgTAC/$)'])
for i in range(len(santa_b_products)):
    p = santa_b_products[i]
    wm_products.loc[len(wm_products)] = [p[0], p[1], '', '', '', float(p[2][1:]), p[3], '', '']

wm_products.sort_values('Price', ascending=True, inplace=True)
wm_products.head(30)

,Strain,Type,TAC,THC,Amount,Price,URL,Dispensary,Value (mgTAC/$)
